Reading from MongoDB

In [1]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Replace 'localhost' and port with your MongoDB server details
db = client['project']  # Replace 'your_database_name' with your actual database name
collection = db['sample_data']  # Replace 'your_collection_name' with your actual collection name

# Query MongoDB collection and convert the result to a pandas DataFrame
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

# Close the MongoDB connection
client.close()

# Display the DataFrame
df.head(15)


,_id,Feature,Label
0,663d742cff37cb97b384af27,"[-505.4991760253906, 0.0, 0.0, 0.0, 0.0, 0.0, ...",000002
1,663d742cff37cb97b384af28,"[-546.3240356445312, 3.503636360168457, 3.3916...",000003
2,663d742cff37cb97b384af29,"[-528.1162719726562, 0.020876940339803696, 0.0...",000005
3,663d742cff37cb97b384af2a,"[-537.1972045898438, 2.363110065460205, 1.7268...",000010
4,663d742cff37cb97b384af2b,"[-496.5435485839844, 8.233688354492188, 7.9586...",000020
5,663d742cff37cb97b384af2c,"[-517.364013671875, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",000026
6,663d742cff37cb97b384af2d,"[-523.1180419921875, 25.760787963867188, 25.01...",000030
7,663d742cff37cb97b384af2e,"[-547.3167724609375, 11.824804306030273, 9.149...",000046
8,663d742cff37cb97b384af2f,"[-523.75146484375, 13.946538925170898, 13.2858...",000048
9,663d742cff37cb97b384af30,"[-497.8307800292969, 1.334374189376831, 1.3210...",000134


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Pandas to PySpark DataFrame") \
    .getOrCreate()

# Remove the '_id' column from the pandas DataFrame
df = df.drop(columns=['_id'])

# Define the schema for the PySpark DataFrame
schema = StructType([
    StructField("Feature", ArrayType(FloatType()), nullable=True),
    StructField("Label", StringType(), nullable=True)
])

# Convert pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(df, schema=schema)
# Filter the PySpark DataFrame to select only the rows where Label matches the first label
#first_label = spark_df.select("Label").distinct().first()[0]
#first_label_features = spark_df.filter(spark_df["Label"] == first_label).select("Feature")

# Show the features of the first label
#first_label_features.show()


24/05/11 05:25:40 WARN Utils: Your hostname, aitazaz-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.18.54 instead (on interface wlo1)
24/05/11 05:25:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 05:25:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Show the PySpark DataFrame
spark_df.show()

# Stop the SparkSession
#spark.stop()


24/05/11 05:25:55 WARN TaskSetManager: Stage 0 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------+
|             Feature| Label|
+--------------------+------+
|[-505.49918, 0.0,...|000002|
|[-546.32404, 3.50...|000003|
|[-528.1163, 0.020...|000005|
|[-537.1972, 2.363...|000010|
|[-496.54355, 8.23...|000020|
|[-517.364, 0.0, 0...|000026|
|[-523.11804, 25.7...|000030|
|[-547.3168, 11.82...|000046|
|[-523.75146, 13.9...|000048|
|[-497.83078, 1.33...|000134|
|[-521.57806, 14.5...|000135|
|[-502.58115, 13.3...|000136|
|[-578.29645, 27.9...|000137|
|[-599.2581, 21.58...|000138|
|[-555.78485, 0.0,...|000139|
|[-592.8958, 0.0, ...|000140|
|[-536.54694, 0.0,...|000141|
|[-496.78558, 0.0,...|000142|
|[-528.72504, 3.82...|000144|
|[-533.13416, 4.79...|000145|
+--------------------+------+
only showing top 20 rows



In [97]:
print(spark_df.schema)


StructType([StructField('Feature', ArrayType(FloatType(), True), True), StructField('Label', StringType(), True)])


Checking are values Correct

In [98]:
# Filter the PySpark DataFrame to select only the rows where Label matches the first label
first_label = spark_df.select("Label").distinct().first()[0]
first_label_features = spark_df.filter(spark_df["Label"] == first_label).select("Feature")

# Collect the features into an array
features_array = first_label_features.collect()

# Print the features array
for row in features_array:
    print(row.Feature)

[-505.4991760253906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -369.0495300292969, 106.3271484375, -3.6316792964935303, -6.880691051483154, 8.13675308227539, 0.5048940181732178, 7.810166835784912, 7.916318893432617, -1.8093026876449585, 10.709980010986328, 8.038970947265625, -21.083293914794922, -27.42470932006836, -15.193597793579102, -20.957069396972656, -28.285276412963867, -15.56839656829834, -8.181970596313477, -12.256559371948242, -18.827842712402344, -244.5655975341797, 136.04049682617188, -33.93807601928711, 13.452998161315918, 4.5233683586120605, -0.28139162063598633, -4.978228569030762, 8.261866569519043, -5.819221019744873, 7.027866363525391, 6.264196395874023, -17.067081451416016, -18.826589584350586, -15.85010051727295, -7.298053741455078, -20.223766326904297, -4.052706718444824, -5.869179725646973, -13.106864929199219, -18.857725143432617, -213.77084350585938, 134.9466552734375, -44.71919631958008, 17.682056427001953, 2

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType
from pyspark.ml.linalg import VectorUDT, Vectors
from typing import List

def convert_array_to_vector(features: List[float]) -> Vectors:
    return Vectors.dense(features)

# Define UDF
convert_array_to_vector_udf = udf(convert_array_to_vector, VectorUDT())

# Apply the UDF to the "features" column
result_df = spark_df.withColumn("feature", convert_array_to_vector_udf(spark_df["feature"]))


In [108]:
print(result_df.schema)

StructType([StructField('feature', VectorUDT(), True), StructField('Label', StringType(), True)])


In [5]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
# Initialize the StandardScaler
scaler = StandardScaler(inputCol="feature", outputCol="NormalizedFeature", withStd=True, withMean=True)

# Fit the DataFrame to the scaler
scaler_model = scaler.fit(result_df)


# Transform the DataFrame
normalized_df = scaler_model.transform(result_df)
# Replace the original "Feature" column with the normalized values
normalized_df = normalized_df.withColumn("Feature", normalized_df["NormalizedFeature"]).drop("NormalizedFeature")



24/05/11 05:26:43 WARN TaskSetManager: Stage 1 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


In [43]:
# Show the first 5 rows of the DataFrame
#normalized_df.show(1, truncate=False)
# Alternatively, you can print the content of each row separately
for row in normalized_df.take(1):
    print(row)

24/05/10 07:20:34 WARN TaskSetManager: Stage 40 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


Row(Feature=DenseVector([0.3687, -0.4593, -0.6877, -0.5762, -0.5355, -0.4242, -0.2775, -0.1768, -0.1154, -0.1924, -0.3219, -0.4489, -0.4848, -0.4263, -0.2145, -0.026, 0.142, 0.1115, 0.0431, -0.0365, -0.0556, -0.4712, -0.1519, -0.8916, -0.0392, -0.3035, 0.7606, 0.567, -0.2822, 0.9036, 0.6981, -1.9313, -2.5581, -1.5252, -2.041, -2.7436, -1.3173, -0.7162, -1.2103, -2.0367, 0.0946, -0.4021, -0.465, -0.3044, -0.03, -0.5118, -0.0188, 0.4068, -0.4036, 0.6611, 0.6911, -1.4575, -1.8957, -1.602, -0.6792, -1.9977, -0.2236, -0.4297, -1.4055, -2.0964, 0.0076, -0.6619, -0.5873, -0.2786, -0.0107, -0.5344, -0.1176, 0.8594, -0.1422, 0.5414, 0.399, -1.0238, -1.6774, -1.5393, 0.5757, -1.0542, 0.4936, -0.5029, -1.2949, -2.3596, -0.1498, -0.6503, -0.5701, -0.508, -0.1246, -0.7537, 0.7058, 1.1563, 0.2547, -0.1799, 0.0459, -0.4559, -0.9585, -1.4476, 0.037, -0.8084, 0.0429, -0.2147, -1.259, -1.8921, -0.0196, -1.3158, -0.2831, -0.1706, -1.2122, 0.0929, -0.0932, 1.0193, -0.3755, -0.4607, 0.4599, 0.1286, -0.5184

SAVING FOR LATER USE

In [6]:
# Save DataFrame as Parquet
normalized_df.write.parquet("dataFrame")

24/05/11 05:29:36 WARN TaskSetManager: Stage 4 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


TRAINING ON LSH

In [6]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
bucketLength = 0.1
numHashTables = 5
ann = BucketedRandomProjectionLSH(inputCol="feature", outputCol="hashes", bucketLength=bucketLength, numHashTables=numHashTables)
model = ann.fit(result_df)

24/05/10 06:16:52 WARN TaskSetManager: Stage 1 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 06:16:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [53]:
# Find nearest neighbors
key = Vectors.dense([0.3687, -0.4593, -0.6877, -0.5762, -0.5355, -0.4242, -0.2775, -0.1768, -0.1154, -0.1924, -0.3219, -0.4489, -0.4848, -0.4263, -0.2145, -0.026, 0.142, 0.1115, 0.0431, -0.0365, -0.0556, -0.4712, -0.1519, -0.8916, -0.0392, -0.3035, 0.7606, 0.567, -0.2822, 0.9036, 0.6981, -1.9313, -2.5581, -1.5252, -2.041, -2.7436, -1.3173, -0.7162, -1.2103, -2.0367, 0.0946, -0.4021, -0.465, -0.3044, -0.03, -0.5118, -0.0188, 0.4068, -0.4036, 0.6611, 0.6911, -1.4575, -1.8957, -1.602, -0.6792, -1.9977, -0.2236, -0.4297, -1.4055, -2.0964, 0.0076, -0.6619, -0.5873, -0.2786, -0.0107, -0.5344, -0.1176, 0.8594, -0.1422, 0.5414, 0.399, -1.0238, -1.6774, -1.5393, 0.5757, -1.0542, 0.4936, -0.5029, -1.2949, -2.3596, -0.1498, -0.6503, -0.5701, -0.508, -0.1246, -0.7537, 0.7058, 1.1563, 0.2547, -0.1799, 0.0459, -0.4559, -0.9585, -1.4476, 0.037, -0.8084, 0.0429, -0.2147, -1.259, -1.8921, -0.0196, -1.3158, -0.2831, -0.1706, -1.2122, 0.0929, -0.0932, 1.0193, -0.3755, -0.4607, 0.4599, 0.1286, -0.5184, -1.3204, 0.0448, -1.2295, 0.0407, -0.9118, -0.5245, -0.8292, 0.3632, -2.4081, 0.0873, 0.4737, -2.7994, 0.7302, -0.9686, 0.9292, -1.2973, 0.6701, -0.4648, 0.2735, -1.0622, -1.0698, -0.4702, -0.0012, 0.7226, -1.1289, -0.151, -0.8505, 0.7794, -2.8934, -0.0731, 0.684, -3.4813, 1.1418, -1.0225, 1.1804, -1.6186, 1.4671, -1.0776, 0.4895, -0.6777, -0.2691, -0.6725, 1.1864, 1.2358, -1.3886, -0.9698, -1.4763, 0.9571, -3.1312, -0.4062, 0.912, -3.5766, 1.3403, -1.1462, 1.221, -1.3262, 1.7898, -0.9695, 0.9772, -0.5834, -0.2691, -0.6226, 0.9437, 0.9324, -1.4291, -0.8846, -1.6406, 0.9506, -3.1125, -0.7057, 0.6582, -3.5863, 1.8527, -1.0584, 1.7124, -0.5402, 2.295, 0.1913, 1.4564, -0.1174, -0.1234, -0.1487, 0.5431, 1.7121, -1.0165, -0.9328, -3.0532, 0.9745, -3.3147, -0.7998, 0.515, -3.2892, 1.8453, -1.5855, 1.7595, -0.5811, 2.5335, 0.8792, 1.1102, -0.453, -0.047, -0.4541, 0.3702, 2.0888, -0.8558, -0.8073, -3.2638, 1.0068, -3.2709, -0.6572, 0.972, -3.289, 2.0754, -2.0184, 1.0477, -1.0296, 2.1025, 0.6709, 0.7793, -1.2896, -0.4506, -0.9221, -0.1698, 2.345, -0.7364, -0.1292, -2.1351, 1.0391, -2.9416, -0.5284, 1.0884, -3.249, 2.074, -1.5945, 1.0114, -0.869, 1.282, -0.2756, 0.0001, -1.686, -0.8311, -1.1737, -0.7553, 1.1466, -0.774, 0.028, -1.4506, 0.9159, -2.3517, -0.6716, 0.7068, -2.6154, 1.7338, -1.0099, 0.633, -0.5511, 0.8794, -0.4624, -0.0854, -1.351, -0.6588, -0.9831, -0.9399, 0.3329, -0.8446, -0.281, -1.4498, 0.7378, -1.711, -0.8379, -0.3079, -1.9533, 1.2535, -0.7155, 0.5131, -0.5129, 0.5119, -0.9829, -0.1698, -1.5693, -0.8703, -0.3826, -0.7091, 0.816, -0.2404, -0.1478, -0.5131, 0.6584, -1.3685, -0.8107, -0.5147, -1.3347, 1.0516, -1.0076, 0.2584, -0.2522, -0.0269, -2.0463, -0.1882, -1.0009, -0.0861, -0.001, -0.4461, 2.0751, -0.0477, 0.3403, 0.2972, 0.5771, -1.1409, -0.8947, -0.3291, -0.8412, 0.4363, -0.9712, 0.3723, -0.0824, -0.2221, -2.1849, -0.0506, -1.2801, 0.1807, 1.0013, -0.4214, 1.9082, -0.211, 0.4279, 0.4764, 0.4887, -1.1708, -0.9421, -0.2622, -0.5362, -0.0906, -1.326, 0.3664, -0.8802, -0.2472, -1.088, -0.1167, -2.2247, -0.603, 1.3691, -0.0433, 1.7387, -0.5482, 0.4435, 0.3103, 0.4864, -1.1069, -0.9243, -0.1312, -0.8052, -0.8505, -1.5993, 0.4103, -1.077, -0.4738, -0.5255, -0.0903, -3.0168, -0.8853, 0.6721, 0.2325, 2.0972, -0.657, 0.8242, 0.5287, 0.4261, -0.9309, -0.9068, -0.255, -0.7973, -1.338, -1.5638, 0.5159, -1.4171, -0.8209, -0.9254, -0.1471, -2.4611, -0.5545, 0.8649, 0.0493, 1.4477, -1.0372, -0.0674, 0.1581, 0.2004, -0.9252, -1.0752, 0.0886, -0.4356, -1.6494, -0.8973, 0.6786, -0.3827, -0.6998, -0.8398, -0.9349, -2.258, -0.9762, 1.3014, -0.7152, 0.2428, -0.7849, -0.3569, 1.0729, 0.052, -1.0073, -1.0686, 0.3583, -0.0347, -1.4951, -0.5164, 0.9731, 0.2948, -0.6915, -0.5903, -1.4553, -2.2134, -2.111, 0.9996, -1.1586, -0.4948, -1.148, -1.2535, 1.3716, 0.0324, -0.8909, -0.9404, 0.3188, 0.0976, -1.2089, -0.0934, 1.2038, 0.1882, -0.8935, -0.1802, -1.8386, -2.1965, -2.0055, 0.6414, -0.1089, -0.7213, -1.168, -1.2413, 0.729, 0.0376, -0.8805, -0.9288, 0.2007, 0.0783, -1.0855, 0.1291, 1.4211, -0.14, -0.9976, 0.2465, -1.542, -2.1521, -2.3735, 0.4819, 0.6749, -0.4034, -0.625, -0.4903, -0.0532, 0.093, -0.9529, -0.7619, 0.493, -0.0235, -0.9085, 0.0898, 1.6212, -0.1069, -1.0678, 0.3583, -1.4637, -2.2918, -2.7509, 0.3165, 0.8915, 0.2453, -0.5459, -0.7299, -0.8381, 0.112, -1.0038, -0.7581, 0.3655, -0.1312, -0.7611, -0.2286, 1.3548, -0.3505, -1.2213, 0.2678, -0.8743, -1.5257, -2.1552, -0.2294, 0.2828, 0.3931, -0.617, -0.7829, -0.5133, 0.2829, -0.9843, -0.6129, 0.3129, -0.3108, -0.1487, -0.5865, 0.568, -1.792, -1.7647, -0.7888, -0.3008, -1.2438, -1.4535, 0.0121, -1.1174, 0.4579, -0.4477, 0.5554, -0.181, 0.5077, -1.0741, -0.6514, 0.3222, -0.9457, 0.2662, -0.8881, 0.001, -1.626, -1.3591, -1.2111, -0.1697, -1.2987, -1.1373, -0.0798, -1.1858, 1.5337, -0.0554, 0.6577, -0.2278, 0.529, -1.4377, -0.7468, 0.5132, -1.0611, 0.6466, -0.8676, -0.2607, -1.1797, -0.7852, -1.2472, 0.2023, -0.7473, -0.9499, -0.033, -0.6414, 2.1306, 0.0385, 1.0047, -0.8003, 0.5931, -1.552, -0.8626, 0.6591, -1.1082, 0.8943, -1.0997, -0.9489, -0.9986, -0.7636, -1.5091, 0.4199, -0.4743, -1.1798, -0.0054, -0.6109, 1.7478, 0.0614, 0.9346, -0.8294, 0.6683, -1.5729, -1.0328, 0.6053, -1.0838, 0.8082, -1.2874, -1.2959, -0.7957, -0.5288, -1.2806, 0.6108, -0.748, -0.5067, -0.4702, -0.9097, 1.4778, 0.6569, 0.4589, -0.9754, 0.59, -1.6916, -1.0646, 0.1323, -0.9861, 1.0513, -1.5404, -1.2466, -0.8546, -0.624, -1.4933, 0.0763, -1.8381, 0.1561, -0.0134, -0.886, 1.0551, 0.8032, 0.7191, -0.7444, 0.4598, -1.7023, -0.8683, 0.0466, -0.9587, 1.3063, -1.4545, -0.9898, -0.7312, -0.5834, -0.4226, -0.0261, -2.7474, -0.7096, -0.557, -1.1274, 0.543, 1.216, 1.348, -0.9433, 0.1572, -1.3257, -0.528, -0.0694, -0.8378, 0.9035, -0.8902, -0.7348, -0.312, -0.6482, 0.4493, -0.3716, -2.1149, -1.1178, -1.1911, -1.3328, 0.0834, 0.8497, 1.5119, -0.2746, -0.2299, -0.7365, -0.3959, -0.2828, -0.0622, -0.2544, -0.0206, -0.6859, -0.3957, -0.4441, 0.8631, -0.6163, -0.4919, -1.2863, -1.8456, -0.8701, -0.3028, 0.1741, 0.4021, -0.0432, -0.3547, -0.6136, -0.3236, -0.4098, 0.294, -0.5177, 0.2833, -0.2499, 0.158, -0.1416, 1.2316, 0.0569, -0.8299, -1.3404, -1.2285, -0.1805, -0.4034, -0.2986, -0.2587, -0.5468, -0.3927, -0.4366, -0.1873, -0.5066, 0.3108, -0.5783, 0.9075, 0.0089, 0.0258, -0.1351, 0.7038, -0.2022, -0.8455, -1.0124, -0.8408, -0.1551, -0.2004, -0.1756, -0.6076, -0.8564, -0.3496, -0.5517, 0.0383, -0.5405, 0.4054, -0.6611, 0.534, 0.7904, -0.3768, 0.2354, 0.3061, 0.3077, -0.744, -0.5235, -0.3916, -0.8452, 0.0666, -0.463, -0.6565, -1.5788, 0.4337, -0.9099, 0.1785, 0.585, -0.2845, 0.8582, 0.5665, 0.9334, -0.0045, 0.4724, -0.3679, 0.3383, -1.2919, -0.7204, -0.6669, 0.2815, 0.8991, -0.2302, 0.3318, 0.4513, 1.1981, -1.0155, 0.4928, 1.8009, -0.6184, 1.2603, 0.1501, 0.7278, -0.4163, 1.2417, 0.2176, 1.1859, -0.8553, -0.0914, -0.1166, 1.1634, 0.4804, -0.1433, 0.6958, 1.8329, 1.3189, -0.8632, 0.4702, 2.0319, -1.0389, 1.6713, 0.0083, 1.0389, -0.5953, 1.8693, 0.4066, 1.7675, -0.0481, 0.2737, 0.7573, 1.2662, 0.5055, -0.1606, 0.7973, 0.6524, 1.2543, -0.5501, 0.2251, 1.755, -1.467, 1.5717, -0.1806, 1.8804, 0.4315, 2.4289, 0.4554, 0.5236, -0.2149, -0.1924, 1.3758, 1.122, 1.6266, 0.9823, 1.3446, -0.7652, 1.1958, -0.5591, 0.1137, 1.3124, -1.8184, 1.4418, -0.3173, 1.7924, 0.8865, 2.4332, 0.8166, -0.2662, -0.562, 0.0859, 1.6889, 1.5556, 2.7208, 1.5615, 0.5819, -1.3567, 1.1359, -0.5905, -0.081, 1.1174, -1.9768, 1.1818, -0.6981, 2.0208, 1.3472, 2.083, 1.1038, 0.2363, 0.7494, 0.7436, 1.6638, 1.6625, 3.3878, 0.7327, -0.3559, -1.4503, 1.1545, -0.4326, -0.1093, 0.9687, -1.9952, 0.6675, -0.8422, 2.4277, 1.435, 2.1796, 1.1882, 0.4295, 0.7933, 0.3345, 1.7738, 1.7122, 3.6883, 0.7982, -0.2895, -1.6428, 1.1178, -0.4384, -0.115, 1.0592, -1.7205, 0.5812, -0.81, 2.1415, 1.1445, 2.3031, 0.8717, -0.2628, 0.1908, 0.2902, 2.4474, 2.1367, 3.7013, 0.2655, -0.5323, -1.377, 1.0709, -0.4802, -0.1723, 0.8128, -1.7192, 0.4819, -1.0615, 2.0514, 1.2051, 2.2957, 0.313, -0.6836, -0.033, 0.4169, 2.8504, 1.6155, 3.0938, -0.4046, -0.6363, -1.1795, 1.0368, -0.6906, -0.1637, 0.5578, -1.9146, 0.3461, -1.3132, 2.0007, 0.8724, 2.5853, -0.0028, -1.1487, -0.2587, 0.2322, 2.7946, 1.0164, 2.177, -0.8126, -0.2182, -1.4878, 1.0276, -1.0142, -0.0572, 0.5427, -2.2242, 0.0508, -1.4961, 2.3011, 0.5408, 3.0567, 0.0424, -0.9934, -0.6887, -0.7903, 2.2525, 0.5611, 2.1373, -0.7686, 0.7484, -0.4825, 1.0252, -1.0478, 0.1736, 0.5539, -2.1557, 0.1547, -1.5104, 2.5212, 0.3962, 3.3048, -0.0474, -0.1171, -0.0454, -1.2029, 1.2678, 0.3965, 2.3937, -0.1974, 1.225, 0.1345, 0.9363, -1.1107, 0.3619, 0.6437, -1.9605, 0.2997, -1.5956, 2.3545, -0.1498, 3.3173, 0.6266, -0.214, 0.0857, -1.0803, 0.6621, -0.0226, 1.6913, 0.3638, 1.1897, -0.4917, 0.7602, -0.9654, 0.3238, 0.6875, -1.3503, 0.6358, -1.1309, 1.7582, -0.9218, 2.5869, 0.9243, -0.3248, 0.1382, -0.5534, 1.1725, 0.1752, 1.3819, 0.4164, 1.1057, -0.6657, 0.3861, -0.545, 0.0789, 0.4126, -0.2702, 1.0379, 0.379, 1.4375, -1.0642, 0.8805, 0.7524, -0.7823, 0.3424, -0.2198, 1.3907, 0.2364, 0.6631, 0.4272, 0.1377, 0.3145, 0.0931, -0.0715, -0.226, 0.1465, 0.3025, 0.7084, 1.4934, 1.0627, 0.1501, 0.1079, 1.2686, -0.9539, 0.4395, 0.4365, 1.407, 0.3846, 0.3633, 0.7227, -0.4255, 0.6623, 0.0342, 0.0584, -0.3716, -0.0068, 0.8037, 0.3419, 1.7175, 1.0442, 0.1647, 0.282, 1.387, -0.5027, 0.6724, 0.6925, 1.091, 0.1417, -0.0113, 1.2953, 0.4388, 0.997, 0.0344, 0.2354, -0.4133, -0.3553, 1.1061, 0.2982, 1.766, 1.0268, -0.307, 0.3221, 1.6229, -0.3509, 0.6222, 0.8926, 0.7618, -0.2323, 0.0231, 1.0648, 0.2877, 0.7517, 0.0748, 0.2776, -0.572, -0.5934, 1.1956, 0.4174, 1.8387, 0.859, -0.2176, 0.2245, 1.6551, 0.3416, 0.824, 0.7349, 0.5896, -0.1029, 0.1325, 0.4715, -0.1372, 0.25, 0.3174, -0.467, -0.2527, -0.3555, 0.472, 1.6103, 0.8784, 1.7005, -0.1432, -0.0863, 1.1246, 0.7331, -0.2969, 0.6544, 1.003, -0.2949, 1.175, 0.0634, 0.3919, 0.3135, 0.7204, -1.4033, 0.0852, 0.3049, -1.1923, 2.5257, 0.6758, 1.8626, 0.1036, 0.1698, -0.124, 1.1631, -1.9092, 0.9121, 2.0496, 0.007, 1.4091, -0.2139, -0.1627, -0.2411, 0.918, -1.4992, 0.0675, 0.4802, -1.6444, 2.4365, 0.5895, 1.6954, -0.3762, 0.0817, -0.4783, 1.0068, -1.7801, 0.4402, 1.5199, 0.4891, 1.0175, -0.8148, -0.0054, -0.8314, 0.9299, -1.1095, 0.0419, 0.4019, -1.5647, 1.96, 0.1623, 1.4416, -0.0959, 1.0013, 0.1789, 0.4288, -0.8429, -0.136, 0.6473, 1.2283, 0.8659, -0.7563, 0.9492, -1.4416, 0.8857, -0.8842, 0.1051, 0.4686, -1.316, 1.6428, -0.6939, 0.8411, 0.5275, 1.4048, 0.7546, 0.228, -0.3548, -0.0743, 0.0844, 1.0231, 1.3733, -0.1368, 1.3514, -1.0874, 0.839, -0.709, 0.0882, 0.6325, -0.9121, 1.5662, -0.9832, 0.1807, 0.3023, 0.9762, 0.7513, 0.0708, -0.1139, 0.1095, 0.149, 0.5037, 1.1645, 0.631, 1.4643, -0.358, 0.7184, -0.6987, 0.0558, 0.6575, -0.3748, 1.56, -0.7391, 0.3349, -0.2161, 1.0657, 0.7344, 0.0179, 0.7085, 0.593, 0.4515, -0.2864, 0.7635, 0.2557, 0.7379, -0.0391, 0.5509, -0.6382, -0.0412, 0.3255, -0.0794, 1.3573, -0.3778, 1.2443, -0.0257, 1.2573, 0.9413, -0.5083, 0.6097, 0.3735, 0.2657, -1.1992, 0.5321, -0.17, 0.7989, 0.2348, 0.4595, -0.5936, -0.1815, 0.2953, 0.253, 1.5955, 0.0495, 1.8259, -0.2545, 0.4688, 0.8222, -0.8558, 0.5086, 0.6458, 1.7773, -0.6476, 0.7695, -0.0539, 0.9782, -0.1859, 0.4282, -0.55, -0.2026, 0.1204, 0.1198, 2.1208, 0.3974, 1.3161, -0.2901, 0.19, 0.9442, -0.7826, 0.2445, 0.9068, 2.0891, -0.527, 0.6839, 0.1426, 0.7382, -0.2419, 0.4275, -0.7375, -0.2605, 0.0438, -0.1604, 2.1792, 0.568, 1.0226, -0.2503, 0.5757, 1.1191, -0.3965, 0.7791, 0.9187, 1.7524, -0.2164, 1.1716, 0.4248, 0.6978, 0.3028, 0.7828, -1.9751, -0.0704, 0.7749, -0.9487, 2.6691, 0.3301, 1.2736, -1.0175, 1.3876, 0.8572, 1.205, 1.0406, 1.7447, 1.1407, 0.689, 1.1916, -0.1259, 0.216, -0.1071, 1.1169, -2.791, -0.0584, 1.0179, -2.0053, 2.8846, 0.1751, 2.024, -1.1597, 2.1699, 0.682, 1.6263, 0.6831, 1.6911, 0.4232, 0.7502, 0.8094, -0.5483, 0.5496, 0.2093, 1.2215, -3.0476, -0.1105, 1.2594, -2.4878, 2.9943, -0.1836, 2.2423, -0.9316, 2.3157, 0.4836, 1.7963, 1.0356, 1.7074, 0.1856, 0.6985, 1.0858, -0.5628, 0.1961, -0.5304, 1.24, -2.7465, -0.0613, 1.6541, -2.4573, 3.1562, -0.7829, 2.1492, -0.8381, 2.0419, 0.5485, 1.8965, 0.8066, 1.1647, 0.7294, 0.7657, 1.8918, -1.0361, 0.469, -0.4393, 1.3704, -2.4292, 0.0639, 2.105, -2.2832, 2.9903, -0.8747, 2.3786, -0.9775, 1.9981, 0.5227, 1.7551, 0.5165, 1.0316, 0.7965, 0.8904, 2.1656, -1.4983, -0.02, -0.3298, 1.4345, -2.4818, -0.0026, 2.1898, -2.125, 3.1973, -0.8566, 2.1745, -1.0027, 2.1937, 0.5167, 1.5877, 0.4169, 1.1916, 0.5955, 0.7622, 2.6377, -0.4135, 1.0515, -0.9504, 1.4113, -2.4314, -0.1712, 2.1283, -2.2673, 2.8893, -0.7412, 1.8878, -1.2291, 1.6104, 0.5261, 1.2417, -0.3404, 0.9571, 0.3244, 0.2616, 2.3766, -0.1118, 1.2462, -0.7539, 1.3136, -1.805, -0.1476, 1.8103, -2.2017, 2.1627, -0.3534, 1.8083, -0.6333, 0.5152, -0.4268, 0.3403, -1.1225, 0.3278, 0.0921, 0.452, 1.8701, -0.4417, 0.8272, -0.3961, 1.1669, -0.9936, -0.0113, 0.9989, -1.655, 1.7341, -0.0616, 1.5569, 0.0898, 0.8215, -1.3265, 0.2557, -0.3587, 0.411, 0.9068, 1.1375, 1.0554, -0.3523, 0.9726, -0.6031, 1.1196, -0.818, -0.1845, 0.5467, -1.5223, 1.5505, 0.2227, 1.5854, 0.2048, 0.6241, -2.0862, 0.0369, 0.216, 1.5964, 1.8019, 1.2701, 1.3295, -0.2858, 1.0933, 0.1399, 1.0085, -0.6676, -0.2742, 0.2835, -1.4929, 1.1673, -0.0374, 1.1193, 0.457, 0.6972, -2.5796, -0.3225, 0.2957, 2.2344, 1.9457, 0.5317, 1.4563, -0.314, 0.8121, 0.3129, 0.9114, -0.6389, -0.2788, 0.2326, -1.4535, 0.6356, -0.5295, 1.1574, 0.479, 1.0109, -1.3492, -0.4952, -0.466, 1.5711, 2.0233, 0.6899, 1.8986, -0.252, 0.6101, -0.0115, 0.8871, -0.4954, -0.1877, 0.3501, -1.1209, 0.4549, -0.4767, 1.1857, 0.3706, 0.9063, -0.2849, -0.0637, -0.8571, 1.1658, 1.5023, 1.0215, 2.5286, 0.2557, 0.97, -0.0453, 0.794, -0.3752, -0.1211, 0.1656, -1.059, 0.0446, -0.3074, 1.177, -0.0054, 0.927, 0.241, 0.4385, -1.0024, 0.8103, 1.7652, 1.6429, 2.4461, 0.0362, 1.3214, -0.167, 0.6369, -0.1824, -0.2391, 0.1648, -0.3331, 0.2767, 0.1753, 1.4241, -0.1809, 1.0464, 0.3744, -0.6759, -0.2605, 0.2133, 1.0882, 0.8271, 1.0721, -0.2883, 0.9667, -0.1169, 0.5459, -0.0777, -0.4593, 0.2071, 0.0647, 0.7385, 0.8362, 1.7758, 0.0955, 1.0004, 1.2392, -1.3337, 0.1551, -0.3478, 0.8827, 0.1627, 0.3446, -0.2653, 0.5751, -0.3425, 0.5441, -0.2321, -0.6239, 0.1502, -0.3049, 0.8682, 1.1465, 1.9078, 0.6049, 1.1321, 1.8494, -1.64, -0.4699, -0.1764, 0.4844, 0.0593, 0.1868, -0.4195, 0.7512, 0.2693, 0.566, -0.3602, -0.6163, 0.3236, -0.1078, 1.1125, 1.1391, 1.989, 0.2943, 0.7868, 1.5557, -1.7656, -0.5313, 0.2353, 0.627, 0.2594, 0.6627, 0.0946, 1.2597, 0.2794, 0.6124, -0.3621, -0.6076, 0.5665, -0.2989, 0.9543, 1.2528, 2.1683, 0.0758, 0.2325, 1.2862, -1.7963, -0.4933, -0.2621, 0.3356, 0.5928, 1.3377, 0.454, 1.3858, 0.05, 0.6516, -0.3926, -0.7027, 0.5912, -0.5815, 0.8717, 1.342, 1.9473, 0.161, 0.2718, 1.186, -1.6457, 0.0726, -0.5909, -0.4587, 0.7923, 1.7474, 0.3889, 1.5802, 0.0693, 0.7413, -0.4249, -0.5599, 0.8703, -0.6692, 1.0788, 1.3269, 1.6623, -0.2773, 0.2031, 0.0492, -1.5339, 0.2656, -0.6825, -0.5747, 0.0727, 2.0815, -0.1694, 1.3183, 0.1556, 0.9259, -0.5279, -0.4242, 1.081, -0.9036, 1.5089, 0.5782, 0.7443, -0.7759, -0.116, -1.4777, -0.7288, 0.0895, 0.061, 0.2107, -0.0234, 2.8985, -0.2839, 0.8452, -0.0362, 1.0188, -0.7889, -0.5518, 1.4656, -0.9425, 1.733, 0.0028, -0.0281, -0.6147, 0.4687, -1.2421, 0.3796, 1.014, 0.5179, 0.4278, 0.0355, 2.8434, -0.4965, 0.7904, -0.6138, 1.0841, -0.9049, -0.7519, 1.3437, -1.1382, 2.0242, -0.1576, -0.0108, -0.1782, 1.1228, -0.5368, 0.2164, 0.638, 0.0549, 0.5184, -0.0501, 3.076, 0.1078, 1.1005, -0.2756, 1.0945, -0.9892, -0.8458, 1.1266, -0.7926, 2.3496, -0.3633, -0.1893, 0.4628, 1.5414, -0.0005, 0.2209, -0.2539, 0.8661, 1.0774, 0.3633, 2.2256, 0.4879, 1.1496, 0.3481, 1.0527, -1.0324, -0.778, 1.0023, -0.7303, 2.422, -0.4655, -0.1523, 0.5452, 1.1009, 0.1888, 0.3828, -1.1077, 1.5603, 0.812, 0.6068, 1.7069, 0.2298, 1.5494, 0.665, 0.9336, -1.0262, -0.5919, 1.0408, -0.6783, 2.3882, 0.0555, 0.9049, 0.8348, 0.869, 0.2272, 0.6168, -1.7728, 1.1911, 1.0174, 0.6708, 1.9189, -0.0559, 1.4913, 0.5669, 0.713, -0.6804, -0.4457, 1.1061, -0.553, 1.468, 0.2886, 1.0782, 0.9407, 1.2071, 1.544, 0.5953, -1.0599, 0.6838, 0.8638, 1.0221, 1.7442, -0.1988, 1.6838, 0.9124, 0.3867, -0.0755, -0.5811, 0.6343, 0.1275, -0.1424, 0.7757, 1.1419, 0.9866, 1.1663, 1.9693, -0.9387, -1.0524, -0.2849, 0.5274, 1.2912, 0.8211, -0.199, 1.8315, 1.9112, 0.3099, 0.2161, -0.6344, 0.4752, 0.5974, -0.1253, 1.6082, 1.302, 1.2609, 1.0831, 1.9267, -1.2319, -0.6693, -0.1742, 0.0598, 1.4308, 0.9139, 0.3096, 1.2478, 1.4646, 0.3019, 0.0575, -0.6604, 0.3509, 0.6039, 0.2923, 1.5461, 1.1647, 0.743, 0.8553, 1.5028, -1.0384, 0.2058, 0.2602, 0.7791, 1.6296, 1.1526, 0.6645, 1.0497, 1.3305, 0.4082, -0.8149, -0.3843, 0.0724, 0.0776, 1.5285, 0.2824, 2.0891, -0.5037, 0.8366, 0.7441, -0.4518, 0.6026, 1.348, 1.5027, 2.2564, 1.4967, -0.7744, 1.1499, -0.6181, 0.6245, -1.0337, -0.226, -0.2935, -0.7091, 1.8139, -0.443, 2.6187, -0.3431, 1.5065, 0.8648, -0.1537, -0.7024, 1.1086, 1.2595, 1.9974, 2.7068, -0.5562, 1.2318, -0.779, 1.0086, -0.4053, -0.2057, 0.0565, -1.315, 0.985, -0.6399, 2.0592, -0.1148, 1.7533, 0.9811, 0.397, -0.9852, 0.0167, 0.5835, 0.8456, 1.4814, -1.5383, -0.5621, -1.2762, 1.1019, -0.3451, -0.242, 0.5071, -1.5232, 1.047, -0.5389, 1.7038, 0.0606, 1.5627, 0.513, 0.3233, -0.5221, -0.2258, -0.0573, 0.0916, 0.8813, -1.4996, -0.6382, -1.5138, 0.9522, -0.7923, -0.1548, 0.8616, -1.3872, 1.5692, -0.4021, 1.2438, -0.1795, 1.3196, -0.0236, 0.7061, -0.388, -0.2612, -0.5176, -0.0731, 0.9313, -1.7908, -0.6317, -1.5498, 0.8652, -1.1373, -0.2999, 0.92, -1.4545, 1.42, -0.3486, 1.2888, -0.5398, 0.635, -1.2796, 0.5213, -0.8132, -0.6735, -0.1301, 0.1142, 1.1026, -1.4129, -0.9189, -2.5099, 0.8487, -1.4136, -0.3351, 1.0148, -1.4503, 0.9551, -1.0401, 1.0618, -1.0025, 0.5773, -1.3386, 0.3195, -1.5163, -1.0512, -0.1745, 0.3191, 0.6663, -1.6095, -1.0947, -2.8747, 0.8687, -1.2357, -0.1405, 0.9538, -1.4752, 0.4909, -1.6566, 0.8341, -1.0401, 0.5866, -1.7472, 0.0738, -1.1501, -0.5471, 0.3609, 0.866, 1.0868, -1.3167, -0.7595, -2.6456, 0.8681, -0.8785, -0.1378, 0.4326, -2.2675, -0.6014, -1.4036, 0.8035, -0.2433, 1.2703, -1.8337, 0.2638, -1.3329, -1.2632, 0.1141, 1.0963, 1.8351, -0.2031, -0.2941, -2.1689, 0.8536, -0.7944, -0.4375, -0.484, -2.9656, -0.9228, -0.9231, 0.9515, 0.3625, 1.5924, -1.7286, 0.0412, -1.7567, -0.2907, 1.0799, 1.6403, 1.9903, -0.2148, -0.6888, -2.1676, 0.9291, -0.7554, -0.627, -1.0232, -3.0563, -0.3167, -0.9446, 1.4724, 0.3905, 1.2287, -0.7362, 0.3025, -1.6476, 0.2811, 1.6153, 1.2583, 1.0494, -2.0133, -0.3904, -1.7841, 0.9572, -0.8704, -0.7531, -1.4296, -3.2751, 0.0679, -1.4369, 1.9174, 0.2492, 0.7848, -0.5231, 0.0741, -1.1621, 1.0495, 1.8263, 0.5485, -0.1793, -2.647, -0.0365, -1.6391, 0.9499, -1.1288, -0.8709, -1.1592, -3.2446, -0.002, -1.335, 2.4761, 0.7638, 1.32, 0.003, -0.6166, -1.2612, 1.4561, 1.2878, -0.159, 0.0674, -1.7498, 0.9053, -1.5051, 1.0294, -1.3433, -1.1835, -0.8753, -3.4027, -0.2257, -1.1573, 2.7171, 0.8108, 1.2938, 1.2159, -0.3134, -0.6106, 1.8079, 0.0733, -0.8275, -0.4389, -1.5486, 1.8157, -1.8419, 1.0998, -1.5313, -1.4255, -0.5782, -3.7928, -0.6963, -1.5396, 2.1933, 0.8289, 1.1292, 1.6997, -0.1602, -0.3337, 2.0026, -0.5048, -0.9671, -0.2621, -1.5017, 1.7426, -1.0212, 1.1494, -1.5678, -1.4983, -0.144, -3.8286, -0.4773, -1.4193, 2.0579, 0.9243, 0.8698, 1.7328, -0.4218, -0.6549, 1.4584, -0.6366, -1.3604, -0.0554, -1.4161, 1.4507, -0.5539, 1.1537, -1.6509, -1.5533, -0.0249, -3.8607, -0.3311, -0.9902, 1.7675, 0.5039, 0.2434, 1.263, -1.1117, -1.172, 1.5389, -0.7873, -1.4873, 0.601, -1.5037, 1.2021, -0.1833, 1.1496, -1.6671, -1.4643, 0.1105, -4.0153, -0.3843, -1.2137, 1.581, 0.701, -0.1002, 0.675, -1.5944, -1.5843, 1.8834, -0.6102, -1.961, 0.4113, -1.7365, 0.8928, -0.3676, 1.078, -1.5552, -1.37, 0.4415, -3.8802, -0.2728, -1.2398, 1.345, 0.9692, -0.2734, 1.3945, -1.6713, -1.2207, 1.7115, -0.4082, -2.191, 0.3661, -1.6127, 0.6535, -0.709, 1.007, -1.5177, -1.3052, 0.6458, -3.9685, -0.3424, -0.9979, 1.4266, 0.7889, -0.3822, 1.9439, -1.3064, -0.9182, 1.4906, -0.9252, -2.1683, 1.028, -0.7495, 1.2096, -0.4421, 1.0244, -1.6156, -1.1664, 0.5954, -3.7872, -0.2865, -0.9373, 1.7916, 0.4916, 0.0913, 1.2397, -0.5858, -1.4275, 1.8856, -1.2504, -1.9856, 1.6734, -0.4246, 1.6068, -0.9052, 1.0256, -1.7495, -1.2065, 0.6595, -3.8224, -0.5289, -1.1692, 1.5639, -0.4421, -0.1027, 0.6638, -0.5848, -0.9379, 2.1965, -0.9199, -2.4276, 1.6514, -0.1238, 1.2383, -1.114, 1.0491, -1.7161, -1.2404, 0.6389, -3.9461, -0.9418, -1.5706, 1.4114, -0.6164, -0.1669, -0.1491, -1.0129, -0.6072, 2.311, 0.2139, -2.6592, 0.9681, -0.7585, 0.737, -1.2207, 1.0919, -1.3587, -0.9276, 0.3945, -3.8133, -1.1629, -2.1152, 1.0817, -0.7351, -0.0802, -0.1858, -0.8023, -0.6887, 2.4559, 0.996, -1.8734, 1.6107, -1.1968, -0.1378, -1.2812, 0.9378, -0.9844, -0.6245, 0.196, -3.3311, -1.1348, -2.1026, 0.7094, -0.6058, -0.1949, 0.1468, -1.0727, -0.6863, 2.6231, 0.892, -1.3076, 1.441, -1.6235, -1.1444, -1.7785, 0.478, -0.6313, -0.5552, -0.3356, -2.3756, -1.4277, -1.2409, 0.3962, 0.0663, -0.3062, 0.644, -0.4801, -0.3497, 1.6483, -0.1746, -1.2961, 0.7208, -1.9528, -1.1281, -1.313, 0.0164, -0.2299, -0.5522, -0.8183, -1.3597, -1.5575, 0.3054, 0.0819, 0.2742, -0.4037, 1.4251, 0.2598, 0.0572, 1.5813, 0.24, -0.2459, 0.184, -1.7743, -0.5226, -0.655, -0.1327, -0.1764, -0.6611, -0.8967, -0.8349, -1.8816, 0.7037, -0.0423, -0.1108, -0.6435, 1.174, 0.6622, -0.2261, 1.4022, 0.4501, 0.8606, 0.9868, -1.0612, -0.3406, -0.5667, -0.142, -0.3902, -0.4156, -0.7552, -0.7475, -1.6921, 0.0779, 0.2866, -0.6132, 0.139, 1.5286, 1.0276, -0.9628, -0.2139, 0.8272, 0.4333, 0.6173, -0.3868, -0.5355, -0.2098, 0.1914, -0.9787, 0.3323, -0.4388, -1.145, -0.07, -0.8416, 1.7963, -0.9069, 0.5568, 0.0984, 1.2245, -1.1446, -0.1134, 0.3606, -0.1381, -0.1016, -1.6699, -0.5585, -1.0814, 0.4046, -1.3144, 0.6172, -0.0351, -1.1555, 0.7345, -1.1937, 2.6598, -1.0855, 0.2283, -0.7545, 0.8462, -1.0034, 0.1573, -0.2624, 0.2123, 0.2655, -1.3317, 0.7657, -0.8447, 0.521, -1.5406, 0.5667, 0.251, -0.8178, 1.1933, -1.5732, 2.8022, -1.1441, 0.5963, -1.4129, 0.0116, -1.1693, 0.1371, -0.1437, -0.2644, -0.006, -0.8157, 0.7568, -0.8621, 0.6432, -1.6019, 0.1437, 0.0868, -0.6406, 1.4207, -1.3319, 2.1728, -1.4234, 1.0505, -1.4819, 0.0313, -1.131, 0.4272, 0.1465, -0.0166, 0.3285, -0.6844, -0.5684, -2.5495, 0.7153, -1.6739, -0.0921, -0.0777, -0.7419, 1.3762, -1.5011, 1.6878, -1.2268, 1.2994, -0.903, 0.7003, -0.9494, 0.4825, -0.0464, 0.1886, 0.8268, -0.4051, 0.1664, -1.6393, 0.7663, -1.7693, -0.3547, 0.0561, -0.5133, 1.9398, -1.2626, 1.3881, -0.5908, 1.7814, -0.7486, 0.1594, -1.5461, 0.0043, -0.0173, -0.1735, 0.8441, 0.1201, -0.3234, -1.7691, 0.8784, -1.7689, -0.6332, 0.462, -0.5597, 2.0103, -1.4796, 0.5421, -0.6909, 1.5299, -0.8057, -0.8359, -2.3705, -1.2046, -0.3972, -0.5748, 0.9159, 0.5752, -1.0318, -1.9677, 0.8983, -1.7385, -0.9096, 0.8523, -0.4657, 2.1201, -0.7965, -0.0209, -0.7335, 1.4284, -0.6259, -0.8641, -2.0812, -2.5055, -1.2845, -0.1861, 1.7847, 0.0649, -1.1564, -1.256, 0.884, -1.5193, -1.0025, 0.7573, -0.8046, 1.7275, -0.4904, -0.7556, -0.4651, 1.0564, -0.8977, -0.622, -1.1049, -2.7783, -1.8776, -0.5798, 1.1085, -0.5485, -1.5946, -0.8102, 0.9302, -1.3418, -0.9411, 0.6899, -1.3971, 1.3693, -0.3077, -0.9093, -0.4951, 0.5518, -0.9752, -0.5571, -0.1169, -1.9091, -1.6105, -0.7198, 0.349, -0.6788, -1.0451, -1.2582, 0.8982, -1.1415, -0.9425, 0.2532, -1.9814, 0.9834, 0.2555, -0.565, -0.6163, 0.0529, -1.4593, -0.8886, -0.1761, -1.4983, -1.6114, -1.1067, -0.0708, -0.6616, 0.3652, -0.6917, 0.8172, -0.9654, -0.9497, 0.0108, -2.0416, 0.7046, 0.3612, -0.5174, -0.8002, -0.723, -2.0731, -0.9734, -0.1033, -1.2492, -1.3738, -1.8637, -0.4405, -0.4947, 0.2572, -0.1617, 0.7021, -0.7039, -0.9183, 0.0122, -1.8668, 0.18, 0.4877, -0.3016, -0.8289, -0.8971, -1.4706, -0.8147, -0.7263, -1.445, -0.4014, -1.8984, -0.5211, -0.4147, 0.3842, 0.0709, 0.6215, -0.6404, -1.0085, 0.0985, -1.5681, -0.2446, 0.1951, -0.0121, -0.5375, -0.7333, -0.8247, -0.957, -1.4083, -1.6677, 1.1644, -1.0533, -1.0487, -1.4759, -1.0013, -0.8946, 0.6208, -0.7157, -1.1083, 0.3963, -1.363, -0.4249, -0.3323, 0.3376, -0.2861, -0.7593, -0.3283, -1.248, -1.21, -1.5282, 1.3861, -0.4627, -1.0371, -1.772, -1.1882, -0.4043, 0.5962, -0.9033, -0.9825, 0.3984, -1.4771, -0.5549, -0.8856, 0.3515, -0.6796, -1.1264, -0.4803, -0.9074, -0.8089, -0.8903, 0.973, 0.0947, -0.6522, -1.5072, -0.7432, -0.2997, 0.5182, -0.99, -0.7993, 0.6926, -1.4821, -0.7964, -1.1488, 0.3084, -0.5985, -1.331, -0.6379, -1.297, -1.6387, -1.4101, 0.2704, -0.3254, -1.0272, -1.8715, -0.7028, -0.0691, 0.4377, -0.8993, -0.6115, 1.054, -1.223, -1.1089, -1.4502, 0.3321, 0.0589, -1.6458, -0.7852, -1.3151, -1.6438, -1.5136, 0.2083, -0.1467, -0.9643, -1.9393, -1.1924, 0.082, 0.3638, -0.8708, -0.4762, 1.1074, -0.9985, -1.1651, -1.0137, 0.7796, 0.0419, -1.505, -0.508, -0.4811, -1.2247, -1.4744, -0.0679, -0.5329, -0.5582, -2.2587, -1.3313, -0.207, 0.3323, -0.7279, -0.3634, 0.9818, -0.9128, -1.3127, -0.6292, 0.9024, -0.2845, -0.964, 0.2242, 0.4162, -1.2575, -2.1096, 0.0283, -0.8175, -0.4614, -1.2532, -0.2664, -0.2633, 0.252, -0.5633, -0.4249, 0.6425, -0.8321, -1.5518, -0.6361, 0.8338, -0.1012, -0.8245, 1.0289, 1.0554, -1.1213, -2.0884, -0.0303, -0.7206, -0.704, -0.6549, -0.1895, -0.6008, 0.24, -0.4905, -0.4745, 0.4411, -0.804, -1.5397, -0.7546, 0.3593, -0.0825, -0.8292, 0.7274, 0.7254, -1.0197, -1.5681, -0.3942, -0.5479, -0.9626, -1.0677, -1.0266, -0.8532, 0.1481, -0.5636, -0.5184, 0.467, -0.4594, -1.3869, -0.8854, -0.1551, 0.1944, -0.1997, 1.0415, 0.8544, -0.6771, -1.1413, -0.6719, -0.7861, -1.9425, -1.4275, -0.8922, -1.0299, 0.0583, -0.6318, -0.4965, 0.8191, -0.1657, -1.1748, -0.5179, -0.2612, 0.478, 0.3322, 1.4132, 0.3333, -0.7879, -1.2791, -0.3743, -0.4269, -1.7837, -0.627, -0.1879, -0.8025, -0.0402, -0.5685, -0.4311, 0.8948, 0.0848, -1.1324, -0.0119, -0.2546, 0.2004, 0.2487, 1.3878, 0.2193, -0.9841, -1.7119, -0.7209, -0.4149, -1.2501, -0.2228, -0.1508, -0.5338, -0.0981, -0.5542, -0.3744, 0.7875, 0.0509, -1.1764, -0.0361, -0.1463, 0.2361, -0.3892, 1.1565, 0.6074, -0.901, -1.7718, -0.7515, -0.2364, -0.6447, -0.0376, -0.3933, -0.786, -0.0937, -0.5981, -0.3574, 0.4124, -0.2002, -0.9025, -0.0499, -0.1197, 0.1527, -0.5013, 1.3078, 0.676, -0.7352, -1.6446, -0.6945, -0.0287, -0.0655, 0.3957, -0.7282, -1.4444, -0.0369, -0.9064, -0.2124, 0.1033, -0.6864, -0.414, -0.5288, 0.3658, -0.7824, -0.3104, 0.9683, -0.4981, -0.5122, -1.8856, -0.3151, -0.282, 0.3385, 0.0701, -0.8612, -1.0825, 0.0406, -1.2674, -0.0079, 0.0499, -0.9346, 0.1433, -1.4372, 0.8355, -1.1363, -0.638, 0.3613, -0.3957, -0.1187, -2.3197, 0.0079, -0.3505, 0.758, 0.2329, -0.4601, -0.9519, 0.1939, -1.5786, 0.2373, 0.1494, -1.302, 1.0087, -1.7239, 0.9516, -0.8964, -0.2204, -0.388, -0.1136, -0.1283, -1.9415, -0.4791, -1.2839, 0.3711, -0.0819, 0.0216, -0.1974, 0.2827, -1.7884, 0.3248, 0.5472, -1.5589, 1.6268, -1.4222, 0.6314, -0.7214, 0.2945, -0.9331, -0.5108, 0.2714, -1.4586, -0.3343, -1.2593, 0.4191, -0.7681, 0.5456, 0.1629, 0.4358, -1.8801, -0.0815, 0.9101, -1.3042, 1.8785, -1.2209, -0.2368, -1.2951, 0.6731, -0.2481, -0.0137, 0.5088, -2.3657, -0.3688, -0.9194, 0.4583, -1.2061, 0.2644, -0.1161, 0.5047, -1.7059, -0.4396, 0.6983, -1.5096, 1.6277, -0.5186, -0.6421, -1.7401, 0.5987, -0.2731, -0.2444, 0.6409, -2.7141, -1.2129, -0.8761, 0.7195, -0.9582, -0.2086, -0.0441, 0.4309, -1.4576, -0.571, 0.5951, -1.35, 1.059, -0.1812, -1.0037, -1.5586, 0.4668, -1.2601, -1.0372, 0.4653, -2.5622, -1.0728, -0.5019, -0.2269, -1.1603, -1.3813, -0.6712, 0.3713, -1.2016, -0.6477, 0.4221, -1.0853, 0.4362, -0.0888, -0.9027, -1.2163, 0.5495, -1.8186, -1.3751, 0.5233, -2.2214, -0.5071, -0.1176, -1.2911, -1.1918, -1.0774, -0.6533, 0.4955, -0.5279, -0.2858, 0.6418, -0.249, 1.1465, 0.6185, 0.3524, 0.3629, 1.2604, -0.4631, -0.5756, 0.8205, -1.3431, 0.4319, 1.1408, -0.1462, -0.8864, -0.7293, -0.783, 0.5596, -0.145, 0.0255, 0.925, -0.1158, 1.3666, 0.6494, 0.5927, 0.9349, 0.9129, 0.1157, -0.3353, 1.2072, -0.8479, 0.7573, 1.0758, 0.3884, -0.3533, -1.0227, -0.9812, 0.6391, -0.8252, -0.083, 1.5046, -0.5022, 1.9328, 0.6798, 1.0682, 0.7129, 0.9163, 0.8418, 0.0053, 1.0717, -0.6134, 0.8804, 0.0219, -0.1502, 0.3817, -0.1656, -0.5294, 0.642, -1.2765, -0.4489, 1.2558, -0.887, 2.0963, 0.6628, 1.2333, 0.2264, 0.4692, 0.8743, -0.3197, 0.038, -0.7467, 0.5117, -0.3735, 0.8579, 1.039, 0.5151, 0.2708, 0.507, -1.4006, -0.6286, 0.7514, -0.9275, 2.0985, 0.3913, 0.5786, 0.1637, 0.6462, 1.124, -0.2495, 0.3497, -0.0924, 0.6043, 0.1181, 1.6224, 0.5115, 0.2592, 0.595, 0.482, -1.3684, -0.4673, 0.4803, -0.7723, 1.8154, -0.1162, 0.9594, 0.027, 1.0364, 1.3612, 0.2435, 0.3546, -0.1557, 0.8926, -0.0366, 0.6891, -0.2894, -0.126, 0.0673, 0.4723, -1.2254, -0.3731, 0.2975, -1.0271, 1.4558, 0.1648, 1.4099, 0.36, 0.9315, 0.8837, 0.192, -0.2271, -0.0921, 1.329, 0.064, 0.9391, -0.6965, 0.0151, 0.1328, 0.5113, -1.1927, -0.5049, 0.3014, -1.5644, 0.842, 0.0823, 1.2173, 0.3703, 0.7298, 0.1829, -0.2138, -0.3247, 0.707, 2.4412, 1.2804, 1.5252, -1.2064, 0.1172, 0.1351, 0.6936, -1.2442, -0.5773, 0.2195, -2.345, 0.6856, -0.5094, 0.8089, -0.1338, 0.0399, -0.5549, -0.5384, 1.3951, 2.6013, 3.0209, 0.817, 0.491, -2.2424, -0.0118, -1.3698, 0.842, -1.2809, -0.7588, -0.1247, -3.2217, 0.1994, -0.9811, 0.5735, -0.1248, -0.0891, -0.5991, -0.5361, 2.0129, 3.0148, 2.4306, -0.4408, -0.2113, -2.3352, 0.1717, -2.0229, 0.8071, -1.3862, -0.8728, 0.1748, -3.0119, 0.0164, -0.484, 0.7309, 0.1977, -0.2823, -0.3191, -0.1691, 1.29, 3.3594, 0.8109, -1.8963, -0.2551, -1.5091, -0.0567, -1.2085, 0.7054, -1.7037, -1.2665, 0.554, -2.3852, 0.398, -0.5157, -0.0119, -0.651, -1.1187, 0.6522, -0.1634, 1.3404, 2.7537, -0.1106, -2.2938, -0.9698, -1.146, -1.0348, -0.4634, 0.7443, -1.6677, -1.2534, 0.4875, -1.8818, 0.6438, -0.8664, -0.1796, -1.5933, -0.9617, 1.153, 0.1747, 1.5001, 1.7803, -0.0891, -1.6656, -0.2855, -1.0146, -0.3318, -0.3019, 0.7935, -1.5958, -1.0281, 0.5073, -1.3291, 0.9048, -1.2863, -0.4724, -2.1882, -0.3182, 1.3919, 0.5391, 0.5259, 1.1248, 0.405, -1.0279, -0.144, -0.8586, 0.6838, -0.4199, 0.7989, -1.7796, -0.8769, 0.7876, -1.0528, 1.4072, -1.5142, -0.8437, -2.4786, -0.1595, 0.8939, 0.4331, -0.6812, 0.2065, 1.7221, -0.9569, -1.0166, -0.4114, 1.3571, -0.3838, 0.5445, -1.8011, -0.5465, 0.5516, -0.7385, 1.8338, -0.8813, -0.7368, -2.0261, 0.1023, -0.047, 0.1573, -1.5872, -0.2747, 2.3519, -1.315, -2.3781, -0.283, 1.5822, -0.1509, 0.1164, -1.3578, -0.0722, 0.258, -0.4261, 0.9941, -0.5969, -0.8097, -1.3071, 0.4756, 0.2115, 0.4081, -1.3698, 0.107, 1.9156, -0.6706, -2.0445, -0.9774, 0.7552, -0.7594, -0.2603, -0.6344, -0.1861, -0.0593, 0.3339, -0.2842, 0.4533, -0.6865, -0.6758, -0.4781, 0.7677, 0.3443, -0.8597, 0.4172, 0.9634, -0.2831, -0.8871, -1.2932, -0.2274, -0.7121, -0.452, -0.4342, -0.2711, -0.3469, 0.8074, -1.2737, 0.6598, -0.2699, -0.1169, -0.1818, 0.4275, -0.1684, -1.1776, 0.0675, 0.2852, -0.5194, -0.2633, -0.5511, -0.2127, -0.7483, -0.5504, -0.387, -0.1709, -0.5297, 0.9472, -1.3618, 0.9323, 0.2017, -0.3973, -0.168, 0.1739, -0.6851, -1.0928, -0.1534, 0.0501, -0.4198, 0.0009, 0.1973, -0.1912, -1.2022, -0.245, -1.2782, 0.2933, -0.3085, -0.3487, 0.0543, -0.3635, 0.8278, -0.9186, -0.0295, -0.2491, -0.241, -0.9738, -0.2584, 0.3014, -0.856, 0.3607, -0.1191, -0.6067, -1.8064, 0.1395, -2.0819, 0.482, 0.269, -1.4654, 1.1979, -0.4079, 0.9474, -1.6339, 0.4995, -1.2482, -0.3021, -1.3023, -0.793, -0.6813, -1.4175, 0.026, -0.7824, -0.0792, -1.7989, 0.3741, -2.0744, 0.2077, 0.4296, -1.2255, 1.465, -0.6259, 0.3427, -2.1584, 0.7973, -1.5537, -0.056, -1.0676, -1.3599, -1.9092, -1.7154, 0.1794, -0.6281, 0.984, -1.375, 0.3911, -2.0725, -0.1231, 0.341, -1.0228, 1.424, -0.8175, -0.5168, -1.8793, 1.0316, -1.7217, 0.4155, -0.6761, -1.1477, -2.1298, -1.4311, 0.3695, -0.8817, 1.4692, -1.3296, 0.3781, -1.9972, -0.2263, 0.2052, -0.8218, 1.5764, -0.7035, -0.5007, -1.5323, 0.8502, -1.5626, 0.19, -0.6482, -1.1374, -1.9, -0.4535, 0.836, -0.4068, 1.4851, -1.3903, 0.3709, -1.7406, -0.243, 0.4313, -0.7748, 1.4108, -0.2421, -0.531, -1.4312, -0.4397, -2.0327, 0.0026, -0.6504, -0.5965, -0.8059, -0.5944, 0.4319, -0.3782, 0.6999, -1.4931, 0.3325, -1.6968, -0.0994, 0.7216, -1.2194, 0.7786, 0.0965, 0.0913, -0.8311, -0.6293, -1.5664, 0.4613, -1.3186, -0.3641, -0.3349, -1.3828, 0.1414, -0.1416, 1.1756, -1.1572, 0.015, -1.5108, 0.1688, 0.636, -1.1418, 0.4073, 0.5203, 0.3717, -0.275, -0.5765, -0.7981, 0.5707, -1.2365, -0.7203, -0.1554, -0.895, -0.0313, -0.8024, 0.2932, -0.5009, -0.4417, -0.6737, 0.174, -0.3739, -0.2922, 0.0112, 0.6651, -0.1668, 0.4913, -1.3141, -0.2877, 0.4517, -0.2605, -0.3194, -0.3779, -0.6659, 0.5257, -0.3602, -1.1897, 0.2136, -0.086, -1.2112, -0.2329, 0.0302, -0.9919, 0.4172, 0.8579, -0.0124, -0.0994, -0.4323, -0.2614, 1.1569, 0.3261, -0.7939, 0.7997, -1.3575, -0.2619, -0.4577, -1.3314, -0.3423, 0.4857, -2.3859, -0.7635, 0.9146, -1.8181, 1.1718, 0.4784, 0.8823, -0.5413, 0.3316, -0.618, 1.3976, 0.1175, -1.025, 0.715, -1.6284, 0.1554, -1.4528, -0.6529, -0.4567, 0.7121, -3.0568, -1.1168, 1.4431, -2.1262, 1.4192, -0.043, 1.4146, -0.7112, 0.7505, -0.3496, 1.7928, 0.1996, -0.6754, 0.8883, -1.1954, 0.8368, -2.0986, -0.3833, -1.1258, 0.9331, -3.0745, -1.1155, 2.2997, -1.8512, 1.8205, 0.015, 1.338, -1.1317, 0.819, -0.0947, 1.6198, 0.0079, -0.3815, 1.1543, -1.4192, 1.4498, -1.8175, 0.3957, -0.4489, 1.185, -2.3934, -0.4003, 2.6616, -1.7596, 2.1151, -0.2998, 1.4356, -1.0813, 0.937, 0.4536, 2.2519, 0.8128, 0.6062, 1.9907, -0.5878, 2.3954, -0.6435, 0.4399, -0.588, 1.183, -2.225, -0.0705, 2.4371, -2.0076, 2.171, -0.2807, 1.9549, -0.819, 1.1205, 0.2337, 2.0892, 1.6522, 0.5717, 1.4163, -0.2468, 2.8901, -0.2699, 0.5587, -1.2576, 1.1486, -2.3012, -0.1014, 1.8348, -2.3859, 1.8775, 0.0299, 2.5881, -0.7725, 1.6325, -0.5341, 1.5321, 1.7909, 0.6001, 0.8692, -0.5672, 3.0938, 0.3303, 0.8057, -1.9667, 1.0373, -2.4058, -0.2203, 1.2936, -2.4409, 1.5628, -0.2269, 2.5589, -1.0788, 1.6116, -0.8714, 0.657, 0.7104, 0.1907, 0.8373, -0.0998, 2.4659, -0.4576, 0.8494, -1.4759, 0.9554, -2.028, -0.196, 0.8893, -2.0835, 1.6993, -0.0699, 2.5004, -0.6432, 1.6871, -0.889, 0.3444, 0.6272, 0.0523, 1.076, -0.0117, 1.7077, -0.7338, 0.2177, -1.6865, 0.7057, -1.2279, -0.0711, 0.0994, -1.7138, 1.3179, -0.2246, 1.847, 0.0465, 0.8834, -0.9079, 0.7905, 1.1438, 0.0866, 1.1643, -0.2974, 1.1326, -0.5243, -0.2383, -1.0808, 0.5572, -0.9316, -0.0984, -0.6506, -2.027, 0.6802, -0.429, 1.5573, 0.8914, 0.0088, -1.6161, 0.6278, 0.7913, 0.6001, 1.7278, -0.0191, 0.6837, -0.8247, -1.1073, -0.1896, 0.6637, -1.0888, -0.2568, -0.7839, -2.358, 0.7732, -0.8207, 1.2156, 0.2548, -0.1832, -1.2692, 0.7464, 0.9509, 2.0497, 1.9202, -0.4425, 0.6725, -1.4756, -0.0466, -0.2445, 0.7952, -1.0209, -0.4141, -1.0122, -2.6657, 0.4534, -1.2735, 1.3232, -0.1395, 0.085, -0.8328, 0.3929, 1.553, 2.6415, 1.3905, -0.7983, 0.8691, -1.7987, 1.3365, -0.8494, 0.7606, -0.9308, -0.4892, -0.8964, -2.4671, -0.0122, -0.9868, 1.7783, -0.4169, 0.6316, 0.0588, -0.3119, 1.197, 2.3032, 0.87, -1.0596, 0.1975, -1.333, 1.6591, -1.7651, 0.7023, -1.1868, -0.7254, -0.3443, -2.5564, -0.5978, -0.3102, 2.0024, 0.0069, 1.0055, 0.8715, -0.4864, 0.2283, 1.5934, 0.126, -1.3148, 0.1153, -0.4123, 1.7103, -1.4413, 0.7369, -1.3342, -1.1298, -0.1479, -3.0067, -0.8245, 0.2568, 1.6844, 0.7116, -0.0584, 1.2734, -0.5257, -0.6419, 2.0566, -1.1273, -1.9478, -0.0332, -0.0873, 1.508, -0.9697, 0.6886, -1.3995, -1.2162, 0.1538, -3.1325, -0.395, 0.3151, 1.7551, 1.7501, -0.9277, 1.741, -0.9164, -0.5858, 2.5647, -1.0492, -1.9338, 0.394, 0.158, 1.3892, -0.3976, 0.629, -1.4929, -1.2175, 0.524, -3.1322, 0.1008, -0.0069, 1.6349, 2.0757, -1.6022, 1.8678, -1.1562, -0.3795, 2.647, 0.2446, -1.0205, 0.8774, 0.4509, 1.6184, -0.2065, 0.6167, -1.4888, -1.2978, 0.6273, -3.0994, 0.0132, 0.0704, 1.546, 2.2891, -1.5791, 2.3369, -1.089, -0.7791, 2.3331, 0.3659, -0.2754, 1.3283, 0.6898, 1.5212, -0.4434, 0.5906, -1.5686, -1.3939, 0.3693, -3.4249, -0.23, -0.0056, 1.4483, 2.5419, -1.6096, 2.3859, -1.0526, -0.7583, 2.3392, 0.2106, -0.1028, 1.5009, 0.8359, 1.4354, -0.8591, 0.6005, -1.5748, -1.4641, 0.2581, -3.4759, -0.0911, 0.1473, 1.5788, 2.903, -1.6792, 2.2916, -1.2329, -1.3452, 1.9576, -0.1689, -0.4778, 0.8906, 0.7853, 1.752, -0.3774, 0.6501, -1.3409, -1.3791, 0.5958, -2.9161, 0.2232, 0.9372, 1.7171, 2.7143, -2.2648, 2.026, -0.9312, -1.6446, 2.4649, 0.0987, 0.1119, 1.5594, 0.3282, 0.6362, -1.7912, 0.7862, -1.1131, -0.8795, 1.2135, -2.4376, 0.5056, 1.0711, 2.2019, 2.7391, -1.2906, 2.456, -0.0712, -1.2037, 2.9391, 0.426, -0.6094, 1.5696, -0.0184, 0.7033, -2.5061, 0.8421, -1.3531, -0.5166, 1.5884, -2.3167, 0.8332, 0.7193, 2.4517, 2.308, -0.3863, 2.5604, 0.274, -0.4482, 2.8715, 0.533, -1.8877, 0.8308, -0.4529, 1.6671, -2.1006, 0.838, -1.5509, -0.4575, 1.5144, -2.2809, 1.172, 0.211, 2.5308, 1.3904, 0.1037, 2.4182, -0.1115, -0.0661, 2.1688, 0.5663, -2.0002, 1.5473, -0.3019, 1.4417, -1.3761, 0.9041, -1.627, -0.4035, 1.2847, -2.1723, 0.9316, -0.3427, 2.4259, 0.3051, 0.3634, 2.023, 0.0736, 0.0796, 1.9971, 0.3619, -1.5038, 2.164, -0.5378, 0.3796, -0.5976, 0.9771, -1.5089, -0.4743, 1.0103, -2.0292, 0.634, -0.2846, 1.98, 0.14, 0.513, 1.2686, 0.6606, 0.3333, 2.4405, 0.3306, -0.9088, 2.1428, -1.4657, -0.0054, -0.9815, 0.8705, -1.1737, -0.2404, 0.7449, -1.8705, 0.2081, -0.4642, 1.3941, 0.3886, 0.7418, 0.7535, 1.0891, 0.5736, 2.7471, 0.5055, -0.9259, 1.1541, -2.1797, 0.3974, -0.9494, 0.5061, -0.8759, 0.2998, 0.1256, -1.7004, 0.066, -0.5718, 1.4053, 0.6322, 1.4486, 0.2969, 0.784, 0.8393, 2.3155, 0.973, -1.4994, -0.1935, -1.5392, 1.0161, -0.4521, 0.1589, -1.0478, 0.7879, -0.4618, -1.2216, 0.1934, -0.7244, 2.5022, 0.0293, 1.8658, 0.5856, 1.1716, 0.6581, 1.1615, 1.3461, -1.4682, -0.2212, -0.7792, 1.5464, -0.2603, 0.1935, -1.5569, 1.306, -0.3939, -1.2576, 0.9766, -1.4957, 2.8485, -1.0262, 1.3458, 1.183, 1.401, 0.7898, 0.0759, 1.434, -0.6183, 0.2676, -0.5156, 0.2661, -0.3536, 0.1766, -2.1522, 1.5792, 0.1684, -1.3378, 1.7947, -1.5405, 2.8611, -1.3765, 0.6175, 0.7131, 0.7255, 0.7096, -0.0155, 0.9741, -0.0936, -0.0799, -0.3954, 0.0358, -0.7839, -0.1447, -2.9117, 1.8119, 0.1233, -1.5185, 1.577, -1.4026, 2.1869, -0.7352, 0.6799, 0.822, 0.6832, 0.8725, 0.5947, 0.7186, 0.6839, -0.1894, -0.0293, -0.5788, -1.4034, 0.6407, -2.5146, 0.6516, -1.355, -1.7915, 0.1922, -1.122, 2.8472, -0.3966, 1.0265, 1.0195, -0.5222, -1.5263, -1.1271, 0.1105, 1.7791, -0.9474, -0.3797, -2.149, -1.6827, 1.2117, -1.9442, 0.3531, -0.655, -1.6744, 0.03, -1.0962, 2.538, -0.8205, 1.5819, 0.3389, -0.0004, -0.9002, -1.53, -0.8165, 0.8213, -1.2126, -1.5827, -2.7583, -2.5188, 1.2315, -1.7587, 0.1607, 0.1922, -1.796, 0.9789, -1.8319, 2.0507, -1.3029, 1.7023, -0.8467, 0.3031, -0.8308, -1.4845, -0.8789, -0.7075, 0.1451, -2.248, -1.565, -2.3432, 1.1709, -1.7047, -0.0611, 0.6172, -1.7757, 1.2306, -1.7483, 1.8723, -1.1752, 1.329, -1.5758, 0.0331, -0.9249, -0.7619, -0.1648, -0.7527, 0.6141, -1.8299, -1.0559, -2.2805, 1.1301, -1.3932, -0.4403, 0.6781, -2.0475, 0.6283, -1.0619, 1.1403, -1.1051, 0.1516, -1.759, 0.1457, -0.3415, -0.0778, 0.1802, -0.2102, 0.5528, -0.894, -0.7383, -2.6031, 1.139, -1.2923, -0.6929, 0.5274, -2.1104, 0.4654, -1.192, 0.3359, -1.6595, -0.5015, -1.266, 0.8097, 0.2545, 0.9276, 0.9271, 0.3728, 1.0299, -0.7395, -0.4662, -2.6697, 1.1555, -1.3075, -0.8065, 0.1669, -2.3295, 0.1831, -1.5722, 0.0425, -1.663, 0.0434, -0.9703, 1.6861, 0.8701, 1.5269, 0.2028, -0.2069, 1.4089, -1.2755, -1.1965, -3.7654, 1.1839, -1.2303, -0.7618, -0.3126, -2.4567, 0.0652, -1.7824, 0.7062, -1.9493, 0.0847, -0.8737, 1.7572, 1.3687, 1.6785, -0.2128, -1.1192, 1.0696, -1.0701, -1.2066, -3.687, 1.0594, -1.2768, -0.73, -0.5459, -2.6831, -0.1164, -1.2238, 1.5276, -1.2909, 0.9208, -0.0066, 1.8319, 1.3063, 0.6877, -0.9843, -2.2005, -0.093, -0.7379, -0.6541, -2.7838, 0.9307, -1.2508, -0.832, -0.5707, -2.822, -0.5216, -1.1957, 1.1855, -0.4279, 1.198, 0.1645, 0.9938, 0.4305, 0.1441, -1.1055, -1.6773, 0.1659, -0.2274, 0.2282, -2.1347, 0.8719, -1.3139, -0.8113, -0.2504, -3.1782, -0.6359, -1.4168, 0.6588, 0.0961, 0.7115, 0.5308, 0.2226, -0.3579, 0.1676, -1.396, -1.6737, 0.4176, -0.8349, 0.9265, -1.1514, 0.922, -1.6312, -0.8638, 0.1111, -3.6526, -0.1624, -1.2048, 1.2207, 0.2102, 0.4977, 1.1275, -0.6938, -0.5641, -0.5089, -2.1113, -2.4103, 0.4902, -0.3436, 1.6, -0.1082, 0.9355, -1.8576, -1.057, 0.1081, -3.7844, 0.2426, -1.2451, 1.7355, 0.3548, 0.3576, 1.4187, -0.9059, 0.5165, 0.0401, -1.278, -2.3544, 0.9867, 0.0433, 1.3575, -0.0615, 0.9352, -1.9627, -1.0763, 0.1299, -3.5576, 0.6806, -1.3212, 1.8675, 0.1556, 0.8591, 1.5172, -0.63, 0.3803, 0.6042, -0.8923, -3.2791, 0.557, -0.5022, 1.3332, -0.5495, 0.9565, -1.9304, -1.1061, 0.0621, -3.2179, 0.5922, -1.4424, 1.2052, -0.4939, 1.0316, 1.3879, 0.1056, 0.6687, 1.3649, -0.7404, -2.8534, 0.3819, -0.9772, 0.8428, -1.0396, 0.936, -1.8968, -1.1537, -0.1717, -3.2716, -0.0221, -1.4586, 0.7513, -1.4296, 0.9405, 1.0569, 0.2075, 1.2481, 1.5597, -0.7348, -2.3486, -0.3304, -0.9062, 0.2924, -0.6668, 0.9054, -1.5766, -1.1489, -0.1199, -3.1121, -0.1578, -0.6281, 0.1759, -1.7019, 0.604, 0.6388, -0.1989, 1.0204, 1.2532, -1.4715, -2.5294, -0.4621, -0.8189, 0.1465, 0.1382, 0.9003, -1.3545, -0.939, 0.1795, -3.0248, -0.0711, -0.3431, -0.8752, -1.9996, 0.4199, 0.1907, -0.3406, -0.0634, 0.9798, -1.7811, -2.8021, 0.2466, -0.4483, 1.2884, 0.4712, 0.9043, -1.357, -0.6067, 0.5195, -2.934, 0.1407, -0.8753, -0.7731, -1.3934, 0.5516, -0.1382, -0.3509, -0.0766, 0.6542, -1.3562, -2.8634, 0.0819, -0.983, 1.3728, 0.8436, 0.8631, -1.4815, -0.6352, 0.6724, -2.6402, 0.5109, -0.6922, -0.4817, -1.1382, 0.3354, -0.6703, -0.094, 0.1379, 0.0316, -1.4467, -2.7335, 1.265, -0.5958, 1.3371, 0.567, 0.6832, -1.5788, -0.5281, 0.5591, -2.2396, 0.5978, -0.4646, -0.5515, -1.7098, -0.5502, -1.1639, -0.1979, -0.5121, -0.2856, -1.988, -2.5884, 1.6746, 0.4247, 1.3173, -0.3628, 0.3869, -1.2468, -0.1246, 0.2545, -1.999, 0.2346, -0.687, -0.6137, -0.9567, -0.8281, -0.4516, -0.1571, -1.776, -0.6134, -1.3301, -1.476, 1.1995, -0.2474, 0.2773, -0.5779, -0.0087, -0.8136, -0.0397, -0.0759, -1.459, -0.599, -0.1634, 0.0053, -0.3273, -0.8044, 0.4696, -0.7696, -2.2287, -1.7858, -0.7565, -0.5483, 1.1027, -0.2803, -0.7604, -0.7059, -0.2901, -0.4092, -0.2377, -0.382, -0.6093, -1.5884, 0.7663, 0.3223, 0.4006, -0.4895, 1.0767, -0.3239, -2.3117, -1.6769, -0.9458, 0.32, 1.0407, -0.2813, -1.2145, -1.4884, -0.2541, -0.858, -0.3838, -0.1474, -0.4951, -0.8464, 0.6928, 0.5248, 0.4641, -0.6464, 0.663, -0.4678, -1.4317, -1.0451, -0.1746, 0.2297, 1.0048, 0.1969, -1.5504, -2.0671, 0.0452, -1.4901, -0.5693, 0.0684, -0.8502, -0.2243, -0.166, 0.8089, 0.4032, -0.1992, 0.0605, -0.9232, -1.5233, -0.4613, 0.0125, -1.0621, 1.0656, 0.8037, -1.1305, -2.382, 0.2832, -2.2414, -0.2747, 0.188, -1.3623, 0.5161, -1.0161, 1.3016, -0.2578, 0.8269, -0.3201, -0.419, -1.9268, -1.2283, -0.2512, -1.1192, 1.2487, 0.2194, -0.3998, -2.4967, 0.4608, -2.6984, -0.1386, 0.4531, -1.7694, 1.1161, -1.4462, 1.5425, -1.0273, 0.9123, -0.9144, -0.161, -2.0875, -1.5841, -0.095, -1.0552, 0.9754, -0.1135, 0.0914, -2.4315, 0.6062, -2.5833, -0.2265, 0.5913, -1.5209, 1.58, -1.504, 1.4803, -1.2603, 0.6731, -1.6699, -0.6713, -1.7211, -1.1904, -0.1904, -1.5521, 0.2164, -0.7676, -0.1009, -2.1482, 0.5409, -2.1807, -0.2239, 0.3874, -1.2428, 1.5541, -0.9679, 0.8829, -1.2498, 0.5322, -1.4248, -0.4709, -1.9212, -1.4041, 0.0332, -1.3502, -0.5931, -1.9788, -0.5174, -1.0231, 0.3873, -1.4483, -0.2177, -0.3935, -1.4248, 0.9063, 0.0065, 1.1835, -0.5696, -0.0999, -1.376, -0.3794, -1.3761, -1.1991, 0.1008, -1.3818, -0.4921, -2.2098, -1.7786, -1.0252, 0.3945, -1.1028, -0.4148, -0.7282, -1.6348, 0.5535, -0.309, 1.1226, 0.4897, -0.2219, -0.723, -0.2627, -0.675, -0.7532, -0.1494, -1.3772, -0.3496, -2.0643, -2.4655, -1.6828, 0.3768, -1.1893, -0.5542, -0.4633, -1.5051, 0.5303, -0.75, 0.1363, 1.12, 0.1289, -0.3542, 0.4927, -1.1136, -1.8492, -1.0761, -1.1134, 0.2206, -1.0588, -2.1188, -2.2032, 0.4835, -1.6556, -0.673, -0.1338, -2.033, 0.7735, -1.4123, 0.0919, 0.4227, 0.7648, 0.2773, -0.7726, -1.5012, -2.3359, -1.4133, -1.3555, 0.674, -0.1572, -1.465, -1.9194, 0.6275, -1.9564, -0.7689, 0.2637, -2.2679, 0.836, -1.6332, 0.3473, -0.1955, 1.1395, 0.7058, -0.8196, -1.6215, -1.7267, -0.7041, -0.9141, 1.2708, 0.2351, -0.5137, -1.1834, 0.5522, -1.9702, -0.923, -0.0212, -2.119, 0.9356, -1.5954, 0.0149, -0.3324, 1.1226, 1.126, -0.1331, -1.0137, -1.5774, -0.9312, -0.8611, 1.521, 0.7971, 0.7871, -0.0973, 0.4447, -1.62, -0.9546, -0.1945, -1.7888, 1.1339, -1.1192, -0.2717, 0.3644, 0.7523, 0.9273, -0.7422, -0.9921, -1.4899, -0.3898, -1.0727, 0.6556, 0.2522, 0.3807, 0.0097, 0.331, -1.4615, -1.0205, -0.307, -1.4544, 0.9265, -0.8842, -0.2413, 0.6494, 0.0263, 0.5015, -0.9995, -1.2073, -1.3203, 0.0047, -1.204, 0.4128, 0.0214, -0.2729, 0.372, 0.2147, -1.3743, -1.0393, -0.5472, -1.0837, 0.6128, -0.995, -0.2301, 0.5296, -0.2916, 0.5842, -0.9805, -2.0282, -1.9303, -0.4539, -1.5663, 0.4563, 0.1479, -1.1658, 0.2872, 0.1972, -1.0531, -0.9536, -0.7753, -0.919, 0.2669, -0.7934, -0.0008, 0.4026, -0.6713, 0.5723, -1.4433, -1.9755, -1.6209, -0.4943, -1.115, 0.3345, -0.0498, -1.8131, 0.3565, 0.1685, -0.9199, -1.0218, -0.6324, -0.6691, -0.2252, -0.0106, 0.1317, -0.0531, -1.1067, 0.6707, -1.2559, -2.0214, -1.5982, 0.0864, -0.1759, -0.1797, -0.1785, -1.219, 0.4953, 0.3033, -0.9233, -1.0931, -0.3376, -0.2556, -0.5276, 0.1654, 0.2592, -1.5114, -1.6218, -0.4477, -0.9206, -1.8623, -0.99, 0.6432, -0.3301, 0.2709, -0.5268, -0.1658, 0.2961, 0.5444, -1.1192, -1.2271, -0.3669, -0.5818, -0.3492, -0.3408, 0.6882, -1.6816, -1.9394, -1.4241, -0.66, -1.543, -0.2712, 0.9927, -0.6655, 1.2691, -0.215, 1.014, 0.4122, 0.638, -1.6128, -1.4122, -0.3915, -1.4251, -0.2042, -0.9589, -0.0219, -1.7164, -1.5302, -1.5257, -0.492, -1.3844, -0.8556, 0.6263, -0.8502, 1.8537, -0.3088, 1.0517, -0.0247, 0.6813, -1.8273, -1.348, 0.0375, -1.5816, -0.2103, -1.4816, -0.4702, -1.8213, -1.1018, -1.4691, -0.3679, -1.5325, -1.2999, 0.145, -0.9034, 2.1471, -0.6063, 0.5355, -0.8994, 0.6814, -1.7581, -1.2489, 0.0245, -1.4856, 0.1534, -1.299, 0.0663, -1.5119, -0.8706, -1.4415, -0.1622, -1.1843, -0.9911, 0.3615, -0.5903, 2.3614, -0.6843, -0.1758, -1.7648, 0.6363, -1.8366, -1.1415, -0.0293, -1.5683, 0.2999, -1.5485, 0.774, -1.1379, -0.8951, -1.0466, -0.3993, -0.805, -1.0174, 0.007, -0.418, 2.6317, -0.3539, -0.6979, -2.5077, 0.6746, -1.9826, -1.0934, 0.1235, -1.6939, 0.3803, -1.7098, 1.1618, -1.2108, -1.0056, -0.6711, -0.4335, -1.0678, -1.5623, -0.0442, -0.3013, 2.9714, -0.2464, -0.8257, -1.8832, 0.7078, -2.0716, -1.0643, 0.2888, -1.6401, 0.4571, -1.9071, 1.2952, -1.5301, -0.7351, -0.7827, -0.4627, -1.1248, -1.7946, 0.359, -0.4515, 2.8585, -0.7021, -0.9506, -1.6948, 0.7086, -2.1567, -1.0008, 0.1923, -1.7908, 0.5055, -2.1002, 1.1807, -1.851, -0.1429, -0.7424, -0.0394, -0.5382, -1.5465, 0.252, -1.0168, 2.531, -1.1424, -0.9327, -1.752, 0.6695, -2.1922, -0.9708, 0.1044, -1.7185, 0.6082, -2.0494, 0.6616, -2.018, 0.5184, -0.6245, -0.1727, -0.4335, -0.8631, 0.6043, -1.4641, 2.7393, -0.9711, -0.4872, -1.7565, 0.6535, -2.0131, -0.8731, 0.2855, -1.6113, 0.3683, -1.9167, 0.3431, -1.7707, 1.0396, -0.6222, -0.2656, -0.4881, -0.7843, 0.4566, -1.5716, 2.7665, -0.8623, 0.0769, -1.3441, 0.5246, -2.1462, -0.8344, 0.3943, -1.4152, 0.3801, -1.6971, 0.3995, -1.5987, 0.6809, -1.1449, 0.4656, -0.3859, -0.7802, 0.812, -1.4846, 2.9507, -0.3494, -0.0693, -1.9132, 0.4651, -1.9468, -0.7838, 0.1717, -1.3255, 0.3296, -1.8752, -0.0346, -1.7425, 0.3326, -1.5275, 0.9019, -0.467, -1.0067, 1.6193, -1.7868, 2.9657, -0.1525, -0.4633, -1.418, 0.4556, -1.8127, -0.835, 0.0699, -1.4195, -0.1177, -2.0363, -0.2466, -1.8593, 0.9534, -1.3213, 0.4829, -0.4811, -0.7846, 1.4809, -1.9449, 2.9611, -0.0339, -0.1847, -1.0833, 0.3977, -1.7498, -0.6179, 0.0525, -1.4344, -0.0236, -1.8173, -0.3623, -2.1415, 1.0192, -1.4751, -0.0648, -0.6147, -0.5171, 1.0033, -1.6512, 2.2228, 0.1462, 0.0475, -1.4299, 0.4322, -1.6345, -0.4453, 0.124, -1.4242, 0.1258, -1.9348, -0.6703, -1.8895, 1.0538, -1.4802, 0.0541, -0.9372, -0.8564, 0.6188, -0.7037, 2.0343, -0.071, 0.4004, -1.9418, 0.4802, -1.5925, -0.5326, -0.0487, -1.6719, 0.3291, -1.9141, -0.6612, -1.5367, 0.2392, -2.4723, 0.0484, -0.4949, -0.7546, 0.502, -1.2606, 1.57, 0.3291, 1.0777, -1.0122, 0.4826, -1.6502, -0.5037, -0.1685, -2.114, 0.6463, -1.9356, -0.461, -1.5152, -0.7074, -2.859, -0.034, 0.4942, 0.0511, 1.3537, -1.0662, 0.963, -0.2933, 1.2762, -0.0245, 0.4278, -1.5925, -0.3324, -0.4482, -2.4546, 0.6377, -1.9469, -0.179, -1.0252, -0.7024, -3.0492, -0.6136, 0.805, 0.1784, 1.8179, 0.181, 1.5076, -0.3722, 1.0383, 0.4954, 0.2924, -1.4979, -0.1531, -0.4377, -2.492, 0.002, -2.0541, -0.0923, -0.1561, -0.4112, -3.0427, -1.9888, 0.484, -0.327, 1.8215, 1.3112, 1.9645, -0.0372, 0.6466, -0.1179, 0.1411, -1.4603, -0.0211, -0.6066, -2.2816, -0.4216, -2.2141, 0.0308, -0.5123, -0.261, -2.1558, -1.5757, -0.0474, -0.9652, 1.4248, 0.9949, 1.6003, -0.1351, 0.242, -0.4661, 0.2313, -1.1118, -0.0226, -0.6113, -1.5108, 0.7741, -1.0539, 0.4569, -0.2956, 0.5473, -0.2504, 0.297, 0.2883, -0.6391, 0.5232, -0.2711, 0.7815, -1.0119, 0.3902, 0.1216, 0.3944, -0.3431, 0.0884, 0.1425, -0.3943, 0.9493, -0.2778, 0.7531, 0.8256, 0.9593, 1.0293, 1.2086, 0.8412, 0.3264, 0.543, -0.0574, 0.8592, -1.2785, 0.0876, -0.1489, 0.389, -0.4746, 0.1487, 0.7046, -0.2915, 1.373, -0.0474, 0.8096, 1.0818, 0.9164, 1.4448, 1.0654, 1.7011, 0.4046, 1.2384, 0.649, 0.8581, 0.0385, -0.2801, -0.2601, 0.5285, -1.6188, -0.174, 1.2533, -0.5986, 2.3994, 0.2973, 1.2385, 0.3467, 1.0357, 1.7614, 1.0611, 1.4134, 0.5239, 1.3174, 0.132, 0.6438, 0.6472, 0.4592, 0.4378, 0.5122, -1.9818, -0.6444, 1.1242, -1.0217, 2.6209, 0.2819, 1.1013, 0.1201, 1.0944, 1.4489, -0.075, 0.5465, -0.1718, 0.7651, -0.4262, 1.5688, 0.6201, 0.3987, -0.0218, 0.3777, -2.0874, -0.7939, 0.8379, -0.9205, 2.5866, 0.1271, 0.8687, 0.7353, 1.4552, 1.7066, -0.0015, 0.7512, -0.2711, 0.4819, -0.4063, 1.2591, -0.2283, -0.3767, -0.2648, 0.2887, -1.9108, -0.7166, 0.6949, -0.809, 1.9354, -0.0002, 1.2268, 0.5484, 1.233, 2.0762, 0.3192, 1.3202, 0.1859, 1.4336, -0.3403, 0.7936, -0.5539, -0.6088, -0.0571, 0.2254, -1.6498, -0.6639, 0.39, -0.6644, 1.5695, 0.47, 0.9911, 0.4111, 0.515, 1.2162, -0.1383, 0.2174, 0.4828, 1.7156, 0.5563, 1.5596, 0.1127, -0.0818, 0.2956, 0.1403, -1.5439, -0.6196, 0.0229, -0.5215, 1.4298, 0.3614, 0.9448, 0.4739, 0.1843, 1.1715, 0.174, 0.2576, 1.4021, 1.3726, 0.6558, 1.0058, -0.8137, -0.3869, -0.1743, -0.0132, -1.7541, -0.7623, -0.4193, -0.3036, 1.4325, 0.3916, 1.0292, 0.4436, 0.0994, 0.7388, 0.9286, 0.5335, 1.3562, 1.1234, 0.0354, 0.4358, -1.2766, -0.5159, -0.2953, -0.0649, -1.6435, -0.5875, -0.628, -0.7984, 0.8109, 0.4808, 1.0232, 0.4663, -0.1579, -0.4077, 1.0219, -0.0108, 0.7364, 1.2863, -0.0563, 0.4812, -0.9524, -0.3164, -0.29, -0.1869, -1.6642, -0.1781, -0.24, -0.9793, 0.0932, 0.5735, 1.0192, 0.4628, -0.013, -1.0234, 0.7954, -0.4241, 0.6173, 1.1446, 0.0609, 1.0875, -0.6355, -0.1496, -0.7713, -0.3481, -1.9038, -0.1304, 0.2979, -0.3328, -0.196, 0.1932, 0.6703, -0.6535, -0.3854, -1.5877, 0.3043, -0.6069, 0.4644, 0.8949, 0.0215, 1.3145, -0.252, -0.0229, -0.9344, -0.3465, -1.8168, -0.119, 0.3296, -0.0754, 0.0746, 0.1626, 0.8297, -0.6153, -0.1917, -0.6072, 0.8027, -0.6673, 0.4238, 1.2299, -0.6317, 0.6314, -0.1752, 0.5908, -0.2426, -0.3641, -1.7651, -0.198, 0.0761, -0.1308, -0.1458, 0.1657, 0.8899, 0.1128, 0.2783, -0.3957, 0.5439, -0.808, 0.3782, 0.859, -1.027, 0.9873, 0.1864, 0.9685, -0.0129, -0.4761, -1.6025, -0.055, 0.0558, -0.0651, -0.4811, 0.134, 0.6191, 0.0191, 0.0996, -0.3312, 0.3399, -0.5409, 0.9758, 0.9659, -0.6106, 1.3742, -0.0263, 0.1609, -0.4046, -0.5613, -1.4181, 0.0876, 0.0924, -0.0935, -0.5894, 0.5861, 0.8391, 0.1135, -0.3195, -0.7578, 0.0493, -0.6943, 0.8268, 1.2798, -0.4827, 1.4191, -0.323, 0.1198, -0.6637, -0.5847, -1.3189, -0.088, 0.0113, 0.2189, -0.4166, 0.8556, 0.8058, -0.4251, -0.9557, -0.3114, -0.2713, -0.8843, 0.7361, 1.6854, 0.3174, 1.0722, -0.6162, 0.006, -0.9192, -0.6047, -1.0666, -0.1786, -0.2478, 0.4597, -0.7523, 0.7537, 0.6118, -0.5948, -0.8041, 0.3789, -0.0386, -0.8229, 0.3378, 0.9468, 0.3323, 0.8707, -0.5184, -0.0276, -0.4443, -0.6749, -0.8883, -0.302, -0.5425, 0.5643, -1.1409, 1.0362, 0.8503, -0.3006, -0.7136, 0.401, -0.425, -1.4626, -0.3317, 0.3344, 0.6286, 1.403, -0.0019, 0.1054, -0.2468, -0.7001, -0.7323, -0.2393, -0.5857, 0.5902, -1.3982, 1.3133, 0.706, -0.5792, -0.3637, 0.568, -0.2413, -1.101, -0.373, 0.1312, 0.5257, 0.9228, 0.3269, -0.24, -1.1497, -0.7316, -0.7132, -0.1787, -0.6771, 0.7321, -1.3458, 1.5843, 0.8427, -0.8187, -0.5972, 0.3523, -0.4352, -1.2535, -0.0796, 0.4224, 0.4536, 1.1542, 0.6534, -0.5216, -1.2708, -0.8085, -0.7707, -0.054, -0.6141, 0.6879, -1.332, 1.7002, 1.0604, -0.6178, -0.8655, -0.0296, -0.3001, -0.8607, -0.4608, -0.2386, 0.1817, 1.3041, 0.7808, -0.5378, -0.9656, -0.7989, -0.6221, 0.0228, -0.6228, 0.5768, -1.6094, 1.3545, 1.0325, -0.5526, -1.1446, -0.1132, 0.6445, -0.272, -0.6755, -0.3768, 0.1208, 1.077, 0.6081, -0.7483, -0.8028, -0.7914, -0.7001, 0.0295, -0.6658, 0.6335, -1.3146, 0.554, 0.0613, -0.7819, -0.8551, 0.3629, 1.0469, -0.6031, -0.6622, -0.0469, 0.3287, 1.4662, 1.0594, 0.0559, -1.0085, -0.4549, -0.8005, 0.3181, -0.4429, -0.2534, -0.3985, 0.6304, 0.2976, -0.3874, -0.5318, -1.0686, 0.005, -1.6576, -0.9694, 0.8374, 0.1785, 0.7689, 0.0373, -0.3239, -1.1571, -0.1155, -0.9788, 0.2501, -0.6416, -1.2364, 0.5311, 0.5941, 0.3057, -0.1407, -0.9051, -2.1199, -0.5279, -1.4416, -0.6566, 0.3683, -0.0042, 0.375, -0.9626, -0.3633, -0.9651, 0.0168, -1.3501, 0.0215, -0.6047, -0.9389, 1.2141, 0.0364, 0.141, -0.3111, -1.2148, -2.0148, -0.3908, -1.5153, -0.1999, -0.2454, -0.279, 1.1546, -0.6785, -1.5207, -1.1065, 0.0105, -1.7769, -0.1614, -0.3624, -0.5297, 1.4298, -1.0077, -0.4739, -0.7832, -1.191, -1.4994, -0.1812, -1.5812, -0.7596, -0.8429, -0.1342, 1.3657, -0.3989, -2.0141, -1.0863, 0.0379, -1.7504, -0.3161, -0.2695, -0.1869, 1.5135, -1.3009, -0.4755, -0.7098, -0.5849, -1.5456, -0.2932, -0.8481, -1.1459, -0.4826, -0.3021, 0.9132, -1.2032, -2.0254, -0.3637, 0.0546, -1.4407, -0.4121, -0.526, 0.1725, 1.3744, -0.7727, 0.5157, -1.0472, -0.8075, -1.2055, -0.0979, -0.1549, -0.9341, -0.0158, -0.536, 0.2153, -1.6607, -1.4256, -0.1553, -0.0463, -0.913, -0.728, -1.0906, 0.4708, 0.6568, -0.2728, 0.6065, -1.3757, -0.8091, -0.4996, -0.3884, 0.2058, -0.7505, 0.2179, -0.5516, 0.5259, -0.7995, -0.6232, -0.3023, -0.2185, -0.4793, -1.1821, -1.358, 1.2834, -0.1834, 0.5609, 0.1159, -1.03, -0.3973, 0.0312, 0.0045, 0.4768, -0.7374, -0.5324, -0.7081, 0.2242, -0.4571, -0.9142, 0.1131, 0.235, -1.0203, -0.3144, -0.6867, 0.9507, 1.3584, -0.6706, 1.5678, -1.1305, 0.7539, 0.1786, 0.9245, 0.4348, 0.3453, 0.1699, -0.1719, 1.2949, -0.5116, 0.0053, -0.4158, 0.7239, -1.07, 0.072, 0.571, 0.0042, 1.833, -0.6888, 1.2766, -0.5316, 1.3908, -0.2523, 1.1574, 0.4747, 0.5974, 0.9294, 0.3901, 1.5786, 0.0222, 0.537, -0.7997, 0.9239, -1.2614, -0.1372, 1.2451, -0.707, 1.9731, -0.4552, 1.0661, -0.204, 1.6376, 0.5038, 0.721, 0.4852, 0.2163, 0.3883, 0.1258, 1.5422, 0.8701, 0.9868, -1.0119, 0.9468, -1.208, -0.4451, 0.8509, -1.4671, 1.3219, -0.5978, 0.9639, -1.0743, 0.9687, 0.6516, 0.486, 1.5006, 0.7695, 0.9119, 0.5293, 1.877, 1.1606, 0.9111, -1.8459, 0.9832, -1.2475, -0.7764, 0.2911, -2.1965, 1.057, -0.7077, 1.0119, -1.2437, 0.6594, 0.3864, 0.4232, 2.8226, 1.5044, 1.8869, 0.916, 1.5929, 0.8714, 0.3149, -1.9095, 1.0611, -1.0741, -1.0377, 0.0759, -2.7582, 0.6521, -0.7738, 1.1959, -0.5842, 0.7841, 0.9652, 1.0617, 3.9576, 2.5183, 2.4848, 0.3937, 0.8181, 0.5641, 0.8999, -1.0067, 0.9463, -1.1657, -1.1305, 0.1439, -2.7187, 0.2987, 0.0681, 1.4599, -0.5647, 1.0148, 1.3389, 0.9481, 3.1115, 2.9341, 1.5757, -1.2421, 0.4951, -0.4024, 1.8963, 0.2862, 0.8924, -1.1946, -1.2968, 0.1747, -2.9566, -0.2486, 0.5338, 1.0171, 0.2201, 1.0486, 1.2134, 0.5715, 1.3339, 2.5844, 0.7043, -1.862, 0.8633, -0.7516, 1.9794, 0.8909, 0.8261, -1.3122, -1.2913, 0.3323, -2.7826, -0.2244, 0.6714, 0.7127, 0.1809, 0.4453, 0.7924, 0.5005, 1.3902, 2.8182, 0.4783, -2.1206, 1.0777, -0.2783, 2.0515, 1.0269, 0.8281, -1.4676, -1.1685, 0.4136, -2.8376, -0.0611, 0.5375, 1.4189, -0.0096, 0.1021, 0.5958, 0.1458, 0.7274, 3.0121, 0.7793, -2.0943, 1.0676, -0.0218, 2.3897, 1.2064, 0.8615, -1.6546, -1.1575, 0.3416, -2.9893, -0.116, 0.0997, 2.0771, -0.0252, 0.0266, 0.4569, -0.2009, 0.4911, 2.5834, 0.8675, -2.0185, 1.0329, 0.1515, 2.3842, 1.1622, 0.8583, -1.8851, -1.131, 0.2167, -3.1567, -0.0785, 0.07, 2.2028, -0.0154, 0.1841, 0.0275, -0.8505, 0.1641, 2.2763, 1.048, -1.5486, 1.1876, 0.3807, 1.5912, 0.9085, 0.8312, -2.0255, -1.0961, 0.3838, -3.2983, 0.0475, -0.2079, 1.8445, -0.1699, -0.0886, -0.3409, -0.8494, 0.3856, 1.6174, 1.0631, -1.4689, 1.4644, 0.1497, 1.251, 0.9722, 0.8161, -1.9128, -0.8346, 0.5849, -3.1036, 0.2789, -0.7185, 1.179, 0.0786, -0.2951, -1.3624, -0.2915, 1.6966, 1.923, 1.7214, -0.5824, 1.7401, -0.518, 0.681, 1.3045, 0.9271, -0.7336, -0.0889, 1.1947, -1.8298, 1.2473, -0.0316, 0.9979, 0.4683, -0.2514, -1.5291, -0.4188, 1.1054, 0.8928, 1.4122, -0.945, 0.8456, -0.4053, 0.0162, 0.4941, 0.8762, 0.3034, 0.1784, 0.7943, -0.2818, 1.4153, 0.6476, 1.4181, 0.9596, -0.0518, -0.7572, 0.3473, -0.4729, 0.5842, 1.9926, -0.4963, 1.3493, 1.0792, 1.2737, -0.3619, 0.9419, -0.0429, 0.3034, 0.6554, -0.212, 1.8609, 1.0308, 1.8887, 0.5589, 0.3015, -1.0608, 1.3216, -0.7836, 1.8825, 2.1114, -0.6814, 0.7778, 2.0025, 2.3586, -2.7938, 1.0009, -0.4116, 0.0525, 0.4199, -0.6743, 1.9827, 1.1689, 2.0924, 0.4723, 0.5156, -0.6497, 1.8283, -0.2643, 2.4167, 2.0292, -0.7402, 0.9695, 1.5068, 1.7906, -2.9726, 0.9234, -0.7257, 0.1687, 0.7273, -1.2745, 1.7649, 1.5426, 2.0885, 0.4247, 0.8498, -0.2913, 1.7736, 0.9799, 1.8236, 1.567, -0.2358, 1.9458, 0.5028, 0.8943, -1.595, 0.7978, -1.7292, 0.5834, 1.4195, -1.969, 2.2097, 1.5927, 2.4798, 0.7497, 1.5814, 0.3411, 1.7623, 1.2276, 1.7105, 1.3197, 0.455, 1.7531, -0.084, 1.5097, -1.1691, 0.6762, -2.3212, 0.9976, 2.2068, -2.0808, 2.0014, 1.0794, 3.183, 1.2231, 2.0983, 0.351, 2.1585, 0.8838, 1.78, 1.29, 0.4922, 0.7225, -0.3879, 2.5277, -1.2145, 0.6466, -2.465, 0.9807, 2.1353, -1.9909, 2.4095, 0.7091, 3.1925, 0.3767, 1.6151, 0.7365, 2.3599, 0.7282, 0.87, 0.9919, 0.7802, 1.274, -0.0394, 2.5437, -0.274, 0.649, -2.5954, 1.0056, 1.9545, -2.6375, 2.3196, 0.5339, 3.5053, 0.2325, 1.4054, 0.3766, 1.8546, -0.4487, -0.4127, 1.2311, 1.1038, 2.7335, 0.7443, 1.6845, -0.1358, 0.6297, -2.8429, 0.9891, 2.0544, -3.0159, 2.0625, 0.5328, 3.0397, -0.215, 1.9774, 0.6316, 1.927, -1.0586, -0.8205, 1.1221, 0.8575, 2.2373, -0.2994, 0.4713, -0.5173, 0.6404, -2.9198, 0.9411, 2.1406, -3.1664, 1.694, -0.2336, 2.8396, -0.5878, 2.2049, 1.2077, 1.4928, -0.7696, -0.9225, 0.9896, 0.6088, 1.3475, -0.5077, 0.519, -0.049, 0.5629, -2.7682, 1.0607, 1.9557, -2.8481, 1.4869, -0.874, 3.1234, -0.3783, 2.3701, 1.4596, 0.8977, -0.6577, -1.1312, 0.5856, -0.2677, 0.9344, -0.0617, 1.0816, 0.3619, 0.2512, -1.9414, 1.0574, 0.9814, -1.9158, 1.0839, -1.0956, 2.8304, 0.1927, 1.719, 1.3105, 0.8314, -0.5316, -1.4686, 0.4601, -0.2674, 0.7657, 0.4524, 1.1693, 0.1516, -0.1032, -0.9084, 0.7659, 0.4381, -0.6704, 0.3772, 0.0641, 1.6455, 0.3144, 0.3404, 0.9028, 1.0627, 0.1885, -1.2693, 0.8417, 0.5117, -0.035, 0.6532, 0.8507, -0.4352, -0.4233, -0.5347, 0.5296, 0.2228, -0.0539, -0.0995, 1.0507, 1.01, 0.8818, -0.4428, 0.9231, 0.6511, 0.0172, -0.8531, 0.6741, 1.0445, -0.0501, 0.7596, 0.3895, -0.4268, -0.7128, -1.0801, 0.3876, 0.0562, -0.3632, -0.1225, 1.612, 0.4248, 0.6982, -0.2275, 0.5433, -0.0403, 0.303, 0.0133, 0.0209, 1.7439, 1.1599, 0.5004, 0.5183, 0.0794, 1.0566, -1.0684, -0.544, -0.184, -2.0503, -0.1465, 0.1297, 0.0654, -0.1403, 0.4999, -0.6749, -0.221, -1.8966, -0.3442, -0.3743, 1.4875, 0.1378, -1.4842, -0.4385, -1.0081, 1.6274, -0.8946, -0.6356, 0.222, -1.9909, 0.4063, -0.2644, 0.4854, -0.12, 0.8454, -0.3326, 0.3341, -1.6974, -0.8016, -0.0472, 1.7307, 0.3144, -1.373, -1.0097, -1.4918, 1.6065, -0.5821, -0.8791, 0.4214, -2.0858, 1.1772, -0.527, 0.8363, -0.7305, 0.6997, -0.5442, 0.5442, -0.7737, -0.6738, 0.3306, 0.9413, 0.3846, -1.0417, -0.3051, -2.0137, 1.4788, -0.5812, -1.0941, 0.2319, -1.7563, 1.5405, -0.5702, 0.883, -1.1222, 0.623, -0.4009, 0.6315, 0.4847, 0.2333, 0.7168, 0.3905, 0.6486, -1.2319, 0.2771, -1.7377, 1.3747, -0.5651, -1.3178, -0.1484, -1.7937, 1.234, -0.8336, 0.9132, -0.991, 0.9301, 0.0717, -0.1241, 0.1123, -0.3122, 1.2468, 0.3915, 1.1732, -1.4986, 0.4402, -0.3996, 1.2854, -0.6386, -1.3546, -0.1376, -2.0574, 1.0119, -0.9988, 0.1319, -1.6982, 0.6043, 0.2489, 0.2883, -0.1637, -0.6638, 1.5629, -0.1349, 1.6612, -0.3945, 1.0254, -0.9419, 1.1655, -1.027, -1.4333, 0.3298, -2.4603, 0.8441, -0.576, -0.0653, -1.1618, 0.6867, -0.6298, 0.4637, 0.3421, 0.2183, 1.6551, -0.8553, 2.0461, -0.2015, 1.2998, -1.4847, 1.0665, -1.2583, -1.2745, 0.6419, -2.4529, 0.6389, -0.2419, 0.2274, -0.8015, 1.1263, -1.1165, 0.4109, 0.3869, 0.2074, 1.8659, -1.2266, 1.88, -0.4931, 0.5399, -1.6658, 0.9895, -1.2665, -1.2376, 0.5151, -2.1188, 0.1447, 0.0746, 0.4486, -0.5808, 1.5214, -0.8779, 0.6793, -0.6203, 0.5159, 2.0831, -1.2702, 0.9109, -0.7328, 0.0044, -2.4109, 0.8654, -1.197, -1.1669, 0.3824, -1.6291, 0.0141, 0.2164, 0.2907, -1.273, 0.983, -0.9891, 0.6275, -1.1764, 0.5957, 1.5011, -1.1129, 0.7633, 0.2316, 0.7575, -2.277, 0.7732, -1.1324, -0.7495, 0.151, -1.813, 0.3572, -0.6966, 0.4343, -0.9657, 0.0976, -0.9516, 0.1584, -0.949, -0.1581, 0.8213, -0.7967, 0.7247, 0.7829, 0.7027, -1.922, 0.76, -1.6336, -0.3498, 0.5047, -2.3347, 0.8909, -0.9142, 1.24, -0.7062, 0.7048, -0.6144, 0.2554, -1.5244, -0.5785, 1.0239, 0.0008, 0.7024, -0.2611, 0.0189, -1.9661, 0.8084, -2.3774, -0.3299, 0.8829, -2.7822, 0.9421, -0.8457, 2.1751, -0.8165, 1.8656, -0.4111, 0.3296, -1.6896, -0.5524, 0.9473, 0.2547, 1.0341, -0.8018, 0.0863, -1.8424, 0.7837, -2.8225, -0.2891, 1.1776, -2.9058, 1.0858, -0.6483, 2.4805, -1.5039, 2.0847, 0.1265, 0.8228, -2.1863, -1.4983, 0.2554, 0.1034, 2.0793, -0.1815, 1.047, -1.4431, 0.752, -2.973, -0.314, 1.4473, -2.8402, 1.1235, -0.6414, 2.5554, -1.7063, 2.2159, 0.1693, 0.1694, -2.7977, -2.0719, 0.1934, -0.3632, 2.035, 0.0003, 1.6357, -0.562, 0.7417, -2.9335, -0.4384, 1.3681, -2.8286, 1.1323, -0.7559, 2.2781, -1.6015, 2.1837, 0.2151, 0.3377, -1.8345, -1.9484, -0.5116, -0.4594, 1.1204, -0.0439, 1.4094, -0.2111, 0.657, -2.9391, -0.3299, 1.3441, -2.9624, 1.1556, -0.4365, 2.5738, -1.0891, 2.0438, 0.193, 0.949, -1.2231, -2.1006, -0.9053, -0.5764, 0.2563, -0.0886, 0.357, 0.6838, 0.6127, -2.8981, -0.2034, 1.5128, -3.0442, 1.0551, -0.2508, 2.5584, -1.2328, 1.6989, 0.5975, 1.4216, -1.2437, -2.3588, -0.6439, -0.1887, 0.8479, -0.1078, -0.0457, 0.7145, 0.5745, -2.9069, -0.2316, 1.6105, -3.0384, 1.2452, -0.1514, 2.6252, -0.9543, 1.5125, 0.7008, 1.5576, -0.9053, -2.9573, -1.5222, -0.3068, 0.8224, -0.7949, -0.0229, 0.4506, 0.5793, -2.9142, -0.2464, 1.7348, -3.0595, 0.8552, -0.7727, 1.7855, -0.8734, 1.4776, 0.7844, 1.1744, -1.0609, -2.7794, -1.7363, -0.4107, 1.0955, -0.3742, 0.2497, 0.1313, 0.5791, -2.9161, -0.4097, 1.7958, -3.2576, 0.7367, -0.1986, 1.7733, -0.5567, 0.9562, 0.2773, 0.9935, -0.5213, -1.8885, -0.7895, -0.5427, 1.3643, 0.1227, -0.3726, -0.4172, 0.5708, -2.5037, -0.4242, 1.4887, -3.117, 0.5997, -0.1375, 1.6138, -0.2914, 0.3967, -0.4461, -0.0234, -0.4767, -2.0528, -0.5402, -0.6048, 0.9965, 0.2161, 0.47, 0.0264, 0.4986, -1.6151, -0.4875, 0.6253, -2.4157, 0.1607, -0.5264, 1.3873, 0.6826, -0.1104, -1.123, -0.5949, 0.6732, -1.6417, -0.3748, -0.075, 0.7431, -0.9783, 0.2106, 0.5498, 0.3174, -1.1602, -0.5213, -0.0204, -1.5592, -0.3128, -0.8659, 1.5165, 1.2677, -0.4305, -1.6437, -0.522, 0.6305, -1.6449, -0.5727, -0.2313, 1.0541, -1.9479, -0.1982, 0.0711, 0.153, -1.2331, -0.454, 0.0359, -0.9133, -0.0004, -0.6887, 1.4394, 0.6289, -0.5691, -1.0295, -0.2815, -0.0788, -2.271, -0.5318, -0.833, 0.9568, -1.6954, -0.5219, 0.0474, 0.0456, -0.9201, -0.3239, -0.0278, -0.9439, -0.3796, -0.141, 1.1287, 0.8314, -0.2444, 0.1888, -0.1895, -0.7263, -1.4671, -0.1271, -0.6175, 0.6731, -1.7832, -0.6299, 0.4767, 0.0007, -0.7597, -0.3499, 0.0376, -0.7199, -0.805, -0.5097, 0.546, 0.6194, -0.0365, 0.3923, -0.7504, -0.7955, -1.5306, -0.6544, -0.8538, 0.9915, -0.5823, -0.2612, 0.5548, 0.0164, -0.6741, -0.4926, -0.1115, -0.4621, -0.9105, -0.8914, 0.4399, 0.7683, -0.0043, -0.055, -1.2257, -0.9475, -1.849, -0.7389, -0.9951, 0.6834, -0.1837, -0.6147, -0.0125, -0.0409, -0.5336, -0.5128, -0.2182, -0.2352, -0.9162, -0.446, 0.4628, 0.828, 0.0851, -0.1197, -0.6681, -0.0857, -1.733, -0.9551, -1.4454, -0.2133, 0.3196, -0.6791, 0.0155, 0.0257, -0.5947, -0.3017, -0.0867, 0.0678, -1.1077, -1.3682, 0.1127, -0.8614, 0.0412, -1.0241, -0.1928, 0.4292, -2.0859, -0.521, -1.1647, 0.2625, 0.0935, -0.047, 0.61, 0.2924, -0.8542, -0.1082, 0.0662, -0.6213, -1.1458, -1.9765, -0.4961, -0.6312, -0.1462, -2.0565, -0.1417, -0.5845, -1.4811, -0.771, -0.0384, 1.1379, -0.638, -0.0877, 0.6639, 0.8358, -1.9466, -0.0877, 0.3964, -1.867, -0.8912, -0.8143, -0.9248, -0.429, 0.1397, -1.4932, 0.4277, -1.2292, -0.1208, -1.4953, 1.1601, 0.7051, -0.3854, -0.7344, 0.4588, 1.0417, -2.4667, -0.0673, 0.7483, -2.0353, -0.3408, -0.7304, -0.9027, -1.5721, 0.7361, -0.6554, 0.3417, -1.4963, -0.112, -1.6496, 0.8058, 0.5938, 0.4148, -1.0383, -0.1943, 1.1057, -2.3647, -0.3728, 0.7827, -2.289, 0.1086, -1.2059, -0.2404, -1.8473, 1.6819, -0.0759, 0.1759, -0.8955, -0.087, -0.7532, 0.7252, 1.5318, 0.581, -1.0097, -1.3751, 1.0596, -2.1784, -0.7745, 1.2343, -2.6337, 0.4523, -2.1442, 0.3008, -1.8358, 2.0402, 0.1794, 0.0218, -0.5679, -0.5393, -0.0054, -0.2509, 1.8807, -0.524, 0.1823, -2.1867, 1.1112, -2.1784, -0.7133, 1.1335, -2.5449, 0.7184, -2.0793, 0.173, -1.9429, 2.1363, 0.1614, 0.7365, -0.7828, -0.9212, -0.5194, -0.8159, 2.0279, -0.6041, 0.5221, -1.2508, 1.1577, -2.1848, -0.6597, 1.1076, -2.6527, 0.5933, -2.126, 0.1855, -2.2102, 2.3439, -0.5518, 0.2963, -0.7091, -0.8816, 0.0793, -0.4278, 1.4839, -0.7977, 0.0884, -1.5677, 1.16, -2.1304, -0.6021, 1.1747, -2.822, 0.6185, -2.1492, 0.5486, -2.0298, 1.8973, -0.8785, -0.5778, -1.4374, -0.649, 0.6166, -0.1985, 1.1988, -0.8325, -0.0996, -2.1812, 1.0975, -2.4062, -0.7466, 1.0866, -2.7506, 1.2828, -1.6524, 0.955, -2.2965, 1.1991, -1.2001, -0.3402, -1.483, -0.9962, -0.1531, -0.9013, 0.9711, -0.754, -0.0246, -2.0944, 1.0316, -2.6833, -0.6013, 1.248, -2.7526, 1.7229, -1.671, 1.0159, -2.1352, 1.3834, -0.7624, 0.5847, -0.9252, -0.7733, -1.0645, -1.0871, 1.1603, -0.9917, -0.016, -1.5764, 1.0235, -2.9442, -0.6095, 1.2781, -3.007, 1.577, -1.5349, 1.2328, -2.2237, 1.4139, -0.1815, 0.9755, -1.0688, -0.5306, -0.5684, -0.3521, 1.1183, -1.5544, -0.2465, -1.2139, 0.9836, -3.3285, -0.9047, 1.2735, -2.8237, 1.562, -1.3599, 1.3387, -2.3036, 1.1369, -0.6028, 0.5315, -1.9986, -0.8497, -0.6971, -0.8578, -0.0871, -2.332, -0.09, -1.776, 1.0352, -3.3976, -0.9161, 1.0947, -2.9031, 1.4542, -1.6677, 1.7744, -1.916, 1.0368, -1.0287, 0.4422, -2.0855, -0.4212, -0.4267, 0.1076, 0.022, -2.5636, -0.0979, -1.7279, 1.0477, -3.4952, -0.6953, 1.0973, -3.2085, 1.14, -1.6535, 1.9375, -1.8646, 0.5019, -1.6037, 0.5455, -1.9345, -0.0599, -0.7172, 0.6131, -0.1736, -2.0407, -0.5253, -1.3355, 0.9281, -3.6175, -0.5329, 1.5349, -3.3856, 0.9369, -1.4826, 1.9739, -1.5461, 0.0484, -1.8927, 0.3312, -2.3792, -0.4102, -1.0777, 0.2851, -1.1739, -2.1211, -0.6847, -0.7285, 0.7771, -3.5808, -0.4177, 1.6093, -3.0107, 1.0615, -1.685, 2.0563, -0.7548, -0.2134, -1.9666, -0.0673, -2.8023, -0.5252, -0.737, 0.9389, -0.6882, -2.4728, -0.6082, -0.4749, 0.5843, -3.3369, -0.5275, 1.2726, -2.5653, 1.4472, -1.6018, 2.1614, 0.1717, -0.457, -0.7628, -0.1282, -2.7853, -1.1149, -0.334, 0.7208, -0.0276, -2.4073, -0.5295, -1.0341, 0.2958, -2.5024, -0.8208, 0.7443, -1.6674, 0.7418, -0.994, 1.9157, 1.0066, -0.6267, 1.1016, 0.0358, -2.5032, -1.6049, 0.0427, 0.4991, -0.1571, -1.5529, -0.0373, -1.2994, 0.0481, -1.555, -1.0768, 0.3577, -0.7799, -0.571, -0.0741, 1.3749, 0.7439, -0.8975, 1.7932, 0.0055, -1.9222, -1.5566, 0.0002, 0.5704, -0.2266, -0.988, -0.4719, -1.2483, -0.0023, -1.4869, -1.0435, 0.0729, -0.4834, -0.8614, -0.3296, 1.1438, -0.0195, -0.7424, 1.3921, 0.2758, -1.3166, -1.1331, -0.4049, 0.4389, -0.0967, -0.8513, -0.8612, -1.2289, 0.291, -2.4551, -0.2387, 0.0681, -0.6744, 0.0207, -1.0757, 1.4108, -1.3645, 0.8597, -0.4158, 1.1415, -2.1523, -0.1782, -0.986, 0.3466, -0.5872, -0.984, 0.0669, -0.7815, 0.5789, -2.7316, -0.0711, 0.659, -1.3141, 0.4754, -1.0854, 1.0406, -2.1095, 1.3985, -1.0681, 0.6341, -2.199, -0.2585, -1.437, -0.265, -0.2046, -0.1693, 0.2136, -0.182, 0.7113, -2.4653, -0.2108, 0.9223, -1.5629, 1.269, -0.4525, 1.1817, -2.7572, 0.5623, -0.8341, 0.1249, -1.4886, -0.844, -1.3202, -0.637, -0.4235, -0.8142, 0.4959, -0.5224, 0.8632, -1.9906, -0.6106, 0.8821, -1.5688, 1.6258, -0.4336, 0.958, -2.391, 0.3555, -1.5894, -0.6501, -1.0229, -1.4061, -1.1649, -1.3356, -0.6095, -1.3722, 1.1901, -1.1869, 0.935, -1.6474, -0.9848, 0.3971, -1.6735, 1.4129, -0.0665, 0.4725, -2.15, -0.089, -2.4393, -0.4516, -0.7965, -2.1064, -1.3116, -0.9625, 0.9887, -0.511, 1.5771, -0.6528, 0.9545, -1.6089, -1.2212, -0.0225, -2.0555, 1.4535, -0.03, -0.0527, -2.222, -0.5056, -2.0519, -0.4911, -0.3829, -1.5869, -1.2229, -0.432, 1.3679, 0.1234, 1.2171, -0.4676, 0.9919, -1.802, -1.3844, -0.3313, -2.3472, 1.2141, -1.0263, -0.7282, -2.5293, -0.2965, -1.682, -0.4292, 0.0424, -1.2721, -1.0685, -0.3775, 1.4513, 0.3678, 1.4687, 0.1703, 1.0422, -1.6654, -1.3734, -0.4486, -2.2948, 0.6245, -1.6476, -0.8371, -2.2101, 0.2774, -2.281, -0.5399, -0.3456, -1.1997, -1.2902, -0.3632, 1.149, -0.3392, 1.1236, 0.6334, 0.9459, -1.3118, -1.2601, -0.1253, -1.9819, 0.3011, -1.555, -1.3305, -2.0323, 0.1346, -2.4562, -0.0444, -0.0634, -0.9448, -1.1602, 0.0091, 1.5844, 0.1258, 1.3614, 0.5058, 0.8547, -1.2645, -1.2381, 0.2477, -1.6764, 0.5804, -1.4902, -1.9739, -1.8473, -0.0336, -2.5171, -0.6509, -0.2378, -0.3902, -1.0539, 0.2173, 1.5939, 0.717, 1.3015, 0.1378, 0.9699, -1.4455, -1.1521, 0.695, -1.9063, 0.8942, -1.9088, -1.7716, -1.5691, -0.3611, -2.596, -1.2935, -0.383, -0.8329, -0.7656, -0.1154, 1.1535, 0.8875, 1.1161, 0.23, 1.0256, -1.6255, -1.2071, 0.5638, -2.0126, 1.0898, -2.2668, -1.6296, -1.6624, -0.5771, -2.2808, -0.9186, -0.1037, -0.905, -0.6106, 0.1303, 1.4542, 0.8042, 1.0086, 0.2103, 1.2078, -0.9505, -0.489, 1.2515, -1.057, 2.3063, -0.405, -0.487, -0.8494, 0.535, -1.1866, 0.0934, 0.1402, 0.0573, -0.8643, 0.3304, 1.9335, 0.3754, 0.656, -0.488, 1.3987, -0.7683, -0.0686, 1.7698, -0.9065, 2.7148, 0.408, 0.2036, -0.4263, 1.7146, -0.7987, 1.0297, 0.1552, 0.5024, -0.9302, 0.6974, 2.7158, 0.6924, 1.5672, -0.7401, 1.3995, -0.7242, 0.061, 1.5823, -0.8714, 2.7083, 0.3947, 0.7295, -0.258, 1.7072, -0.6888, 1.417, 0.7817, 1.1686, 0.6513, 1.5657, 2.1721, -0.0297, 0.8802, -0.9993, 1.3093, -0.763, -0.0314, 1.4229, -0.7626, 2.6581, 0.4067, 1.1056, -0.2264, 1.4267, -0.0655, 1.1, 0.7565, 0.6155, 1.3108, 1.5972, 1.8828, 0.4318, 0.5904, -1.1851, 1.2245, -0.8436, -0.1474, 1.3239, -0.5259, 2.8818, 0.1517, 0.8032, -0.5344, 1.2522, 0.1166, 1.1981, 1.15, 0.7459, 1.3959, 1.2003, 2.7539, 0.8901, 0.891, -0.4566, 1.1086, -0.8883, -0.1462, 1.1094, -0.429, 2.8567, -0.1397, 0.5374, -0.7978, 0.7922, -0.1075, 0.8824, 1.4596, 1.1513, 1.7349, 1.2734, 3.1228, 1.4453, 1.3079, 0.8537, 1.0457, -0.7889, -0.0575, 1.2317, -0.2523, 2.5571, -0.1018, 0.615, -1.3428, 0.3847, -0.1564, 0.8101, 1.7698, 1.4314, 1.681, 1.4191, 3.1234, 2.0906, 1.831, 0.5503, 0.9707, -0.8141, -0.0538, 1.019, -0.5051, 2.3499, 0.3637, 0.2173, -1.9519, 0.1772, -0.2719, 0.963, 1.4847, 1.9001, 1.548, 1.5224, 2.9959, 2.2401, 2.3293, 0.7818, 0.852, -0.8804, -0.1568, 0.7996, -0.5958, 2.0765, 0.0744, -0.069, -1.6837, -0.4152, -1.0449, 0.4884, 1.4362, 2.107, 1.5466, 1.114, 1.9232, 1.3235, 2.128, 0.8189, 0.6746, -1.0716, -0.0884, 0.5313, -1.0645, 1.5172, -0.7221, -0.1202, -1.5589, -1.5744, -2.29, -0.9104, 0.7185, 1.6722, 1.258, 0.322, 0.269, 0.3172, 1.7307, 0.9389, 0.6213, -0.8812, 0.1721, 0.1208, -1.8206, 0.9687, -0.8878, 0.0268, -1.368, -1.7061, -2.7084, -1.0599, -0.1037, 0.6477, 1.1966, 0.1327, 0.3702, 0.079, 1.1483, 0.5902, 0.5084, -0.5762, 0.4705, 0.253, -2.105, 0.5277, -0.6684, -0.5127, -0.7249, -1.2395, -2.7746, -1.214, -0.4719, -0.4972, 1.09, 0.4768, 0.5249, 0.947, 0.9869, -0.3749, 0.3187, -0.5672, 0.5111, 0.4277, -1.7417, 0.2459, -0.4132, -0.2768, -0.8809, -1.1816, -2.2458, -1.5608, -0.9953, -1.6097, 1.2206, -0.0096, 0.4288, 1.8545, 1.2879, -0.7024, 0.2275, -0.5496, 0.5973, 0.4745, -1.5402, -0.0885, -0.517, 0.22, -0.8568, -0.3747, -0.7104, -1.027, -1.0787, -1.902, 0.5423, -1.0795, 0.0538, 1.9625, 1.4081, -0.272, 0.0858, -0.6442, 0.5679, 0.4134, -1.2936, -0.5631, -1.1144, -0.5017, -1.8215, -0.5629, -0.3807, -0.773, -0.1566, -0.8901, 0.2836, -1.6609, -0.3706, 1.4633, 1.2671, 0.1104, -0.09, -0.5575, 0.2973, 0.1006, -0.633, -0.6684, -0.7135, -0.8347, -1.1896, -0.5094, -0.7851, -0.6642, -0.1033, -0.2893, 0.1192, -1.3531, -0.5523, 0.4186, 1.1911, 0.4554, -0.2233, -0.5011, 0.2164, 0.3245, -0.086, -0.7286, -0.0663, -0.2461, -0.5963, -0.9095, -1.0418, -1.3269, -0.24, 0.4863, 0.9901, -0.2441, -0.6334, 0.1644, 0.7113, 0.5633, -0.1757, -0.6262, 0.0074, 0.5407, 0.083, -0.5029, 0.1357, 0.3296, -0.82, -1.0768, -0.39, -0.8403, 0.2651, 0.3223, 1.435, 0.0372, -0.7608, -0.0895, 0.3865, 0.6413, -0.1491, -0.8334, -0.0261, 0.5295, -0.1898, -0.2244, -0.0435, 0.5192, -0.4572, -1.1105, -0.1708, -0.5065, -0.1996, 0.4175, 1.2616, -0.1364, -0.6469, -0.1749, 0.833, 0.587, -0.0755, -1.2371, -0.1144, 0.4122, -0.5615, 0.3261, -0.2752, 0.9844, -0.497, -1.1584, -0.7426, -0.634, -1.1021, -0.6362, 0.871, 0.0846, 0.977, 0.4445, 1.0869, -0.2953, 0.0284, -1.4086, -0.0098, 0.1328, -0.8691, 0.6062, -0.2404, 1.3807, -0.8103, -0.6744, -0.9886, -0.5228, -0.7313, 0.1204, 1.4475, -0.2346, 0.9888, 0.9068, 0.5087, -0.7545, 0.09, -1.5794, 0.0963, 0.2855, -0.9284, 0.5651, -0.3627, 1.285, -0.9869, -0.3927, -0.7929, -0.1126, -0.6163, 0.2052, 1.188, -0.568, 0.3728, 0.8322, 0.1772, -1.079, 0.1389, -1.7409, 0.2138, 0.2255, -0.9663, 0.36, -0.1998, 1.2938, -0.8646, -0.1434, -0.8127, -0.0318, -1.0184, -0.0517, -0.2131, -0.1694, -0.089, 0.8032, -0.4043, -0.986, 0.1013, -1.7432, 0.1689, 0.0662, -0.8048, 0.3943, -0.2477, 0.9727, -0.9289, -0.0686, -1.0204, 0.013, -0.5737, 0.2645, -1.3067, -0.4694, -0.4276, 1.41, -0.2093, -0.8855, 0.0368, -1.7152, -0.0385, -0.0489, -0.8979, 0.3705, -0.7159, 0.7514, -0.4328, 0.0819, -0.7944, 0.1218, 0.2299, 0.7, -0.9824, -0.3268, 0.1066, 1.1969, -0.3484, -1.5325, -0.0902, -1.5819, -0.0065, -0.0462, -0.8965, 0.4698, -0.6829, 0.6761, -0.6661, -0.0571, -0.3648, -0.1218, 0.0853, -0.3899, -0.4935, -0.0334, 0.6479, 1.1693, 0.1202, -1.6503, -0.2333, -1.4564, -0.0169, -0.2506, -0.6236, 0.8352, -0.1951, 0.3631, -0.8324, -0.2212, -0.1857, -0.1716, 0.343, -0.1201, -0.055, -0.0547, 0.4849, 0.7027, 0.3432, -1.2059, -0.4092, -1.1782, 0.0417, -0.5394, -0.641, 0.4754, 0.3854, 0.6205, -0.4271, -0.5597, -0.0398, -0.4904, 0.0682, -0.0259, -0.156, 0.394, 0.8875, 0.4875, 0.4865, -0.8691, -0.6818, -0.7771, 0.0682, -0.725, -0.1888, -0.3768, 0.54, 0.2824, -0.5766, -1.2488, 0.2801, -0.1368, 0.049, -0.2937, -0.4236, 0.652, 1.118, 0.506, 0.1784, -0.4806, -0.7029, -0.4818, -0.0269, -1.0586, -0.0094, -0.6999, 0.9396, 0.255, -0.4718, -1.0967, 0.4687, 0.5916, 1.0765, 0.3845, -0.3223, 0.9205, 1.0404, 0.7535, 0.5338, -0.0124, -0.5207, -0.3959, -0.7154, -1.4352, 0.5948, -0.35, 1.1522, 0.1847, -0.2621, -0.6115, 0.5925, 0.8086, 1.0333, 0.7196, 0.3997, 0.6188, 0.5211, 0.3088, -0.0847, -0.4274, 0.598, -0.9823, 0.1187, 0.7043, -0.4826, 1.0517, 1.2729, 0.2123, 0.3309, 0.3096, -0.6312, 0.0432, -1.1369, -0.0462, -1.343, 0.3173, 0.5676, 0.2459, 1.0088, -0.0128, 1.3226, -1.2291, 0.6029, 1.6209, -0.9676, 1.2593, 1.2875, 0.8046, 0.7768, 1.1898, -0.7335, 0.3927, -1.1446, 0.4591, -0.7244, 1.2718, 0.9697, 0.7731, 1.0438, 0.1966, 1.4237, -1.1691, 0.5055, 1.7048, -1.2378, 1.6375, 1.1334, 1.7243, 0.7325, 1.5577, -1.1162, 0.8851, -0.6421, 0.7761, 0.7321, 1.9449, 0.9148, -0.3719, 0.4876, -0.5303, 1.2742, -1.1567, 0.4367, 1.8172, -1.1589, 1.928, 0.7123, 2.6559, 0.0369, 1.7416, -0.6626, 1.048, 0.5277, 0.8949, 1.8704, 1.3266, 1.5898, -1.037, -0.3702, -1.5601, 1.1522, -1.3067, 0.4729, 1.5021, -1.2667, 1.9551, 0.3487, 3.1265, 0.1927, 1.7143, 0.3693, 1.9204, 1.096, 0.6323, 1.2297, 0.7774, 2.3725, -0.7195, 0.1155, -1.4727, 1.0122, -1.3006, 0.2671, 0.8771, -1.2923, 1.9148, 0.2318, 2.8031, 0.434, 1.6368, 0.6228, 1.47, 0.5886, 0.1453, 0.8456, 0.4934, 2.2972, -0.1611, 0.5681, -1.0746, 0.8823, -0.9674, 0.291, 1.1291, -1.4111, 1.4908, 0.585, 2.2325, 1.5195, 1.7459, 0.9643, 1.1507, 0.3111, -0.496, 1.0259, 0.8402, 2.2837, 0.3745, 0.2183, -0.3495, 0.8402, -0.6895, 0.1534, 1.1146, -1.7346, 0.9586, 0.6645, 2.0555, 1.8326, 1.1372, 0.7961, 1.415, 0.8249, -1.1897, 1.5058, 1.1996, 2.7669, 0.5351, 0.6545, 0.5203, 0.9309, -1.0774, -0.0772, 1.1124, -2.2963, 0.7735, 0.1894, 2.2563, 1.6653, 1.2295, 0.1854, 0.6237, 1.5744, -0.7114, 2.0062, 1.4556, 3.0824, 0.7004, 0.8649, 0.6172, 1.0626, -1.5353, -0.2328, 1.1448, -3.2724, 0.1802, -1.0075, 1.9925, 1.4722, 1.2373, -0.9518, -0.6733, 2.4865, 1.0006, 2.8164, 0.5411, 2.1711, -0.0313, -0.1649, -0.4055, 1.0453, -1.8201, -0.4873, 1.0142, -3.3471, 0.3502, -1.0129, 1.6591, 0.3075, 0.2317, -1.5545, -1.0153, 2.4042, 1.0645, 2.2797, -0.0435, 1.2656, -0.3397, -0.5982, -0.8435, 1.0163, -1.6821, -0.7028, 1.019, -2.7745, 0.2802, -0.5765, 1.6399, -0.2222, -0.0942, -1.3589, -0.2707, 2.5261, 0.7779, 1.7061, -1.2249, 0.2753, 0.295, -0.2284, -0.3725, 0.9619, -1.3216, -0.7065, 0.5523, -2.1504, -0.1958, -0.6028, 1.2897, -0.8645, 0.1412, -1.8866, -0.1265, 2.2711, 0.1488, 1.1169, -1.0053, 0.334, 1.3422, 0.1611, -1.2086, 0.8825, -1.8151, -0.1796, 0.517, -2.5854, 0.8784, -1.3351, 1.1497, -0.9351, -0.0217, -1.7518, 0.0298, 0.9266, -0.4114, -0.0726, -1.131, 0.842, 0.5581, 0.9343, -1.9138, 0.7649, -2.6762, 0.4368, 0.7314, -3.3746, 1.751, -1.4691, 1.4555, -1.0731, 0.5051, -0.9281, 1.0462, -0.2103, -1.3108, -0.2312, -0.6908, 0.6438, -0.6035, 0.6339, -1.7754, 0.786, -3.1514, 0.6156, 0.9702, -3.4889, 2.3852, -0.5865, 2.4781, -1.1548, 0.5747, -0.9631, 1.2281, -0.9368, -1.439, 0.3216, 0.0838, 0.9221, -0.9493, 0.6617, -1.4711, 0.8338, -3.3939, 0.5852, 1.0407, -3.7705, 2.474, -0.442, 3.1351, -0.8497, 0.3216, -0.8935, 1.0574, -1.6943, -1.2132, 0.8625, 0.2185, 1.0696, -0.4465, 1.523, -1.6919, 0.8405, -3.4931, 0.5986, 0.8426, -4.2255, 2.416, -0.3076, 3.6913, -0.9117, 0.4433, -0.1389, 1.6479, -1.6847, -1.4597, 0.5409, -0.9696, 0.3489, 0.0093, 2.3065, -1.8264, 0.8364, -3.6501, 0.3973, 0.5605, -4.2918, 2.5611, 0.1624, 4.2112, -1.3501, 0.5048, -0.1207, 1.6129, -2.0093, -2.6173, -0.3373, -0.7928, 0.4663, -0.5325, 2.0416, -1.6212, 0.8724, -3.4509, 0.2718, 0.5143, -3.9033, 2.4479, 0.1496, 4.0652, -1.4376, 0.4925, -0.7792, 1.3403, -1.6411, -2.6129, -0.6613, -0.0097, 0.994, -0.7669, 1.5603, -1.3301, 0.9331, -2.991, 0.3979, 0.794, -3.391, 2.4683, 0.431, 3.8331, -1.0309, 1.4592, -0.4811, 1.3769, -0.5142, -1.7105, -0.2822, -0.1874, 0.9035, -1.0541, 0.5013, -1.5916, 0.882, -2.6107, 0.612, 1.4018, -2.5925, 2.5684, 0.1014, 3.2032, -0.685, 1.937, 0.1138, 1.4019, 0.6935, 0.196, 0.9863, -0.5055, 0.4438, -1.4782, 0.403, -1.5814, 0.9392, -1.84, 0.4309, 1.3377, -1.8772, 2.2282, 0.0851, 2.567, -0.5085, 1.2025, -0.1812, 0.9245, 0.781, 0.5053, 1.1645, -0.5418, -0.5363, -1.445, 2.1709, -0.3913, 0.891, -1.1871, -0.1705, 1.0155, -1.1539, 1.3524, 0.5179, 2.0579, 0.3092, 0.7163, -0.8766, 0.5858, 0.724, 0.3198, 0.2781, -0.4701, -0.0086, -1.0412, 2.29, 0.645, 0.8391, -0.9518, -0.4119, 0.9632, -0.8804, 0.7645, 0.3255, 1.6155, 0.6684, 0.5781, -0.853, 0.404, 0.7049, 0.4638, -0.0074, 0.1536, 0.4761, 0.0926, 1.771, 0.5702, 0.7288, -0.8559, -0.3149, 0.886, -0.6163, 0.7234, 0.2, 1.7566, 0.4041, 0.3229, -0.1768, 0.2682, 0.2563, 0.1378, 0.033, 0.5824, 0.3792, 0.3073, 1.3307, -0.0656, 0.5337, -0.7431, 0.0097, 0.5426, -0.316, 0.4589, -0.3612, 1.8919, 0.1799, 0.3292, 0.2757, 1.2201, -0.0596, 0.1325, 0.3297, -0.1543, -0.1813, 0.204, 2.3266, 0.1275, 0.274, -0.4945, 0.2072, 0.0509, -0.1198, 0.3098, -0.2269, 1.9934, 0.6127, 0.7448, 0.5998, 0.7797, -0.4709, 0.0045, 0.3804, -0.8918, -0.504, 0.5191, 2.6212, -0.1689, 0.0646, -0.4127, 0.223, 0.2178, -0.3046, 0.6912, 0.5805, 1.6253, 1.289, 0.827, 1.5944, -0.7039, -0.0773, -0.0937, -0.3317, -1.6958, 0.0824, 1.0555, 1.7838, -0.0976, -0.0823, -0.5005, 0.316, 0.6685, 0.0627, 0.533, 0.4231, 0.9776, 1.1727, 0.5452, 1.5978, -0.8948, 0.4912, 0.7221, -0.1729, -1.7608, -0.6128, 0.6145, 1.0769, -0.1973, -0.4155, -1.1648, 0.304, 0.7134, 0.1466, 0.7424, 0.5053, 0.5248, 0.4862, 0.0756, 1.3596, -0.7844, 0.7675, 1.2274, -0.6181, -1.8648, -1.0819, -0.0358, 1.072, -0.6627, 0.3666, -1.189, -0.3293, -0.2224, -1.3648, -0.1574, 0.2406, 0.0093, -0.352, 0.0638, -0.6619, -1.9095, -1.4406, 0.0445, -0.9525, 0.389, -0.5226, -0.8403, -0.4182, -1.281, 1.4968, -1.1336, -0.5462, -0.0643, -1.722, -0.0546, -0.1571, 0.1018, -0.0841, 0.8844, -0.8025, -0.1321, -1.6859, 0.0887, -0.7715, 1.8454, 0.091, -0.8339, -0.99, -1.6337, 1.7253, -0.9894, -0.6456, 0.084, -1.6879, 0.4817, -0.4775, 0.3588, -0.3574, 0.8894, -0.9354, 0.3496, -1.5236, -0.1283, -0.5743, 1.5566, 0.3512, -0.5493, -1.0422, -1.9134, 1.614, -0.7072, -1.0157, 0.236, -1.8029, 1.6474, -0.3948, 0.9432, -1.4396, 0.4416, -0.8797, 0.6491, 0.0692, 0.4253, 0.8993, 0.685, 0.8517, -0.6253, 0.1077, -2.3374, 1.5351, -0.7946, -1.2238, 0.0009, -1.6059, 1.5958, -0.6129, 0.8688, -1.2279, 0.5079, -0.4038, 0.6662, 1.0768, 0.7526, 1.0539, 0.4115, 1.2559, -0.6284, -0.0529, -1.487, 1.4458, -0.8888, -1.43, -0.1458, -2.0932, 1.177, -0.4033, 0.6702, -1.0036, 0.2627, 0.1494, 0.7803, 0.772, 0.2923, 0.6804, 0.3112, 1.1736, -0.8031, 0.0616, -0.7942, 1.2051, -1.1235, -1.3272, 0.076, -2.3833, 1.116, 0.3514, 1.0238, -0.629, -0.578, -0.074, 0.5481, 0.2845, 0.2461, 0.4423, 0.2053, 1.1484, -0.9603, 0.2335, -1.5567, 1.0374, -1.382, -1.3116, 0.1841, -2.6062, 0.8403, 0.5858, 1.6429, 1.3085, -0.4038, -0.2307, -0.3769, -0.3185, 0.9012, -0.6568, 0.0577, 0.77, -1.404, 0.0249, -1.3308, 0.9965, -1.3344, -1.2281, 0.2985, -2.8445, 0.5012, 0.6277, 1.5768, 1.726, -0.8138, -0.0601, -0.6653, -0.0218, 1.9426, -0.5523, 0.3547, 0.7709, -1.1372, -0.255, -1.0934, 0.9642, -1.3738, -1.3261, 0.3767, -2.9003, 0.5356, 0.9177, 1.4567, 1.3741, -1.0356, 0.0776, -0.9785, 0.2244, 1.8168, -0.0564, 0.4365, 0.1289, -1.8679, -0.5584, -0.7362, 0.8878, -1.3764, -1.4539, 0.3648, -2.8834, 0.4051, 0.7101, 1.4702, 2.0519, -0.9104, 0.3943, -1.3884, 0.425, 1.0882, -0.375, 0.135, 0.3332, -1.2841, -0.6182, 0.51, 0.8365, -1.3941, -1.4363, 0.4532, -3.0796, 0.2379, 0.3502, 1.3079, 2.4391, -0.7326, 0.8961, -2.0571, 0.8595, 1.5752, 0.3918, -0.6794, -0.3566, -0.895, -0.6121, 0.7459, 0.8492, -1.7555, -1.2991, 0.3409, -3.1528, 0.2008, -0.112, 1.6658, 2.1209, 0.0022, 0.4885, -2.1169, 0.5318, 1.7967, 1.4723, -0.4102, 0.0271, -1.4679, -0.2307, 0.767, 0.8642, -1.9925, -1.3098, 0.4204, -3.1342, 0.0733, -0.3141, 1.6269, 1.5607, 0.2656, 0.3662, -2.4215, -0.0565, 0.7906, 1.2386, -0.3281, 0.4338, -1.682, -0.221, 1.3303, 0.9058, -1.9783, -1.1835, 0.386, -2.9568, 0.2212, -0.4797, 1.4088, 1.0482, 0.7537, 0.3954, -1.8445, -0.627, 0.2038, 0.9912, -0.5542, 0.4068, -2.2158, -0.21, 1.2398, 0.9093, -1.9948, -1.1126, 0.1697, -3.0204, 0.1904, -0.6529, 1.3315, 0.5508, 1.6431, 0.0217, -1.0039, -0.4891, -0.007, 1.2787, -0.328, 0.9904, -2.4332, -0.5695, 1.0765, 0.9215, -1.7091, -0.9739, 0.1192, -3.0821, -0.0588, -1.1468, 0.6008, -0.3547, 1.3959, -0.2811, -0.839, -0.1867, 0.8827, 1.7833, -0.2133, 1.4815, -2.7525, -1.5469, 0.1746, 0.7248, -1.2428, -0.8628, 0.1353, -2.4824, -0.518, -1.2627, 0.0565, -0.4842, 0.1761, -0.0416, -0.6065, -0.381, 0.8207, 1.9225, 0.3415, 1.6001, -2.4352, -1.7236, 0.0082, 0.2268, -0.8224, -0.5604, -0.0869, -1.5987, -0.9492, -0.5078, 0.0332, -0.1281, -0.3252, 1.4468, 0.3357, -0.7017, -0.4368, 0.7212, 0.4283, 0.9078, -1.6321, -1.4381, 0.8286, -0.0911, -0.6602, -0.4532, -0.4274, -1.218, -1.0193, 0.6433, 0.5005, 0.5288, -0.1255, 2.3217, 0.6325, -0.483, -1.326, -0.9525, 0.2948, -0.0812, -0.7485, -1.0548, 0.3612, -0.1757, -0.8951, -0.4481, -0.2777, -1.2756, -1.2552, 0.2088, 0.2188, 0.7268, 0.022, 2.2258, 0.7296, -0.1137, -1.2015, -0.9451, 0.3332, 0.5307, 0.2421, -0.8618, -0.8645, -0.2738, -0.9815, -0.5082, -0.4675, -1.2887, -1.3713, 0.5077, 0.4508, 0.5271, -0.3051, 1.751, 0.0583, -1.0238, -1.451, -1.0215, 0.0962, 0.4249, 0.1214, -0.9639, -0.8335, -0.2556, -0.8746, -0.5566, -0.5126, -1.4643, -1.4044, 0.8874, 0.6941, 0.9663, -0.2771, 1.5738, -0.0468, -1.3609, -1.7803, -1.3467, -0.0026, 0.0211, -0.5858, -1.4892, -0.8774, -0.2536, -0.8531, -0.6528, -0.5968, -1.3178, -1.6518, 0.6563, 0.6269, 1.0551, -0.3043, 1.2339, -0.1698, -1.4815, -1.6743, -0.9912, 0.1833, 0.2199, -0.6271, -1.681, -1.5538, -0.2169, -1.1595, -0.2926, -0.677, -0.9051, -1.3295, 0.0926, 1.0175, 0.2457, 0.1618, 0.5925, 0.0377, -1.7234, -0.847, -0.771, 0.865, 0.7233, -0.1253, -1.3477, -1.7687, 0.3511, -1.7208, 0.4358, -0.2246, -0.4797, 0.2799, -0.3021, 2.5045, -0.9345, 0.8467, -0.9303, 0.4578, -1.4763, 0.4302, 0.2939, 1.5549, -0.393, -0.0118, -0.7443, -0.0213])  # Example feature vector
k =8 # Number of nearest neighbors to find
nearest_neighbors = model.approxNearestNeighbors(normalized_df, key, k)

nearest_neighbors.show()

24/05/10 07:42:06 WARN TaskSetManager: Stage 46 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------+--------------------+--------------------+
|             Feature| Label|              hashes|             distCol|
+--------------------+------+--------------------+--------------------+
|[0.36867056582057...|000002|[[-4.0], [-3.0], ...|0.002907392808289832|
|[0.35350788704256...|000237|[[-4.0], [8.0], [...|  132.28162478263482|
|[0.01978337383415...|000172|[[-6.0], [-3.0], ...|   132.8116696678195|
|[0.31870555364281...|000228|[[-6.0], [18.0], ...|   135.2893767406185|
|[4.23758335180759...|000176|[[-4.0], [2.0], [...|  145.88679347882152|
|[-0.2272291045045...|000255|[[-1.0], [2.0], [...|  147.55635896749158|
|[-0.1803237575068...|000252|[[17.0], [11.0], ...|   152.4403701758318|
|[-1.1309648640403...|000167|[[8.0], [3.0], [1...|  154.02075860561393|
+--------------------+------+--------------------+--------------------+



TRAINING ON Annoy (Approximate Nearest Neighbors Oh Yeah)

In [75]:
import annoy

def create_annoy_index(df, n_trees):
    index = annoy.AnnoyIndex(df.select('Feature').first()[0].shape[0], "angular")
    for i, row in enumerate(df.collect()):
        feature = row['Feature']
        index.add_item(i, feature)
    index.build(n_trees)
    return index


In [76]:
annoy_index = create_annoy_index(normalized_df, 10)

24/05/10 18:04:06 WARN TaskSetManager: Stage 55 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:04:06 WARN TaskSetManager: Stage 56 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


In [78]:
def nearest_neighbor_search(annoy_index, query_feature_vector, spark_df, k=5):
    nearest_indices = annoy_index.get_nns_by_vector(query_feature_vector, k)
    nearest_labels = [spark_df.select('Label').collect()[i][0] for i in nearest_indices]
    return nearest_indices, nearest_labels

# Assuming 'query_feature_vector' is the feature vector you want to search for
query_feature_vector = spark_df.select('Feature').first()[0]
nearest_indices, nearest_labels = nearest_neighbor_search(annoy_index, query_feature_vector, spark_df)

print("Nearest neighbors indices:", nearest_indices)
print("Corresponding labels:", nearest_labels)


24/05/10 18:07:00 WARN TaskSetManager: Stage 58 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:07:00 WARN TaskSetManager: Stage 59 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:07:01 WARN TaskSetManager: Stage 60 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:07:01 WARN TaskSetManager: Stage 61 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:07:01 WARN TaskSetManager: Stage 62 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 18:07:02 WARN TaskSetManager: Stage 63 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


Nearest neighbors indices: [40, 35, 36, 93, 41]
Corresponding labels: ['000166', '000161', '000162', '000249', '000167']


In [79]:
annoy_index.save("music.ann")   #   Saving the AnnoyIndex object into a separate file.

True

TRAINING ON KNN_MODEL

In [12]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [54]:
# Step 3: Split the data
train_data, test_data = normalized_df.randomSplit([0.5, 0.5], seed=123)

In [55]:
# Prepare data for scikit-learn
train_features = train_data.select("feature").rdd.map(lambda row: row[0]).collect()
train_labels = train_data.select("Label").rdd.map(lambda row: row[0]).collect()

test_features = test_data.select("feature").rdd.map(lambda row: row[0]).collect()
test_labels = test_data.select("Label").rdd.map(lambda row: row[0]).collect()


24/05/10 07:46:58 WARN TaskSetManager: Stage 47 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 07:47:01 WARN TaskSetManager: Stage 48 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 07:47:02 WARN TaskSetManager: Stage 49 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 07:47:03 WARN TaskSetManager: Stage 50 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.


In [58]:
from pyspark.mllib.regression import LabeledPoint

# Split the data
train_data, test_data = normalized_df.randomSplit([0.5, 0.5], seed=123)

# Prepare data for scikit-learn
def to_labeled_point(row):
    label = row["Label"]
    features = row["Feature"]
    return LabeledPoint(label, features)

train_labeled_points = train_data.rdd.map(to_labeled_point)
test_labeled_points = test_data.rdd.map(to_labeled_point)

# Convert labeled points to lists for scikit-learn if needed
train_features = train_labeled_points.map(lambda lp: lp.features.toArray()).collect()
train_labels = train_labeled_points.map(lambda lp: lp.label).collect()

test_features = test_labeled_points.map(lambda lp: lp.features.toArray()).collect()
test_labels = test_labeled_points.map(lambda lp: lp.label).collect()


24/05/10 07:49:33 WARN TaskSetManager: Stage 51 contains a task of very large size (2205 KiB). The maximum recommended task size is 1000 KiB.
24/05/10 07:49:35 ERROR Executor: Exception in task 3.0 in stage 51.0 (TID 138)]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_3414/95638

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 51.0 failed 1 times, most recent failure: Lost task 3.0 in stage 51.0 (TID 138) (192.168.18.54 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/aitazaz/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line

In [60]:
knn = KNeighborsClassifier(n_neighbors=5)

knn_model =knn.fit(train_features, train_labels)


In [61]:
# Predict
predictions = knn_model.predict(test_features)

# Evaluate
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.0


In [62]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

# Reshape the input array into a 2D array
query_features_2d = np.array([0.3687, -0.4593, -0.6877, -0.5762, -0.5355, -0.4242, -0.2775, -0.1768, -0.1154, -0.1924, -0.3219, -0.4489, -0.4848, -0.4263, -0.2145, -0.026, 0.142, 0.1115, 0.0431, -0.0365, -0.0556, -0.4712, -0.1519, -0.8916, -0.0392, -0.3035, 0.7606, 0.567, -0.2822, 0.9036, 0.6981, -1.9313, -2.5581, -1.5252, -2.041, -2.7436, -1.3173, -0.7162, -1.2103, -2.0367, 0.0946, -0.4021, -0.465, -0.3044, -0.03, -0.5118, -0.0188, 0.4068, -0.4036, 0.6611, 0.6911, -1.4575, -1.8957, -1.602, -0.6792, -1.9977, -0.2236, -0.4297, -1.4055, -2.0964, 0.0076, -0.6619, -0.5873, -0.2786, -0.0107, -0.5344, -0.1176, 0.8594, -0.1422, 0.5414, 0.399, -1.0238, -1.6774, -1.5393, 0.5757, -1.0542, 0.4936, -0.5029, -1.2949, -2.3596, -0.1498, -0.6503, -0.5701, -0.508, -0.1246, -0.7537, 0.7058, 1.1563, 0.2547, -0.1799, 0.0459, -0.4559, -0.9585, -1.4476, 0.037, -0.8084, 0.0429, -0.2147, -1.259, -1.8921, -0.0196, -1.3158, -0.2831, -0.1706, -1.2122, 0.0929, -0.0932, 1.0193, -0.3755, -0.4607, 0.4599, 0.1286, -0.5184, -1.3204, 0.0448, -1.2295, 0.0407, -0.9118, -0.5245, -0.8292, 0.3632, -2.4081, 0.0873, 0.4737, -2.7994, 0.7302, -0.9686, 0.9292, -1.2973, 0.6701, -0.4648, 0.2735, -1.0622, -1.0698, -0.4702, -0.0012, 0.7226, -1.1289, -0.151, -0.8505, 0.7794, -2.8934, -0.0731, 0.684, -3.4813, 1.1418, -1.0225, 1.1804, -1.6186, 1.4671, -1.0776, 0.4895, -0.6777, -0.2691, -0.6725, 1.1864, 1.2358, -1.3886, -0.9698, -1.4763, 0.9571, -3.1312, -0.4062, 0.912, -3.5766, 1.3403, -1.1462, 1.221, -1.3262, 1.7898, -0.9695, 0.9772, -0.5834, -0.2691, -0.6226, 0.9437, 0.9324, -1.4291, -0.8846, -1.6406, 0.9506, -3.1125, -0.7057, 0.6582, -3.5863, 1.8527, -1.0584, 1.7124, -0.5402, 2.295, 0.1913, 1.4564, -0.1174, -0.1234, -0.1487, 0.5431, 1.7121, -1.0165, -0.9328, -3.0532, 0.9745, -3.3147, -0.7998, 0.515, -3.2892, 1.8453, -1.5855, 1.7595, -0.5811, 2.5335, 0.8792, 1.1102, -0.453, -0.047, -0.4541, 0.3702, 2.0888, -0.8558, -0.8073, -3.2638, 1.0068, -3.2709, -0.6572, 0.972, -3.289, 2.0754, -2.0184, 1.0477, -1.0296, 2.1025, 0.6709, 0.7793, -1.2896, -0.4506, -0.9221, -0.1698, 2.345, -0.7364, -0.1292, -2.1351, 1.0391, -2.9416, -0.5284, 1.0884, -3.249, 2.074, -1.5945, 1.0114, -0.869, 1.282, -0.2756, 0.0001, -1.686, -0.8311, -1.1737, -0.7553, 1.1466, -0.774, 0.028, -1.4506, 0.9159, -2.3517, -0.6716, 0.7068, -2.6154, 1.7338, -1.0099, 0.633, -0.5511, 0.8794, -0.4624, -0.0854, -1.351, -0.6588, -0.9831, -0.9399, 0.3329, -0.8446, -0.281, -1.4498, 0.7378, -1.711, -0.8379, -0.3079, -1.9533, 1.2535, -0.7155, 0.5131, -0.5129, 0.5119, -0.9829, -0.1698, -1.5693, -0.8703, -0.3826, -0.7091, 0.816, -0.2404, -0.1478, -0.5131, 0.6584, -1.3685, -0.8107, -0.5147, -1.3347, 1.0516, -1.0076, 0.2584, -0.2522, -0.0269, -2.0463, -0.1882, -1.0009, -0.0861, -0.001, -0.4461, 2.0751, -0.0477, 0.3403, 0.2972, 0.5771, -1.1409, -0.8947, -0.3291, -0.8412, 0.4363, -0.9712, 0.3723, -0.0824, -0.2221, -2.1849, -0.0506, -1.2801, 0.1807, 1.0013, -0.4214, 1.9082, -0.211, 0.4279, 0.4764, 0.4887, -1.1708, -0.9421, -0.2622, -0.5362, -0.0906, -1.326, 0.3664, -0.8802, -0.2472, -1.088, -0.1167, -2.2247, -0.603, 1.3691, -0.0433, 1.7387, -0.5482, 0.4435, 0.3103, 0.4864, -1.1069, -0.9243, -0.1312, -0.8052, -0.8505, -1.5993, 0.4103, -1.077, -0.4738, -0.5255, -0.0903, -3.0168, -0.8853, 0.6721, 0.2325, 2.0972, -0.657, 0.8242, 0.5287, 0.4261, -0.9309, -0.9068, -0.255, -0.7973, -1.338, -1.5638, 0.5159, -1.4171, -0.8209, -0.9254, -0.1471, -2.4611, -0.5545, 0.8649, 0.0493, 1.4477, -1.0372, -0.0674, 0.1581, 0.2004, -0.9252, -1.0752, 0.0886, -0.4356, -1.6494, -0.8973, 0.6786, -0.3827, -0.6998, -0.8398, -0.9349, -2.258, -0.9762, 1.3014, -0.7152, 0.2428, -0.7849, -0.3569, 1.0729, 0.052, -1.0073, -1.0686, 0.3583, -0.0347, -1.4951, -0.5164, 0.9731, 0.2948, -0.6915, -0.5903, -1.4553, -2.2134, -2.111, 0.9996, -1.1586, -0.4948, -1.148, -1.2535, 1.3716, 0.0324, -0.8909, -0.9404, 0.3188, 0.0976, -1.2089, -0.0934, 1.2038, 0.1882, -0.8935, -0.1802, -1.8386, -2.1965, -2.0055, 0.6414, -0.1089, -0.7213, -1.168, -1.2413, 0.729, 0.0376, -0.8805, -0.9288, 0.2007, 0.0783, -1.0855, 0.1291, 1.4211, -0.14, -0.9976, 0.2465, -1.542, -2.1521, -2.3735, 0.4819, 0.6749, -0.4034, -0.625, -0.4903, -0.0532, 0.093, -0.9529, -0.7619, 0.493, -0.0235, -0.9085, 0.0898, 1.6212, -0.1069, -1.0678, 0.3583, -1.4637, -2.2918, -2.7509, 0.3165, 0.8915, 0.2453, -0.5459, -0.7299, -0.8381, 0.112, -1.0038, -0.7581, 0.3655, -0.1312, -0.7611, -0.2286, 1.3548, -0.3505, -1.2213, 0.2678, -0.8743, -1.5257, -2.1552, -0.2294, 0.2828, 0.3931, -0.617, -0.7829, -0.5133, 0.2829, -0.9843, -0.6129, 0.3129, -0.3108, -0.1487, -0.5865, 0.568, -1.792, -1.7647, -0.7888, -0.3008, -1.2438, -1.4535, 0.0121, -1.1174, 0.4579, -0.4477, 0.5554, -0.181, 0.5077, -1.0741, -0.6514, 0.3222, -0.9457, 0.2662, -0.8881, 0.001, -1.626, -1.3591, -1.2111, -0.1697, -1.2987, -1.1373, -0.0798, -1.1858, 1.5337, -0.0554, 0.6577, -0.2278, 0.529, -1.4377, -0.7468, 0.5132, -1.0611, 0.6466, -0.8676, -0.2607, -1.1797, -0.7852, -1.2472, 0.2023, -0.7473, -0.9499, -0.033, -0.6414, 2.1306, 0.0385, 1.0047, -0.8003, 0.5931, -1.552, -0.8626, 0.6591, -1.1082, 0.8943, -1.0997, -0.9489, -0.9986, -0.7636, -1.5091, 0.4199, -0.4743, -1.1798, -0.0054, -0.6109, 1.7478, 0.0614, 0.9346, -0.8294, 0.6683, -1.5729, -1.0328, 0.6053, -1.0838, 0.8082, -1.2874, -1.2959, -0.7957, -0.5288, -1.2806, 0.6108, -0.748, -0.5067, -0.4702, -0.9097, 1.4778, 0.6569, 0.4589, -0.9754, 0.59, -1.6916, -1.0646, 0.1323, -0.9861, 1.0513, -1.5404, -1.2466, -0.8546, -0.624, -1.4933, 0.0763, -1.8381, 0.1561, -0.0134, -0.886, 1.0551, 0.8032, 0.7191, -0.7444, 0.4598, -1.7023, -0.8683, 0.0466, -0.9587, 1.3063, -1.4545, -0.9898, -0.7312, -0.5834, -0.4226, -0.0261, -2.7474, -0.7096, -0.557, -1.1274, 0.543, 1.216, 1.348, -0.9433, 0.1572, -1.3257, -0.528, -0.0694, -0.8378, 0.9035, -0.8902, -0.7348, -0.312, -0.6482, 0.4493, -0.3716, -2.1149, -1.1178, -1.1911, -1.3328, 0.0834, 0.8497, 1.5119, -0.2746, -0.2299, -0.7365, -0.3959, -0.2828, -0.0622, -0.2544, -0.0206, -0.6859, -0.3957, -0.4441, 0.8631, -0.6163, -0.4919, -1.2863, -1.8456, -0.8701, -0.3028, 0.1741, 0.4021, -0.0432, -0.3547, -0.6136, -0.3236, -0.4098, 0.294, -0.5177, 0.2833, -0.2499, 0.158, -0.1416, 1.2316, 0.0569, -0.8299, -1.3404, -1.2285, -0.1805, -0.4034, -0.2986, -0.2587, -0.5468, -0.3927, -0.4366, -0.1873, -0.5066, 0.3108, -0.5783, 0.9075, 0.0089, 0.0258, -0.1351, 0.7038, -0.2022, -0.8455, -1.0124, -0.8408, -0.1551, -0.2004, -0.1756, -0.6076, -0.8564, -0.3496, -0.5517, 0.0383, -0.5405, 0.4054, -0.6611, 0.534, 0.7904, -0.3768, 0.2354, 0.3061, 0.3077, -0.744, -0.5235, -0.3916, -0.8452, 0.0666, -0.463, -0.6565, -1.5788, 0.4337, -0.9099, 0.1785, 0.585, -0.2845, 0.8582, 0.5665, 0.9334, -0.0045, 0.4724, -0.3679, 0.3383, -1.2919, -0.7204, -0.6669, 0.2815, 0.8991, -0.2302, 0.3318, 0.4513, 1.1981, -1.0155, 0.4928, 1.8009, -0.6184, 1.2603, 0.1501, 0.7278, -0.4163, 1.2417, 0.2176, 1.1859, -0.8553, -0.0914, -0.1166, 1.1634, 0.4804, -0.1433, 0.6958, 1.8329, 1.3189, -0.8632, 0.4702, 2.0319, -1.0389, 1.6713, 0.0083, 1.0389, -0.5953, 1.8693, 0.4066, 1.7675, -0.0481, 0.2737, 0.7573, 1.2662, 0.5055, -0.1606, 0.7973, 0.6524, 1.2543, -0.5501, 0.2251, 1.755, -1.467, 1.5717, -0.1806, 1.8804, 0.4315, 2.4289, 0.4554, 0.5236, -0.2149, -0.1924, 1.3758, 1.122, 1.6266, 0.9823, 1.3446, -0.7652, 1.1958, -0.5591, 0.1137, 1.3124, -1.8184, 1.4418, -0.3173, 1.7924, 0.8865, 2.4332, 0.8166, -0.2662, -0.562, 0.0859, 1.6889, 1.5556, 2.7208, 1.5615, 0.5819, -1.3567, 1.1359, -0.5905, -0.081, 1.1174, -1.9768, 1.1818, -0.6981, 2.0208, 1.3472, 2.083, 1.1038, 0.2363, 0.7494, 0.7436, 1.6638, 1.6625, 3.3878, 0.7327, -0.3559, -1.4503, 1.1545, -0.4326, -0.1093, 0.9687, -1.9952, 0.6675, -0.8422, 2.4277, 1.435, 2.1796, 1.1882, 0.4295, 0.7933, 0.3345, 1.7738, 1.7122, 3.6883, 0.7982, -0.2895, -1.6428, 1.1178, -0.4384, -0.115, 1.0592, -1.7205, 0.5812, -0.81, 2.1415, 1.1445, 2.3031, 0.8717, -0.2628, 0.1908, 0.2902, 2.4474, 2.1367, 3.7013, 0.2655, -0.5323, -1.377, 1.0709, -0.4802, -0.1723, 0.8128, -1.7192, 0.4819, -1.0615, 2.0514, 1.2051, 2.2957, 0.313, -0.6836, -0.033, 0.4169, 2.8504, 1.6155, 3.0938, -0.4046, -0.6363, -1.1795, 1.0368, -0.6906, -0.1637, 0.5578, -1.9146, 0.3461, -1.3132, 2.0007, 0.8724, 2.5853, -0.0028, -1.1487, -0.2587, 0.2322, 2.7946, 1.0164, 2.177, -0.8126, -0.2182, -1.4878, 1.0276, -1.0142, -0.0572, 0.5427, -2.2242, 0.0508, -1.4961, 2.3011, 0.5408, 3.0567, 0.0424, -0.9934, -0.6887, -0.7903, 2.2525, 0.5611, 2.1373, -0.7686, 0.7484, -0.4825, 1.0252, -1.0478, 0.1736, 0.5539, -2.1557, 0.1547, -1.5104, 2.5212, 0.3962, 3.3048, -0.0474, -0.1171, -0.0454, -1.2029, 1.2678, 0.3965, 2.3937, -0.1974, 1.225, 0.1345, 0.9363, -1.1107, 0.3619, 0.6437, -1.9605, 0.2997, -1.5956, 2.3545, -0.1498, 3.3173, 0.6266, -0.214, 0.0857, -1.0803, 0.6621, -0.0226, 1.6913, 0.3638, 1.1897, -0.4917, 0.7602, -0.9654, 0.3238, 0.6875, -1.3503, 0.6358, -1.1309, 1.7582, -0.9218, 2.5869, 0.9243, -0.3248, 0.1382, -0.5534, 1.1725, 0.1752, 1.3819, 0.4164, 1.1057, -0.6657, 0.3861, -0.545, 0.0789, 0.4126, -0.2702, 1.0379, 0.379, 1.4375, -1.0642, 0.8805, 0.7524, -0.7823, 0.3424, -0.2198, 1.3907, 0.2364, 0.6631, 0.4272, 0.1377, 0.3145, 0.0931, -0.0715, -0.226, 0.1465, 0.3025, 0.7084, 1.4934, 1.0627, 0.1501, 0.1079, 1.2686, -0.9539, 0.4395, 0.4365, 1.407, 0.3846, 0.3633, 0.7227, -0.4255, 0.6623, 0.0342, 0.0584, -0.3716, -0.0068, 0.8037, 0.3419, 1.7175, 1.0442, 0.1647, 0.282, 1.387, -0.5027, 0.6724, 0.6925, 1.091, 0.1417, -0.0113, 1.2953, 0.4388, 0.997, 0.0344, 0.2354, -0.4133, -0.3553, 1.1061, 0.2982, 1.766, 1.0268, -0.307, 0.3221, 1.6229, -0.3509, 0.6222, 0.8926, 0.7618, -0.2323, 0.0231, 1.0648, 0.2877, 0.7517, 0.0748, 0.2776, -0.572, -0.5934, 1.1956, 0.4174, 1.8387, 0.859, -0.2176, 0.2245, 1.6551, 0.3416, 0.824, 0.7349, 0.5896, -0.1029, 0.1325, 0.4715, -0.1372, 0.25, 0.3174, -0.467, -0.2527, -0.3555, 0.472, 1.6103, 0.8784, 1.7005, -0.1432, -0.0863, 1.1246, 0.7331, -0.2969, 0.6544, 1.003, -0.2949, 1.175, 0.0634, 0.3919, 0.3135, 0.7204, -1.4033, 0.0852, 0.3049, -1.1923, 2.5257, 0.6758, 1.8626, 0.1036, 0.1698, -0.124, 1.1631, -1.9092, 0.9121, 2.0496, 0.007, 1.4091, -0.2139, -0.1627, -0.2411, 0.918, -1.4992, 0.0675, 0.4802, -1.6444, 2.4365, 0.5895, 1.6954, -0.3762, 0.0817, -0.4783, 1.0068, -1.7801, 0.4402, 1.5199, 0.4891, 1.0175, -0.8148, -0.0054, -0.8314, 0.9299, -1.1095, 0.0419, 0.4019, -1.5647, 1.96, 0.1623, 1.4416, -0.0959, 1.0013, 0.1789, 0.4288, -0.8429, -0.136, 0.6473, 1.2283, 0.8659, -0.7563, 0.9492, -1.4416, 0.8857, -0.8842, 0.1051, 0.4686, -1.316, 1.6428, -0.6939, 0.8411, 0.5275, 1.4048, 0.7546, 0.228, -0.3548, -0.0743, 0.0844, 1.0231, 1.3733, -0.1368, 1.3514, -1.0874, 0.839, -0.709, 0.0882, 0.6325, -0.9121, 1.5662, -0.9832, 0.1807, 0.3023, 0.9762, 0.7513, 0.0708, -0.1139, 0.1095, 0.149, 0.5037, 1.1645, 0.631, 1.4643, -0.358, 0.7184, -0.6987, 0.0558, 0.6575, -0.3748, 1.56, -0.7391, 0.3349, -0.2161, 1.0657, 0.7344, 0.0179, 0.7085, 0.593, 0.4515, -0.2864, 0.7635, 0.2557, 0.7379, -0.0391, 0.5509, -0.6382, -0.0412, 0.3255, -0.0794, 1.3573, -0.3778, 1.2443, -0.0257, 1.2573, 0.9413, -0.5083, 0.6097, 0.3735, 0.2657, -1.1992, 0.5321, -0.17, 0.7989, 0.2348, 0.4595, -0.5936, -0.1815, 0.2953, 0.253, 1.5955, 0.0495, 1.8259, -0.2545, 0.4688, 0.8222, -0.8558, 0.5086, 0.6458, 1.7773, -0.6476, 0.7695, -0.0539, 0.9782, -0.1859, 0.4282, -0.55, -0.2026, 0.1204, 0.1198, 2.1208, 0.3974, 1.3161, -0.2901, 0.19, 0.9442, -0.7826, 0.2445, 0.9068, 2.0891, -0.527, 0.6839, 0.1426, 0.7382, -0.2419, 0.4275, -0.7375, -0.2605, 0.0438, -0.1604, 2.1792, 0.568, 1.0226, -0.2503, 0.5757, 1.1191, -0.3965, 0.7791, 0.9187, 1.7524, -0.2164, 1.1716, 0.4248, 0.6978, 0.3028, 0.7828, -1.9751, -0.0704, 0.7749, -0.9487, 2.6691, 0.3301, 1.2736, -1.0175, 1.3876, 0.8572, 1.205, 1.0406, 1.7447, 1.1407, 0.689, 1.1916, -0.1259, 0.216, -0.1071, 1.1169, -2.791, -0.0584, 1.0179, -2.0053, 2.8846, 0.1751, 2.024, -1.1597, 2.1699, 0.682, 1.6263, 0.6831, 1.6911, 0.4232, 0.7502, 0.8094, -0.5483, 0.5496, 0.2093, 1.2215, -3.0476, -0.1105, 1.2594, -2.4878, 2.9943, -0.1836, 2.2423, -0.9316, 2.3157, 0.4836, 1.7963, 1.0356, 1.7074, 0.1856, 0.6985, 1.0858, -0.5628, 0.1961, -0.5304, 1.24, -2.7465, -0.0613, 1.6541, -2.4573, 3.1562, -0.7829, 2.1492, -0.8381, 2.0419, 0.5485, 1.8965, 0.8066, 1.1647, 0.7294, 0.7657, 1.8918, -1.0361, 0.469, -0.4393, 1.3704, -2.4292, 0.0639, 2.105, -2.2832, 2.9903, -0.8747, 2.3786, -0.9775, 1.9981, 0.5227, 1.7551, 0.5165, 1.0316, 0.7965, 0.8904, 2.1656, -1.4983, -0.02, -0.3298, 1.4345, -2.4818, -0.0026, 2.1898, -2.125, 3.1973, -0.8566, 2.1745, -1.0027, 2.1937, 0.5167, 1.5877, 0.4169, 1.1916, 0.5955, 0.7622, 2.6377, -0.4135, 1.0515, -0.9504, 1.4113, -2.4314, -0.1712, 2.1283, -2.2673, 2.8893, -0.7412, 1.8878, -1.2291, 1.6104, 0.5261, 1.2417, -0.3404, 0.9571, 0.3244, 0.2616, 2.3766, -0.1118, 1.2462, -0.7539, 1.3136, -1.805, -0.1476, 1.8103, -2.2017, 2.1627, -0.3534, 1.8083, -0.6333, 0.5152, -0.4268, 0.3403, -1.1225, 0.3278, 0.0921, 0.452, 1.8701, -0.4417, 0.8272, -0.3961, 1.1669, -0.9936, -0.0113, 0.9989, -1.655, 1.7341, -0.0616, 1.5569, 0.0898, 0.8215, -1.3265, 0.2557, -0.3587, 0.411, 0.9068, 1.1375, 1.0554, -0.3523, 0.9726, -0.6031, 1.1196, -0.818, -0.1845, 0.5467, -1.5223, 1.5505, 0.2227, 1.5854, 0.2048, 0.6241, -2.0862, 0.0369, 0.216, 1.5964, 1.8019, 1.2701, 1.3295, -0.2858, 1.0933, 0.1399, 1.0085, -0.6676, -0.2742, 0.2835, -1.4929, 1.1673, -0.0374, 1.1193, 0.457, 0.6972, -2.5796, -0.3225, 0.2957, 2.2344, 1.9457, 0.5317, 1.4563, -0.314, 0.8121, 0.3129, 0.9114, -0.6389, -0.2788, 0.2326, -1.4535, 0.6356, -0.5295, 1.1574, 0.479, 1.0109, -1.3492, -0.4952, -0.466, 1.5711, 2.0233, 0.6899, 1.8986, -0.252, 0.6101, -0.0115, 0.8871, -0.4954, -0.1877, 0.3501, -1.1209, 0.4549, -0.4767, 1.1857, 0.3706, 0.9063, -0.2849, -0.0637, -0.8571, 1.1658, 1.5023, 1.0215, 2.5286, 0.2557, 0.97, -0.0453, 0.794, -0.3752, -0.1211, 0.1656, -1.059, 0.0446, -0.3074, 1.177, -0.0054, 0.927, 0.241, 0.4385, -1.0024, 0.8103, 1.7652, 1.6429, 2.4461, 0.0362, 1.3214, -0.167, 0.6369, -0.1824, -0.2391, 0.1648, -0.3331, 0.2767, 0.1753, 1.4241, -0.1809, 1.0464, 0.3744, -0.6759, -0.2605, 0.2133, 1.0882, 0.8271, 1.0721, -0.2883, 0.9667, -0.1169, 0.5459, -0.0777, -0.4593, 0.2071, 0.0647, 0.7385, 0.8362, 1.7758, 0.0955, 1.0004, 1.2392, -1.3337, 0.1551, -0.3478, 0.8827, 0.1627, 0.3446, -0.2653, 0.5751, -0.3425, 0.5441, -0.2321, -0.6239, 0.1502, -0.3049, 0.8682, 1.1465, 1.9078, 0.6049, 1.1321, 1.8494, -1.64, -0.4699, -0.1764, 0.4844, 0.0593, 0.1868, -0.4195, 0.7512, 0.2693, 0.566, -0.3602, -0.6163, 0.3236, -0.1078, 1.1125, 1.1391, 1.989, 0.2943, 0.7868, 1.5557, -1.7656, -0.5313, 0.2353, 0.627, 0.2594, 0.6627, 0.0946, 1.2597, 0.2794, 0.6124, -0.3621, -0.6076, 0.5665, -0.2989, 0.9543, 1.2528, 2.1683, 0.0758, 0.2325, 1.2862, -1.7963, -0.4933, -0.2621, 0.3356, 0.5928, 1.3377, 0.454, 1.3858, 0.05, 0.6516, -0.3926, -0.7027, 0.5912, -0.5815, 0.8717, 1.342, 1.9473, 0.161, 0.2718, 1.186, -1.6457, 0.0726, -0.5909, -0.4587, 0.7923, 1.7474, 0.3889, 1.5802, 0.0693, 0.7413, -0.4249, -0.5599, 0.8703, -0.6692, 1.0788, 1.3269, 1.6623, -0.2773, 0.2031, 0.0492, -1.5339, 0.2656, -0.6825, -0.5747, 0.0727, 2.0815, -0.1694, 1.3183, 0.1556, 0.9259, -0.5279, -0.4242, 1.081, -0.9036, 1.5089, 0.5782, 0.7443, -0.7759, -0.116, -1.4777, -0.7288, 0.0895, 0.061, 0.2107, -0.0234, 2.8985, -0.2839, 0.8452, -0.0362, 1.0188, -0.7889, -0.5518, 1.4656, -0.9425, 1.733, 0.0028, -0.0281, -0.6147, 0.4687, -1.2421, 0.3796, 1.014, 0.5179, 0.4278, 0.0355, 2.8434, -0.4965, 0.7904, -0.6138, 1.0841, -0.9049, -0.7519, 1.3437, -1.1382, 2.0242, -0.1576, -0.0108, -0.1782, 1.1228, -0.5368, 0.2164, 0.638, 0.0549, 0.5184, -0.0501, 3.076, 0.1078, 1.1005, -0.2756, 1.0945, -0.9892, -0.8458, 1.1266, -0.7926, 2.3496, -0.3633, -0.1893, 0.4628, 1.5414, -0.0005, 0.2209, -0.2539, 0.8661, 1.0774, 0.3633, 2.2256, 0.4879, 1.1496, 0.3481, 1.0527, -1.0324, -0.778, 1.0023, -0.7303, 2.422, -0.4655, -0.1523, 0.5452, 1.1009, 0.1888, 0.3828, -1.1077, 1.5603, 0.812, 0.6068, 1.7069, 0.2298, 1.5494, 0.665, 0.9336, -1.0262, -0.5919, 1.0408, -0.6783, 2.3882, 0.0555, 0.9049, 0.8348, 0.869, 0.2272, 0.6168, -1.7728, 1.1911, 1.0174, 0.6708, 1.9189, -0.0559, 1.4913, 0.5669, 0.713, -0.6804, -0.4457, 1.1061, -0.553, 1.468, 0.2886, 1.0782, 0.9407, 1.2071, 1.544, 0.5953, -1.0599, 0.6838, 0.8638, 1.0221, 1.7442, -0.1988, 1.6838, 0.9124, 0.3867, -0.0755, -0.5811, 0.6343, 0.1275, -0.1424, 0.7757, 1.1419, 0.9866, 1.1663, 1.9693, -0.9387, -1.0524, -0.2849, 0.5274, 1.2912, 0.8211, -0.199, 1.8315, 1.9112, 0.3099, 0.2161, -0.6344, 0.4752, 0.5974, -0.1253, 1.6082, 1.302, 1.2609, 1.0831, 1.9267, -1.2319, -0.6693, -0.1742, 0.0598, 1.4308, 0.9139, 0.3096, 1.2478, 1.4646, 0.3019, 0.0575, -0.6604, 0.3509, 0.6039, 0.2923, 1.5461, 1.1647, 0.743, 0.8553, 1.5028, -1.0384, 0.2058, 0.2602, 0.7791, 1.6296, 1.1526, 0.6645, 1.0497, 1.3305, 0.4082, -0.8149, -0.3843, 0.0724, 0.0776, 1.5285, 0.2824, 2.0891, -0.5037, 0.8366, 0.7441, -0.4518, 0.6026, 1.348, 1.5027, 2.2564, 1.4967, -0.7744, 1.1499, -0.6181, 0.6245, -1.0337, -0.226, -0.2935, -0.7091, 1.8139, -0.443, 2.6187, -0.3431, 1.5065, 0.8648, -0.1537, -0.7024, 1.1086, 1.2595, 1.9974, 2.7068, -0.5562, 1.2318, -0.779, 1.0086, -0.4053, -0.2057, 0.0565, -1.315, 0.985, -0.6399, 2.0592, -0.1148, 1.7533, 0.9811, 0.397, -0.9852, 0.0167, 0.5835, 0.8456, 1.4814, -1.5383, -0.5621, -1.2762, 1.1019, -0.3451, -0.242, 0.5071, -1.5232, 1.047, -0.5389, 1.7038, 0.0606, 1.5627, 0.513, 0.3233, -0.5221, -0.2258, -0.0573, 0.0916, 0.8813, -1.4996, -0.6382, -1.5138, 0.9522, -0.7923, -0.1548, 0.8616, -1.3872, 1.5692, -0.4021, 1.2438, -0.1795, 1.3196, -0.0236, 0.7061, -0.388, -0.2612, -0.5176, -0.0731, 0.9313, -1.7908, -0.6317, -1.5498, 0.8652, -1.1373, -0.2999, 0.92, -1.4545, 1.42, -0.3486, 1.2888, -0.5398, 0.635, -1.2796, 0.5213, -0.8132, -0.6735, -0.1301, 0.1142, 1.1026, -1.4129, -0.9189, -2.5099, 0.8487, -1.4136, -0.3351, 1.0148, -1.4503, 0.9551, -1.0401, 1.0618, -1.0025, 0.5773, -1.3386, 0.3195, -1.5163, -1.0512, -0.1745, 0.3191, 0.6663, -1.6095, -1.0947, -2.8747, 0.8687, -1.2357, -0.1405, 0.9538, -1.4752, 0.4909, -1.6566, 0.8341, -1.0401, 0.5866, -1.7472, 0.0738, -1.1501, -0.5471, 0.3609, 0.866, 1.0868, -1.3167, -0.7595, -2.6456, 0.8681, -0.8785, -0.1378, 0.4326, -2.2675, -0.6014, -1.4036, 0.8035, -0.2433, 1.2703, -1.8337, 0.2638, -1.3329, -1.2632, 0.1141, 1.0963, 1.8351, -0.2031, -0.2941, -2.1689, 0.8536, -0.7944, -0.4375, -0.484, -2.9656, -0.9228, -0.9231, 0.9515, 0.3625, 1.5924, -1.7286, 0.0412, -1.7567, -0.2907, 1.0799, 1.6403, 1.9903, -0.2148, -0.6888, -2.1676, 0.9291, -0.7554, -0.627, -1.0232, -3.0563, -0.3167, -0.9446, 1.4724, 0.3905, 1.2287, -0.7362, 0.3025, -1.6476, 0.2811, 1.6153, 1.2583, 1.0494, -2.0133, -0.3904, -1.7841, 0.9572, -0.8704, -0.7531, -1.4296, -3.2751, 0.0679, -1.4369, 1.9174, 0.2492, 0.7848, -0.5231, 0.0741, -1.1621, 1.0495, 1.8263, 0.5485, -0.1793, -2.647, -0.0365, -1.6391, 0.9499, -1.1288, -0.8709, -1.1592, -3.2446, -0.002, -1.335, 2.4761, 0.7638, 1.32, 0.003, -0.6166, -1.2612, 1.4561, 1.2878, -0.159, 0.0674, -1.7498, 0.9053, -1.5051, 1.0294, -1.3433, -1.1835, -0.8753, -3.4027, -0.2257, -1.1573, 2.7171, 0.8108, 1.2938, 1.2159, -0.3134, -0.6106, 1.8079, 0.0733, -0.8275, -0.4389, -1.5486, 1.8157, -1.8419, 1.0998, -1.5313, -1.4255, -0.5782, -3.7928, -0.6963, -1.5396, 2.1933, 0.8289, 1.1292, 1.6997, -0.1602, -0.3337, 2.0026, -0.5048, -0.9671, -0.2621, -1.5017, 1.7426, -1.0212, 1.1494, -1.5678, -1.4983, -0.144, -3.8286, -0.4773, -1.4193, 2.0579, 0.9243, 0.8698, 1.7328, -0.4218, -0.6549, 1.4584, -0.6366, -1.3604, -0.0554, -1.4161, 1.4507, -0.5539, 1.1537, -1.6509, -1.5533, -0.0249, -3.8607, -0.3311, -0.9902, 1.7675, 0.5039, 0.2434, 1.263, -1.1117, -1.172, 1.5389, -0.7873, -1.4873, 0.601, -1.5037, 1.2021, -0.1833, 1.1496, -1.6671, -1.4643, 0.1105, -4.0153, -0.3843, -1.2137, 1.581, 0.701, -0.1002, 0.675, -1.5944, -1.5843, 1.8834, -0.6102, -1.961, 0.4113, -1.7365, 0.8928, -0.3676, 1.078, -1.5552, -1.37, 0.4415, -3.8802, -0.2728, -1.2398, 1.345, 0.9692, -0.2734, 1.3945, -1.6713, -1.2207, 1.7115, -0.4082, -2.191, 0.3661, -1.6127, 0.6535, -0.709, 1.007, -1.5177, -1.3052, 0.6458, -3.9685, -0.3424, -0.9979, 1.4266, 0.7889, -0.3822, 1.9439, -1.3064, -0.9182, 1.4906, -0.9252, -2.1683, 1.028, -0.7495, 1.2096, -0.4421, 1.0244, -1.6156, -1.1664, 0.5954, -3.7872, -0.2865, -0.9373, 1.7916, 0.4916, 0.0913, 1.2397, -0.5858, -1.4275, 1.8856, -1.2504, -1.9856, 1.6734, -0.4246, 1.6068, -0.9052, 1.0256, -1.7495, -1.2065, 0.6595, -3.8224, -0.5289, -1.1692, 1.5639, -0.4421, -0.1027, 0.6638, -0.5848, -0.9379, 2.1965, -0.9199, -2.4276, 1.6514, -0.1238, 1.2383, -1.114, 1.0491, -1.7161, -1.2404, 0.6389, -3.9461, -0.9418, -1.5706, 1.4114, -0.6164, -0.1669, -0.1491, -1.0129, -0.6072, 2.311, 0.2139, -2.6592, 0.9681, -0.7585, 0.737, -1.2207, 1.0919, -1.3587, -0.9276, 0.3945, -3.8133, -1.1629, -2.1152, 1.0817, -0.7351, -0.0802, -0.1858, -0.8023, -0.6887, 2.4559, 0.996, -1.8734, 1.6107, -1.1968, -0.1378, -1.2812, 0.9378, -0.9844, -0.6245, 0.196, -3.3311, -1.1348, -2.1026, 0.7094, -0.6058, -0.1949, 0.1468, -1.0727, -0.6863, 2.6231, 0.892, -1.3076, 1.441, -1.6235, -1.1444, -1.7785, 0.478, -0.6313, -0.5552, -0.3356, -2.3756, -1.4277, -1.2409, 0.3962, 0.0663, -0.3062, 0.644, -0.4801, -0.3497, 1.6483, -0.1746, -1.2961, 0.7208, -1.9528, -1.1281, -1.313, 0.0164, -0.2299, -0.5522, -0.8183, -1.3597, -1.5575, 0.3054, 0.0819, 0.2742, -0.4037, 1.4251, 0.2598, 0.0572, 1.5813, 0.24, -0.2459, 0.184, -1.7743, -0.5226, -0.655, -0.1327, -0.1764, -0.6611, -0.8967, -0.8349, -1.8816, 0.7037, -0.0423, -0.1108, -0.6435, 1.174, 0.6622, -0.2261, 1.4022, 0.4501, 0.8606, 0.9868, -1.0612, -0.3406, -0.5667, -0.142, -0.3902, -0.4156, -0.7552, -0.7475, -1.6921, 0.0779, 0.2866, -0.6132, 0.139, 1.5286, 1.0276, -0.9628, -0.2139, 0.8272, 0.4333, 0.6173, -0.3868, -0.5355, -0.2098, 0.1914, -0.9787, 0.3323, -0.4388, -1.145, -0.07, -0.8416, 1.7963, -0.9069, 0.5568, 0.0984, 1.2245, -1.1446, -0.1134, 0.3606, -0.1381, -0.1016, -1.6699, -0.5585, -1.0814, 0.4046, -1.3144, 0.6172, -0.0351, -1.1555, 0.7345, -1.1937, 2.6598, -1.0855, 0.2283, -0.7545, 0.8462, -1.0034, 0.1573, -0.2624, 0.2123, 0.2655, -1.3317, 0.7657, -0.8447, 0.521, -1.5406, 0.5667, 0.251, -0.8178, 1.1933, -1.5732, 2.8022, -1.1441, 0.5963, -1.4129, 0.0116, -1.1693, 0.1371, -0.1437, -0.2644, -0.006, -0.8157, 0.7568, -0.8621, 0.6432, -1.6019, 0.1437, 0.0868, -0.6406, 1.4207, -1.3319, 2.1728, -1.4234, 1.0505, -1.4819, 0.0313, -1.131, 0.4272, 0.1465, -0.0166, 0.3285, -0.6844, -0.5684, -2.5495, 0.7153, -1.6739, -0.0921, -0.0777, -0.7419, 1.3762, -1.5011, 1.6878, -1.2268, 1.2994, -0.903, 0.7003, -0.9494, 0.4825, -0.0464, 0.1886, 0.8268, -0.4051, 0.1664, -1.6393, 0.7663, -1.7693, -0.3547, 0.0561, -0.5133, 1.9398, -1.2626, 1.3881, -0.5908, 1.7814, -0.7486, 0.1594, -1.5461, 0.0043, -0.0173, -0.1735, 0.8441, 0.1201, -0.3234, -1.7691, 0.8784, -1.7689, -0.6332, 0.462, -0.5597, 2.0103, -1.4796, 0.5421, -0.6909, 1.5299, -0.8057, -0.8359, -2.3705, -1.2046, -0.3972, -0.5748, 0.9159, 0.5752, -1.0318, -1.9677, 0.8983, -1.7385, -0.9096, 0.8523, -0.4657, 2.1201, -0.7965, -0.0209, -0.7335, 1.4284, -0.6259, -0.8641, -2.0812, -2.5055, -1.2845, -0.1861, 1.7847, 0.0649, -1.1564, -1.256, 0.884, -1.5193, -1.0025, 0.7573, -0.8046, 1.7275, -0.4904, -0.7556, -0.4651, 1.0564, -0.8977, -0.622, -1.1049, -2.7783, -1.8776, -0.5798, 1.1085, -0.5485, -1.5946, -0.8102, 0.9302, -1.3418, -0.9411, 0.6899, -1.3971, 1.3693, -0.3077, -0.9093, -0.4951, 0.5518, -0.9752, -0.5571, -0.1169, -1.9091, -1.6105, -0.7198, 0.349, -0.6788, -1.0451, -1.2582, 0.8982, -1.1415, -0.9425, 0.2532, -1.9814, 0.9834, 0.2555, -0.565, -0.6163, 0.0529, -1.4593, -0.8886, -0.1761, -1.4983, -1.6114, -1.1067, -0.0708, -0.6616, 0.3652, -0.6917, 0.8172, -0.9654, -0.9497, 0.0108, -2.0416, 0.7046, 0.3612, -0.5174, -0.8002, -0.723, -2.0731, -0.9734, -0.1033, -1.2492, -1.3738, -1.8637, -0.4405, -0.4947, 0.2572, -0.1617, 0.7021, -0.7039, -0.9183, 0.0122, -1.8668, 0.18, 0.4877, -0.3016, -0.8289, -0.8971, -1.4706, -0.8147, -0.7263, -1.445, -0.4014, -1.8984, -0.5211, -0.4147, 0.3842, 0.0709, 0.6215, -0.6404, -1.0085, 0.0985, -1.5681, -0.2446, 0.1951, -0.0121, -0.5375, -0.7333, -0.8247, -0.957, -1.4083, -1.6677, 1.1644, -1.0533, -1.0487, -1.4759, -1.0013, -0.8946, 0.6208, -0.7157, -1.1083, 0.3963, -1.363, -0.4249, -0.3323, 0.3376, -0.2861, -0.7593, -0.3283, -1.248, -1.21, -1.5282, 1.3861, -0.4627, -1.0371, -1.772, -1.1882, -0.4043, 0.5962, -0.9033, -0.9825, 0.3984, -1.4771, -0.5549, -0.8856, 0.3515, -0.6796, -1.1264, -0.4803, -0.9074, -0.8089, -0.8903, 0.973, 0.0947, -0.6522, -1.5072, -0.7432, -0.2997, 0.5182, -0.99, -0.7993, 0.6926, -1.4821, -0.7964, -1.1488, 0.3084, -0.5985, -1.331, -0.6379, -1.297, -1.6387, -1.4101, 0.2704, -0.3254, -1.0272, -1.8715, -0.7028, -0.0691, 0.4377, -0.8993, -0.6115, 1.054, -1.223, -1.1089, -1.4502, 0.3321, 0.0589, -1.6458, -0.7852, -1.3151, -1.6438, -1.5136, 0.2083, -0.1467, -0.9643, -1.9393, -1.1924, 0.082, 0.3638, -0.8708, -0.4762, 1.1074, -0.9985, -1.1651, -1.0137, 0.7796, 0.0419, -1.505, -0.508, -0.4811, -1.2247, -1.4744, -0.0679, -0.5329, -0.5582, -2.2587, -1.3313, -0.207, 0.3323, -0.7279, -0.3634, 0.9818, -0.9128, -1.3127, -0.6292, 0.9024, -0.2845, -0.964, 0.2242, 0.4162, -1.2575, -2.1096, 0.0283, -0.8175, -0.4614, -1.2532, -0.2664, -0.2633, 0.252, -0.5633, -0.4249, 0.6425, -0.8321, -1.5518, -0.6361, 0.8338, -0.1012, -0.8245, 1.0289, 1.0554, -1.1213, -2.0884, -0.0303, -0.7206, -0.704, -0.6549, -0.1895, -0.6008, 0.24, -0.4905, -0.4745, 0.4411, -0.804, -1.5397, -0.7546, 0.3593, -0.0825, -0.8292, 0.7274, 0.7254, -1.0197, -1.5681, -0.3942, -0.5479, -0.9626, -1.0677, -1.0266, -0.8532, 0.1481, -0.5636, -0.5184, 0.467, -0.4594, -1.3869, -0.8854, -0.1551, 0.1944, -0.1997, 1.0415, 0.8544, -0.6771, -1.1413, -0.6719, -0.7861, -1.9425, -1.4275, -0.8922, -1.0299, 0.0583, -0.6318, -0.4965, 0.8191, -0.1657, -1.1748, -0.5179, -0.2612, 0.478, 0.3322, 1.4132, 0.3333, -0.7879, -1.2791, -0.3743, -0.4269, -1.7837, -0.627, -0.1879, -0.8025, -0.0402, -0.5685, -0.4311, 0.8948, 0.0848, -1.1324, -0.0119, -0.2546, 0.2004, 0.2487, 1.3878, 0.2193, -0.9841, -1.7119, -0.7209, -0.4149, -1.2501, -0.2228, -0.1508, -0.5338, -0.0981, -0.5542, -0.3744, 0.7875, 0.0509, -1.1764, -0.0361, -0.1463, 0.2361, -0.3892, 1.1565, 0.6074, -0.901, -1.7718, -0.7515, -0.2364, -0.6447, -0.0376, -0.3933, -0.786, -0.0937, -0.5981, -0.3574, 0.4124, -0.2002, -0.9025, -0.0499, -0.1197, 0.1527, -0.5013, 1.3078, 0.676, -0.7352, -1.6446, -0.6945, -0.0287, -0.0655, 0.3957, -0.7282, -1.4444, -0.0369, -0.9064, -0.2124, 0.1033, -0.6864, -0.414, -0.5288, 0.3658, -0.7824, -0.3104, 0.9683, -0.4981, -0.5122, -1.8856, -0.3151, -0.282, 0.3385, 0.0701, -0.8612, -1.0825, 0.0406, -1.2674, -0.0079, 0.0499, -0.9346, 0.1433, -1.4372, 0.8355, -1.1363, -0.638, 0.3613, -0.3957, -0.1187, -2.3197, 0.0079, -0.3505, 0.758, 0.2329, -0.4601, -0.9519, 0.1939, -1.5786, 0.2373, 0.1494, -1.302, 1.0087, -1.7239, 0.9516, -0.8964, -0.2204, -0.388, -0.1136, -0.1283, -1.9415, -0.4791, -1.2839, 0.3711, -0.0819, 0.0216, -0.1974, 0.2827, -1.7884, 0.3248, 0.5472, -1.5589, 1.6268, -1.4222, 0.6314, -0.7214, 0.2945, -0.9331, -0.5108, 0.2714, -1.4586, -0.3343, -1.2593, 0.4191, -0.7681, 0.5456, 0.1629, 0.4358, -1.8801, -0.0815, 0.9101, -1.3042, 1.8785, -1.2209, -0.2368, -1.2951, 0.6731, -0.2481, -0.0137, 0.5088, -2.3657, -0.3688, -0.9194, 0.4583, -1.2061, 0.2644, -0.1161, 0.5047, -1.7059, -0.4396, 0.6983, -1.5096, 1.6277, -0.5186, -0.6421, -1.7401, 0.5987, -0.2731, -0.2444, 0.6409, -2.7141, -1.2129, -0.8761, 0.7195, -0.9582, -0.2086, -0.0441, 0.4309, -1.4576, -0.571, 0.5951, -1.35, 1.059, -0.1812, -1.0037, -1.5586, 0.4668, -1.2601, -1.0372, 0.4653, -2.5622, -1.0728, -0.5019, -0.2269, -1.1603, -1.3813, -0.6712, 0.3713, -1.2016, -0.6477, 0.4221, -1.0853, 0.4362, -0.0888, -0.9027, -1.2163, 0.5495, -1.8186, -1.3751, 0.5233, -2.2214, -0.5071, -0.1176, -1.2911, -1.1918, -1.0774, -0.6533, 0.4955, -0.5279, -0.2858, 0.6418, -0.249, 1.1465, 0.6185, 0.3524, 0.3629, 1.2604, -0.4631, -0.5756, 0.8205, -1.3431, 0.4319, 1.1408, -0.1462, -0.8864, -0.7293, -0.783, 0.5596, -0.145, 0.0255, 0.925, -0.1158, 1.3666, 0.6494, 0.5927, 0.9349, 0.9129, 0.1157, -0.3353, 1.2072, -0.8479, 0.7573, 1.0758, 0.3884, -0.3533, -1.0227, -0.9812, 0.6391, -0.8252, -0.083, 1.5046, -0.5022, 1.9328, 0.6798, 1.0682, 0.7129, 0.9163, 0.8418, 0.0053, 1.0717, -0.6134, 0.8804, 0.0219, -0.1502, 0.3817, -0.1656, -0.5294, 0.642, -1.2765, -0.4489, 1.2558, -0.887, 2.0963, 0.6628, 1.2333, 0.2264, 0.4692, 0.8743, -0.3197, 0.038, -0.7467, 0.5117, -0.3735, 0.8579, 1.039, 0.5151, 0.2708, 0.507, -1.4006, -0.6286, 0.7514, -0.9275, 2.0985, 0.3913, 0.5786, 0.1637, 0.6462, 1.124, -0.2495, 0.3497, -0.0924, 0.6043, 0.1181, 1.6224, 0.5115, 0.2592, 0.595, 0.482, -1.3684, -0.4673, 0.4803, -0.7723, 1.8154, -0.1162, 0.9594, 0.027, 1.0364, 1.3612, 0.2435, 0.3546, -0.1557, 0.8926, -0.0366, 0.6891, -0.2894, -0.126, 0.0673, 0.4723, -1.2254, -0.3731, 0.2975, -1.0271, 1.4558, 0.1648, 1.4099, 0.36, 0.9315, 0.8837, 0.192, -0.2271, -0.0921, 1.329, 0.064, 0.9391, -0.6965, 0.0151, 0.1328, 0.5113, -1.1927, -0.5049, 0.3014, -1.5644, 0.842, 0.0823, 1.2173, 0.3703, 0.7298, 0.1829, -0.2138, -0.3247, 0.707, 2.4412, 1.2804, 1.5252, -1.2064, 0.1172, 0.1351, 0.6936, -1.2442, -0.5773, 0.2195, -2.345, 0.6856, -0.5094, 0.8089, -0.1338, 0.0399, -0.5549, -0.5384, 1.3951, 2.6013, 3.0209, 0.817, 0.491, -2.2424, -0.0118, -1.3698, 0.842, -1.2809, -0.7588, -0.1247, -3.2217, 0.1994, -0.9811, 0.5735, -0.1248, -0.0891, -0.5991, -0.5361, 2.0129, 3.0148, 2.4306, -0.4408, -0.2113, -2.3352, 0.1717, -2.0229, 0.8071, -1.3862, -0.8728, 0.1748, -3.0119, 0.0164, -0.484, 0.7309, 0.1977, -0.2823, -0.3191, -0.1691, 1.29, 3.3594, 0.8109, -1.8963, -0.2551, -1.5091, -0.0567, -1.2085, 0.7054, -1.7037, -1.2665, 0.554, -2.3852, 0.398, -0.5157, -0.0119, -0.651, -1.1187, 0.6522, -0.1634, 1.3404, 2.7537, -0.1106, -2.2938, -0.9698, -1.146, -1.0348, -0.4634, 0.7443, -1.6677, -1.2534, 0.4875, -1.8818, 0.6438, -0.8664, -0.1796, -1.5933, -0.9617, 1.153, 0.1747, 1.5001, 1.7803, -0.0891, -1.6656, -0.2855, -1.0146, -0.3318, -0.3019, 0.7935, -1.5958, -1.0281, 0.5073, -1.3291, 0.9048, -1.2863, -0.4724, -2.1882, -0.3182, 1.3919, 0.5391, 0.5259, 1.1248, 0.405, -1.0279, -0.144, -0.8586, 0.6838, -0.4199, 0.7989, -1.7796, -0.8769, 0.7876, -1.0528, 1.4072, -1.5142, -0.8437, -2.4786, -0.1595, 0.8939, 0.4331, -0.6812, 0.2065, 1.7221, -0.9569, -1.0166, -0.4114, 1.3571, -0.3838, 0.5445, -1.8011, -0.5465, 0.5516, -0.7385, 1.8338, -0.8813, -0.7368, -2.0261, 0.1023, -0.047, 0.1573, -1.5872, -0.2747, 2.3519, -1.315, -2.3781, -0.283, 1.5822, -0.1509, 0.1164, -1.3578, -0.0722, 0.258, -0.4261, 0.9941, -0.5969, -0.8097, -1.3071, 0.4756, 0.2115, 0.4081, -1.3698, 0.107, 1.9156, -0.6706, -2.0445, -0.9774, 0.7552, -0.7594, -0.2603, -0.6344, -0.1861, -0.0593, 0.3339, -0.2842, 0.4533, -0.6865, -0.6758, -0.4781, 0.7677, 0.3443, -0.8597, 0.4172, 0.9634, -0.2831, -0.8871, -1.2932, -0.2274, -0.7121, -0.452, -0.4342, -0.2711, -0.3469, 0.8074, -1.2737, 0.6598, -0.2699, -0.1169, -0.1818, 0.4275, -0.1684, -1.1776, 0.0675, 0.2852, -0.5194, -0.2633, -0.5511, -0.2127, -0.7483, -0.5504, -0.387, -0.1709, -0.5297, 0.9472, -1.3618, 0.9323, 0.2017, -0.3973, -0.168, 0.1739, -0.6851, -1.0928, -0.1534, 0.0501, -0.4198, 0.0009, 0.1973, -0.1912, -1.2022, -0.245, -1.2782, 0.2933, -0.3085, -0.3487, 0.0543, -0.3635, 0.8278, -0.9186, -0.0295, -0.2491, -0.241, -0.9738, -0.2584, 0.3014, -0.856, 0.3607, -0.1191, -0.6067, -1.8064, 0.1395, -2.0819, 0.482, 0.269, -1.4654, 1.1979, -0.4079, 0.9474, -1.6339, 0.4995, -1.2482, -0.3021, -1.3023, -0.793, -0.6813, -1.4175, 0.026, -0.7824, -0.0792, -1.7989, 0.3741, -2.0744, 0.2077, 0.4296, -1.2255, 1.465, -0.6259, 0.3427, -2.1584, 0.7973, -1.5537, -0.056, -1.0676, -1.3599, -1.9092, -1.7154, 0.1794, -0.6281, 0.984, -1.375, 0.3911, -2.0725, -0.1231, 0.341, -1.0228, 1.424, -0.8175, -0.5168, -1.8793, 1.0316, -1.7217, 0.4155, -0.6761, -1.1477, -2.1298, -1.4311, 0.3695, -0.8817, 1.4692, -1.3296, 0.3781, -1.9972, -0.2263, 0.2052, -0.8218, 1.5764, -0.7035, -0.5007, -1.5323, 0.8502, -1.5626, 0.19, -0.6482, -1.1374, -1.9, -0.4535, 0.836, -0.4068, 1.4851, -1.3903, 0.3709, -1.7406, -0.243, 0.4313, -0.7748, 1.4108, -0.2421, -0.531, -1.4312, -0.4397, -2.0327, 0.0026, -0.6504, -0.5965, -0.8059, -0.5944, 0.4319, -0.3782, 0.6999, -1.4931, 0.3325, -1.6968, -0.0994, 0.7216, -1.2194, 0.7786, 0.0965, 0.0913, -0.8311, -0.6293, -1.5664, 0.4613, -1.3186, -0.3641, -0.3349, -1.3828, 0.1414, -0.1416, 1.1756, -1.1572, 0.015, -1.5108, 0.1688, 0.636, -1.1418, 0.4073, 0.5203, 0.3717, -0.275, -0.5765, -0.7981, 0.5707, -1.2365, -0.7203, -0.1554, -0.895, -0.0313, -0.8024, 0.2932, -0.5009, -0.4417, -0.6737, 0.174, -0.3739, -0.2922, 0.0112, 0.6651, -0.1668, 0.4913, -1.3141, -0.2877, 0.4517, -0.2605, -0.3194, -0.3779, -0.6659, 0.5257, -0.3602, -1.1897, 0.2136, -0.086, -1.2112, -0.2329, 0.0302, -0.9919, 0.4172, 0.8579, -0.0124, -0.0994, -0.4323, -0.2614, 1.1569, 0.3261, -0.7939, 0.7997, -1.3575, -0.2619, -0.4577, -1.3314, -0.3423, 0.4857, -2.3859, -0.7635, 0.9146, -1.8181, 1.1718, 0.4784, 0.8823, -0.5413, 0.3316, -0.618, 1.3976, 0.1175, -1.025, 0.715, -1.6284, 0.1554, -1.4528, -0.6529, -0.4567, 0.7121, -3.0568, -1.1168, 1.4431, -2.1262, 1.4192, -0.043, 1.4146, -0.7112, 0.7505, -0.3496, 1.7928, 0.1996, -0.6754, 0.8883, -1.1954, 0.8368, -2.0986, -0.3833, -1.1258, 0.9331, -3.0745, -1.1155, 2.2997, -1.8512, 1.8205, 0.015, 1.338, -1.1317, 0.819, -0.0947, 1.6198, 0.0079, -0.3815, 1.1543, -1.4192, 1.4498, -1.8175, 0.3957, -0.4489, 1.185, -2.3934, -0.4003, 2.6616, -1.7596, 2.1151, -0.2998, 1.4356, -1.0813, 0.937, 0.4536, 2.2519, 0.8128, 0.6062, 1.9907, -0.5878, 2.3954, -0.6435, 0.4399, -0.588, 1.183, -2.225, -0.0705, 2.4371, -2.0076, 2.171, -0.2807, 1.9549, -0.819, 1.1205, 0.2337, 2.0892, 1.6522, 0.5717, 1.4163, -0.2468, 2.8901, -0.2699, 0.5587, -1.2576, 1.1486, -2.3012, -0.1014, 1.8348, -2.3859, 1.8775, 0.0299, 2.5881, -0.7725, 1.6325, -0.5341, 1.5321, 1.7909, 0.6001, 0.8692, -0.5672, 3.0938, 0.3303, 0.8057, -1.9667, 1.0373, -2.4058, -0.2203, 1.2936, -2.4409, 1.5628, -0.2269, 2.5589, -1.0788, 1.6116, -0.8714, 0.657, 0.7104, 0.1907, 0.8373, -0.0998, 2.4659, -0.4576, 0.8494, -1.4759, 0.9554, -2.028, -0.196, 0.8893, -2.0835, 1.6993, -0.0699, 2.5004, -0.6432, 1.6871, -0.889, 0.3444, 0.6272, 0.0523, 1.076, -0.0117, 1.7077, -0.7338, 0.2177, -1.6865, 0.7057, -1.2279, -0.0711, 0.0994, -1.7138, 1.3179, -0.2246, 1.847, 0.0465, 0.8834, -0.9079, 0.7905, 1.1438, 0.0866, 1.1643, -0.2974, 1.1326, -0.5243, -0.2383, -1.0808, 0.5572, -0.9316, -0.0984, -0.6506, -2.027, 0.6802, -0.429, 1.5573, 0.8914, 0.0088, -1.6161, 0.6278, 0.7913, 0.6001, 1.7278, -0.0191, 0.6837, -0.8247, -1.1073, -0.1896, 0.6637, -1.0888, -0.2568, -0.7839, -2.358, 0.7732, -0.8207, 1.2156, 0.2548, -0.1832, -1.2692, 0.7464, 0.9509, 2.0497, 1.9202, -0.4425, 0.6725, -1.4756, -0.0466, -0.2445, 0.7952, -1.0209, -0.4141, -1.0122, -2.6657, 0.4534, -1.2735, 1.3232, -0.1395, 0.085, -0.8328, 0.3929, 1.553, 2.6415, 1.3905, -0.7983, 0.8691, -1.7987, 1.3365, -0.8494, 0.7606, -0.9308, -0.4892, -0.8964, -2.4671, -0.0122, -0.9868, 1.7783, -0.4169, 0.6316, 0.0588, -0.3119, 1.197, 2.3032, 0.87, -1.0596, 0.1975, -1.333, 1.6591, -1.7651, 0.7023, -1.1868, -0.7254, -0.3443, -2.5564, -0.5978, -0.3102, 2.0024, 0.0069, 1.0055, 0.8715, -0.4864, 0.2283, 1.5934, 0.126, -1.3148, 0.1153, -0.4123, 1.7103, -1.4413, 0.7369, -1.3342, -1.1298, -0.1479, -3.0067, -0.8245, 0.2568, 1.6844, 0.7116, -0.0584, 1.2734, -0.5257, -0.6419, 2.0566, -1.1273, -1.9478, -0.0332, -0.0873, 1.508, -0.9697, 0.6886, -1.3995, -1.2162, 0.1538, -3.1325, -0.395, 0.3151, 1.7551, 1.7501, -0.9277, 1.741, -0.9164, -0.5858, 2.5647, -1.0492, -1.9338, 0.394, 0.158, 1.3892, -0.3976, 0.629, -1.4929, -1.2175, 0.524, -3.1322, 0.1008, -0.0069, 1.6349, 2.0757, -1.6022, 1.8678, -1.1562, -0.3795, 2.647, 0.2446, -1.0205, 0.8774, 0.4509, 1.6184, -0.2065, 0.6167, -1.4888, -1.2978, 0.6273, -3.0994, 0.0132, 0.0704, 1.546, 2.2891, -1.5791, 2.3369, -1.089, -0.7791, 2.3331, 0.3659, -0.2754, 1.3283, 0.6898, 1.5212, -0.4434, 0.5906, -1.5686, -1.3939, 0.3693, -3.4249, -0.23, -0.0056, 1.4483, 2.5419, -1.6096, 2.3859, -1.0526, -0.7583, 2.3392, 0.2106, -0.1028, 1.5009, 0.8359, 1.4354, -0.8591, 0.6005, -1.5748, -1.4641, 0.2581, -3.4759, -0.0911, 0.1473, 1.5788, 2.903, -1.6792, 2.2916, -1.2329, -1.3452, 1.9576, -0.1689, -0.4778, 0.8906, 0.7853, 1.752, -0.3774, 0.6501, -1.3409, -1.3791, 0.5958, -2.9161, 0.2232, 0.9372, 1.7171, 2.7143, -2.2648, 2.026, -0.9312, -1.6446, 2.4649, 0.0987, 0.1119, 1.5594, 0.3282, 0.6362, -1.7912, 0.7862, -1.1131, -0.8795, 1.2135, -2.4376, 0.5056, 1.0711, 2.2019, 2.7391, -1.2906, 2.456, -0.0712, -1.2037, 2.9391, 0.426, -0.6094, 1.5696, -0.0184, 0.7033, -2.5061, 0.8421, -1.3531, -0.5166, 1.5884, -2.3167, 0.8332, 0.7193, 2.4517, 2.308, -0.3863, 2.5604, 0.274, -0.4482, 2.8715, 0.533, -1.8877, 0.8308, -0.4529, 1.6671, -2.1006, 0.838, -1.5509, -0.4575, 1.5144, -2.2809, 1.172, 0.211, 2.5308, 1.3904, 0.1037, 2.4182, -0.1115, -0.0661, 2.1688, 0.5663, -2.0002, 1.5473, -0.3019, 1.4417, -1.3761, 0.9041, -1.627, -0.4035, 1.2847, -2.1723, 0.9316, -0.3427, 2.4259, 0.3051, 0.3634, 2.023, 0.0736, 0.0796, 1.9971, 0.3619, -1.5038, 2.164, -0.5378, 0.3796, -0.5976, 0.9771, -1.5089, -0.4743, 1.0103, -2.0292, 0.634, -0.2846, 1.98, 0.14, 0.513, 1.2686, 0.6606, 0.3333, 2.4405, 0.3306, -0.9088, 2.1428, -1.4657, -0.0054, -0.9815, 0.8705, -1.1737, -0.2404, 0.7449, -1.8705, 0.2081, -0.4642, 1.3941, 0.3886, 0.7418, 0.7535, 1.0891, 0.5736, 2.7471, 0.5055, -0.9259, 1.1541, -2.1797, 0.3974, -0.9494, 0.5061, -0.8759, 0.2998, 0.1256, -1.7004, 0.066, -0.5718, 1.4053, 0.6322, 1.4486, 0.2969, 0.784, 0.8393, 2.3155, 0.973, -1.4994, -0.1935, -1.5392, 1.0161, -0.4521, 0.1589, -1.0478, 0.7879, -0.4618, -1.2216, 0.1934, -0.7244, 2.5022, 0.0293, 1.8658, 0.5856, 1.1716, 0.6581, 1.1615, 1.3461, -1.4682, -0.2212, -0.7792, 1.5464, -0.2603, 0.1935, -1.5569, 1.306, -0.3939, -1.2576, 0.9766, -1.4957, 2.8485, -1.0262, 1.3458, 1.183, 1.401, 0.7898, 0.0759, 1.434, -0.6183, 0.2676, -0.5156, 0.2661, -0.3536, 0.1766, -2.1522, 1.5792, 0.1684, -1.3378, 1.7947, -1.5405, 2.8611, -1.3765, 0.6175, 0.7131, 0.7255, 0.7096, -0.0155, 0.9741, -0.0936, -0.0799, -0.3954, 0.0358, -0.7839, -0.1447, -2.9117, 1.8119, 0.1233, -1.5185, 1.577, -1.4026, 2.1869, -0.7352, 0.6799, 0.822, 0.6832, 0.8725, 0.5947, 0.7186, 0.6839, -0.1894, -0.0293, -0.5788, -1.4034, 0.6407, -2.5146, 0.6516, -1.355, -1.7915, 0.1922, -1.122, 2.8472, -0.3966, 1.0265, 1.0195, -0.5222, -1.5263, -1.1271, 0.1105, 1.7791, -0.9474, -0.3797, -2.149, -1.6827, 1.2117, -1.9442, 0.3531, -0.655, -1.6744, 0.03, -1.0962, 2.538, -0.8205, 1.5819, 0.3389, -0.0004, -0.9002, -1.53, -0.8165, 0.8213, -1.2126, -1.5827, -2.7583, -2.5188, 1.2315, -1.7587, 0.1607, 0.1922, -1.796, 0.9789, -1.8319, 2.0507, -1.3029, 1.7023, -0.8467, 0.3031, -0.8308, -1.4845, -0.8789, -0.7075, 0.1451, -2.248, -1.565, -2.3432, 1.1709, -1.7047, -0.0611, 0.6172, -1.7757, 1.2306, -1.7483, 1.8723, -1.1752, 1.329, -1.5758, 0.0331, -0.9249, -0.7619, -0.1648, -0.7527, 0.6141, -1.8299, -1.0559, -2.2805, 1.1301, -1.3932, -0.4403, 0.6781, -2.0475, 0.6283, -1.0619, 1.1403, -1.1051, 0.1516, -1.759, 0.1457, -0.3415, -0.0778, 0.1802, -0.2102, 0.5528, -0.894, -0.7383, -2.6031, 1.139, -1.2923, -0.6929, 0.5274, -2.1104, 0.4654, -1.192, 0.3359, -1.6595, -0.5015, -1.266, 0.8097, 0.2545, 0.9276, 0.9271, 0.3728, 1.0299, -0.7395, -0.4662, -2.6697, 1.1555, -1.3075, -0.8065, 0.1669, -2.3295, 0.1831, -1.5722, 0.0425, -1.663, 0.0434, -0.9703, 1.6861, 0.8701, 1.5269, 0.2028, -0.2069, 1.4089, -1.2755, -1.1965, -3.7654, 1.1839, -1.2303, -0.7618, -0.3126, -2.4567, 0.0652, -1.7824, 0.7062, -1.9493, 0.0847, -0.8737, 1.7572, 1.3687, 1.6785, -0.2128, -1.1192, 1.0696, -1.0701, -1.2066, -3.687, 1.0594, -1.2768, -0.73, -0.5459, -2.6831, -0.1164, -1.2238, 1.5276, -1.2909, 0.9208, -0.0066, 1.8319, 1.3063, 0.6877, -0.9843, -2.2005, -0.093, -0.7379, -0.6541, -2.7838, 0.9307, -1.2508, -0.832, -0.5707, -2.822, -0.5216, -1.1957, 1.1855, -0.4279, 1.198, 0.1645, 0.9938, 0.4305, 0.1441, -1.1055, -1.6773, 0.1659, -0.2274, 0.2282, -2.1347, 0.8719, -1.3139, -0.8113, -0.2504, -3.1782, -0.6359, -1.4168, 0.6588, 0.0961, 0.7115, 0.5308, 0.2226, -0.3579, 0.1676, -1.396, -1.6737, 0.4176, -0.8349, 0.9265, -1.1514, 0.922, -1.6312, -0.8638, 0.1111, -3.6526, -0.1624, -1.2048, 1.2207, 0.2102, 0.4977, 1.1275, -0.6938, -0.5641, -0.5089, -2.1113, -2.4103, 0.4902, -0.3436, 1.6, -0.1082, 0.9355, -1.8576, -1.057, 0.1081, -3.7844, 0.2426, -1.2451, 1.7355, 0.3548, 0.3576, 1.4187, -0.9059, 0.5165, 0.0401, -1.278, -2.3544, 0.9867, 0.0433, 1.3575, -0.0615, 0.9352, -1.9627, -1.0763, 0.1299, -3.5576, 0.6806, -1.3212, 1.8675, 0.1556, 0.8591, 1.5172, -0.63, 0.3803, 0.6042, -0.8923, -3.2791, 0.557, -0.5022, 1.3332, -0.5495, 0.9565, -1.9304, -1.1061, 0.0621, -3.2179, 0.5922, -1.4424, 1.2052, -0.4939, 1.0316, 1.3879, 0.1056, 0.6687, 1.3649, -0.7404, -2.8534, 0.3819, -0.9772, 0.8428, -1.0396, 0.936, -1.8968, -1.1537, -0.1717, -3.2716, -0.0221, -1.4586, 0.7513, -1.4296, 0.9405, 1.0569, 0.2075, 1.2481, 1.5597, -0.7348, -2.3486, -0.3304, -0.9062, 0.2924, -0.6668, 0.9054, -1.5766, -1.1489, -0.1199, -3.1121, -0.1578, -0.6281, 0.1759, -1.7019, 0.604, 0.6388, -0.1989, 1.0204, 1.2532, -1.4715, -2.5294, -0.4621, -0.8189, 0.1465, 0.1382, 0.9003, -1.3545, -0.939, 0.1795, -3.0248, -0.0711, -0.3431, -0.8752, -1.9996, 0.4199, 0.1907, -0.3406, -0.0634, 0.9798, -1.7811, -2.8021, 0.2466, -0.4483, 1.2884, 0.4712, 0.9043, -1.357, -0.6067, 0.5195, -2.934, 0.1407, -0.8753, -0.7731, -1.3934, 0.5516, -0.1382, -0.3509, -0.0766, 0.6542, -1.3562, -2.8634, 0.0819, -0.983, 1.3728, 0.8436, 0.8631, -1.4815, -0.6352, 0.6724, -2.6402, 0.5109, -0.6922, -0.4817, -1.1382, 0.3354, -0.6703, -0.094, 0.1379, 0.0316, -1.4467, -2.7335, 1.265, -0.5958, 1.3371, 0.567, 0.6832, -1.5788, -0.5281, 0.5591, -2.2396, 0.5978, -0.4646, -0.5515, -1.7098, -0.5502, -1.1639, -0.1979, -0.5121, -0.2856, -1.988, -2.5884, 1.6746, 0.4247, 1.3173, -0.3628, 0.3869, -1.2468, -0.1246, 0.2545, -1.999, 0.2346, -0.687, -0.6137, -0.9567, -0.8281, -0.4516, -0.1571, -1.776, -0.6134, -1.3301, -1.476, 1.1995, -0.2474, 0.2773, -0.5779, -0.0087, -0.8136, -0.0397, -0.0759, -1.459, -0.599, -0.1634, 0.0053, -0.3273, -0.8044, 0.4696, -0.7696, -2.2287, -1.7858, -0.7565, -0.5483, 1.1027, -0.2803, -0.7604, -0.7059, -0.2901, -0.4092, -0.2377, -0.382, -0.6093, -1.5884, 0.7663, 0.3223, 0.4006, -0.4895, 1.0767, -0.3239, -2.3117, -1.6769, -0.9458, 0.32, 1.0407, -0.2813, -1.2145, -1.4884, -0.2541, -0.858, -0.3838, -0.1474, -0.4951, -0.8464, 0.6928, 0.5248, 0.4641, -0.6464, 0.663, -0.4678, -1.4317, -1.0451, -0.1746, 0.2297, 1.0048, 0.1969, -1.5504, -2.0671, 0.0452, -1.4901, -0.5693, 0.0684, -0.8502, -0.2243, -0.166, 0.8089, 0.4032, -0.1992, 0.0605, -0.9232, -1.5233, -0.4613, 0.0125, -1.0621, 1.0656, 0.8037, -1.1305, -2.382, 0.2832, -2.2414, -0.2747, 0.188, -1.3623, 0.5161, -1.0161, 1.3016, -0.2578, 0.8269, -0.3201, -0.419, -1.9268, -1.2283, -0.2512, -1.1192, 1.2487, 0.2194, -0.3998, -2.4967, 0.4608, -2.6984, -0.1386, 0.4531, -1.7694, 1.1161, -1.4462, 1.5425, -1.0273, 0.9123, -0.9144, -0.161, -2.0875, -1.5841, -0.095, -1.0552, 0.9754, -0.1135, 0.0914, -2.4315, 0.6062, -2.5833, -0.2265, 0.5913, -1.5209, 1.58, -1.504, 1.4803, -1.2603, 0.6731, -1.6699, -0.6713, -1.7211, -1.1904, -0.1904, -1.5521, 0.2164, -0.7676, -0.1009, -2.1482, 0.5409, -2.1807, -0.2239, 0.3874, -1.2428, 1.5541, -0.9679, 0.8829, -1.2498, 0.5322, -1.4248, -0.4709, -1.9212, -1.4041, 0.0332, -1.3502, -0.5931, -1.9788, -0.5174, -1.0231, 0.3873, -1.4483, -0.2177, -0.3935, -1.4248, 0.9063, 0.0065, 1.1835, -0.5696, -0.0999, -1.376, -0.3794, -1.3761, -1.1991, 0.1008, -1.3818, -0.4921, -2.2098, -1.7786, -1.0252, 0.3945, -1.1028, -0.4148, -0.7282, -1.6348, 0.5535, -0.309, 1.1226, 0.4897, -0.2219, -0.723, -0.2627, -0.675, -0.7532, -0.1494, -1.3772, -0.3496, -2.0643, -2.4655, -1.6828, 0.3768, -1.1893, -0.5542, -0.4633, -1.5051, 0.5303, -0.75, 0.1363, 1.12, 0.1289, -0.3542, 0.4927, -1.1136, -1.8492, -1.0761, -1.1134, 0.2206, -1.0588, -2.1188, -2.2032, 0.4835, -1.6556, -0.673, -0.1338, -2.033, 0.7735, -1.4123, 0.0919, 0.4227, 0.7648, 0.2773, -0.7726, -1.5012, -2.3359, -1.4133, -1.3555, 0.674, -0.1572, -1.465, -1.9194, 0.6275, -1.9564, -0.7689, 0.2637, -2.2679, 0.836, -1.6332, 0.3473, -0.1955, 1.1395, 0.7058, -0.8196, -1.6215, -1.7267, -0.7041, -0.9141, 1.2708, 0.2351, -0.5137, -1.1834, 0.5522, -1.9702, -0.923, -0.0212, -2.119, 0.9356, -1.5954, 0.0149, -0.3324, 1.1226, 1.126, -0.1331, -1.0137, -1.5774, -0.9312, -0.8611, 1.521, 0.7971, 0.7871, -0.0973, 0.4447, -1.62, -0.9546, -0.1945, -1.7888, 1.1339, -1.1192, -0.2717, 0.3644, 0.7523, 0.9273, -0.7422, -0.9921, -1.4899, -0.3898, -1.0727, 0.6556, 0.2522, 0.3807, 0.0097, 0.331, -1.4615, -1.0205, -0.307, -1.4544, 0.9265, -0.8842, -0.2413, 0.6494, 0.0263, 0.5015, -0.9995, -1.2073, -1.3203, 0.0047, -1.204, 0.4128, 0.0214, -0.2729, 0.372, 0.2147, -1.3743, -1.0393, -0.5472, -1.0837, 0.6128, -0.995, -0.2301, 0.5296, -0.2916, 0.5842, -0.9805, -2.0282, -1.9303, -0.4539, -1.5663, 0.4563, 0.1479, -1.1658, 0.2872, 0.1972, -1.0531, -0.9536, -0.7753, -0.919, 0.2669, -0.7934, -0.0008, 0.4026, -0.6713, 0.5723, -1.4433, -1.9755, -1.6209, -0.4943, -1.115, 0.3345, -0.0498, -1.8131, 0.3565, 0.1685, -0.9199, -1.0218, -0.6324, -0.6691, -0.2252, -0.0106, 0.1317, -0.0531, -1.1067, 0.6707, -1.2559, -2.0214, -1.5982, 0.0864, -0.1759, -0.1797, -0.1785, -1.219, 0.4953, 0.3033, -0.9233, -1.0931, -0.3376, -0.2556, -0.5276, 0.1654, 0.2592, -1.5114, -1.6218, -0.4477, -0.9206, -1.8623, -0.99, 0.6432, -0.3301, 0.2709, -0.5268, -0.1658, 0.2961, 0.5444, -1.1192, -1.2271, -0.3669, -0.5818, -0.3492, -0.3408, 0.6882, -1.6816, -1.9394, -1.4241, -0.66, -1.543, -0.2712, 0.9927, -0.6655, 1.2691, -0.215, 1.014, 0.4122, 0.638, -1.6128, -1.4122, -0.3915, -1.4251, -0.2042, -0.9589, -0.0219, -1.7164, -1.5302, -1.5257, -0.492, -1.3844, -0.8556, 0.6263, -0.8502, 1.8537, -0.3088, 1.0517, -0.0247, 0.6813, -1.8273, -1.348, 0.0375, -1.5816, -0.2103, -1.4816, -0.4702, -1.8213, -1.1018, -1.4691, -0.3679, -1.5325, -1.2999, 0.145, -0.9034, 2.1471, -0.6063, 0.5355, -0.8994, 0.6814, -1.7581, -1.2489, 0.0245, -1.4856, 0.1534, -1.299, 0.0663, -1.5119, -0.8706, -1.4415, -0.1622, -1.1843, -0.9911, 0.3615, -0.5903, 2.3614, -0.6843, -0.1758, -1.7648, 0.6363, -1.8366, -1.1415, -0.0293, -1.5683, 0.2999, -1.5485, 0.774, -1.1379, -0.8951, -1.0466, -0.3993, -0.805, -1.0174, 0.007, -0.418, 2.6317, -0.3539, -0.6979, -2.5077, 0.6746, -1.9826, -1.0934, 0.1235, -1.6939, 0.3803, -1.7098, 1.1618, -1.2108, -1.0056, -0.6711, -0.4335, -1.0678, -1.5623, -0.0442, -0.3013, 2.9714, -0.2464, -0.8257, -1.8832, 0.7078, -2.0716, -1.0643, 0.2888, -1.6401, 0.4571, -1.9071, 1.2952, -1.5301, -0.7351, -0.7827, -0.4627, -1.1248, -1.7946, 0.359, -0.4515, 2.8585, -0.7021, -0.9506, -1.6948, 0.7086, -2.1567, -1.0008, 0.1923, -1.7908, 0.5055, -2.1002, 1.1807, -1.851, -0.1429, -0.7424, -0.0394, -0.5382, -1.5465, 0.252, -1.0168, 2.531, -1.1424, -0.9327, -1.752, 0.6695, -2.1922, -0.9708, 0.1044, -1.7185, 0.6082, -2.0494, 0.6616, -2.018, 0.5184, -0.6245, -0.1727, -0.4335, -0.8631, 0.6043, -1.4641, 2.7393, -0.9711, -0.4872, -1.7565, 0.6535, -2.0131, -0.8731, 0.2855, -1.6113, 0.3683, -1.9167, 0.3431, -1.7707, 1.0396, -0.6222, -0.2656, -0.4881, -0.7843, 0.4566, -1.5716, 2.7665, -0.8623, 0.0769, -1.3441, 0.5246, -2.1462, -0.8344, 0.3943, -1.4152, 0.3801, -1.6971, 0.3995, -1.5987, 0.6809, -1.1449, 0.4656, -0.3859, -0.7802, 0.812, -1.4846, 2.9507, -0.3494, -0.0693, -1.9132, 0.4651, -1.9468, -0.7838, 0.1717, -1.3255, 0.3296, -1.8752, -0.0346, -1.7425, 0.3326, -1.5275, 0.9019, -0.467, -1.0067, 1.6193, -1.7868, 2.9657, -0.1525, -0.4633, -1.418, 0.4556, -1.8127, -0.835, 0.0699, -1.4195, -0.1177, -2.0363, -0.2466, -1.8593, 0.9534, -1.3213, 0.4829, -0.4811, -0.7846, 1.4809, -1.9449, 2.9611, -0.0339, -0.1847, -1.0833, 0.3977, -1.7498, -0.6179, 0.0525, -1.4344, -0.0236, -1.8173, -0.3623, -2.1415, 1.0192, -1.4751, -0.0648, -0.6147, -0.5171, 1.0033, -1.6512, 2.2228, 0.1462, 0.0475, -1.4299, 0.4322, -1.6345, -0.4453, 0.124, -1.4242, 0.1258, -1.9348, -0.6703, -1.8895, 1.0538, -1.4802, 0.0541, -0.9372, -0.8564, 0.6188, -0.7037, 2.0343, -0.071, 0.4004, -1.9418, 0.4802, -1.5925, -0.5326, -0.0487, -1.6719, 0.3291, -1.9141, -0.6612, -1.5367, 0.2392, -2.4723, 0.0484, -0.4949, -0.7546, 0.502, -1.2606, 1.57, 0.3291, 1.0777, -1.0122, 0.4826, -1.6502, -0.5037, -0.1685, -2.114, 0.6463, -1.9356, -0.461, -1.5152, -0.7074, -2.859, -0.034, 0.4942, 0.0511, 1.3537, -1.0662, 0.963, -0.2933, 1.2762, -0.0245, 0.4278, -1.5925, -0.3324, -0.4482, -2.4546, 0.6377, -1.9469, -0.179, -1.0252, -0.7024, -3.0492, -0.6136, 0.805, 0.1784, 1.8179, 0.181, 1.5076, -0.3722, 1.0383, 0.4954, 0.2924, -1.4979, -0.1531, -0.4377, -2.492, 0.002, -2.0541, -0.0923, -0.1561, -0.4112, -3.0427, -1.9888, 0.484, -0.327, 1.8215, 1.3112, 1.9645, -0.0372, 0.6466, -0.1179, 0.1411, -1.4603, -0.0211, -0.6066, -2.2816, -0.4216, -2.2141, 0.0308, -0.5123, -0.261, -2.1558, -1.5757, -0.0474, -0.9652, 1.4248, 0.9949, 1.6003, -0.1351, 0.242, -0.4661, 0.2313, -1.1118, -0.0226, -0.6113, -1.5108, 0.7741, -1.0539, 0.4569, -0.2956, 0.5473, -0.2504, 0.297, 0.2883, -0.6391, 0.5232, -0.2711, 0.7815, -1.0119, 0.3902, 0.1216, 0.3944, -0.3431, 0.0884, 0.1425, -0.3943, 0.9493, -0.2778, 0.7531, 0.8256, 0.9593, 1.0293, 1.2086, 0.8412, 0.3264, 0.543, -0.0574, 0.8592, -1.2785, 0.0876, -0.1489, 0.389, -0.4746, 0.1487, 0.7046, -0.2915, 1.373, -0.0474, 0.8096, 1.0818, 0.9164, 1.4448, 1.0654, 1.7011, 0.4046, 1.2384, 0.649, 0.8581, 0.0385, -0.2801, -0.2601, 0.5285, -1.6188, -0.174, 1.2533, -0.5986, 2.3994, 0.2973, 1.2385, 0.3467, 1.0357, 1.7614, 1.0611, 1.4134, 0.5239, 1.3174, 0.132, 0.6438, 0.6472, 0.4592, 0.4378, 0.5122, -1.9818, -0.6444, 1.1242, -1.0217, 2.6209, 0.2819, 1.1013, 0.1201, 1.0944, 1.4489, -0.075, 0.5465, -0.1718, 0.7651, -0.4262, 1.5688, 0.6201, 0.3987, -0.0218, 0.3777, -2.0874, -0.7939, 0.8379, -0.9205, 2.5866, 0.1271, 0.8687, 0.7353, 1.4552, 1.7066, -0.0015, 0.7512, -0.2711, 0.4819, -0.4063, 1.2591, -0.2283, -0.3767, -0.2648, 0.2887, -1.9108, -0.7166, 0.6949, -0.809, 1.9354, -0.0002, 1.2268, 0.5484, 1.233, 2.0762, 0.3192, 1.3202, 0.1859, 1.4336, -0.3403, 0.7936, -0.5539, -0.6088, -0.0571, 0.2254, -1.6498, -0.6639, 0.39, -0.6644, 1.5695, 0.47, 0.9911, 0.4111, 0.515, 1.2162, -0.1383, 0.2174, 0.4828, 1.7156, 0.5563, 1.5596, 0.1127, -0.0818, 0.2956, 0.1403, -1.5439, -0.6196, 0.0229, -0.5215, 1.4298, 0.3614, 0.9448, 0.4739, 0.1843, 1.1715, 0.174, 0.2576, 1.4021, 1.3726, 0.6558, 1.0058, -0.8137, -0.3869, -0.1743, -0.0132, -1.7541, -0.7623, -0.4193, -0.3036, 1.4325, 0.3916, 1.0292, 0.4436, 0.0994, 0.7388, 0.9286, 0.5335, 1.3562, 1.1234, 0.0354, 0.4358, -1.2766, -0.5159, -0.2953, -0.0649, -1.6435, -0.5875, -0.628, -0.7984, 0.8109, 0.4808, 1.0232, 0.4663, -0.1579, -0.4077, 1.0219, -0.0108, 0.7364, 1.2863, -0.0563, 0.4812, -0.9524, -0.3164, -0.29, -0.1869, -1.6642, -0.1781, -0.24, -0.9793, 0.0932, 0.5735, 1.0192, 0.4628, -0.013, -1.0234, 0.7954, -0.4241, 0.6173, 1.1446, 0.0609, 1.0875, -0.6355, -0.1496, -0.7713, -0.3481, -1.9038, -0.1304, 0.2979, -0.3328, -0.196, 0.1932, 0.6703, -0.6535, -0.3854, -1.5877, 0.3043, -0.6069, 0.4644, 0.8949, 0.0215, 1.3145, -0.252, -0.0229, -0.9344, -0.3465, -1.8168, -0.119, 0.3296, -0.0754, 0.0746, 0.1626, 0.8297, -0.6153, -0.1917, -0.6072, 0.8027, -0.6673, 0.4238, 1.2299, -0.6317, 0.6314, -0.1752, 0.5908, -0.2426, -0.3641, -1.7651, -0.198, 0.0761, -0.1308, -0.1458, 0.1657, 0.8899, 0.1128, 0.2783, -0.3957, 0.5439, -0.808, 0.3782, 0.859, -1.027, 0.9873, 0.1864, 0.9685, -0.0129, -0.4761, -1.6025, -0.055, 0.0558, -0.0651, -0.4811, 0.134, 0.6191, 0.0191, 0.0996, -0.3312, 0.3399, -0.5409, 0.9758, 0.9659, -0.6106, 1.3742, -0.0263, 0.1609, -0.4046, -0.5613, -1.4181, 0.0876, 0.0924, -0.0935, -0.5894, 0.5861, 0.8391, 0.1135, -0.3195, -0.7578, 0.0493, -0.6943, 0.8268, 1.2798, -0.4827, 1.4191, -0.323, 0.1198, -0.6637, -0.5847, -1.3189, -0.088, 0.0113, 0.2189, -0.4166, 0.8556, 0.8058, -0.4251, -0.9557, -0.3114, -0.2713, -0.8843, 0.7361, 1.6854, 0.3174, 1.0722, -0.6162, 0.006, -0.9192, -0.6047, -1.0666, -0.1786, -0.2478, 0.4597, -0.7523, 0.7537, 0.6118, -0.5948, -0.8041, 0.3789, -0.0386, -0.8229, 0.3378, 0.9468, 0.3323, 0.8707, -0.5184, -0.0276, -0.4443, -0.6749, -0.8883, -0.302, -0.5425, 0.5643, -1.1409, 1.0362, 0.8503, -0.3006, -0.7136, 0.401, -0.425, -1.4626, -0.3317, 0.3344, 0.6286, 1.403, -0.0019, 0.1054, -0.2468, -0.7001, -0.7323, -0.2393, -0.5857, 0.5902, -1.3982, 1.3133, 0.706, -0.5792, -0.3637, 0.568, -0.2413, -1.101, -0.373, 0.1312, 0.5257, 0.9228, 0.3269, -0.24, -1.1497, -0.7316, -0.7132, -0.1787, -0.6771, 0.7321, -1.3458, 1.5843, 0.8427, -0.8187, -0.5972, 0.3523, -0.4352, -1.2535, -0.0796, 0.4224, 0.4536, 1.1542, 0.6534, -0.5216, -1.2708, -0.8085, -0.7707, -0.054, -0.6141, 0.6879, -1.332, 1.7002, 1.0604, -0.6178, -0.8655, -0.0296, -0.3001, -0.8607, -0.4608, -0.2386, 0.1817, 1.3041, 0.7808, -0.5378, -0.9656, -0.7989, -0.6221, 0.0228, -0.6228, 0.5768, -1.6094, 1.3545, 1.0325, -0.5526, -1.1446, -0.1132, 0.6445, -0.272, -0.6755, -0.3768, 0.1208, 1.077, 0.6081, -0.7483, -0.8028, -0.7914, -0.7001, 0.0295, -0.6658, 0.6335, -1.3146, 0.554, 0.0613, -0.7819, -0.8551, 0.3629, 1.0469, -0.6031, -0.6622, -0.0469, 0.3287, 1.4662, 1.0594, 0.0559, -1.0085, -0.4549, -0.8005, 0.3181, -0.4429, -0.2534, -0.3985, 0.6304, 0.2976, -0.3874, -0.5318, -1.0686, 0.005, -1.6576, -0.9694, 0.8374, 0.1785, 0.7689, 0.0373, -0.3239, -1.1571, -0.1155, -0.9788, 0.2501, -0.6416, -1.2364, 0.5311, 0.5941, 0.3057, -0.1407, -0.9051, -2.1199, -0.5279, -1.4416, -0.6566, 0.3683, -0.0042, 0.375, -0.9626, -0.3633, -0.9651, 0.0168, -1.3501, 0.0215, -0.6047, -0.9389, 1.2141, 0.0364, 0.141, -0.3111, -1.2148, -2.0148, -0.3908, -1.5153, -0.1999, -0.2454, -0.279, 1.1546, -0.6785, -1.5207, -1.1065, 0.0105, -1.7769, -0.1614, -0.3624, -0.5297, 1.4298, -1.0077, -0.4739, -0.7832, -1.191, -1.4994, -0.1812, -1.5812, -0.7596, -0.8429, -0.1342, 1.3657, -0.3989, -2.0141, -1.0863, 0.0379, -1.7504, -0.3161, -0.2695, -0.1869, 1.5135, -1.3009, -0.4755, -0.7098, -0.5849, -1.5456, -0.2932, -0.8481, -1.1459, -0.4826, -0.3021, 0.9132, -1.2032, -2.0254, -0.3637, 0.0546, -1.4407, -0.4121, -0.526, 0.1725, 1.3744, -0.7727, 0.5157, -1.0472, -0.8075, -1.2055, -0.0979, -0.1549, -0.9341, -0.0158, -0.536, 0.2153, -1.6607, -1.4256, -0.1553, -0.0463, -0.913, -0.728, -1.0906, 0.4708, 0.6568, -0.2728, 0.6065, -1.3757, -0.8091, -0.4996, -0.3884, 0.2058, -0.7505, 0.2179, -0.5516, 0.5259, -0.7995, -0.6232, -0.3023, -0.2185, -0.4793, -1.1821, -1.358, 1.2834, -0.1834, 0.5609, 0.1159, -1.03, -0.3973, 0.0312, 0.0045, 0.4768, -0.7374, -0.5324, -0.7081, 0.2242, -0.4571, -0.9142, 0.1131, 0.235, -1.0203, -0.3144, -0.6867, 0.9507, 1.3584, -0.6706, 1.5678, -1.1305, 0.7539, 0.1786, 0.9245, 0.4348, 0.3453, 0.1699, -0.1719, 1.2949, -0.5116, 0.0053, -0.4158, 0.7239, -1.07, 0.072, 0.571, 0.0042, 1.833, -0.6888, 1.2766, -0.5316, 1.3908, -0.2523, 1.1574, 0.4747, 0.5974, 0.9294, 0.3901, 1.5786, 0.0222, 0.537, -0.7997, 0.9239, -1.2614, -0.1372, 1.2451, -0.707, 1.9731, -0.4552, 1.0661, -0.204, 1.6376, 0.5038, 0.721, 0.4852, 0.2163, 0.3883, 0.1258, 1.5422, 0.8701, 0.9868, -1.0119, 0.9468, -1.208, -0.4451, 0.8509, -1.4671, 1.3219, -0.5978, 0.9639, -1.0743, 0.9687, 0.6516, 0.486, 1.5006, 0.7695, 0.9119, 0.5293, 1.877, 1.1606, 0.9111, -1.8459, 0.9832, -1.2475, -0.7764, 0.2911, -2.1965, 1.057, -0.7077, 1.0119, -1.2437, 0.6594, 0.3864, 0.4232, 2.8226, 1.5044, 1.8869, 0.916, 1.5929, 0.8714, 0.3149, -1.9095, 1.0611, -1.0741, -1.0377, 0.0759, -2.7582, 0.6521, -0.7738, 1.1959, -0.5842, 0.7841, 0.9652, 1.0617, 3.9576, 2.5183, 2.4848, 0.3937, 0.8181, 0.5641, 0.8999, -1.0067, 0.9463, -1.1657, -1.1305, 0.1439, -2.7187, 0.2987, 0.0681, 1.4599, -0.5647, 1.0148, 1.3389, 0.9481, 3.1115, 2.9341, 1.5757, -1.2421, 0.4951, -0.4024, 1.8963, 0.2862, 0.8924, -1.1946, -1.2968, 0.1747, -2.9566, -0.2486, 0.5338, 1.0171, 0.2201, 1.0486, 1.2134, 0.5715, 1.3339, 2.5844, 0.7043, -1.862, 0.8633, -0.7516, 1.9794, 0.8909, 0.8261, -1.3122, -1.2913, 0.3323, -2.7826, -0.2244, 0.6714, 0.7127, 0.1809, 0.4453, 0.7924, 0.5005, 1.3902, 2.8182, 0.4783, -2.1206, 1.0777, -0.2783, 2.0515, 1.0269, 0.8281, -1.4676, -1.1685, 0.4136, -2.8376, -0.0611, 0.5375, 1.4189, -0.0096, 0.1021, 0.5958, 0.1458, 0.7274, 3.0121, 0.7793, -2.0943, 1.0676, -0.0218, 2.3897, 1.2064, 0.8615, -1.6546, -1.1575, 0.3416, -2.9893, -0.116, 0.0997, 2.0771, -0.0252, 0.0266, 0.4569, -0.2009, 0.4911, 2.5834, 0.8675, -2.0185, 1.0329, 0.1515, 2.3842, 1.1622, 0.8583, -1.8851, -1.131, 0.2167, -3.1567, -0.0785, 0.07, 2.2028, -0.0154, 0.1841, 0.0275, -0.8505, 0.1641, 2.2763, 1.048, -1.5486, 1.1876, 0.3807, 1.5912, 0.9085, 0.8312, -2.0255, -1.0961, 0.3838, -3.2983, 0.0475, -0.2079, 1.8445, -0.1699, -0.0886, -0.3409, -0.8494, 0.3856, 1.6174, 1.0631, -1.4689, 1.4644, 0.1497, 1.251, 0.9722, 0.8161, -1.9128, -0.8346, 0.5849, -3.1036, 0.2789, -0.7185, 1.179, 0.0786, -0.2951, -1.3624, -0.2915, 1.6966, 1.923, 1.7214, -0.5824, 1.7401, -0.518, 0.681, 1.3045, 0.9271, -0.7336, -0.0889, 1.1947, -1.8298, 1.2473, -0.0316, 0.9979, 0.4683, -0.2514, -1.5291, -0.4188, 1.1054, 0.8928, 1.4122, -0.945, 0.8456, -0.4053, 0.0162, 0.4941, 0.8762, 0.3034, 0.1784, 0.7943, -0.2818, 1.4153, 0.6476, 1.4181, 0.9596, -0.0518, -0.7572, 0.3473, -0.4729, 0.5842, 1.9926, -0.4963, 1.3493, 1.0792, 1.2737, -0.3619, 0.9419, -0.0429, 0.3034, 0.6554, -0.212, 1.8609, 1.0308, 1.8887, 0.5589, 0.3015, -1.0608, 1.3216, -0.7836, 1.8825, 2.1114, -0.6814, 0.7778, 2.0025, 2.3586, -2.7938, 1.0009, -0.4116, 0.0525, 0.4199, -0.6743, 1.9827, 1.1689, 2.0924, 0.4723, 0.5156, -0.6497, 1.8283, -0.2643, 2.4167, 2.0292, -0.7402, 0.9695, 1.5068, 1.7906, -2.9726, 0.9234, -0.7257, 0.1687, 0.7273, -1.2745, 1.7649, 1.5426, 2.0885, 0.4247, 0.8498, -0.2913, 1.7736, 0.9799, 1.8236, 1.567, -0.2358, 1.9458, 0.5028, 0.8943, -1.595, 0.7978, -1.7292, 0.5834, 1.4195, -1.969, 2.2097, 1.5927, 2.4798, 0.7497, 1.5814, 0.3411, 1.7623, 1.2276, 1.7105, 1.3197, 0.455, 1.7531, -0.084, 1.5097, -1.1691, 0.6762, -2.3212, 0.9976, 2.2068, -2.0808, 2.0014, 1.0794, 3.183, 1.2231, 2.0983, 0.351, 2.1585, 0.8838, 1.78, 1.29, 0.4922, 0.7225, -0.3879, 2.5277, -1.2145, 0.6466, -2.465, 0.9807, 2.1353, -1.9909, 2.4095, 0.7091, 3.1925, 0.3767, 1.6151, 0.7365, 2.3599, 0.7282, 0.87, 0.9919, 0.7802, 1.274, -0.0394, 2.5437, -0.274, 0.649, -2.5954, 1.0056, 1.9545, -2.6375, 2.3196, 0.5339, 3.5053, 0.2325, 1.4054, 0.3766, 1.8546, -0.4487, -0.4127, 1.2311, 1.1038, 2.7335, 0.7443, 1.6845, -0.1358, 0.6297, -2.8429, 0.9891, 2.0544, -3.0159, 2.0625, 0.5328, 3.0397, -0.215, 1.9774, 0.6316, 1.927, -1.0586, -0.8205, 1.1221, 0.8575, 2.2373, -0.2994, 0.4713, -0.5173, 0.6404, -2.9198, 0.9411, 2.1406, -3.1664, 1.694, -0.2336, 2.8396, -0.5878, 2.2049, 1.2077, 1.4928, -0.7696, -0.9225, 0.9896, 0.6088, 1.3475, -0.5077, 0.519, -0.049, 0.5629, -2.7682, 1.0607, 1.9557, -2.8481, 1.4869, -0.874, 3.1234, -0.3783, 2.3701, 1.4596, 0.8977, -0.6577, -1.1312, 0.5856, -0.2677, 0.9344, -0.0617, 1.0816, 0.3619, 0.2512, -1.9414, 1.0574, 0.9814, -1.9158, 1.0839, -1.0956, 2.8304, 0.1927, 1.719, 1.3105, 0.8314, -0.5316, -1.4686, 0.4601, -0.2674, 0.7657, 0.4524, 1.1693, 0.1516, -0.1032, -0.9084, 0.7659, 0.4381, -0.6704, 0.3772, 0.0641, 1.6455, 0.3144, 0.3404, 0.9028, 1.0627, 0.1885, -1.2693, 0.8417, 0.5117, -0.035, 0.6532, 0.8507, -0.4352, -0.4233, -0.5347, 0.5296, 0.2228, -0.0539, -0.0995, 1.0507, 1.01, 0.8818, -0.4428, 0.9231, 0.6511, 0.0172, -0.8531, 0.6741, 1.0445, -0.0501, 0.7596, 0.3895, -0.4268, -0.7128, -1.0801, 0.3876, 0.0562, -0.3632, -0.1225, 1.612, 0.4248, 0.6982, -0.2275, 0.5433, -0.0403, 0.303, 0.0133, 0.0209, 1.7439, 1.1599, 0.5004, 0.5183, 0.0794, 1.0566, -1.0684, -0.544, -0.184, -2.0503, -0.1465, 0.1297, 0.0654, -0.1403, 0.4999, -0.6749, -0.221, -1.8966, -0.3442, -0.3743, 1.4875, 0.1378, -1.4842, -0.4385, -1.0081, 1.6274, -0.8946, -0.6356, 0.222, -1.9909, 0.4063, -0.2644, 0.4854, -0.12, 0.8454, -0.3326, 0.3341, -1.6974, -0.8016, -0.0472, 1.7307, 0.3144, -1.373, -1.0097, -1.4918, 1.6065, -0.5821, -0.8791, 0.4214, -2.0858, 1.1772, -0.527, 0.8363, -0.7305, 0.6997, -0.5442, 0.5442, -0.7737, -0.6738, 0.3306, 0.9413, 0.3846, -1.0417, -0.3051, -2.0137, 1.4788, -0.5812, -1.0941, 0.2319, -1.7563, 1.5405, -0.5702, 0.883, -1.1222, 0.623, -0.4009, 0.6315, 0.4847, 0.2333, 0.7168, 0.3905, 0.6486, -1.2319, 0.2771, -1.7377, 1.3747, -0.5651, -1.3178, -0.1484, -1.7937, 1.234, -0.8336, 0.9132, -0.991, 0.9301, 0.0717, -0.1241, 0.1123, -0.3122, 1.2468, 0.3915, 1.1732, -1.4986, 0.4402, -0.3996, 1.2854, -0.6386, -1.3546, -0.1376, -2.0574, 1.0119, -0.9988, 0.1319, -1.6982, 0.6043, 0.2489, 0.2883, -0.1637, -0.6638, 1.5629, -0.1349, 1.6612, -0.3945, 1.0254, -0.9419, 1.1655, -1.027, -1.4333, 0.3298, -2.4603, 0.8441, -0.576, -0.0653, -1.1618, 0.6867, -0.6298, 0.4637, 0.3421, 0.2183, 1.6551, -0.8553, 2.0461, -0.2015, 1.2998, -1.4847, 1.0665, -1.2583, -1.2745, 0.6419, -2.4529, 0.6389, -0.2419, 0.2274, -0.8015, 1.1263, -1.1165, 0.4109, 0.3869, 0.2074, 1.8659, -1.2266, 1.88, -0.4931, 0.5399, -1.6658, 0.9895, -1.2665, -1.2376, 0.5151, -2.1188, 0.1447, 0.0746, 0.4486, -0.5808, 1.5214, -0.8779, 0.6793, -0.6203, 0.5159, 2.0831, -1.2702, 0.9109, -0.7328, 0.0044, -2.4109, 0.8654, -1.197, -1.1669, 0.3824, -1.6291, 0.0141, 0.2164, 0.2907, -1.273, 0.983, -0.9891, 0.6275, -1.1764, 0.5957, 1.5011, -1.1129, 0.7633, 0.2316, 0.7575, -2.277, 0.7732, -1.1324, -0.7495, 0.151, -1.813, 0.3572, -0.6966, 0.4343, -0.9657, 0.0976, -0.9516, 0.1584, -0.949, -0.1581, 0.8213, -0.7967, 0.7247, 0.7829, 0.7027, -1.922, 0.76, -1.6336, -0.3498, 0.5047, -2.3347, 0.8909, -0.9142, 1.24, -0.7062, 0.7048, -0.6144, 0.2554, -1.5244, -0.5785, 1.0239, 0.0008, 0.7024, -0.2611, 0.0189, -1.9661, 0.8084, -2.3774, -0.3299, 0.8829, -2.7822, 0.9421, -0.8457, 2.1751, -0.8165, 1.8656, -0.4111, 0.3296, -1.6896, -0.5524, 0.9473, 0.2547, 1.0341, -0.8018, 0.0863, -1.8424, 0.7837, -2.8225, -0.2891, 1.1776, -2.9058, 1.0858, -0.6483, 2.4805, -1.5039, 2.0847, 0.1265, 0.8228, -2.1863, -1.4983, 0.2554, 0.1034, 2.0793, -0.1815, 1.047, -1.4431, 0.752, -2.973, -0.314, 1.4473, -2.8402, 1.1235, -0.6414, 2.5554, -1.7063, 2.2159, 0.1693, 0.1694, -2.7977, -2.0719, 0.1934, -0.3632, 2.035, 0.0003, 1.6357, -0.562, 0.7417, -2.9335, -0.4384, 1.3681, -2.8286, 1.1323, -0.7559, 2.2781, -1.6015, 2.1837, 0.2151, 0.3377, -1.8345, -1.9484, -0.5116, -0.4594, 1.1204, -0.0439, 1.4094, -0.2111, 0.657, -2.9391, -0.3299, 1.3441, -2.9624, 1.1556, -0.4365, 2.5738, -1.0891, 2.0438, 0.193, 0.949, -1.2231, -2.1006, -0.9053, -0.5764, 0.2563, -0.0886, 0.357, 0.6838, 0.6127, -2.8981, -0.2034, 1.5128, -3.0442, 1.0551, -0.2508, 2.5584, -1.2328, 1.6989, 0.5975, 1.4216, -1.2437, -2.3588, -0.6439, -0.1887, 0.8479, -0.1078, -0.0457, 0.7145, 0.5745, -2.9069, -0.2316, 1.6105, -3.0384, 1.2452, -0.1514, 2.6252, -0.9543, 1.5125, 0.7008, 1.5576, -0.9053, -2.9573, -1.5222, -0.3068, 0.8224, -0.7949, -0.0229, 0.4506, 0.5793, -2.9142, -0.2464, 1.7348, -3.0595, 0.8552, -0.7727, 1.7855, -0.8734, 1.4776, 0.7844, 1.1744, -1.0609, -2.7794, -1.7363, -0.4107, 1.0955, -0.3742, 0.2497, 0.1313, 0.5791, -2.9161, -0.4097, 1.7958, -3.2576, 0.7367, -0.1986, 1.7733, -0.5567, 0.9562, 0.2773, 0.9935, -0.5213, -1.8885, -0.7895, -0.5427, 1.3643, 0.1227, -0.3726, -0.4172, 0.5708, -2.5037, -0.4242, 1.4887, -3.117, 0.5997, -0.1375, 1.6138, -0.2914, 0.3967, -0.4461, -0.0234, -0.4767, -2.0528, -0.5402, -0.6048, 0.9965, 0.2161, 0.47, 0.0264, 0.4986, -1.6151, -0.4875, 0.6253, -2.4157, 0.1607, -0.5264, 1.3873, 0.6826, -0.1104, -1.123, -0.5949, 0.6732, -1.6417, -0.3748, -0.075, 0.7431, -0.9783, 0.2106, 0.5498, 0.3174, -1.1602, -0.5213, -0.0204, -1.5592, -0.3128, -0.8659, 1.5165, 1.2677, -0.4305, -1.6437, -0.522, 0.6305, -1.6449, -0.5727, -0.2313, 1.0541, -1.9479, -0.1982, 0.0711, 0.153, -1.2331, -0.454, 0.0359, -0.9133, -0.0004, -0.6887, 1.4394, 0.6289, -0.5691, -1.0295, -0.2815, -0.0788, -2.271, -0.5318, -0.833, 0.9568, -1.6954, -0.5219, 0.0474, 0.0456, -0.9201, -0.3239, -0.0278, -0.9439, -0.3796, -0.141, 1.1287, 0.8314, -0.2444, 0.1888, -0.1895, -0.7263, -1.4671, -0.1271, -0.6175, 0.6731, -1.7832, -0.6299, 0.4767, 0.0007, -0.7597, -0.3499, 0.0376, -0.7199, -0.805, -0.5097, 0.546, 0.6194, -0.0365, 0.3923, -0.7504, -0.7955, -1.5306, -0.6544, -0.8538, 0.9915, -0.5823, -0.2612, 0.5548, 0.0164, -0.6741, -0.4926, -0.1115, -0.4621, -0.9105, -0.8914, 0.4399, 0.7683, -0.0043, -0.055, -1.2257, -0.9475, -1.849, -0.7389, -0.9951, 0.6834, -0.1837, -0.6147, -0.0125, -0.0409, -0.5336, -0.5128, -0.2182, -0.2352, -0.9162, -0.446, 0.4628, 0.828, 0.0851, -0.1197, -0.6681, -0.0857, -1.733, -0.9551, -1.4454, -0.2133, 0.3196, -0.6791, 0.0155, 0.0257, -0.5947, -0.3017, -0.0867, 0.0678, -1.1077, -1.3682, 0.1127, -0.8614, 0.0412, -1.0241, -0.1928, 0.4292, -2.0859, -0.521, -1.1647, 0.2625, 0.0935, -0.047, 0.61, 0.2924, -0.8542, -0.1082, 0.0662, -0.6213, -1.1458, -1.9765, -0.4961, -0.6312, -0.1462, -2.0565, -0.1417, -0.5845, -1.4811, -0.771, -0.0384, 1.1379, -0.638, -0.0877, 0.6639, 0.8358, -1.9466, -0.0877, 0.3964, -1.867, -0.8912, -0.8143, -0.9248, -0.429, 0.1397, -1.4932, 0.4277, -1.2292, -0.1208, -1.4953, 1.1601, 0.7051, -0.3854, -0.7344, 0.4588, 1.0417, -2.4667, -0.0673, 0.7483, -2.0353, -0.3408, -0.7304, -0.9027, -1.5721, 0.7361, -0.6554, 0.3417, -1.4963, -0.112, -1.6496, 0.8058, 0.5938, 0.4148, -1.0383, -0.1943, 1.1057, -2.3647, -0.3728, 0.7827, -2.289, 0.1086, -1.2059, -0.2404, -1.8473, 1.6819, -0.0759, 0.1759, -0.8955, -0.087, -0.7532, 0.7252, 1.5318, 0.581, -1.0097, -1.3751, 1.0596, -2.1784, -0.7745, 1.2343, -2.6337, 0.4523, -2.1442, 0.3008, -1.8358, 2.0402, 0.1794, 0.0218, -0.5679, -0.5393, -0.0054, -0.2509, 1.8807, -0.524, 0.1823, -2.1867, 1.1112, -2.1784, -0.7133, 1.1335, -2.5449, 0.7184, -2.0793, 0.173, -1.9429, 2.1363, 0.1614, 0.7365, -0.7828, -0.9212, -0.5194, -0.8159, 2.0279, -0.6041, 0.5221, -1.2508, 1.1577, -2.1848, -0.6597, 1.1076, -2.6527, 0.5933, -2.126, 0.1855, -2.2102, 2.3439, -0.5518, 0.2963, -0.7091, -0.8816, 0.0793, -0.4278, 1.4839, -0.7977, 0.0884, -1.5677, 1.16, -2.1304, -0.6021, 1.1747, -2.822, 0.6185, -2.1492, 0.5486, -2.0298, 1.8973, -0.8785, -0.5778, -1.4374, -0.649, 0.6166, -0.1985, 1.1988, -0.8325, -0.0996, -2.1812, 1.0975, -2.4062, -0.7466, 1.0866, -2.7506, 1.2828, -1.6524, 0.955, -2.2965, 1.1991, -1.2001, -0.3402, -1.483, -0.9962, -0.1531, -0.9013, 0.9711, -0.754, -0.0246, -2.0944, 1.0316, -2.6833, -0.6013, 1.248, -2.7526, 1.7229, -1.671, 1.0159, -2.1352, 1.3834, -0.7624, 0.5847, -0.9252, -0.7733, -1.0645, -1.0871, 1.1603, -0.9917, -0.016, -1.5764, 1.0235, -2.9442, -0.6095, 1.2781, -3.007, 1.577, -1.5349, 1.2328, -2.2237, 1.4139, -0.1815, 0.9755, -1.0688, -0.5306, -0.5684, -0.3521, 1.1183, -1.5544, -0.2465, -1.2139, 0.9836, -3.3285, -0.9047, 1.2735, -2.8237, 1.562, -1.3599, 1.3387, -2.3036, 1.1369, -0.6028, 0.5315, -1.9986, -0.8497, -0.6971, -0.8578, -0.0871, -2.332, -0.09, -1.776, 1.0352, -3.3976, -0.9161, 1.0947, -2.9031, 1.4542, -1.6677, 1.7744, -1.916, 1.0368, -1.0287, 0.4422, -2.0855, -0.4212, -0.4267, 0.1076, 0.022, -2.5636, -0.0979, -1.7279, 1.0477, -3.4952, -0.6953, 1.0973, -3.2085, 1.14, -1.6535, 1.9375, -1.8646, 0.5019, -1.6037, 0.5455, -1.9345, -0.0599, -0.7172, 0.6131, -0.1736, -2.0407, -0.5253, -1.3355, 0.9281, -3.6175, -0.5329, 1.5349, -3.3856, 0.9369, -1.4826, 1.9739, -1.5461, 0.0484, -1.8927, 0.3312, -2.3792, -0.4102, -1.0777, 0.2851, -1.1739, -2.1211, -0.6847, -0.7285, 0.7771, -3.5808, -0.4177, 1.6093, -3.0107, 1.0615, -1.685, 2.0563, -0.7548, -0.2134, -1.9666, -0.0673, -2.8023, -0.5252, -0.737, 0.9389, -0.6882, -2.4728, -0.6082, -0.4749, 0.5843, -3.3369, -0.5275, 1.2726, -2.5653, 1.4472, -1.6018, 2.1614, 0.1717, -0.457, -0.7628, -0.1282, -2.7853, -1.1149, -0.334, 0.7208, -0.0276, -2.4073, -0.5295, -1.0341, 0.2958, -2.5024, -0.8208, 0.7443, -1.6674, 0.7418, -0.994, 1.9157, 1.0066, -0.6267, 1.1016, 0.0358, -2.5032, -1.6049, 0.0427, 0.4991, -0.1571, -1.5529, -0.0373, -1.2994, 0.0481, -1.555, -1.0768, 0.3577, -0.7799, -0.571, -0.0741, 1.3749, 0.7439, -0.8975, 1.7932, 0.0055, -1.9222, -1.5566, 0.0002, 0.5704, -0.2266, -0.988, -0.4719, -1.2483, -0.0023, -1.4869, -1.0435, 0.0729, -0.4834, -0.8614, -0.3296, 1.1438, -0.0195, -0.7424, 1.3921, 0.2758, -1.3166, -1.1331, -0.4049, 0.4389, -0.0967, -0.8513, -0.8612, -1.2289, 0.291, -2.4551, -0.2387, 0.0681, -0.6744, 0.0207, -1.0757, 1.4108, -1.3645, 0.8597, -0.4158, 1.1415, -2.1523, -0.1782, -0.986, 0.3466, -0.5872, -0.984, 0.0669, -0.7815, 0.5789, -2.7316, -0.0711, 0.659, -1.3141, 0.4754, -1.0854, 1.0406, -2.1095, 1.3985, -1.0681, 0.6341, -2.199, -0.2585, -1.437, -0.265, -0.2046, -0.1693, 0.2136, -0.182, 0.7113, -2.4653, -0.2108, 0.9223, -1.5629, 1.269, -0.4525, 1.1817, -2.7572, 0.5623, -0.8341, 0.1249, -1.4886, -0.844, -1.3202, -0.637, -0.4235, -0.8142, 0.4959, -0.5224, 0.8632, -1.9906, -0.6106, 0.8821, -1.5688, 1.6258, -0.4336, 0.958, -2.391, 0.3555, -1.5894, -0.6501, -1.0229, -1.4061, -1.1649, -1.3356, -0.6095, -1.3722, 1.1901, -1.1869, 0.935, -1.6474, -0.9848, 0.3971, -1.6735, 1.4129, -0.0665, 0.4725, -2.15, -0.089, -2.4393, -0.4516, -0.7965, -2.1064, -1.3116, -0.9625, 0.9887, -0.511, 1.5771, -0.6528, 0.9545, -1.6089, -1.2212, -0.0225, -2.0555, 1.4535, -0.03, -0.0527, -2.222, -0.5056, -2.0519, -0.4911, -0.3829, -1.5869, -1.2229, -0.432, 1.3679, 0.1234, 1.2171, -0.4676, 0.9919, -1.802, -1.3844, -0.3313, -2.3472, 1.2141, -1.0263, -0.7282, -2.5293, -0.2965, -1.682, -0.4292, 0.0424, -1.2721, -1.0685, -0.3775, 1.4513, 0.3678, 1.4687, 0.1703, 1.0422, -1.6654, -1.3734, -0.4486, -2.2948, 0.6245, -1.6476, -0.8371, -2.2101, 0.2774, -2.281, -0.5399, -0.3456, -1.1997, -1.2902, -0.3632, 1.149, -0.3392, 1.1236, 0.6334, 0.9459, -1.3118, -1.2601, -0.1253, -1.9819, 0.3011, -1.555, -1.3305, -2.0323, 0.1346, -2.4562, -0.0444, -0.0634, -0.9448, -1.1602, 0.0091, 1.5844, 0.1258, 1.3614, 0.5058, 0.8547, -1.2645, -1.2381, 0.2477, -1.6764, 0.5804, -1.4902, -1.9739, -1.8473, -0.0336, -2.5171, -0.6509, -0.2378, -0.3902, -1.0539, 0.2173, 1.5939, 0.717, 1.3015, 0.1378, 0.9699, -1.4455, -1.1521, 0.695, -1.9063, 0.8942, -1.9088, -1.7716, -1.5691, -0.3611, -2.596, -1.2935, -0.383, -0.8329, -0.7656, -0.1154, 1.1535, 0.8875, 1.1161, 0.23, 1.0256, -1.6255, -1.2071, 0.5638, -2.0126, 1.0898, -2.2668, -1.6296, -1.6624, -0.5771, -2.2808, -0.9186, -0.1037, -0.905, -0.6106, 0.1303, 1.4542, 0.8042, 1.0086, 0.2103, 1.2078, -0.9505, -0.489, 1.2515, -1.057, 2.3063, -0.405, -0.487, -0.8494, 0.535, -1.1866, 0.0934, 0.1402, 0.0573, -0.8643, 0.3304, 1.9335, 0.3754, 0.656, -0.488, 1.3987, -0.7683, -0.0686, 1.7698, -0.9065, 2.7148, 0.408, 0.2036, -0.4263, 1.7146, -0.7987, 1.0297, 0.1552, 0.5024, -0.9302, 0.6974, 2.7158, 0.6924, 1.5672, -0.7401, 1.3995, -0.7242, 0.061, 1.5823, -0.8714, 2.7083, 0.3947, 0.7295, -0.258, 1.7072, -0.6888, 1.417, 0.7817, 1.1686, 0.6513, 1.5657, 2.1721, -0.0297, 0.8802, -0.9993, 1.3093, -0.763, -0.0314, 1.4229, -0.7626, 2.6581, 0.4067, 1.1056, -0.2264, 1.4267, -0.0655, 1.1, 0.7565, 0.6155, 1.3108, 1.5972, 1.8828, 0.4318, 0.5904, -1.1851, 1.2245, -0.8436, -0.1474, 1.3239, -0.5259, 2.8818, 0.1517, 0.8032, -0.5344, 1.2522, 0.1166, 1.1981, 1.15, 0.7459, 1.3959, 1.2003, 2.7539, 0.8901, 0.891, -0.4566, 1.1086, -0.8883, -0.1462, 1.1094, -0.429, 2.8567, -0.1397, 0.5374, -0.7978, 0.7922, -0.1075, 0.8824, 1.4596, 1.1513, 1.7349, 1.2734, 3.1228, 1.4453, 1.3079, 0.8537, 1.0457, -0.7889, -0.0575, 1.2317, -0.2523, 2.5571, -0.1018, 0.615, -1.3428, 0.3847, -0.1564, 0.8101, 1.7698, 1.4314, 1.681, 1.4191, 3.1234, 2.0906, 1.831, 0.5503, 0.9707, -0.8141, -0.0538, 1.019, -0.5051, 2.3499, 0.3637, 0.2173, -1.9519, 0.1772, -0.2719, 0.963, 1.4847, 1.9001, 1.548, 1.5224, 2.9959, 2.2401, 2.3293, 0.7818, 0.852, -0.8804, -0.1568, 0.7996, -0.5958, 2.0765, 0.0744, -0.069, -1.6837, -0.4152, -1.0449, 0.4884, 1.4362, 2.107, 1.5466, 1.114, 1.9232, 1.3235, 2.128, 0.8189, 0.6746, -1.0716, -0.0884, 0.5313, -1.0645, 1.5172, -0.7221, -0.1202, -1.5589, -1.5744, -2.29, -0.9104, 0.7185, 1.6722, 1.258, 0.322, 0.269, 0.3172, 1.7307, 0.9389, 0.6213, -0.8812, 0.1721, 0.1208, -1.8206, 0.9687, -0.8878, 0.0268, -1.368, -1.7061, -2.7084, -1.0599, -0.1037, 0.6477, 1.1966, 0.1327, 0.3702, 0.079, 1.1483, 0.5902, 0.5084, -0.5762, 0.4705, 0.253, -2.105, 0.5277, -0.6684, -0.5127, -0.7249, -1.2395, -2.7746, -1.214, -0.4719, -0.4972, 1.09, 0.4768, 0.5249, 0.947, 0.9869, -0.3749, 0.3187, -0.5672, 0.5111, 0.4277, -1.7417, 0.2459, -0.4132, -0.2768, -0.8809, -1.1816, -2.2458, -1.5608, -0.9953, -1.6097, 1.2206, -0.0096, 0.4288, 1.8545, 1.2879, -0.7024, 0.2275, -0.5496, 0.5973, 0.4745, -1.5402, -0.0885, -0.517, 0.22, -0.8568, -0.3747, -0.7104, -1.027, -1.0787, -1.902, 0.5423, -1.0795, 0.0538, 1.9625, 1.4081, -0.272, 0.0858, -0.6442, 0.5679, 0.4134, -1.2936, -0.5631, -1.1144, -0.5017, -1.8215, -0.5629, -0.3807, -0.773, -0.1566, -0.8901, 0.2836, -1.6609, -0.3706, 1.4633, 1.2671, 0.1104, -0.09, -0.5575, 0.2973, 0.1006, -0.633, -0.6684, -0.7135, -0.8347, -1.1896, -0.5094, -0.7851, -0.6642, -0.1033, -0.2893, 0.1192, -1.3531, -0.5523, 0.4186, 1.1911, 0.4554, -0.2233, -0.5011, 0.2164, 0.3245, -0.086, -0.7286, -0.0663, -0.2461, -0.5963, -0.9095, -1.0418, -1.3269, -0.24, 0.4863, 0.9901, -0.2441, -0.6334, 0.1644, 0.7113, 0.5633, -0.1757, -0.6262, 0.0074, 0.5407, 0.083, -0.5029, 0.1357, 0.3296, -0.82, -1.0768, -0.39, -0.8403, 0.2651, 0.3223, 1.435, 0.0372, -0.7608, -0.0895, 0.3865, 0.6413, -0.1491, -0.8334, -0.0261, 0.5295, -0.1898, -0.2244, -0.0435, 0.5192, -0.4572, -1.1105, -0.1708, -0.5065, -0.1996, 0.4175, 1.2616, -0.1364, -0.6469, -0.1749, 0.833, 0.587, -0.0755, -1.2371, -0.1144, 0.4122, -0.5615, 0.3261, -0.2752, 0.9844, -0.497, -1.1584, -0.7426, -0.634, -1.1021, -0.6362, 0.871, 0.0846, 0.977, 0.4445, 1.0869, -0.2953, 0.0284, -1.4086, -0.0098, 0.1328, -0.8691, 0.6062, -0.2404, 1.3807, -0.8103, -0.6744, -0.9886, -0.5228, -0.7313, 0.1204, 1.4475, -0.2346, 0.9888, 0.9068, 0.5087, -0.7545, 0.09, -1.5794, 0.0963, 0.2855, -0.9284, 0.5651, -0.3627, 1.285, -0.9869, -0.3927, -0.7929, -0.1126, -0.6163, 0.2052, 1.188, -0.568, 0.3728, 0.8322, 0.1772, -1.079, 0.1389, -1.7409, 0.2138, 0.2255, -0.9663, 0.36, -0.1998, 1.2938, -0.8646, -0.1434, -0.8127, -0.0318, -1.0184, -0.0517, -0.2131, -0.1694, -0.089, 0.8032, -0.4043, -0.986, 0.1013, -1.7432, 0.1689, 0.0662, -0.8048, 0.3943, -0.2477, 0.9727, -0.9289, -0.0686, -1.0204, 0.013, -0.5737, 0.2645, -1.3067, -0.4694, -0.4276, 1.41, -0.2093, -0.8855, 0.0368, -1.7152, -0.0385, -0.0489, -0.8979, 0.3705, -0.7159, 0.7514, -0.4328, 0.0819, -0.7944, 0.1218, 0.2299, 0.7, -0.9824, -0.3268, 0.1066, 1.1969, -0.3484, -1.5325, -0.0902, -1.5819, -0.0065, -0.0462, -0.8965, 0.4698, -0.6829, 0.6761, -0.6661, -0.0571, -0.3648, -0.1218, 0.0853, -0.3899, -0.4935, -0.0334, 0.6479, 1.1693, 0.1202, -1.6503, -0.2333, -1.4564, -0.0169, -0.2506, -0.6236, 0.8352, -0.1951, 0.3631, -0.8324, -0.2212, -0.1857, -0.1716, 0.343, -0.1201, -0.055, -0.0547, 0.4849, 0.7027, 0.3432, -1.2059, -0.4092, -1.1782, 0.0417, -0.5394, -0.641, 0.4754, 0.3854, 0.6205, -0.4271, -0.5597, -0.0398, -0.4904, 0.0682, -0.0259, -0.156, 0.394, 0.8875, 0.4875, 0.4865, -0.8691, -0.6818, -0.7771, 0.0682, -0.725, -0.1888, -0.3768, 0.54, 0.2824, -0.5766, -1.2488, 0.2801, -0.1368, 0.049, -0.2937, -0.4236, 0.652, 1.118, 0.506, 0.1784, -0.4806, -0.7029, -0.4818, -0.0269, -1.0586, -0.0094, -0.6999, 0.9396, 0.255, -0.4718, -1.0967, 0.4687, 0.5916, 1.0765, 0.3845, -0.3223, 0.9205, 1.0404, 0.7535, 0.5338, -0.0124, -0.5207, -0.3959, -0.7154, -1.4352, 0.5948, -0.35, 1.1522, 0.1847, -0.2621, -0.6115, 0.5925, 0.8086, 1.0333, 0.7196, 0.3997, 0.6188, 0.5211, 0.3088, -0.0847, -0.4274, 0.598, -0.9823, 0.1187, 0.7043, -0.4826, 1.0517, 1.2729, 0.2123, 0.3309, 0.3096, -0.6312, 0.0432, -1.1369, -0.0462, -1.343, 0.3173, 0.5676, 0.2459, 1.0088, -0.0128, 1.3226, -1.2291, 0.6029, 1.6209, -0.9676, 1.2593, 1.2875, 0.8046, 0.7768, 1.1898, -0.7335, 0.3927, -1.1446, 0.4591, -0.7244, 1.2718, 0.9697, 0.7731, 1.0438, 0.1966, 1.4237, -1.1691, 0.5055, 1.7048, -1.2378, 1.6375, 1.1334, 1.7243, 0.7325, 1.5577, -1.1162, 0.8851, -0.6421, 0.7761, 0.7321, 1.9449, 0.9148, -0.3719, 0.4876, -0.5303, 1.2742, -1.1567, 0.4367, 1.8172, -1.1589, 1.928, 0.7123, 2.6559, 0.0369, 1.7416, -0.6626, 1.048, 0.5277, 0.8949, 1.8704, 1.3266, 1.5898, -1.037, -0.3702, -1.5601, 1.1522, -1.3067, 0.4729, 1.5021, -1.2667, 1.9551, 0.3487, 3.1265, 0.1927, 1.7143, 0.3693, 1.9204, 1.096, 0.6323, 1.2297, 0.7774, 2.3725, -0.7195, 0.1155, -1.4727, 1.0122, -1.3006, 0.2671, 0.8771, -1.2923, 1.9148, 0.2318, 2.8031, 0.434, 1.6368, 0.6228, 1.47, 0.5886, 0.1453, 0.8456, 0.4934, 2.2972, -0.1611, 0.5681, -1.0746, 0.8823, -0.9674, 0.291, 1.1291, -1.4111, 1.4908, 0.585, 2.2325, 1.5195, 1.7459, 0.9643, 1.1507, 0.3111, -0.496, 1.0259, 0.8402, 2.2837, 0.3745, 0.2183, -0.3495, 0.8402, -0.6895, 0.1534, 1.1146, -1.7346, 0.9586, 0.6645, 2.0555, 1.8326, 1.1372, 0.7961, 1.415, 0.8249, -1.1897, 1.5058, 1.1996, 2.7669, 0.5351, 0.6545, 0.5203, 0.9309, -1.0774, -0.0772, 1.1124, -2.2963, 0.7735, 0.1894, 2.2563, 1.6653, 1.2295, 0.1854, 0.6237, 1.5744, -0.7114, 2.0062, 1.4556, 3.0824, 0.7004, 0.8649, 0.6172, 1.0626, -1.5353, -0.2328, 1.1448, -3.2724, 0.1802, -1.0075, 1.9925, 1.4722, 1.2373, -0.9518, -0.6733, 2.4865, 1.0006, 2.8164, 0.5411, 2.1711, -0.0313, -0.1649, -0.4055, 1.0453, -1.8201, -0.4873, 1.0142, -3.3471, 0.3502, -1.0129, 1.6591, 0.3075, 0.2317, -1.5545, -1.0153, 2.4042, 1.0645, 2.2797, -0.0435, 1.2656, -0.3397, -0.5982, -0.8435, 1.0163, -1.6821, -0.7028, 1.019, -2.7745, 0.2802, -0.5765, 1.6399, -0.2222, -0.0942, -1.3589, -0.2707, 2.5261, 0.7779, 1.7061, -1.2249, 0.2753, 0.295, -0.2284, -0.3725, 0.9619, -1.3216, -0.7065, 0.5523, -2.1504, -0.1958, -0.6028, 1.2897, -0.8645, 0.1412, -1.8866, -0.1265, 2.2711, 0.1488, 1.1169, -1.0053, 0.334, 1.3422, 0.1611, -1.2086, 0.8825, -1.8151, -0.1796, 0.517, -2.5854, 0.8784, -1.3351, 1.1497, -0.9351, -0.0217, -1.7518, 0.0298, 0.9266, -0.4114, -0.0726, -1.131, 0.842, 0.5581, 0.9343, -1.9138, 0.7649, -2.6762, 0.4368, 0.7314, -3.3746, 1.751, -1.4691, 1.4555, -1.0731, 0.5051, -0.9281, 1.0462, -0.2103, -1.3108, -0.2312, -0.6908, 0.6438, -0.6035, 0.6339, -1.7754, 0.786, -3.1514, 0.6156, 0.9702, -3.4889, 2.3852, -0.5865, 2.4781, -1.1548, 0.5747, -0.9631, 1.2281, -0.9368, -1.439, 0.3216, 0.0838, 0.9221, -0.9493, 0.6617, -1.4711, 0.8338, -3.3939, 0.5852, 1.0407, -3.7705, 2.474, -0.442, 3.1351, -0.8497, 0.3216, -0.8935, 1.0574, -1.6943, -1.2132, 0.8625, 0.2185, 1.0696, -0.4465, 1.523, -1.6919, 0.8405, -3.4931, 0.5986, 0.8426, -4.2255, 2.416, -0.3076, 3.6913, -0.9117, 0.4433, -0.1389, 1.6479, -1.6847, -1.4597, 0.5409, -0.9696, 0.3489, 0.0093, 2.3065, -1.8264, 0.8364, -3.6501, 0.3973, 0.5605, -4.2918, 2.5611, 0.1624, 4.2112, -1.3501, 0.5048, -0.1207, 1.6129, -2.0093, -2.6173, -0.3373, -0.7928, 0.4663, -0.5325, 2.0416, -1.6212, 0.8724, -3.4509, 0.2718, 0.5143, -3.9033, 2.4479, 0.1496, 4.0652, -1.4376, 0.4925, -0.7792, 1.3403, -1.6411, -2.6129, -0.6613, -0.0097, 0.994, -0.7669, 1.5603, -1.3301, 0.9331, -2.991, 0.3979, 0.794, -3.391, 2.4683, 0.431, 3.8331, -1.0309, 1.4592, -0.4811, 1.3769, -0.5142, -1.7105, -0.2822, -0.1874, 0.9035, -1.0541, 0.5013, -1.5916, 0.882, -2.6107, 0.612, 1.4018, -2.5925, 2.5684, 0.1014, 3.2032, -0.685, 1.937, 0.1138, 1.4019, 0.6935, 0.196, 0.9863, -0.5055, 0.4438, -1.4782, 0.403, -1.5814, 0.9392, -1.84, 0.4309, 1.3377, -1.8772, 2.2282, 0.0851, 2.567, -0.5085, 1.2025, -0.1812, 0.9245, 0.781, 0.5053, 1.1645, -0.5418, -0.5363, -1.445, 2.1709, -0.3913, 0.891, -1.1871, -0.1705, 1.0155, -1.1539, 1.3524, 0.5179, 2.0579, 0.3092, 0.7163, -0.8766, 0.5858, 0.724, 0.3198, 0.2781, -0.4701, -0.0086, -1.0412, 2.29, 0.645, 0.8391, -0.9518, -0.4119, 0.9632, -0.8804, 0.7645, 0.3255, 1.6155, 0.6684, 0.5781, -0.853, 0.404, 0.7049, 0.4638, -0.0074, 0.1536, 0.4761, 0.0926, 1.771, 0.5702, 0.7288, -0.8559, -0.3149, 0.886, -0.6163, 0.7234, 0.2, 1.7566, 0.4041, 0.3229, -0.1768, 0.2682, 0.2563, 0.1378, 0.033, 0.5824, 0.3792, 0.3073, 1.3307, -0.0656, 0.5337, -0.7431, 0.0097, 0.5426, -0.316, 0.4589, -0.3612, 1.8919, 0.1799, 0.3292, 0.2757, 1.2201, -0.0596, 0.1325, 0.3297, -0.1543, -0.1813, 0.204, 2.3266, 0.1275, 0.274, -0.4945, 0.2072, 0.0509, -0.1198, 0.3098, -0.2269, 1.9934, 0.6127, 0.7448, 0.5998, 0.7797, -0.4709, 0.0045, 0.3804, -0.8918, -0.504, 0.5191, 2.6212, -0.1689, 0.0646, -0.4127, 0.223, 0.2178, -0.3046, 0.6912, 0.5805, 1.6253, 1.289, 0.827, 1.5944, -0.7039, -0.0773, -0.0937, -0.3317, -1.6958, 0.0824, 1.0555, 1.7838, -0.0976, -0.0823, -0.5005, 0.316, 0.6685, 0.0627, 0.533, 0.4231, 0.9776, 1.1727, 0.5452, 1.5978, -0.8948, 0.4912, 0.7221, -0.1729, -1.7608, -0.6128, 0.6145, 1.0769, -0.1973, -0.4155, -1.1648, 0.304, 0.7134, 0.1466, 0.7424, 0.5053, 0.5248, 0.4862, 0.0756, 1.3596, -0.7844, 0.7675, 1.2274, -0.6181, -1.8648, -1.0819, -0.0358, 1.072, -0.6627, 0.3666, -1.189, -0.3293, -0.2224, -1.3648, -0.1574, 0.2406, 0.0093, -0.352, 0.0638, -0.6619, -1.9095, -1.4406, 0.0445, -0.9525, 0.389, -0.5226, -0.8403, -0.4182, -1.281, 1.4968, -1.1336, -0.5462, -0.0643, -1.722, -0.0546, -0.1571, 0.1018, -0.0841, 0.8844, -0.8025, -0.1321, -1.6859, 0.0887, -0.7715, 1.8454, 0.091, -0.8339, -0.99, -1.6337, 1.7253, -0.9894, -0.6456, 0.084, -1.6879, 0.4817, -0.4775, 0.3588, -0.3574, 0.8894, -0.9354, 0.3496, -1.5236, -0.1283, -0.5743, 1.5566, 0.3512, -0.5493, -1.0422, -1.9134, 1.614, -0.7072, -1.0157, 0.236, -1.8029, 1.6474, -0.3948, 0.9432, -1.4396, 0.4416, -0.8797, 0.6491, 0.0692, 0.4253, 0.8993, 0.685, 0.8517, -0.6253, 0.1077, -2.3374, 1.5351, -0.7946, -1.2238, 0.0009, -1.6059, 1.5958, -0.6129, 0.8688, -1.2279, 0.5079, -0.4038, 0.6662, 1.0768, 0.7526, 1.0539, 0.4115, 1.2559, -0.6284, -0.0529, -1.487, 1.4458, -0.8888, -1.43, -0.1458, -2.0932, 1.177, -0.4033, 0.6702, -1.0036, 0.2627, 0.1494, 0.7803, 0.772, 0.2923, 0.6804, 0.3112, 1.1736, -0.8031, 0.0616, -0.7942, 1.2051, -1.1235, -1.3272, 0.076, -2.3833, 1.116, 0.3514, 1.0238, -0.629, -0.578, -0.074, 0.5481, 0.2845, 0.2461, 0.4423, 0.2053, 1.1484, -0.9603, 0.2335, -1.5567, 1.0374, -1.382, -1.3116, 0.1841, -2.6062, 0.8403, 0.5858, 1.6429, 1.3085, -0.4038, -0.2307, -0.3769, -0.3185, 0.9012, -0.6568, 0.0577, 0.77, -1.404, 0.0249, -1.3308, 0.9965, -1.3344, -1.2281, 0.2985, -2.8445, 0.5012, 0.6277, 1.5768, 1.726, -0.8138, -0.0601, -0.6653, -0.0218, 1.9426, -0.5523, 0.3547, 0.7709, -1.1372, -0.255, -1.0934, 0.9642, -1.3738, -1.3261, 0.3767, -2.9003, 0.5356, 0.9177, 1.4567, 1.3741, -1.0356, 0.0776, -0.9785, 0.2244, 1.8168, -0.0564, 0.4365, 0.1289, -1.8679, -0.5584, -0.7362, 0.8878, -1.3764, -1.4539, 0.3648, -2.8834, 0.4051, 0.7101, 1.4702, 2.0519, -0.9104, 0.3943, -1.3884, 0.425, 1.0882, -0.375, 0.135, 0.3332, -1.2841, -0.6182, 0.51, 0.8365, -1.3941, -1.4363, 0.4532, -3.0796, 0.2379, 0.3502, 1.3079, 2.4391, -0.7326, 0.8961, -2.0571, 0.8595, 1.5752, 0.3918, -0.6794, -0.3566, -0.895, -0.6121, 0.7459, 0.8492, -1.7555, -1.2991, 0.3409, -3.1528, 0.2008, -0.112, 1.6658, 2.1209, 0.0022, 0.4885, -2.1169, 0.5318, 1.7967, 1.4723, -0.4102, 0.0271, -1.4679, -0.2307, 0.767, 0.8642, -1.9925, -1.3098, 0.4204, -3.1342, 0.0733, -0.3141, 1.6269, 1.5607, 0.2656, 0.3662, -2.4215, -0.0565, 0.7906, 1.2386, -0.3281, 0.4338, -1.682, -0.221, 1.3303, 0.9058, -1.9783, -1.1835, 0.386, -2.9568, 0.2212, -0.4797, 1.4088, 1.0482, 0.7537, 0.3954, -1.8445, -0.627, 0.2038, 0.9912, -0.5542, 0.4068, -2.2158, -0.21, 1.2398, 0.9093, -1.9948, -1.1126, 0.1697, -3.0204, 0.1904, -0.6529, 1.3315, 0.5508, 1.6431, 0.0217, -1.0039, -0.4891, -0.007, 1.2787, -0.328, 0.9904, -2.4332, -0.5695, 1.0765, 0.9215, -1.7091, -0.9739, 0.1192, -3.0821, -0.0588, -1.1468, 0.6008, -0.3547, 1.3959, -0.2811, -0.839, -0.1867, 0.8827, 1.7833, -0.2133, 1.4815, -2.7525, -1.5469, 0.1746, 0.7248, -1.2428, -0.8628, 0.1353, -2.4824, -0.518, -1.2627, 0.0565, -0.4842, 0.1761, -0.0416, -0.6065, -0.381, 0.8207, 1.9225, 0.3415, 1.6001, -2.4352, -1.7236, 0.0082, 0.2268, -0.8224, -0.5604, -0.0869, -1.5987, -0.9492, -0.5078, 0.0332, -0.1281, -0.3252, 1.4468, 0.3357, -0.7017, -0.4368, 0.7212, 0.4283, 0.9078, -1.6321, -1.4381, 0.8286, -0.0911, -0.6602, -0.4532, -0.4274, -1.218, -1.0193, 0.6433, 0.5005, 0.5288, -0.1255, 2.3217, 0.6325, -0.483, -1.326, -0.9525, 0.2948, -0.0812, -0.7485, -1.0548, 0.3612, -0.1757, -0.8951, -0.4481, -0.2777, -1.2756, -1.2552, 0.2088, 0.2188, 0.7268, 0.022, 2.2258, 0.7296, -0.1137, -1.2015, -0.9451, 0.3332, 0.5307, 0.2421, -0.8618, -0.8645, -0.2738, -0.9815, -0.5082, -0.4675, -1.2887, -1.3713, 0.5077, 0.4508, 0.5271, -0.3051, 1.751, 0.0583, -1.0238, -1.451, -1.0215, 0.0962, 0.4249, 0.1214, -0.9639, -0.8335, -0.2556, -0.8746, -0.5566, -0.5126, -1.4643, -1.4044, 0.8874, 0.6941, 0.9663, -0.2771, 1.5738, -0.0468, -1.3609, -1.7803, -1.3467, -0.0026, 0.0211, -0.5858, -1.4892, -0.8774, -0.2536, -0.8531, -0.6528, -0.5968, -1.3178, -1.6518, 0.6563, 0.6269, 1.0551, -0.3043, 1.2339, -0.1698, -1.4815, -1.6743, -0.9912, 0.1833, 0.2199, -0.6271, -1.681, -1.5538, -0.2169, -1.1595, -0.2926, -0.677, -0.9051, -1.3295, 0.0926, 1.0175, 0.2457, 0.1618, 0.5925, 0.0377, -1.7234, -0.847, -0.771, 0.865, 0.7233, -0.1253, -1.3477, -1.7687, 0.3511, -1.7208, 0.4358, -0.2246, -0.4797, 0.2799, -0.3021, 2.5045, -0.9345, 0.8467, -0.9303, 0.4578, -1.4763, 0.4302, 0.2939, 1.5549, -0.393, -0.0118, -0.7443, -0.0213]).reshape(1, -1)

# Find the nearest neighbors
distances, indices = knn_model.kneighbors(query_features_2d)


# Print the indices of nearest neighbors
print("Indices of nearest neighbors:", indices)

# Optionally, you can print the distances as well
print("Distances to nearest neighbors:", distances)
# Retrieve labels of nearest neighbors using list comprehension
nearest_neighbor_labels = [train_labels[i] for i in indices[0]]

# Print the labels of nearest neighbors
print("Labels of nearest neighbors:", nearest_neighbor_labels)


Indices of nearest neighbors: [[ 8  7 16 53 11]]
Distances to nearest neighbors: [[2.90739311e-03 1.27337164e+02 1.28563800e+02 1.32281625e+02
  1.33552524e+02]]
Labels of nearest neighbors: ['000002', '000135', '000155', '000237', '000161']


In [65]:
from joblib import dump

# Save the model
dump(knn_model, "knn_model.joblib")


['knn_model.joblib']

In [66]:
from joblib import load

# Load the mode
odel = load("aitazaz_model.joblib")

In [67]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

# Reshape the input array into a 2D array
query_features_2d = np.array([0.3687, -0.4593, -0.6877, -0.5762, -0.5355, -0.4242, -0.2775, -0.1768, -0.1154, -0.1924, -0.3219, -0.4489, -0.4848, -0.4263, -0.2145, -0.026, 0.142, 0.1115, 0.0431, -0.0365, -0.0556, -0.4712, -0.1519, -0.8916, -0.0392, -0.3035, 0.7606, 0.567, -0.2822, 0.9036, 0.6981, -1.9313, -2.5581, -1.5252, -2.041, -2.7436, -1.3173, -0.7162, -1.2103, -2.0367, 0.0946, -0.4021, -0.465, -0.3044, -0.03, -0.5118, -0.0188, 0.4068, -0.4036, 0.6611, 0.6911, -1.4575, -1.8957, -1.602, -0.6792, -1.9977, -0.2236, -0.4297, -1.4055, -2.0964, 0.0076, -0.6619, -0.5873, -0.2786, -0.0107, -0.5344, -0.1176, 0.8594, -0.1422, 0.5414, 0.399, -1.0238, -1.6774, -1.5393, 0.5757, -1.0542, 0.4936, -0.5029, -1.2949, -2.3596, -0.1498, -0.6503, -0.5701, -0.508, -0.1246, -0.7537, 0.7058, 1.1563, 0.2547, -0.1799, 0.0459, -0.4559, -0.9585, -1.4476, 0.037, -0.8084, 0.0429, -0.2147, -1.259, -1.8921, -0.0196, -1.3158, -0.2831, -0.1706, -1.2122, 0.0929, -0.0932, 1.0193, -0.3755, -0.4607, 0.4599, 0.1286, -0.5184, -1.3204, 0.0448, -1.2295, 0.0407, -0.9118, -0.5245, -0.8292, 0.3632, -2.4081, 0.0873, 0.4737, -2.7994, 0.7302, -0.9686, 0.9292, -1.2973, 0.6701, -0.4648, 0.2735, -1.0622, -1.0698, -0.4702, -0.0012, 0.7226, -1.1289, -0.151, -0.8505, 0.7794, -2.8934, -0.0731, 0.684, -3.4813, 1.1418, -1.0225, 1.1804, -1.6186, 1.4671, -1.0776, 0.4895, -0.6777, -0.2691, -0.6725, 1.1864, 1.2358, -1.3886, -0.9698, -1.4763, 0.9571, -3.1312, -0.4062, 0.912, -3.5766, 1.3403, -1.1462, 1.221, -1.3262, 1.7898, -0.9695, 0.9772, -0.5834, -0.2691, -0.6226, 0.9437, 0.9324, -1.4291, -0.8846, -1.6406, 0.9506, -3.1125, -0.7057, 0.6582, -3.5863, 1.8527, -1.0584, 1.7124, -0.5402, 2.295, 0.1913, 1.4564, -0.1174, -0.1234, -0.1487, 0.5431, 1.7121, -1.0165, -0.9328, -3.0532, 0.9745, -3.3147, -0.7998, 0.515, -3.2892, 1.8453, -1.5855, 1.7595, -0.5811, 2.5335, 0.8792, 1.1102, -0.453, -0.047, -0.4541, 0.3702, 2.0888, -0.8558, -0.8073, -3.2638, 1.0068, -3.2709, -0.6572, 0.972, -3.289, 2.0754, -2.0184, 1.0477, -1.0296, 2.1025, 0.6709, 0.7793, -1.2896, -0.4506, -0.9221, -0.1698, 2.345, -0.7364, -0.1292, -2.1351, 1.0391, -2.9416, -0.5284, 1.0884, -3.249, 2.074, -1.5945, 1.0114, -0.869, 1.282, -0.2756, 0.0001, -1.686, -0.8311, -1.1737, -0.7553, 1.1466, -0.774, 0.028, -1.4506, 0.9159, -2.3517, -0.6716, 0.7068, -2.6154, 1.7338, -1.0099, 0.633, -0.5511, 0.8794, -0.4624, -0.0854, -1.351, -0.6588, -0.9831, -0.9399, 0.3329, -0.8446, -0.281, -1.4498, 0.7378, -1.711, -0.8379, -0.3079, -1.9533, 1.2535, -0.7155, 0.5131, -0.5129, 0.5119, -0.9829, -0.1698, -1.5693, -0.8703, -0.3826, -0.7091, 0.816, -0.2404, -0.1478, -0.5131, 0.6584, -1.3685, -0.8107, -0.5147, -1.3347, 1.0516, -1.0076, 0.2584, -0.2522, -0.0269, -2.0463, -0.1882, -1.0009, -0.0861, -0.001, -0.4461, 2.0751, -0.0477, 0.3403, 0.2972, 0.5771, -1.1409, -0.8947, -0.3291, -0.8412, 0.4363, -0.9712, 0.3723, -0.0824, -0.2221, -2.1849, -0.0506, -1.2801, 0.1807, 1.0013, -0.4214, 1.9082, -0.211, 0.4279, 0.4764, 0.4887, -1.1708, -0.9421, -0.2622, -0.5362, -0.0906, -1.326, 0.3664, -0.8802, -0.2472, -1.088, -0.1167, -2.2247, -0.603, 1.3691, -0.0433, 1.7387, -0.5482, 0.4435, 0.3103, 0.4864, -1.1069, -0.9243, -0.1312, -0.8052, -0.8505, -1.5993, 0.4103, -1.077, -0.4738, -0.5255, -0.0903, -3.0168, -0.8853, 0.6721, 0.2325, 2.0972, -0.657, 0.8242, 0.5287, 0.4261, -0.9309, -0.9068, -0.255, -0.7973, -1.338, -1.5638, 0.5159, -1.4171, -0.8209, -0.9254, -0.1471, -2.4611, -0.5545, 0.8649, 0.0493, 1.4477, -1.0372, -0.0674, 0.1581, 0.2004, -0.9252, -1.0752, 0.0886, -0.4356, -1.6494, -0.8973, 0.6786, -0.3827, -0.6998, -0.8398, -0.9349, -2.258, -0.9762, 1.3014, -0.7152, 0.2428, -0.7849, -0.3569, 1.0729, 0.052, -1.0073, -1.0686, 0.3583, -0.0347, -1.4951, -0.5164, 0.9731, 0.2948, -0.6915, -0.5903, -1.4553, -2.2134, -2.111, 0.9996, -1.1586, -0.4948, -1.148, -1.2535, 1.3716, 0.0324, -0.8909, -0.9404, 0.3188, 0.0976, -1.2089, -0.0934, 1.2038, 0.1882, -0.8935, -0.1802, -1.8386, -2.1965, -2.0055, 0.6414, -0.1089, -0.7213, -1.168, -1.2413, 0.729, 0.0376, -0.8805, -0.9288, 0.2007, 0.0783, -1.0855, 0.1291, 1.4211, -0.14, -0.9976, 0.2465, -1.542, -2.1521, -2.3735, 0.4819, 0.6749, -0.4034, -0.625, -0.4903, -0.0532, 0.093, -0.9529, -0.7619, 0.493, -0.0235, -0.9085, 0.0898, 1.6212, -0.1069, -1.0678, 0.3583, -1.4637, -2.2918, -2.7509, 0.3165, 0.8915, 0.2453, -0.5459, -0.7299, -0.8381, 0.112, -1.0038, -0.7581, 0.3655, -0.1312, -0.7611, -0.2286, 1.3548, -0.3505, -1.2213, 0.2678, -0.8743, -1.5257, -2.1552, -0.2294, 0.2828, 0.3931, -0.617, -0.7829, -0.5133, 0.2829, -0.9843, -0.6129, 0.3129, -0.3108, -0.1487, -0.5865, 0.568, -1.792, -1.7647, -0.7888, -0.3008, -1.2438, -1.4535, 0.0121, -1.1174, 0.4579, -0.4477, 0.5554, -0.181, 0.5077, -1.0741, -0.6514, 0.3222, -0.9457, 0.2662, -0.8881, 0.001, -1.626, -1.3591, -1.2111, -0.1697, -1.2987, -1.1373, -0.0798, -1.1858, 1.5337, -0.0554, 0.6577, -0.2278, 0.529, -1.4377, -0.7468, 0.5132, -1.0611, 0.6466, -0.8676, -0.2607, -1.1797, -0.7852, -1.2472, 0.2023, -0.7473, -0.9499, -0.033, -0.6414, 2.1306, 0.0385, 1.0047, -0.8003, 0.5931, -1.552, -0.8626, 0.6591, -1.1082, 0.8943, -1.0997, -0.9489, -0.9986, -0.7636, -1.5091, 0.4199, -0.4743, -1.1798, -0.0054, -0.6109, 1.7478, 0.0614, 0.9346, -0.8294, 0.6683, -1.5729, -1.0328, 0.6053, -1.0838, 0.8082, -1.2874, -1.2959, -0.7957, -0.5288, -1.2806, 0.6108, -0.748, -0.5067, -0.4702, -0.9097, 1.4778, 0.6569, 0.4589, -0.9754, 0.59, -1.6916, -1.0646, 0.1323, -0.9861, 1.0513, -1.5404, -1.2466, -0.8546, -0.624, -1.4933, 0.0763, -1.8381, 0.1561, -0.0134, -0.886, 1.0551, 0.8032, 0.7191, -0.7444, 0.4598, -1.7023, -0.8683, 0.0466, -0.9587, 1.3063, -1.4545, -0.9898, -0.7312, -0.5834, -0.4226, -0.0261, -2.7474, -0.7096, -0.557, -1.1274, 0.543, 1.216, 1.348, -0.9433, 0.1572, -1.3257, -0.528, -0.0694, -0.8378, 0.9035, -0.8902, -0.7348, -0.312, -0.6482, 0.4493, -0.3716, -2.1149, -1.1178, -1.1911, -1.3328, 0.0834, 0.8497, 1.5119, -0.2746, -0.2299, -0.7365, -0.3959, -0.2828, -0.0622, -0.2544, -0.0206, -0.6859, -0.3957, -0.4441, 0.8631, -0.6163, -0.4919, -1.2863, -1.8456, -0.8701, -0.3028, 0.1741, 0.4021, -0.0432, -0.3547, -0.6136, -0.3236, -0.4098, 0.294, -0.5177, 0.2833, -0.2499, 0.158, -0.1416, 1.2316, 0.0569, -0.8299, -1.3404, -1.2285, -0.1805, -0.4034, -0.2986, -0.2587, -0.5468, -0.3927, -0.4366, -0.1873, -0.5066, 0.3108, -0.5783, 0.9075, 0.0089, 0.0258, -0.1351, 0.7038, -0.2022, -0.8455, -1.0124, -0.8408, -0.1551, -0.2004, -0.1756, -0.6076, -0.8564, -0.3496, -0.5517, 0.0383, -0.5405, 0.4054, -0.6611, 0.534, 0.7904, -0.3768, 0.2354, 0.3061, 0.3077, -0.744, -0.5235, -0.3916, -0.8452, 0.0666, -0.463, -0.6565, -1.5788, 0.4337, -0.9099, 0.1785, 0.585, -0.2845, 0.8582, 0.5665, 0.9334, -0.0045, 0.4724, -0.3679, 0.3383, -1.2919, -0.7204, -0.6669, 0.2815, 0.8991, -0.2302, 0.3318, 0.4513, 1.1981, -1.0155, 0.4928, 1.8009, -0.6184, 1.2603, 0.1501, 0.7278, -0.4163, 1.2417, 0.2176, 1.1859, -0.8553, -0.0914, -0.1166, 1.1634, 0.4804, -0.1433, 0.6958, 1.8329, 1.3189, -0.8632, 0.4702, 2.0319, -1.0389, 1.6713, 0.0083, 1.0389, -0.5953, 1.8693, 0.4066, 1.7675, -0.0481, 0.2737, 0.7573, 1.2662, 0.5055, -0.1606, 0.7973, 0.6524, 1.2543, -0.5501, 0.2251, 1.755, -1.467, 1.5717, -0.1806, 1.8804, 0.4315, 2.4289, 0.4554, 0.5236, -0.2149, -0.1924, 1.3758, 1.122, 1.6266, 0.9823, 1.3446, -0.7652, 1.1958, -0.5591, 0.1137, 1.3124, -1.8184, 1.4418, -0.3173, 1.7924, 0.8865, 2.4332, 0.8166, -0.2662, -0.562, 0.0859, 1.6889, 1.5556, 2.7208, 1.5615, 0.5819, -1.3567, 1.1359, -0.5905, -0.081, 1.1174, -1.9768, 1.1818, -0.6981, 2.0208, 1.3472, 2.083, 1.1038, 0.2363, 0.7494, 0.7436, 1.6638, 1.6625, 3.3878, 0.7327, -0.3559, -1.4503, 1.1545, -0.4326, -0.1093, 0.9687, -1.9952, 0.6675, -0.8422, 2.4277, 1.435, 2.1796, 1.1882, 0.4295, 0.7933, 0.3345, 1.7738, 1.7122, 3.6883, 0.7982, -0.2895, -1.6428, 1.1178, -0.4384, -0.115, 1.0592, -1.7205, 0.5812, -0.81, 2.1415, 1.1445, 2.3031, 0.8717, -0.2628, 0.1908, 0.2902, 2.4474, 2.1367, 3.7013, 0.2655, -0.5323, -1.377, 1.0709, -0.4802, -0.1723, 0.8128, -1.7192, 0.4819, -1.0615, 2.0514, 1.2051, 2.2957, 0.313, -0.6836, -0.033, 0.4169, 2.8504, 1.6155, 3.0938, -0.4046, -0.6363, -1.1795, 1.0368, -0.6906, -0.1637, 0.5578, -1.9146, 0.3461, -1.3132, 2.0007, 0.8724, 2.5853, -0.0028, -1.1487, -0.2587, 0.2322, 2.7946, 1.0164, 2.177, -0.8126, -0.2182, -1.4878, 1.0276, -1.0142, -0.0572, 0.5427, -2.2242, 0.0508, -1.4961, 2.3011, 0.5408, 3.0567, 0.0424, -0.9934, -0.6887, -0.7903, 2.2525, 0.5611, 2.1373, -0.7686, 0.7484, -0.4825, 1.0252, -1.0478, 0.1736, 0.5539, -2.1557, 0.1547, -1.5104, 2.5212, 0.3962, 3.3048, -0.0474, -0.1171, -0.0454, -1.2029, 1.2678, 0.3965, 2.3937, -0.1974, 1.225, 0.1345, 0.9363, -1.1107, 0.3619, 0.6437, -1.9605, 0.2997, -1.5956, 2.3545, -0.1498, 3.3173, 0.6266, -0.214, 0.0857, -1.0803, 0.6621, -0.0226, 1.6913, 0.3638, 1.1897, -0.4917, 0.7602, -0.9654, 0.3238, 0.6875, -1.3503, 0.6358, -1.1309, 1.7582, -0.9218, 2.5869, 0.9243, -0.3248, 0.1382, -0.5534, 1.1725, 0.1752, 1.3819, 0.4164, 1.1057, -0.6657, 0.3861, -0.545, 0.0789, 0.4126, -0.2702, 1.0379, 0.379, 1.4375, -1.0642, 0.8805, 0.7524, -0.7823, 0.3424, -0.2198, 1.3907, 0.2364, 0.6631, 0.4272, 0.1377, 0.3145, 0.0931, -0.0715, -0.226, 0.1465, 0.3025, 0.7084, 1.4934, 1.0627, 0.1501, 0.1079, 1.2686, -0.9539, 0.4395, 0.4365, 1.407, 0.3846, 0.3633, 0.7227, -0.4255, 0.6623, 0.0342, 0.0584, -0.3716, -0.0068, 0.8037, 0.3419, 1.7175, 1.0442, 0.1647, 0.282, 1.387, -0.5027, 0.6724, 0.6925, 1.091, 0.1417, -0.0113, 1.2953, 0.4388, 0.997, 0.0344, 0.2354, -0.4133, -0.3553, 1.1061, 0.2982, 1.766, 1.0268, -0.307, 0.3221, 1.6229, -0.3509, 0.6222, 0.8926, 0.7618, -0.2323, 0.0231, 1.0648, 0.2877, 0.7517, 0.0748, 0.2776, -0.572, -0.5934, 1.1956, 0.4174, 1.8387, 0.859, -0.2176, 0.2245, 1.6551, 0.3416, 0.824, 0.7349, 0.5896, -0.1029, 0.1325, 0.4715, -0.1372, 0.25, 0.3174, -0.467, -0.2527, -0.3555, 0.472, 1.6103, 0.8784, 1.7005, -0.1432, -0.0863, 1.1246, 0.7331, -0.2969, 0.6544, 1.003, -0.2949, 1.175, 0.0634, 0.3919, 0.3135, 0.7204, -1.4033, 0.0852, 0.3049, -1.1923, 2.5257, 0.6758, 1.8626, 0.1036, 0.1698, -0.124, 1.1631, -1.9092, 0.9121, 2.0496, 0.007, 1.4091, -0.2139, -0.1627, -0.2411, 0.918, -1.4992, 0.0675, 0.4802, -1.6444, 2.4365, 0.5895, 1.6954, -0.3762, 0.0817, -0.4783, 1.0068, -1.7801, 0.4402, 1.5199, 0.4891, 1.0175, -0.8148, -0.0054, -0.8314, 0.9299, -1.1095, 0.0419, 0.4019, -1.5647, 1.96, 0.1623, 1.4416, -0.0959, 1.0013, 0.1789, 0.4288, -0.8429, -0.136, 0.6473, 1.2283, 0.8659, -0.7563, 0.9492, -1.4416, 0.8857, -0.8842, 0.1051, 0.4686, -1.316, 1.6428, -0.6939, 0.8411, 0.5275, 1.4048, 0.7546, 0.228, -0.3548, -0.0743, 0.0844, 1.0231, 1.3733, -0.1368, 1.3514, -1.0874, 0.839, -0.709, 0.0882, 0.6325, -0.9121, 1.5662, -0.9832, 0.1807, 0.3023, 0.9762, 0.7513, 0.0708, -0.1139, 0.1095, 0.149, 0.5037, 1.1645, 0.631, 1.4643, -0.358, 0.7184, -0.6987, 0.0558, 0.6575, -0.3748, 1.56, -0.7391, 0.3349, -0.2161, 1.0657, 0.7344, 0.0179, 0.7085, 0.593, 0.4515, -0.2864, 0.7635, 0.2557, 0.7379, -0.0391, 0.5509, -0.6382, -0.0412, 0.3255, -0.0794, 1.3573, -0.3778, 1.2443, -0.0257, 1.2573, 0.9413, -0.5083, 0.6097, 0.3735, 0.2657, -1.1992, 0.5321, -0.17, 0.7989, 0.2348, 0.4595, -0.5936, -0.1815, 0.2953, 0.253, 1.5955, 0.0495, 1.8259, -0.2545, 0.4688, 0.8222, -0.8558, 0.5086, 0.6458, 1.7773, -0.6476, 0.7695, -0.0539, 0.9782, -0.1859, 0.4282, -0.55, -0.2026, 0.1204, 0.1198, 2.1208, 0.3974, 1.3161, -0.2901, 0.19, 0.9442, -0.7826, 0.2445, 0.9068, 2.0891, -0.527, 0.6839, 0.1426, 0.7382, -0.2419, 0.4275, -0.7375, -0.2605, 0.0438, -0.1604, 2.1792, 0.568, 1.0226, -0.2503, 0.5757, 1.1191, -0.3965, 0.7791, 0.9187, 1.7524, -0.2164, 1.1716, 0.4248, 0.6978, 0.3028, 0.7828, -1.9751, -0.0704, 0.7749, -0.9487, 2.6691, 0.3301, 1.2736, -1.0175, 1.3876, 0.8572, 1.205, 1.0406, 1.7447, 1.1407, 0.689, 1.1916, -0.1259, 0.216, -0.1071, 1.1169, -2.791, -0.0584, 1.0179, -2.0053, 2.8846, 0.1751, 2.024, -1.1597, 2.1699, 0.682, 1.6263, 0.6831, 1.6911, 0.4232, 0.7502, 0.8094, -0.5483, 0.5496, 0.2093, 1.2215, -3.0476, -0.1105, 1.2594, -2.4878, 2.9943, -0.1836, 2.2423, -0.9316, 2.3157, 0.4836, 1.7963, 1.0356, 1.7074, 0.1856, 0.6985, 1.0858, -0.5628, 0.1961, -0.5304, 1.24, -2.7465, -0.0613, 1.6541, -2.4573, 3.1562, -0.7829, 2.1492, -0.8381, 2.0419, 0.5485, 1.8965, 0.8066, 1.1647, 0.7294, 0.7657, 1.8918, -1.0361, 0.469, -0.4393, 1.3704, -2.4292, 0.0639, 2.105, -2.2832, 2.9903, -0.8747, 2.3786, -0.9775, 1.9981, 0.5227, 1.7551, 0.5165, 1.0316, 0.7965, 0.8904, 2.1656, -1.4983, -0.02, -0.3298, 1.4345, -2.4818, -0.0026, 2.1898, -2.125, 3.1973, -0.8566, 2.1745, -1.0027, 2.1937, 0.5167, 1.5877, 0.4169, 1.1916, 0.5955, 0.7622, 2.6377, -0.4135, 1.0515, -0.9504, 1.4113, -2.4314, -0.1712, 2.1283, -2.2673, 2.8893, -0.7412, 1.8878, -1.2291, 1.6104, 0.5261, 1.2417, -0.3404, 0.9571, 0.3244, 0.2616, 2.3766, -0.1118, 1.2462, -0.7539, 1.3136, -1.805, -0.1476, 1.8103, -2.2017, 2.1627, -0.3534, 1.8083, -0.6333, 0.5152, -0.4268, 0.3403, -1.1225, 0.3278, 0.0921, 0.452, 1.8701, -0.4417, 0.8272, -0.3961, 1.1669, -0.9936, -0.0113, 0.9989, -1.655, 1.7341, -0.0616, 1.5569, 0.0898, 0.8215, -1.3265, 0.2557, -0.3587, 0.411, 0.9068, 1.1375, 1.0554, -0.3523, 0.9726, -0.6031, 1.1196, -0.818, -0.1845, 0.5467, -1.5223, 1.5505, 0.2227, 1.5854, 0.2048, 0.6241, -2.0862, 0.0369, 0.216, 1.5964, 1.8019, 1.2701, 1.3295, -0.2858, 1.0933, 0.1399, 1.0085, -0.6676, -0.2742, 0.2835, -1.4929, 1.1673, -0.0374, 1.1193, 0.457, 0.6972, -2.5796, -0.3225, 0.2957, 2.2344, 1.9457, 0.5317, 1.4563, -0.314, 0.8121, 0.3129, 0.9114, -0.6389, -0.2788, 0.2326, -1.4535, 0.6356, -0.5295, 1.1574, 0.479, 1.0109, -1.3492, -0.4952, -0.466, 1.5711, 2.0233, 0.6899, 1.8986, -0.252, 0.6101, -0.0115, 0.8871, -0.4954, -0.1877, 0.3501, -1.1209, 0.4549, -0.4767, 1.1857, 0.3706, 0.9063, -0.2849, -0.0637, -0.8571, 1.1658, 1.5023, 1.0215, 2.5286, 0.2557, 0.97, -0.0453, 0.794, -0.3752, -0.1211, 0.1656, -1.059, 0.0446, -0.3074, 1.177, -0.0054, 0.927, 0.241, 0.4385, -1.0024, 0.8103, 1.7652, 1.6429, 2.4461, 0.0362, 1.3214, -0.167, 0.6369, -0.1824, -0.2391, 0.1648, -0.3331, 0.2767, 0.1753, 1.4241, -0.1809, 1.0464, 0.3744, -0.6759, -0.2605, 0.2133, 1.0882, 0.8271, 1.0721, -0.2883, 0.9667, -0.1169, 0.5459, -0.0777, -0.4593, 0.2071, 0.0647, 0.7385, 0.8362, 1.7758, 0.0955, 1.0004, 1.2392, -1.3337, 0.1551, -0.3478, 0.8827, 0.1627, 0.3446, -0.2653, 0.5751, -0.3425, 0.5441, -0.2321, -0.6239, 0.1502, -0.3049, 0.8682, 1.1465, 1.9078, 0.6049, 1.1321, 1.8494, -1.64, -0.4699, -0.1764, 0.4844, 0.0593, 0.1868, -0.4195, 0.7512, 0.2693, 0.566, -0.3602, -0.6163, 0.3236, -0.1078, 1.1125, 1.1391, 1.989, 0.2943, 0.7868, 1.5557, -1.7656, -0.5313, 0.2353, 0.627, 0.2594, 0.6627, 0.0946, 1.2597, 0.2794, 0.6124, -0.3621, -0.6076, 0.5665, -0.2989, 0.9543, 1.2528, 2.1683, 0.0758, 0.2325, 1.2862, -1.7963, -0.4933, -0.2621, 0.3356, 0.5928, 1.3377, 0.454, 1.3858, 0.05, 0.6516, -0.3926, -0.7027, 0.5912, -0.5815, 0.8717, 1.342, 1.9473, 0.161, 0.2718, 1.186, -1.6457, 0.0726, -0.5909, -0.4587, 0.7923, 1.7474, 0.3889, 1.5802, 0.0693, 0.7413, -0.4249, -0.5599, 0.8703, -0.6692, 1.0788, 1.3269, 1.6623, -0.2773, 0.2031, 0.0492, -1.5339, 0.2656, -0.6825, -0.5747, 0.0727, 2.0815, -0.1694, 1.3183, 0.1556, 0.9259, -0.5279, -0.4242, 1.081, -0.9036, 1.5089, 0.5782, 0.7443, -0.7759, -0.116, -1.4777, -0.7288, 0.0895, 0.061, 0.2107, -0.0234, 2.8985, -0.2839, 0.8452, -0.0362, 1.0188, -0.7889, -0.5518, 1.4656, -0.9425, 1.733, 0.0028, -0.0281, -0.6147, 0.4687, -1.2421, 0.3796, 1.014, 0.5179, 0.4278, 0.0355, 2.8434, -0.4965, 0.7904, -0.6138, 1.0841, -0.9049, -0.7519, 1.3437, -1.1382, 2.0242, -0.1576, -0.0108, -0.1782, 1.1228, -0.5368, 0.2164, 0.638, 0.0549, 0.5184, -0.0501, 3.076, 0.1078, 1.1005, -0.2756, 1.0945, -0.9892, -0.8458, 1.1266, -0.7926, 2.3496, -0.3633, -0.1893, 0.4628, 1.5414, -0.0005, 0.2209, -0.2539, 0.8661, 1.0774, 0.3633, 2.2256, 0.4879, 1.1496, 0.3481, 1.0527, -1.0324, -0.778, 1.0023, -0.7303, 2.422, -0.4655, -0.1523, 0.5452, 1.1009, 0.1888, 0.3828, -1.1077, 1.5603, 0.812, 0.6068, 1.7069, 0.2298, 1.5494, 0.665, 0.9336, -1.0262, -0.5919, 1.0408, -0.6783, 2.3882, 0.0555, 0.9049, 0.8348, 0.869, 0.2272, 0.6168, -1.7728, 1.1911, 1.0174, 0.6708, 1.9189, -0.0559, 1.4913, 0.5669, 0.713, -0.6804, -0.4457, 1.1061, -0.553, 1.468, 0.2886, 1.0782, 0.9407, 1.2071, 1.544, 0.5953, -1.0599, 0.6838, 0.8638, 1.0221, 1.7442, -0.1988, 1.6838, 0.9124, 0.3867, -0.0755, -0.5811, 0.6343, 0.1275, -0.1424, 0.7757, 1.1419, 0.9866, 1.1663, 1.9693, -0.9387, -1.0524, -0.2849, 0.5274, 1.2912, 0.8211, -0.199, 1.8315, 1.9112, 0.3099, 0.2161, -0.6344, 0.4752, 0.5974, -0.1253, 1.6082, 1.302, 1.2609, 1.0831, 1.9267, -1.2319, -0.6693, -0.1742, 0.0598, 1.4308, 0.9139, 0.3096, 1.2478, 1.4646, 0.3019, 0.0575, -0.6604, 0.3509, 0.6039, 0.2923, 1.5461, 1.1647, 0.743, 0.8553, 1.5028, -1.0384, 0.2058, 0.2602, 0.7791, 1.6296, 1.1526, 0.6645, 1.0497, 1.3305, 0.4082, -0.8149, -0.3843, 0.0724, 0.0776, 1.5285, 0.2824, 2.0891, -0.5037, 0.8366, 0.7441, -0.4518, 0.6026, 1.348, 1.5027, 2.2564, 1.4967, -0.7744, 1.1499, -0.6181, 0.6245, -1.0337, -0.226, -0.2935, -0.7091, 1.8139, -0.443, 2.6187, -0.3431, 1.5065, 0.8648, -0.1537, -0.7024, 1.1086, 1.2595, 1.9974, 2.7068, -0.5562, 1.2318, -0.779, 1.0086, -0.4053, -0.2057, 0.0565, -1.315, 0.985, -0.6399, 2.0592, -0.1148, 1.7533, 0.9811, 0.397, -0.9852, 0.0167, 0.5835, 0.8456, 1.4814, -1.5383, -0.5621, -1.2762, 1.1019, -0.3451, -0.242, 0.5071, -1.5232, 1.047, -0.5389, 1.7038, 0.0606, 1.5627, 0.513, 0.3233, -0.5221, -0.2258, -0.0573, 0.0916, 0.8813, -1.4996, -0.6382, -1.5138, 0.9522, -0.7923, -0.1548, 0.8616, -1.3872, 1.5692, -0.4021, 1.2438, -0.1795, 1.3196, -0.0236, 0.7061, -0.388, -0.2612, -0.5176, -0.0731, 0.9313, -1.7908, -0.6317, -1.5498, 0.8652, -1.1373, -0.2999, 0.92, -1.4545, 1.42, -0.3486, 1.2888, -0.5398, 0.635, -1.2796, 0.5213, -0.8132, -0.6735, -0.1301, 0.1142, 1.1026, -1.4129, -0.9189, -2.5099, 0.8487, -1.4136, -0.3351, 1.0148, -1.4503, 0.9551, -1.0401, 1.0618, -1.0025, 0.5773, -1.3386, 0.3195, -1.5163, -1.0512, -0.1745, 0.3191, 0.6663, -1.6095, -1.0947, -2.8747, 0.8687, -1.2357, -0.1405, 0.9538, -1.4752, 0.4909, -1.6566, 0.8341, -1.0401, 0.5866, -1.7472, 0.0738, -1.1501, -0.5471, 0.3609, 0.866, 1.0868, -1.3167, -0.7595, -2.6456, 0.8681, -0.8785, -0.1378, 0.4326, -2.2675, -0.6014, -1.4036, 0.8035, -0.2433, 1.2703, -1.8337, 0.2638, -1.3329, -1.2632, 0.1141, 1.0963, 1.8351, -0.2031, -0.2941, -2.1689, 0.8536, -0.7944, -0.4375, -0.484, -2.9656, -0.9228, -0.9231, 0.9515, 0.3625, 1.5924, -1.7286, 0.0412, -1.7567, -0.2907, 1.0799, 1.6403, 1.9903, -0.2148, -0.6888, -2.1676, 0.9291, -0.7554, -0.627, -1.0232, -3.0563, -0.3167, -0.9446, 1.4724, 0.3905, 1.2287, -0.7362, 0.3025, -1.6476, 0.2811, 1.6153, 1.2583, 1.0494, -2.0133, -0.3904, -1.7841, 0.9572, -0.8704, -0.7531, -1.4296, -3.2751, 0.0679, -1.4369, 1.9174, 0.2492, 0.7848, -0.5231, 0.0741, -1.1621, 1.0495, 1.8263, 0.5485, -0.1793, -2.647, -0.0365, -1.6391, 0.9499, -1.1288, -0.8709, -1.1592, -3.2446, -0.002, -1.335, 2.4761, 0.7638, 1.32, 0.003, -0.6166, -1.2612, 1.4561, 1.2878, -0.159, 0.0674, -1.7498, 0.9053, -1.5051, 1.0294, -1.3433, -1.1835, -0.8753, -3.4027, -0.2257, -1.1573, 2.7171, 0.8108, 1.2938, 1.2159, -0.3134, -0.6106, 1.8079, 0.0733, -0.8275, -0.4389, -1.5486, 1.8157, -1.8419, 1.0998, -1.5313, -1.4255, -0.5782, -3.7928, -0.6963, -1.5396, 2.1933, 0.8289, 1.1292, 1.6997, -0.1602, -0.3337, 2.0026, -0.5048, -0.9671, -0.2621, -1.5017, 1.7426, -1.0212, 1.1494, -1.5678, -1.4983, -0.144, -3.8286, -0.4773, -1.4193, 2.0579, 0.9243, 0.8698, 1.7328, -0.4218, -0.6549, 1.4584, -0.6366, -1.3604, -0.0554, -1.4161, 1.4507, -0.5539, 1.1537, -1.6509, -1.5533, -0.0249, -3.8607, -0.3311, -0.9902, 1.7675, 0.5039, 0.2434, 1.263, -1.1117, -1.172, 1.5389, -0.7873, -1.4873, 0.601, -1.5037, 1.2021, -0.1833, 1.1496, -1.6671, -1.4643, 0.1105, -4.0153, -0.3843, -1.2137, 1.581, 0.701, -0.1002, 0.675, -1.5944, -1.5843, 1.8834, -0.6102, -1.961, 0.4113, -1.7365, 0.8928, -0.3676, 1.078, -1.5552, -1.37, 0.4415, -3.8802, -0.2728, -1.2398, 1.345, 0.9692, -0.2734, 1.3945, -1.6713, -1.2207, 1.7115, -0.4082, -2.191, 0.3661, -1.6127, 0.6535, -0.709, 1.007, -1.5177, -1.3052, 0.6458, -3.9685, -0.3424, -0.9979, 1.4266, 0.7889, -0.3822, 1.9439, -1.3064, -0.9182, 1.4906, -0.9252, -2.1683, 1.028, -0.7495, 1.2096, -0.4421, 1.0244, -1.6156, -1.1664, 0.5954, -3.7872, -0.2865, -0.9373, 1.7916, 0.4916, 0.0913, 1.2397, -0.5858, -1.4275, 1.8856, -1.2504, -1.9856, 1.6734, -0.4246, 1.6068, -0.9052, 1.0256, -1.7495, -1.2065, 0.6595, -3.8224, -0.5289, -1.1692, 1.5639, -0.4421, -0.1027, 0.6638, -0.5848, -0.9379, 2.1965, -0.9199, -2.4276, 1.6514, -0.1238, 1.2383, -1.114, 1.0491, -1.7161, -1.2404, 0.6389, -3.9461, -0.9418, -1.5706, 1.4114, -0.6164, -0.1669, -0.1491, -1.0129, -0.6072, 2.311, 0.2139, -2.6592, 0.9681, -0.7585, 0.737, -1.2207, 1.0919, -1.3587, -0.9276, 0.3945, -3.8133, -1.1629, -2.1152, 1.0817, -0.7351, -0.0802, -0.1858, -0.8023, -0.6887, 2.4559, 0.996, -1.8734, 1.6107, -1.1968, -0.1378, -1.2812, 0.9378, -0.9844, -0.6245, 0.196, -3.3311, -1.1348, -2.1026, 0.7094, -0.6058, -0.1949, 0.1468, -1.0727, -0.6863, 2.6231, 0.892, -1.3076, 1.441, -1.6235, -1.1444, -1.7785, 0.478, -0.6313, -0.5552, -0.3356, -2.3756, -1.4277, -1.2409, 0.3962, 0.0663, -0.3062, 0.644, -0.4801, -0.3497, 1.6483, -0.1746, -1.2961, 0.7208, -1.9528, -1.1281, -1.313, 0.0164, -0.2299, -0.5522, -0.8183, -1.3597, -1.5575, 0.3054, 0.0819, 0.2742, -0.4037, 1.4251, 0.2598, 0.0572, 1.5813, 0.24, -0.2459, 0.184, -1.7743, -0.5226, -0.655, -0.1327, -0.1764, -0.6611, -0.8967, -0.8349, -1.8816, 0.7037, -0.0423, -0.1108, -0.6435, 1.174, 0.6622, -0.2261, 1.4022, 0.4501, 0.8606, 0.9868, -1.0612, -0.3406, -0.5667, -0.142, -0.3902, -0.4156, -0.7552, -0.7475, -1.6921, 0.0779, 0.2866, -0.6132, 0.139, 1.5286, 1.0276, -0.9628, -0.2139, 0.8272, 0.4333, 0.6173, -0.3868, -0.5355, -0.2098, 0.1914, -0.9787, 0.3323, -0.4388, -1.145, -0.07, -0.8416, 1.7963, -0.9069, 0.5568, 0.0984, 1.2245, -1.1446, -0.1134, 0.3606, -0.1381, -0.1016, -1.6699, -0.5585, -1.0814, 0.4046, -1.3144, 0.6172, -0.0351, -1.1555, 0.7345, -1.1937, 2.6598, -1.0855, 0.2283, -0.7545, 0.8462, -1.0034, 0.1573, -0.2624, 0.2123, 0.2655, -1.3317, 0.7657, -0.8447, 0.521, -1.5406, 0.5667, 0.251, -0.8178, 1.1933, -1.5732, 2.8022, -1.1441, 0.5963, -1.4129, 0.0116, -1.1693, 0.1371, -0.1437, -0.2644, -0.006, -0.8157, 0.7568, -0.8621, 0.6432, -1.6019, 0.1437, 0.0868, -0.6406, 1.4207, -1.3319, 2.1728, -1.4234, 1.0505, -1.4819, 0.0313, -1.131, 0.4272, 0.1465, -0.0166, 0.3285, -0.6844, -0.5684, -2.5495, 0.7153, -1.6739, -0.0921, -0.0777, -0.7419, 1.3762, -1.5011, 1.6878, -1.2268, 1.2994, -0.903, 0.7003, -0.9494, 0.4825, -0.0464, 0.1886, 0.8268, -0.4051, 0.1664, -1.6393, 0.7663, -1.7693, -0.3547, 0.0561, -0.5133, 1.9398, -1.2626, 1.3881, -0.5908, 1.7814, -0.7486, 0.1594, -1.5461, 0.0043, -0.0173, -0.1735, 0.8441, 0.1201, -0.3234, -1.7691, 0.8784, -1.7689, -0.6332, 0.462, -0.5597, 2.0103, -1.4796, 0.5421, -0.6909, 1.5299, -0.8057, -0.8359, -2.3705, -1.2046, -0.3972, -0.5748, 0.9159, 0.5752, -1.0318, -1.9677, 0.8983, -1.7385, -0.9096, 0.8523, -0.4657, 2.1201, -0.7965, -0.0209, -0.7335, 1.4284, -0.6259, -0.8641, -2.0812, -2.5055, -1.2845, -0.1861, 1.7847, 0.0649, -1.1564, -1.256, 0.884, -1.5193, -1.0025, 0.7573, -0.8046, 1.7275, -0.4904, -0.7556, -0.4651, 1.0564, -0.8977, -0.622, -1.1049, -2.7783, -1.8776, -0.5798, 1.1085, -0.5485, -1.5946, -0.8102, 0.9302, -1.3418, -0.9411, 0.6899, -1.3971, 1.3693, -0.3077, -0.9093, -0.4951, 0.5518, -0.9752, -0.5571, -0.1169, -1.9091, -1.6105, -0.7198, 0.349, -0.6788, -1.0451, -1.2582, 0.8982, -1.1415, -0.9425, 0.2532, -1.9814, 0.9834, 0.2555, -0.565, -0.6163, 0.0529, -1.4593, -0.8886, -0.1761, -1.4983, -1.6114, -1.1067, -0.0708, -0.6616, 0.3652, -0.6917, 0.8172, -0.9654, -0.9497, 0.0108, -2.0416, 0.7046, 0.3612, -0.5174, -0.8002, -0.723, -2.0731, -0.9734, -0.1033, -1.2492, -1.3738, -1.8637, -0.4405, -0.4947, 0.2572, -0.1617, 0.7021, -0.7039, -0.9183, 0.0122, -1.8668, 0.18, 0.4877, -0.3016, -0.8289, -0.8971, -1.4706, -0.8147, -0.7263, -1.445, -0.4014, -1.8984, -0.5211, -0.4147, 0.3842, 0.0709, 0.6215, -0.6404, -1.0085, 0.0985, -1.5681, -0.2446, 0.1951, -0.0121, -0.5375, -0.7333, -0.8247, -0.957, -1.4083, -1.6677, 1.1644, -1.0533, -1.0487, -1.4759, -1.0013, -0.8946, 0.6208, -0.7157, -1.1083, 0.3963, -1.363, -0.4249, -0.3323, 0.3376, -0.2861, -0.7593, -0.3283, -1.248, -1.21, -1.5282, 1.3861, -0.4627, -1.0371, -1.772, -1.1882, -0.4043, 0.5962, -0.9033, -0.9825, 0.3984, -1.4771, -0.5549, -0.8856, 0.3515, -0.6796, -1.1264, -0.4803, -0.9074, -0.8089, -0.8903, 0.973, 0.0947, -0.6522, -1.5072, -0.7432, -0.2997, 0.5182, -0.99, -0.7993, 0.6926, -1.4821, -0.7964, -1.1488, 0.3084, -0.5985, -1.331, -0.6379, -1.297, -1.6387, -1.4101, 0.2704, -0.3254, -1.0272, -1.8715, -0.7028, -0.0691, 0.4377, -0.8993, -0.6115, 1.054, -1.223, -1.1089, -1.4502, 0.3321, 0.0589, -1.6458, -0.7852, -1.3151, -1.6438, -1.5136, 0.2083, -0.1467, -0.9643, -1.9393, -1.1924, 0.082, 0.3638, -0.8708, -0.4762, 1.1074, -0.9985, -1.1651, -1.0137, 0.7796, 0.0419, -1.505, -0.508, -0.4811, -1.2247, -1.4744, -0.0679, -0.5329, -0.5582, -2.2587, -1.3313, -0.207, 0.3323, -0.7279, -0.3634, 0.9818, -0.9128, -1.3127, -0.6292, 0.9024, -0.2845, -0.964, 0.2242, 0.4162, -1.2575, -2.1096, 0.0283, -0.8175, -0.4614, -1.2532, -0.2664, -0.2633, 0.252, -0.5633, -0.4249, 0.6425, -0.8321, -1.5518, -0.6361, 0.8338, -0.1012, -0.8245, 1.0289, 1.0554, -1.1213, -2.0884, -0.0303, -0.7206, -0.704, -0.6549, -0.1895, -0.6008, 0.24, -0.4905, -0.4745, 0.4411, -0.804, -1.5397, -0.7546, 0.3593, -0.0825, -0.8292, 0.7274, 0.7254, -1.0197, -1.5681, -0.3942, -0.5479, -0.9626, -1.0677, -1.0266, -0.8532, 0.1481, -0.5636, -0.5184, 0.467, -0.4594, -1.3869, -0.8854, -0.1551, 0.1944, -0.1997, 1.0415, 0.8544, -0.6771, -1.1413, -0.6719, -0.7861, -1.9425, -1.4275, -0.8922, -1.0299, 0.0583, -0.6318, -0.4965, 0.8191, -0.1657, -1.1748, -0.5179, -0.2612, 0.478, 0.3322, 1.4132, 0.3333, -0.7879, -1.2791, -0.3743, -0.4269, -1.7837, -0.627, -0.1879, -0.8025, -0.0402, -0.5685, -0.4311, 0.8948, 0.0848, -1.1324, -0.0119, -0.2546, 0.2004, 0.2487, 1.3878, 0.2193, -0.9841, -1.7119, -0.7209, -0.4149, -1.2501, -0.2228, -0.1508, -0.5338, -0.0981, -0.5542, -0.3744, 0.7875, 0.0509, -1.1764, -0.0361, -0.1463, 0.2361, -0.3892, 1.1565, 0.6074, -0.901, -1.7718, -0.7515, -0.2364, -0.6447, -0.0376, -0.3933, -0.786, -0.0937, -0.5981, -0.3574, 0.4124, -0.2002, -0.9025, -0.0499, -0.1197, 0.1527, -0.5013, 1.3078, 0.676, -0.7352, -1.6446, -0.6945, -0.0287, -0.0655, 0.3957, -0.7282, -1.4444, -0.0369, -0.9064, -0.2124, 0.1033, -0.6864, -0.414, -0.5288, 0.3658, -0.7824, -0.3104, 0.9683, -0.4981, -0.5122, -1.8856, -0.3151, -0.282, 0.3385, 0.0701, -0.8612, -1.0825, 0.0406, -1.2674, -0.0079, 0.0499, -0.9346, 0.1433, -1.4372, 0.8355, -1.1363, -0.638, 0.3613, -0.3957, -0.1187, -2.3197, 0.0079, -0.3505, 0.758, 0.2329, -0.4601, -0.9519, 0.1939, -1.5786, 0.2373, 0.1494, -1.302, 1.0087, -1.7239, 0.9516, -0.8964, -0.2204, -0.388, -0.1136, -0.1283, -1.9415, -0.4791, -1.2839, 0.3711, -0.0819, 0.0216, -0.1974, 0.2827, -1.7884, 0.3248, 0.5472, -1.5589, 1.6268, -1.4222, 0.6314, -0.7214, 0.2945, -0.9331, -0.5108, 0.2714, -1.4586, -0.3343, -1.2593, 0.4191, -0.7681, 0.5456, 0.1629, 0.4358, -1.8801, -0.0815, 0.9101, -1.3042, 1.8785, -1.2209, -0.2368, -1.2951, 0.6731, -0.2481, -0.0137, 0.5088, -2.3657, -0.3688, -0.9194, 0.4583, -1.2061, 0.2644, -0.1161, 0.5047, -1.7059, -0.4396, 0.6983, -1.5096, 1.6277, -0.5186, -0.6421, -1.7401, 0.5987, -0.2731, -0.2444, 0.6409, -2.7141, -1.2129, -0.8761, 0.7195, -0.9582, -0.2086, -0.0441, 0.4309, -1.4576, -0.571, 0.5951, -1.35, 1.059, -0.1812, -1.0037, -1.5586, 0.4668, -1.2601, -1.0372, 0.4653, -2.5622, -1.0728, -0.5019, -0.2269, -1.1603, -1.3813, -0.6712, 0.3713, -1.2016, -0.6477, 0.4221, -1.0853, 0.4362, -0.0888, -0.9027, -1.2163, 0.5495, -1.8186, -1.3751, 0.5233, -2.2214, -0.5071, -0.1176, -1.2911, -1.1918, -1.0774, -0.6533, 0.4955, -0.5279, -0.2858, 0.6418, -0.249, 1.1465, 0.6185, 0.3524, 0.3629, 1.2604, -0.4631, -0.5756, 0.8205, -1.3431, 0.4319, 1.1408, -0.1462, -0.8864, -0.7293, -0.783, 0.5596, -0.145, 0.0255, 0.925, -0.1158, 1.3666, 0.6494, 0.5927, 0.9349, 0.9129, 0.1157, -0.3353, 1.2072, -0.8479, 0.7573, 1.0758, 0.3884, -0.3533, -1.0227, -0.9812, 0.6391, -0.8252, -0.083, 1.5046, -0.5022, 1.9328, 0.6798, 1.0682, 0.7129, 0.9163, 0.8418, 0.0053, 1.0717, -0.6134, 0.8804, 0.0219, -0.1502, 0.3817, -0.1656, -0.5294, 0.642, -1.2765, -0.4489, 1.2558, -0.887, 2.0963, 0.6628, 1.2333, 0.2264, 0.4692, 0.8743, -0.3197, 0.038, -0.7467, 0.5117, -0.3735, 0.8579, 1.039, 0.5151, 0.2708, 0.507, -1.4006, -0.6286, 0.7514, -0.9275, 2.0985, 0.3913, 0.5786, 0.1637, 0.6462, 1.124, -0.2495, 0.3497, -0.0924, 0.6043, 0.1181, 1.6224, 0.5115, 0.2592, 0.595, 0.482, -1.3684, -0.4673, 0.4803, -0.7723, 1.8154, -0.1162, 0.9594, 0.027, 1.0364, 1.3612, 0.2435, 0.3546, -0.1557, 0.8926, -0.0366, 0.6891, -0.2894, -0.126, 0.0673, 0.4723, -1.2254, -0.3731, 0.2975, -1.0271, 1.4558, 0.1648, 1.4099, 0.36, 0.9315, 0.8837, 0.192, -0.2271, -0.0921, 1.329, 0.064, 0.9391, -0.6965, 0.0151, 0.1328, 0.5113, -1.1927, -0.5049, 0.3014, -1.5644, 0.842, 0.0823, 1.2173, 0.3703, 0.7298, 0.1829, -0.2138, -0.3247, 0.707, 2.4412, 1.2804, 1.5252, -1.2064, 0.1172, 0.1351, 0.6936, -1.2442, -0.5773, 0.2195, -2.345, 0.6856, -0.5094, 0.8089, -0.1338, 0.0399, -0.5549, -0.5384, 1.3951, 2.6013, 3.0209, 0.817, 0.491, -2.2424, -0.0118, -1.3698, 0.842, -1.2809, -0.7588, -0.1247, -3.2217, 0.1994, -0.9811, 0.5735, -0.1248, -0.0891, -0.5991, -0.5361, 2.0129, 3.0148, 2.4306, -0.4408, -0.2113, -2.3352, 0.1717, -2.0229, 0.8071, -1.3862, -0.8728, 0.1748, -3.0119, 0.0164, -0.484, 0.7309, 0.1977, -0.2823, -0.3191, -0.1691, 1.29, 3.3594, 0.8109, -1.8963, -0.2551, -1.5091, -0.0567, -1.2085, 0.7054, -1.7037, -1.2665, 0.554, -2.3852, 0.398, -0.5157, -0.0119, -0.651, -1.1187, 0.6522, -0.1634, 1.3404, 2.7537, -0.1106, -2.2938, -0.9698, -1.146, -1.0348, -0.4634, 0.7443, -1.6677, -1.2534, 0.4875, -1.8818, 0.6438, -0.8664, -0.1796, -1.5933, -0.9617, 1.153, 0.1747, 1.5001, 1.7803, -0.0891, -1.6656, -0.2855, -1.0146, -0.3318, -0.3019, 0.7935, -1.5958, -1.0281, 0.5073, -1.3291, 0.9048, -1.2863, -0.4724, -2.1882, -0.3182, 1.3919, 0.5391, 0.5259, 1.1248, 0.405, -1.0279, -0.144, -0.8586, 0.6838, -0.4199, 0.7989, -1.7796, -0.8769, 0.7876, -1.0528, 1.4072, -1.5142, -0.8437, -2.4786, -0.1595, 0.8939, 0.4331, -0.6812, 0.2065, 1.7221, -0.9569, -1.0166, -0.4114, 1.3571, -0.3838, 0.5445, -1.8011, -0.5465, 0.5516, -0.7385, 1.8338, -0.8813, -0.7368, -2.0261, 0.1023, -0.047, 0.1573, -1.5872, -0.2747, 2.3519, -1.315, -2.3781, -0.283, 1.5822, -0.1509, 0.1164, -1.3578, -0.0722, 0.258, -0.4261, 0.9941, -0.5969, -0.8097, -1.3071, 0.4756, 0.2115, 0.4081, -1.3698, 0.107, 1.9156, -0.6706, -2.0445, -0.9774, 0.7552, -0.7594, -0.2603, -0.6344, -0.1861, -0.0593, 0.3339, -0.2842, 0.4533, -0.6865, -0.6758, -0.4781, 0.7677, 0.3443, -0.8597, 0.4172, 0.9634, -0.2831, -0.8871, -1.2932, -0.2274, -0.7121, -0.452, -0.4342, -0.2711, -0.3469, 0.8074, -1.2737, 0.6598, -0.2699, -0.1169, -0.1818, 0.4275, -0.1684, -1.1776, 0.0675, 0.2852, -0.5194, -0.2633, -0.5511, -0.2127, -0.7483, -0.5504, -0.387, -0.1709, -0.5297, 0.9472, -1.3618, 0.9323, 0.2017, -0.3973, -0.168, 0.1739, -0.6851, -1.0928, -0.1534, 0.0501, -0.4198, 0.0009, 0.1973, -0.1912, -1.2022, -0.245, -1.2782, 0.2933, -0.3085, -0.3487, 0.0543, -0.3635, 0.8278, -0.9186, -0.0295, -0.2491, -0.241, -0.9738, -0.2584, 0.3014, -0.856, 0.3607, -0.1191, -0.6067, -1.8064, 0.1395, -2.0819, 0.482, 0.269, -1.4654, 1.1979, -0.4079, 0.9474, -1.6339, 0.4995, -1.2482, -0.3021, -1.3023, -0.793, -0.6813, -1.4175, 0.026, -0.7824, -0.0792, -1.7989, 0.3741, -2.0744, 0.2077, 0.4296, -1.2255, 1.465, -0.6259, 0.3427, -2.1584, 0.7973, -1.5537, -0.056, -1.0676, -1.3599, -1.9092, -1.7154, 0.1794, -0.6281, 0.984, -1.375, 0.3911, -2.0725, -0.1231, 0.341, -1.0228, 1.424, -0.8175, -0.5168, -1.8793, 1.0316, -1.7217, 0.4155, -0.6761, -1.1477, -2.1298, -1.4311, 0.3695, -0.8817, 1.4692, -1.3296, 0.3781, -1.9972, -0.2263, 0.2052, -0.8218, 1.5764, -0.7035, -0.5007, -1.5323, 0.8502, -1.5626, 0.19, -0.6482, -1.1374, -1.9, -0.4535, 0.836, -0.4068, 1.4851, -1.3903, 0.3709, -1.7406, -0.243, 0.4313, -0.7748, 1.4108, -0.2421, -0.531, -1.4312, -0.4397, -2.0327, 0.0026, -0.6504, -0.5965, -0.8059, -0.5944, 0.4319, -0.3782, 0.6999, -1.4931, 0.3325, -1.6968, -0.0994, 0.7216, -1.2194, 0.7786, 0.0965, 0.0913, -0.8311, -0.6293, -1.5664, 0.4613, -1.3186, -0.3641, -0.3349, -1.3828, 0.1414, -0.1416, 1.1756, -1.1572, 0.015, -1.5108, 0.1688, 0.636, -1.1418, 0.4073, 0.5203, 0.3717, -0.275, -0.5765, -0.7981, 0.5707, -1.2365, -0.7203, -0.1554, -0.895, -0.0313, -0.8024, 0.2932, -0.5009, -0.4417, -0.6737, 0.174, -0.3739, -0.2922, 0.0112, 0.6651, -0.1668, 0.4913, -1.3141, -0.2877, 0.4517, -0.2605, -0.3194, -0.3779, -0.6659, 0.5257, -0.3602, -1.1897, 0.2136, -0.086, -1.2112, -0.2329, 0.0302, -0.9919, 0.4172, 0.8579, -0.0124, -0.0994, -0.4323, -0.2614, 1.1569, 0.3261, -0.7939, 0.7997, -1.3575, -0.2619, -0.4577, -1.3314, -0.3423, 0.4857, -2.3859, -0.7635, 0.9146, -1.8181, 1.1718, 0.4784, 0.8823, -0.5413, 0.3316, -0.618, 1.3976, 0.1175, -1.025, 0.715, -1.6284, 0.1554, -1.4528, -0.6529, -0.4567, 0.7121, -3.0568, -1.1168, 1.4431, -2.1262, 1.4192, -0.043, 1.4146, -0.7112, 0.7505, -0.3496, 1.7928, 0.1996, -0.6754, 0.8883, -1.1954, 0.8368, -2.0986, -0.3833, -1.1258, 0.9331, -3.0745, -1.1155, 2.2997, -1.8512, 1.8205, 0.015, 1.338, -1.1317, 0.819, -0.0947, 1.6198, 0.0079, -0.3815, 1.1543, -1.4192, 1.4498, -1.8175, 0.3957, -0.4489, 1.185, -2.3934, -0.4003, 2.6616, -1.7596, 2.1151, -0.2998, 1.4356, -1.0813, 0.937, 0.4536, 2.2519, 0.8128, 0.6062, 1.9907, -0.5878, 2.3954, -0.6435, 0.4399, -0.588, 1.183, -2.225, -0.0705, 2.4371, -2.0076, 2.171, -0.2807, 1.9549, -0.819, 1.1205, 0.2337, 2.0892, 1.6522, 0.5717, 1.4163, -0.2468, 2.8901, -0.2699, 0.5587, -1.2576, 1.1486, -2.3012, -0.1014, 1.8348, -2.3859, 1.8775, 0.0299, 2.5881, -0.7725, 1.6325, -0.5341, 1.5321, 1.7909, 0.6001, 0.8692, -0.5672, 3.0938, 0.3303, 0.8057, -1.9667, 1.0373, -2.4058, -0.2203, 1.2936, -2.4409, 1.5628, -0.2269, 2.5589, -1.0788, 1.6116, -0.8714, 0.657, 0.7104, 0.1907, 0.8373, -0.0998, 2.4659, -0.4576, 0.8494, -1.4759, 0.9554, -2.028, -0.196, 0.8893, -2.0835, 1.6993, -0.0699, 2.5004, -0.6432, 1.6871, -0.889, 0.3444, 0.6272, 0.0523, 1.076, -0.0117, 1.7077, -0.7338, 0.2177, -1.6865, 0.7057, -1.2279, -0.0711, 0.0994, -1.7138, 1.3179, -0.2246, 1.847, 0.0465, 0.8834, -0.9079, 0.7905, 1.1438, 0.0866, 1.1643, -0.2974, 1.1326, -0.5243, -0.2383, -1.0808, 0.5572, -0.9316, -0.0984, -0.6506, -2.027, 0.6802, -0.429, 1.5573, 0.8914, 0.0088, -1.6161, 0.6278, 0.7913, 0.6001, 1.7278, -0.0191, 0.6837, -0.8247, -1.1073, -0.1896, 0.6637, -1.0888, -0.2568, -0.7839, -2.358, 0.7732, -0.8207, 1.2156, 0.2548, -0.1832, -1.2692, 0.7464, 0.9509, 2.0497, 1.9202, -0.4425, 0.6725, -1.4756, -0.0466, -0.2445, 0.7952, -1.0209, -0.4141, -1.0122, -2.6657, 0.4534, -1.2735, 1.3232, -0.1395, 0.085, -0.8328, 0.3929, 1.553, 2.6415, 1.3905, -0.7983, 0.8691, -1.7987, 1.3365, -0.8494, 0.7606, -0.9308, -0.4892, -0.8964, -2.4671, -0.0122, -0.9868, 1.7783, -0.4169, 0.6316, 0.0588, -0.3119, 1.197, 2.3032, 0.87, -1.0596, 0.1975, -1.333, 1.6591, -1.7651, 0.7023, -1.1868, -0.7254, -0.3443, -2.5564, -0.5978, -0.3102, 2.0024, 0.0069, 1.0055, 0.8715, -0.4864, 0.2283, 1.5934, 0.126, -1.3148, 0.1153, -0.4123, 1.7103, -1.4413, 0.7369, -1.3342, -1.1298, -0.1479, -3.0067, -0.8245, 0.2568, 1.6844, 0.7116, -0.0584, 1.2734, -0.5257, -0.6419, 2.0566, -1.1273, -1.9478, -0.0332, -0.0873, 1.508, -0.9697, 0.6886, -1.3995, -1.2162, 0.1538, -3.1325, -0.395, 0.3151, 1.7551, 1.7501, -0.9277, 1.741, -0.9164, -0.5858, 2.5647, -1.0492, -1.9338, 0.394, 0.158, 1.3892, -0.3976, 0.629, -1.4929, -1.2175, 0.524, -3.1322, 0.1008, -0.0069, 1.6349, 2.0757, -1.6022, 1.8678, -1.1562, -0.3795, 2.647, 0.2446, -1.0205, 0.8774, 0.4509, 1.6184, -0.2065, 0.6167, -1.4888, -1.2978, 0.6273, -3.0994, 0.0132, 0.0704, 1.546, 2.2891, -1.5791, 2.3369, -1.089, -0.7791, 2.3331, 0.3659, -0.2754, 1.3283, 0.6898, 1.5212, -0.4434, 0.5906, -1.5686, -1.3939, 0.3693, -3.4249, -0.23, -0.0056, 1.4483, 2.5419, -1.6096, 2.3859, -1.0526, -0.7583, 2.3392, 0.2106, -0.1028, 1.5009, 0.8359, 1.4354, -0.8591, 0.6005, -1.5748, -1.4641, 0.2581, -3.4759, -0.0911, 0.1473, 1.5788, 2.903, -1.6792, 2.2916, -1.2329, -1.3452, 1.9576, -0.1689, -0.4778, 0.8906, 0.7853, 1.752, -0.3774, 0.6501, -1.3409, -1.3791, 0.5958, -2.9161, 0.2232, 0.9372, 1.7171, 2.7143, -2.2648, 2.026, -0.9312, -1.6446, 2.4649, 0.0987, 0.1119, 1.5594, 0.3282, 0.6362, -1.7912, 0.7862, -1.1131, -0.8795, 1.2135, -2.4376, 0.5056, 1.0711, 2.2019, 2.7391, -1.2906, 2.456, -0.0712, -1.2037, 2.9391, 0.426, -0.6094, 1.5696, -0.0184, 0.7033, -2.5061, 0.8421, -1.3531, -0.5166, 1.5884, -2.3167, 0.8332, 0.7193, 2.4517, 2.308, -0.3863, 2.5604, 0.274, -0.4482, 2.8715, 0.533, -1.8877, 0.8308, -0.4529, 1.6671, -2.1006, 0.838, -1.5509, -0.4575, 1.5144, -2.2809, 1.172, 0.211, 2.5308, 1.3904, 0.1037, 2.4182, -0.1115, -0.0661, 2.1688, 0.5663, -2.0002, 1.5473, -0.3019, 1.4417, -1.3761, 0.9041, -1.627, -0.4035, 1.2847, -2.1723, 0.9316, -0.3427, 2.4259, 0.3051, 0.3634, 2.023, 0.0736, 0.0796, 1.9971, 0.3619, -1.5038, 2.164, -0.5378, 0.3796, -0.5976, 0.9771, -1.5089, -0.4743, 1.0103, -2.0292, 0.634, -0.2846, 1.98, 0.14, 0.513, 1.2686, 0.6606, 0.3333, 2.4405, 0.3306, -0.9088, 2.1428, -1.4657, -0.0054, -0.9815, 0.8705, -1.1737, -0.2404, 0.7449, -1.8705, 0.2081, -0.4642, 1.3941, 0.3886, 0.7418, 0.7535, 1.0891, 0.5736, 2.7471, 0.5055, -0.9259, 1.1541, -2.1797, 0.3974, -0.9494, 0.5061, -0.8759, 0.2998, 0.1256, -1.7004, 0.066, -0.5718, 1.4053, 0.6322, 1.4486, 0.2969, 0.784, 0.8393, 2.3155, 0.973, -1.4994, -0.1935, -1.5392, 1.0161, -0.4521, 0.1589, -1.0478, 0.7879, -0.4618, -1.2216, 0.1934, -0.7244, 2.5022, 0.0293, 1.8658, 0.5856, 1.1716, 0.6581, 1.1615, 1.3461, -1.4682, -0.2212, -0.7792, 1.5464, -0.2603, 0.1935, -1.5569, 1.306, -0.3939, -1.2576, 0.9766, -1.4957, 2.8485, -1.0262, 1.3458, 1.183, 1.401, 0.7898, 0.0759, 1.434, -0.6183, 0.2676, -0.5156, 0.2661, -0.3536, 0.1766, -2.1522, 1.5792, 0.1684, -1.3378, 1.7947, -1.5405, 2.8611, -1.3765, 0.6175, 0.7131, 0.7255, 0.7096, -0.0155, 0.9741, -0.0936, -0.0799, -0.3954, 0.0358, -0.7839, -0.1447, -2.9117, 1.8119, 0.1233, -1.5185, 1.577, -1.4026, 2.1869, -0.7352, 0.6799, 0.822, 0.6832, 0.8725, 0.5947, 0.7186, 0.6839, -0.1894, -0.0293, -0.5788, -1.4034, 0.6407, -2.5146, 0.6516, -1.355, -1.7915, 0.1922, -1.122, 2.8472, -0.3966, 1.0265, 1.0195, -0.5222, -1.5263, -1.1271, 0.1105, 1.7791, -0.9474, -0.3797, -2.149, -1.6827, 1.2117, -1.9442, 0.3531, -0.655, -1.6744, 0.03, -1.0962, 2.538, -0.8205, 1.5819, 0.3389, -0.0004, -0.9002, -1.53, -0.8165, 0.8213, -1.2126, -1.5827, -2.7583, -2.5188, 1.2315, -1.7587, 0.1607, 0.1922, -1.796, 0.9789, -1.8319, 2.0507, -1.3029, 1.7023, -0.8467, 0.3031, -0.8308, -1.4845, -0.8789, -0.7075, 0.1451, -2.248, -1.565, -2.3432, 1.1709, -1.7047, -0.0611, 0.6172, -1.7757, 1.2306, -1.7483, 1.8723, -1.1752, 1.329, -1.5758, 0.0331, -0.9249, -0.7619, -0.1648, -0.7527, 0.6141, -1.8299, -1.0559, -2.2805, 1.1301, -1.3932, -0.4403, 0.6781, -2.0475, 0.6283, -1.0619, 1.1403, -1.1051, 0.1516, -1.759, 0.1457, -0.3415, -0.0778, 0.1802, -0.2102, 0.5528, -0.894, -0.7383, -2.6031, 1.139, -1.2923, -0.6929, 0.5274, -2.1104, 0.4654, -1.192, 0.3359, -1.6595, -0.5015, -1.266, 0.8097, 0.2545, 0.9276, 0.9271, 0.3728, 1.0299, -0.7395, -0.4662, -2.6697, 1.1555, -1.3075, -0.8065, 0.1669, -2.3295, 0.1831, -1.5722, 0.0425, -1.663, 0.0434, -0.9703, 1.6861, 0.8701, 1.5269, 0.2028, -0.2069, 1.4089, -1.2755, -1.1965, -3.7654, 1.1839, -1.2303, -0.7618, -0.3126, -2.4567, 0.0652, -1.7824, 0.7062, -1.9493, 0.0847, -0.8737, 1.7572, 1.3687, 1.6785, -0.2128, -1.1192, 1.0696, -1.0701, -1.2066, -3.687, 1.0594, -1.2768, -0.73, -0.5459, -2.6831, -0.1164, -1.2238, 1.5276, -1.2909, 0.9208, -0.0066, 1.8319, 1.3063, 0.6877, -0.9843, -2.2005, -0.093, -0.7379, -0.6541, -2.7838, 0.9307, -1.2508, -0.832, -0.5707, -2.822, -0.5216, -1.1957, 1.1855, -0.4279, 1.198, 0.1645, 0.9938, 0.4305, 0.1441, -1.1055, -1.6773, 0.1659, -0.2274, 0.2282, -2.1347, 0.8719, -1.3139, -0.8113, -0.2504, -3.1782, -0.6359, -1.4168, 0.6588, 0.0961, 0.7115, 0.5308, 0.2226, -0.3579, 0.1676, -1.396, -1.6737, 0.4176, -0.8349, 0.9265, -1.1514, 0.922, -1.6312, -0.8638, 0.1111, -3.6526, -0.1624, -1.2048, 1.2207, 0.2102, 0.4977, 1.1275, -0.6938, -0.5641, -0.5089, -2.1113, -2.4103, 0.4902, -0.3436, 1.6, -0.1082, 0.9355, -1.8576, -1.057, 0.1081, -3.7844, 0.2426, -1.2451, 1.7355, 0.3548, 0.3576, 1.4187, -0.9059, 0.5165, 0.0401, -1.278, -2.3544, 0.9867, 0.0433, 1.3575, -0.0615, 0.9352, -1.9627, -1.0763, 0.1299, -3.5576, 0.6806, -1.3212, 1.8675, 0.1556, 0.8591, 1.5172, -0.63, 0.3803, 0.6042, -0.8923, -3.2791, 0.557, -0.5022, 1.3332, -0.5495, 0.9565, -1.9304, -1.1061, 0.0621, -3.2179, 0.5922, -1.4424, 1.2052, -0.4939, 1.0316, 1.3879, 0.1056, 0.6687, 1.3649, -0.7404, -2.8534, 0.3819, -0.9772, 0.8428, -1.0396, 0.936, -1.8968, -1.1537, -0.1717, -3.2716, -0.0221, -1.4586, 0.7513, -1.4296, 0.9405, 1.0569, 0.2075, 1.2481, 1.5597, -0.7348, -2.3486, -0.3304, -0.9062, 0.2924, -0.6668, 0.9054, -1.5766, -1.1489, -0.1199, -3.1121, -0.1578, -0.6281, 0.1759, -1.7019, 0.604, 0.6388, -0.1989, 1.0204, 1.2532, -1.4715, -2.5294, -0.4621, -0.8189, 0.1465, 0.1382, 0.9003, -1.3545, -0.939, 0.1795, -3.0248, -0.0711, -0.3431, -0.8752, -1.9996, 0.4199, 0.1907, -0.3406, -0.0634, 0.9798, -1.7811, -2.8021, 0.2466, -0.4483, 1.2884, 0.4712, 0.9043, -1.357, -0.6067, 0.5195, -2.934, 0.1407, -0.8753, -0.7731, -1.3934, 0.5516, -0.1382, -0.3509, -0.0766, 0.6542, -1.3562, -2.8634, 0.0819, -0.983, 1.3728, 0.8436, 0.8631, -1.4815, -0.6352, 0.6724, -2.6402, 0.5109, -0.6922, -0.4817, -1.1382, 0.3354, -0.6703, -0.094, 0.1379, 0.0316, -1.4467, -2.7335, 1.265, -0.5958, 1.3371, 0.567, 0.6832, -1.5788, -0.5281, 0.5591, -2.2396, 0.5978, -0.4646, -0.5515, -1.7098, -0.5502, -1.1639, -0.1979, -0.5121, -0.2856, -1.988, -2.5884, 1.6746, 0.4247, 1.3173, -0.3628, 0.3869, -1.2468, -0.1246, 0.2545, -1.999, 0.2346, -0.687, -0.6137, -0.9567, -0.8281, -0.4516, -0.1571, -1.776, -0.6134, -1.3301, -1.476, 1.1995, -0.2474, 0.2773, -0.5779, -0.0087, -0.8136, -0.0397, -0.0759, -1.459, -0.599, -0.1634, 0.0053, -0.3273, -0.8044, 0.4696, -0.7696, -2.2287, -1.7858, -0.7565, -0.5483, 1.1027, -0.2803, -0.7604, -0.7059, -0.2901, -0.4092, -0.2377, -0.382, -0.6093, -1.5884, 0.7663, 0.3223, 0.4006, -0.4895, 1.0767, -0.3239, -2.3117, -1.6769, -0.9458, 0.32, 1.0407, -0.2813, -1.2145, -1.4884, -0.2541, -0.858, -0.3838, -0.1474, -0.4951, -0.8464, 0.6928, 0.5248, 0.4641, -0.6464, 0.663, -0.4678, -1.4317, -1.0451, -0.1746, 0.2297, 1.0048, 0.1969, -1.5504, -2.0671, 0.0452, -1.4901, -0.5693, 0.0684, -0.8502, -0.2243, -0.166, 0.8089, 0.4032, -0.1992, 0.0605, -0.9232, -1.5233, -0.4613, 0.0125, -1.0621, 1.0656, 0.8037, -1.1305, -2.382, 0.2832, -2.2414, -0.2747, 0.188, -1.3623, 0.5161, -1.0161, 1.3016, -0.2578, 0.8269, -0.3201, -0.419, -1.9268, -1.2283, -0.2512, -1.1192, 1.2487, 0.2194, -0.3998, -2.4967, 0.4608, -2.6984, -0.1386, 0.4531, -1.7694, 1.1161, -1.4462, 1.5425, -1.0273, 0.9123, -0.9144, -0.161, -2.0875, -1.5841, -0.095, -1.0552, 0.9754, -0.1135, 0.0914, -2.4315, 0.6062, -2.5833, -0.2265, 0.5913, -1.5209, 1.58, -1.504, 1.4803, -1.2603, 0.6731, -1.6699, -0.6713, -1.7211, -1.1904, -0.1904, -1.5521, 0.2164, -0.7676, -0.1009, -2.1482, 0.5409, -2.1807, -0.2239, 0.3874, -1.2428, 1.5541, -0.9679, 0.8829, -1.2498, 0.5322, -1.4248, -0.4709, -1.9212, -1.4041, 0.0332, -1.3502, -0.5931, -1.9788, -0.5174, -1.0231, 0.3873, -1.4483, -0.2177, -0.3935, -1.4248, 0.9063, 0.0065, 1.1835, -0.5696, -0.0999, -1.376, -0.3794, -1.3761, -1.1991, 0.1008, -1.3818, -0.4921, -2.2098, -1.7786, -1.0252, 0.3945, -1.1028, -0.4148, -0.7282, -1.6348, 0.5535, -0.309, 1.1226, 0.4897, -0.2219, -0.723, -0.2627, -0.675, -0.7532, -0.1494, -1.3772, -0.3496, -2.0643, -2.4655, -1.6828, 0.3768, -1.1893, -0.5542, -0.4633, -1.5051, 0.5303, -0.75, 0.1363, 1.12, 0.1289, -0.3542, 0.4927, -1.1136, -1.8492, -1.0761, -1.1134, 0.2206, -1.0588, -2.1188, -2.2032, 0.4835, -1.6556, -0.673, -0.1338, -2.033, 0.7735, -1.4123, 0.0919, 0.4227, 0.7648, 0.2773, -0.7726, -1.5012, -2.3359, -1.4133, -1.3555, 0.674, -0.1572, -1.465, -1.9194, 0.6275, -1.9564, -0.7689, 0.2637, -2.2679, 0.836, -1.6332, 0.3473, -0.1955, 1.1395, 0.7058, -0.8196, -1.6215, -1.7267, -0.7041, -0.9141, 1.2708, 0.2351, -0.5137, -1.1834, 0.5522, -1.9702, -0.923, -0.0212, -2.119, 0.9356, -1.5954, 0.0149, -0.3324, 1.1226, 1.126, -0.1331, -1.0137, -1.5774, -0.9312, -0.8611, 1.521, 0.7971, 0.7871, -0.0973, 0.4447, -1.62, -0.9546, -0.1945, -1.7888, 1.1339, -1.1192, -0.2717, 0.3644, 0.7523, 0.9273, -0.7422, -0.9921, -1.4899, -0.3898, -1.0727, 0.6556, 0.2522, 0.3807, 0.0097, 0.331, -1.4615, -1.0205, -0.307, -1.4544, 0.9265, -0.8842, -0.2413, 0.6494, 0.0263, 0.5015, -0.9995, -1.2073, -1.3203, 0.0047, -1.204, 0.4128, 0.0214, -0.2729, 0.372, 0.2147, -1.3743, -1.0393, -0.5472, -1.0837, 0.6128, -0.995, -0.2301, 0.5296, -0.2916, 0.5842, -0.9805, -2.0282, -1.9303, -0.4539, -1.5663, 0.4563, 0.1479, -1.1658, 0.2872, 0.1972, -1.0531, -0.9536, -0.7753, -0.919, 0.2669, -0.7934, -0.0008, 0.4026, -0.6713, 0.5723, -1.4433, -1.9755, -1.6209, -0.4943, -1.115, 0.3345, -0.0498, -1.8131, 0.3565, 0.1685, -0.9199, -1.0218, -0.6324, -0.6691, -0.2252, -0.0106, 0.1317, -0.0531, -1.1067, 0.6707, -1.2559, -2.0214, -1.5982, 0.0864, -0.1759, -0.1797, -0.1785, -1.219, 0.4953, 0.3033, -0.9233, -1.0931, -0.3376, -0.2556, -0.5276, 0.1654, 0.2592, -1.5114, -1.6218, -0.4477, -0.9206, -1.8623, -0.99, 0.6432, -0.3301, 0.2709, -0.5268, -0.1658, 0.2961, 0.5444, -1.1192, -1.2271, -0.3669, -0.5818, -0.3492, -0.3408, 0.6882, -1.6816, -1.9394, -1.4241, -0.66, -1.543, -0.2712, 0.9927, -0.6655, 1.2691, -0.215, 1.014, 0.4122, 0.638, -1.6128, -1.4122, -0.3915, -1.4251, -0.2042, -0.9589, -0.0219, -1.7164, -1.5302, -1.5257, -0.492, -1.3844, -0.8556, 0.6263, -0.8502, 1.8537, -0.3088, 1.0517, -0.0247, 0.6813, -1.8273, -1.348, 0.0375, -1.5816, -0.2103, -1.4816, -0.4702, -1.8213, -1.1018, -1.4691, -0.3679, -1.5325, -1.2999, 0.145, -0.9034, 2.1471, -0.6063, 0.5355, -0.8994, 0.6814, -1.7581, -1.2489, 0.0245, -1.4856, 0.1534, -1.299, 0.0663, -1.5119, -0.8706, -1.4415, -0.1622, -1.1843, -0.9911, 0.3615, -0.5903, 2.3614, -0.6843, -0.1758, -1.7648, 0.6363, -1.8366, -1.1415, -0.0293, -1.5683, 0.2999, -1.5485, 0.774, -1.1379, -0.8951, -1.0466, -0.3993, -0.805, -1.0174, 0.007, -0.418, 2.6317, -0.3539, -0.6979, -2.5077, 0.6746, -1.9826, -1.0934, 0.1235, -1.6939, 0.3803, -1.7098, 1.1618, -1.2108, -1.0056, -0.6711, -0.4335, -1.0678, -1.5623, -0.0442, -0.3013, 2.9714, -0.2464, -0.8257, -1.8832, 0.7078, -2.0716, -1.0643, 0.2888, -1.6401, 0.4571, -1.9071, 1.2952, -1.5301, -0.7351, -0.7827, -0.4627, -1.1248, -1.7946, 0.359, -0.4515, 2.8585, -0.7021, -0.9506, -1.6948, 0.7086, -2.1567, -1.0008, 0.1923, -1.7908, 0.5055, -2.1002, 1.1807, -1.851, -0.1429, -0.7424, -0.0394, -0.5382, -1.5465, 0.252, -1.0168, 2.531, -1.1424, -0.9327, -1.752, 0.6695, -2.1922, -0.9708, 0.1044, -1.7185, 0.6082, -2.0494, 0.6616, -2.018, 0.5184, -0.6245, -0.1727, -0.4335, -0.8631, 0.6043, -1.4641, 2.7393, -0.9711, -0.4872, -1.7565, 0.6535, -2.0131, -0.8731, 0.2855, -1.6113, 0.3683, -1.9167, 0.3431, -1.7707, 1.0396, -0.6222, -0.2656, -0.4881, -0.7843, 0.4566, -1.5716, 2.7665, -0.8623, 0.0769, -1.3441, 0.5246, -2.1462, -0.8344, 0.3943, -1.4152, 0.3801, -1.6971, 0.3995, -1.5987, 0.6809, -1.1449, 0.4656, -0.3859, -0.7802, 0.812, -1.4846, 2.9507, -0.3494, -0.0693, -1.9132, 0.4651, -1.9468, -0.7838, 0.1717, -1.3255, 0.3296, -1.8752, -0.0346, -1.7425, 0.3326, -1.5275, 0.9019, -0.467, -1.0067, 1.6193, -1.7868, 2.9657, -0.1525, -0.4633, -1.418, 0.4556, -1.8127, -0.835, 0.0699, -1.4195, -0.1177, -2.0363, -0.2466, -1.8593, 0.9534, -1.3213, 0.4829, -0.4811, -0.7846, 1.4809, -1.9449, 2.9611, -0.0339, -0.1847, -1.0833, 0.3977, -1.7498, -0.6179, 0.0525, -1.4344, -0.0236, -1.8173, -0.3623, -2.1415, 1.0192, -1.4751, -0.0648, -0.6147, -0.5171, 1.0033, -1.6512, 2.2228, 0.1462, 0.0475, -1.4299, 0.4322, -1.6345, -0.4453, 0.124, -1.4242, 0.1258, -1.9348, -0.6703, -1.8895, 1.0538, -1.4802, 0.0541, -0.9372, -0.8564, 0.6188, -0.7037, 2.0343, -0.071, 0.4004, -1.9418, 0.4802, -1.5925, -0.5326, -0.0487, -1.6719, 0.3291, -1.9141, -0.6612, -1.5367, 0.2392, -2.4723, 0.0484, -0.4949, -0.7546, 0.502, -1.2606, 1.57, 0.3291, 1.0777, -1.0122, 0.4826, -1.6502, -0.5037, -0.1685, -2.114, 0.6463, -1.9356, -0.461, -1.5152, -0.7074, -2.859, -0.034, 0.4942, 0.0511, 1.3537, -1.0662, 0.963, -0.2933, 1.2762, -0.0245, 0.4278, -1.5925, -0.3324, -0.4482, -2.4546, 0.6377, -1.9469, -0.179, -1.0252, -0.7024, -3.0492, -0.6136, 0.805, 0.1784, 1.8179, 0.181, 1.5076, -0.3722, 1.0383, 0.4954, 0.2924, -1.4979, -0.1531, -0.4377, -2.492, 0.002, -2.0541, -0.0923, -0.1561, -0.4112, -3.0427, -1.9888, 0.484, -0.327, 1.8215, 1.3112, 1.9645, -0.0372, 0.6466, -0.1179, 0.1411, -1.4603, -0.0211, -0.6066, -2.2816, -0.4216, -2.2141, 0.0308, -0.5123, -0.261, -2.1558, -1.5757, -0.0474, -0.9652, 1.4248, 0.9949, 1.6003, -0.1351, 0.242, -0.4661, 0.2313, -1.1118, -0.0226, -0.6113, -1.5108, 0.7741, -1.0539, 0.4569, -0.2956, 0.5473, -0.2504, 0.297, 0.2883, -0.6391, 0.5232, -0.2711, 0.7815, -1.0119, 0.3902, 0.1216, 0.3944, -0.3431, 0.0884, 0.1425, -0.3943, 0.9493, -0.2778, 0.7531, 0.8256, 0.9593, 1.0293, 1.2086, 0.8412, 0.3264, 0.543, -0.0574, 0.8592, -1.2785, 0.0876, -0.1489, 0.389, -0.4746, 0.1487, 0.7046, -0.2915, 1.373, -0.0474, 0.8096, 1.0818, 0.9164, 1.4448, 1.0654, 1.7011, 0.4046, 1.2384, 0.649, 0.8581, 0.0385, -0.2801, -0.2601, 0.5285, -1.6188, -0.174, 1.2533, -0.5986, 2.3994, 0.2973, 1.2385, 0.3467, 1.0357, 1.7614, 1.0611, 1.4134, 0.5239, 1.3174, 0.132, 0.6438, 0.6472, 0.4592, 0.4378, 0.5122, -1.9818, -0.6444, 1.1242, -1.0217, 2.6209, 0.2819, 1.1013, 0.1201, 1.0944, 1.4489, -0.075, 0.5465, -0.1718, 0.7651, -0.4262, 1.5688, 0.6201, 0.3987, -0.0218, 0.3777, -2.0874, -0.7939, 0.8379, -0.9205, 2.5866, 0.1271, 0.8687, 0.7353, 1.4552, 1.7066, -0.0015, 0.7512, -0.2711, 0.4819, -0.4063, 1.2591, -0.2283, -0.3767, -0.2648, 0.2887, -1.9108, -0.7166, 0.6949, -0.809, 1.9354, -0.0002, 1.2268, 0.5484, 1.233, 2.0762, 0.3192, 1.3202, 0.1859, 1.4336, -0.3403, 0.7936, -0.5539, -0.6088, -0.0571, 0.2254, -1.6498, -0.6639, 0.39, -0.6644, 1.5695, 0.47, 0.9911, 0.4111, 0.515, 1.2162, -0.1383, 0.2174, 0.4828, 1.7156, 0.5563, 1.5596, 0.1127, -0.0818, 0.2956, 0.1403, -1.5439, -0.6196, 0.0229, -0.5215, 1.4298, 0.3614, 0.9448, 0.4739, 0.1843, 1.1715, 0.174, 0.2576, 1.4021, 1.3726, 0.6558, 1.0058, -0.8137, -0.3869, -0.1743, -0.0132, -1.7541, -0.7623, -0.4193, -0.3036, 1.4325, 0.3916, 1.0292, 0.4436, 0.0994, 0.7388, 0.9286, 0.5335, 1.3562, 1.1234, 0.0354, 0.4358, -1.2766, -0.5159, -0.2953, -0.0649, -1.6435, -0.5875, -0.628, -0.7984, 0.8109, 0.4808, 1.0232, 0.4663, -0.1579, -0.4077, 1.0219, -0.0108, 0.7364, 1.2863, -0.0563, 0.4812, -0.9524, -0.3164, -0.29, -0.1869, -1.6642, -0.1781, -0.24, -0.9793, 0.0932, 0.5735, 1.0192, 0.4628, -0.013, -1.0234, 0.7954, -0.4241, 0.6173, 1.1446, 0.0609, 1.0875, -0.6355, -0.1496, -0.7713, -0.3481, -1.9038, -0.1304, 0.2979, -0.3328, -0.196, 0.1932, 0.6703, -0.6535, -0.3854, -1.5877, 0.3043, -0.6069, 0.4644, 0.8949, 0.0215, 1.3145, -0.252, -0.0229, -0.9344, -0.3465, -1.8168, -0.119, 0.3296, -0.0754, 0.0746, 0.1626, 0.8297, -0.6153, -0.1917, -0.6072, 0.8027, -0.6673, 0.4238, 1.2299, -0.6317, 0.6314, -0.1752, 0.5908, -0.2426, -0.3641, -1.7651, -0.198, 0.0761, -0.1308, -0.1458, 0.1657, 0.8899, 0.1128, 0.2783, -0.3957, 0.5439, -0.808, 0.3782, 0.859, -1.027, 0.9873, 0.1864, 0.9685, -0.0129, -0.4761, -1.6025, -0.055, 0.0558, -0.0651, -0.4811, 0.134, 0.6191, 0.0191, 0.0996, -0.3312, 0.3399, -0.5409, 0.9758, 0.9659, -0.6106, 1.3742, -0.0263, 0.1609, -0.4046, -0.5613, -1.4181, 0.0876, 0.0924, -0.0935, -0.5894, 0.5861, 0.8391, 0.1135, -0.3195, -0.7578, 0.0493, -0.6943, 0.8268, 1.2798, -0.4827, 1.4191, -0.323, 0.1198, -0.6637, -0.5847, -1.3189, -0.088, 0.0113, 0.2189, -0.4166, 0.8556, 0.8058, -0.4251, -0.9557, -0.3114, -0.2713, -0.8843, 0.7361, 1.6854, 0.3174, 1.0722, -0.6162, 0.006, -0.9192, -0.6047, -1.0666, -0.1786, -0.2478, 0.4597, -0.7523, 0.7537, 0.6118, -0.5948, -0.8041, 0.3789, -0.0386, -0.8229, 0.3378, 0.9468, 0.3323, 0.8707, -0.5184, -0.0276, -0.4443, -0.6749, -0.8883, -0.302, -0.5425, 0.5643, -1.1409, 1.0362, 0.8503, -0.3006, -0.7136, 0.401, -0.425, -1.4626, -0.3317, 0.3344, 0.6286, 1.403, -0.0019, 0.1054, -0.2468, -0.7001, -0.7323, -0.2393, -0.5857, 0.5902, -1.3982, 1.3133, 0.706, -0.5792, -0.3637, 0.568, -0.2413, -1.101, -0.373, 0.1312, 0.5257, 0.9228, 0.3269, -0.24, -1.1497, -0.7316, -0.7132, -0.1787, -0.6771, 0.7321, -1.3458, 1.5843, 0.8427, -0.8187, -0.5972, 0.3523, -0.4352, -1.2535, -0.0796, 0.4224, 0.4536, 1.1542, 0.6534, -0.5216, -1.2708, -0.8085, -0.7707, -0.054, -0.6141, 0.6879, -1.332, 1.7002, 1.0604, -0.6178, -0.8655, -0.0296, -0.3001, -0.8607, -0.4608, -0.2386, 0.1817, 1.3041, 0.7808, -0.5378, -0.9656, -0.7989, -0.6221, 0.0228, -0.6228, 0.5768, -1.6094, 1.3545, 1.0325, -0.5526, -1.1446, -0.1132, 0.6445, -0.272, -0.6755, -0.3768, 0.1208, 1.077, 0.6081, -0.7483, -0.8028, -0.7914, -0.7001, 0.0295, -0.6658, 0.6335, -1.3146, 0.554, 0.0613, -0.7819, -0.8551, 0.3629, 1.0469, -0.6031, -0.6622, -0.0469, 0.3287, 1.4662, 1.0594, 0.0559, -1.0085, -0.4549, -0.8005, 0.3181, -0.4429, -0.2534, -0.3985, 0.6304, 0.2976, -0.3874, -0.5318, -1.0686, 0.005, -1.6576, -0.9694, 0.8374, 0.1785, 0.7689, 0.0373, -0.3239, -1.1571, -0.1155, -0.9788, 0.2501, -0.6416, -1.2364, 0.5311, 0.5941, 0.3057, -0.1407, -0.9051, -2.1199, -0.5279, -1.4416, -0.6566, 0.3683, -0.0042, 0.375, -0.9626, -0.3633, -0.9651, 0.0168, -1.3501, 0.0215, -0.6047, -0.9389, 1.2141, 0.0364, 0.141, -0.3111, -1.2148, -2.0148, -0.3908, -1.5153, -0.1999, -0.2454, -0.279, 1.1546, -0.6785, -1.5207, -1.1065, 0.0105, -1.7769, -0.1614, -0.3624, -0.5297, 1.4298, -1.0077, -0.4739, -0.7832, -1.191, -1.4994, -0.1812, -1.5812, -0.7596, -0.8429, -0.1342, 1.3657, -0.3989, -2.0141, -1.0863, 0.0379, -1.7504, -0.3161, -0.2695, -0.1869, 1.5135, -1.3009, -0.4755, -0.7098, -0.5849, -1.5456, -0.2932, -0.8481, -1.1459, -0.4826, -0.3021, 0.9132, -1.2032, -2.0254, -0.3637, 0.0546, -1.4407, -0.4121, -0.526, 0.1725, 1.3744, -0.7727, 0.5157, -1.0472, -0.8075, -1.2055, -0.0979, -0.1549, -0.9341, -0.0158, -0.536, 0.2153, -1.6607, -1.4256, -0.1553, -0.0463, -0.913, -0.728, -1.0906, 0.4708, 0.6568, -0.2728, 0.6065, -1.3757, -0.8091, -0.4996, -0.3884, 0.2058, -0.7505, 0.2179, -0.5516, 0.5259, -0.7995, -0.6232, -0.3023, -0.2185, -0.4793, -1.1821, -1.358, 1.2834, -0.1834, 0.5609, 0.1159, -1.03, -0.3973, 0.0312, 0.0045, 0.4768, -0.7374, -0.5324, -0.7081, 0.2242, -0.4571, -0.9142, 0.1131, 0.235, -1.0203, -0.3144, -0.6867, 0.9507, 1.3584, -0.6706, 1.5678, -1.1305, 0.7539, 0.1786, 0.9245, 0.4348, 0.3453, 0.1699, -0.1719, 1.2949, -0.5116, 0.0053, -0.4158, 0.7239, -1.07, 0.072, 0.571, 0.0042, 1.833, -0.6888, 1.2766, -0.5316, 1.3908, -0.2523, 1.1574, 0.4747, 0.5974, 0.9294, 0.3901, 1.5786, 0.0222, 0.537, -0.7997, 0.9239, -1.2614, -0.1372, 1.2451, -0.707, 1.9731, -0.4552, 1.0661, -0.204, 1.6376, 0.5038, 0.721, 0.4852, 0.2163, 0.3883, 0.1258, 1.5422, 0.8701, 0.9868, -1.0119, 0.9468, -1.208, -0.4451, 0.8509, -1.4671, 1.3219, -0.5978, 0.9639, -1.0743, 0.9687, 0.6516, 0.486, 1.5006, 0.7695, 0.9119, 0.5293, 1.877, 1.1606, 0.9111, -1.8459, 0.9832, -1.2475, -0.7764, 0.2911, -2.1965, 1.057, -0.7077, 1.0119, -1.2437, 0.6594, 0.3864, 0.4232, 2.8226, 1.5044, 1.8869, 0.916, 1.5929, 0.8714, 0.3149, -1.9095, 1.0611, -1.0741, -1.0377, 0.0759, -2.7582, 0.6521, -0.7738, 1.1959, -0.5842, 0.7841, 0.9652, 1.0617, 3.9576, 2.5183, 2.4848, 0.3937, 0.8181, 0.5641, 0.8999, -1.0067, 0.9463, -1.1657, -1.1305, 0.1439, -2.7187, 0.2987, 0.0681, 1.4599, -0.5647, 1.0148, 1.3389, 0.9481, 3.1115, 2.9341, 1.5757, -1.2421, 0.4951, -0.4024, 1.8963, 0.2862, 0.8924, -1.1946, -1.2968, 0.1747, -2.9566, -0.2486, 0.5338, 1.0171, 0.2201, 1.0486, 1.2134, 0.5715, 1.3339, 2.5844, 0.7043, -1.862, 0.8633, -0.7516, 1.9794, 0.8909, 0.8261, -1.3122, -1.2913, 0.3323, -2.7826, -0.2244, 0.6714, 0.7127, 0.1809, 0.4453, 0.7924, 0.5005, 1.3902, 2.8182, 0.4783, -2.1206, 1.0777, -0.2783, 2.0515, 1.0269, 0.8281, -1.4676, -1.1685, 0.4136, -2.8376, -0.0611, 0.5375, 1.4189, -0.0096, 0.1021, 0.5958, 0.1458, 0.7274, 3.0121, 0.7793, -2.0943, 1.0676, -0.0218, 2.3897, 1.2064, 0.8615, -1.6546, -1.1575, 0.3416, -2.9893, -0.116, 0.0997, 2.0771, -0.0252, 0.0266, 0.4569, -0.2009, 0.4911, 2.5834, 0.8675, -2.0185, 1.0329, 0.1515, 2.3842, 1.1622, 0.8583, -1.8851, -1.131, 0.2167, -3.1567, -0.0785, 0.07, 2.2028, -0.0154, 0.1841, 0.0275, -0.8505, 0.1641, 2.2763, 1.048, -1.5486, 1.1876, 0.3807, 1.5912, 0.9085, 0.8312, -2.0255, -1.0961, 0.3838, -3.2983, 0.0475, -0.2079, 1.8445, -0.1699, -0.0886, -0.3409, -0.8494, 0.3856, 1.6174, 1.0631, -1.4689, 1.4644, 0.1497, 1.251, 0.9722, 0.8161, -1.9128, -0.8346, 0.5849, -3.1036, 0.2789, -0.7185, 1.179, 0.0786, -0.2951, -1.3624, -0.2915, 1.6966, 1.923, 1.7214, -0.5824, 1.7401, -0.518, 0.681, 1.3045, 0.9271, -0.7336, -0.0889, 1.1947, -1.8298, 1.2473, -0.0316, 0.9979, 0.4683, -0.2514, -1.5291, -0.4188, 1.1054, 0.8928, 1.4122, -0.945, 0.8456, -0.4053, 0.0162, 0.4941, 0.8762, 0.3034, 0.1784, 0.7943, -0.2818, 1.4153, 0.6476, 1.4181, 0.9596, -0.0518, -0.7572, 0.3473, -0.4729, 0.5842, 1.9926, -0.4963, 1.3493, 1.0792, 1.2737, -0.3619, 0.9419, -0.0429, 0.3034, 0.6554, -0.212, 1.8609, 1.0308, 1.8887, 0.5589, 0.3015, -1.0608, 1.3216, -0.7836, 1.8825, 2.1114, -0.6814, 0.7778, 2.0025, 2.3586, -2.7938, 1.0009, -0.4116, 0.0525, 0.4199, -0.6743, 1.9827, 1.1689, 2.0924, 0.4723, 0.5156, -0.6497, 1.8283, -0.2643, 2.4167, 2.0292, -0.7402, 0.9695, 1.5068, 1.7906, -2.9726, 0.9234, -0.7257, 0.1687, 0.7273, -1.2745, 1.7649, 1.5426, 2.0885, 0.4247, 0.8498, -0.2913, 1.7736, 0.9799, 1.8236, 1.567, -0.2358, 1.9458, 0.5028, 0.8943, -1.595, 0.7978, -1.7292, 0.5834, 1.4195, -1.969, 2.2097, 1.5927, 2.4798, 0.7497, 1.5814, 0.3411, 1.7623, 1.2276, 1.7105, 1.3197, 0.455, 1.7531, -0.084, 1.5097, -1.1691, 0.6762, -2.3212, 0.9976, 2.2068, -2.0808, 2.0014, 1.0794, 3.183, 1.2231, 2.0983, 0.351, 2.1585, 0.8838, 1.78, 1.29, 0.4922, 0.7225, -0.3879, 2.5277, -1.2145, 0.6466, -2.465, 0.9807, 2.1353, -1.9909, 2.4095, 0.7091, 3.1925, 0.3767, 1.6151, 0.7365, 2.3599, 0.7282, 0.87, 0.9919, 0.7802, 1.274, -0.0394, 2.5437, -0.274, 0.649, -2.5954, 1.0056, 1.9545, -2.6375, 2.3196, 0.5339, 3.5053, 0.2325, 1.4054, 0.3766, 1.8546, -0.4487, -0.4127, 1.2311, 1.1038, 2.7335, 0.7443, 1.6845, -0.1358, 0.6297, -2.8429, 0.9891, 2.0544, -3.0159, 2.0625, 0.5328, 3.0397, -0.215, 1.9774, 0.6316, 1.927, -1.0586, -0.8205, 1.1221, 0.8575, 2.2373, -0.2994, 0.4713, -0.5173, 0.6404, -2.9198, 0.9411, 2.1406, -3.1664, 1.694, -0.2336, 2.8396, -0.5878, 2.2049, 1.2077, 1.4928, -0.7696, -0.9225, 0.9896, 0.6088, 1.3475, -0.5077, 0.519, -0.049, 0.5629, -2.7682, 1.0607, 1.9557, -2.8481, 1.4869, -0.874, 3.1234, -0.3783, 2.3701, 1.4596, 0.8977, -0.6577, -1.1312, 0.5856, -0.2677, 0.9344, -0.0617, 1.0816, 0.3619, 0.2512, -1.9414, 1.0574, 0.9814, -1.9158, 1.0839, -1.0956, 2.8304, 0.1927, 1.719, 1.3105, 0.8314, -0.5316, -1.4686, 0.4601, -0.2674, 0.7657, 0.4524, 1.1693, 0.1516, -0.1032, -0.9084, 0.7659, 0.4381, -0.6704, 0.3772, 0.0641, 1.6455, 0.3144, 0.3404, 0.9028, 1.0627, 0.1885, -1.2693, 0.8417, 0.5117, -0.035, 0.6532, 0.8507, -0.4352, -0.4233, -0.5347, 0.5296, 0.2228, -0.0539, -0.0995, 1.0507, 1.01, 0.8818, -0.4428, 0.9231, 0.6511, 0.0172, -0.8531, 0.6741, 1.0445, -0.0501, 0.7596, 0.3895, -0.4268, -0.7128, -1.0801, 0.3876, 0.0562, -0.3632, -0.1225, 1.612, 0.4248, 0.6982, -0.2275, 0.5433, -0.0403, 0.303, 0.0133, 0.0209, 1.7439, 1.1599, 0.5004, 0.5183, 0.0794, 1.0566, -1.0684, -0.544, -0.184, -2.0503, -0.1465, 0.1297, 0.0654, -0.1403, 0.4999, -0.6749, -0.221, -1.8966, -0.3442, -0.3743, 1.4875, 0.1378, -1.4842, -0.4385, -1.0081, 1.6274, -0.8946, -0.6356, 0.222, -1.9909, 0.4063, -0.2644, 0.4854, -0.12, 0.8454, -0.3326, 0.3341, -1.6974, -0.8016, -0.0472, 1.7307, 0.3144, -1.373, -1.0097, -1.4918, 1.6065, -0.5821, -0.8791, 0.4214, -2.0858, 1.1772, -0.527, 0.8363, -0.7305, 0.6997, -0.5442, 0.5442, -0.7737, -0.6738, 0.3306, 0.9413, 0.3846, -1.0417, -0.3051, -2.0137, 1.4788, -0.5812, -1.0941, 0.2319, -1.7563, 1.5405, -0.5702, 0.883, -1.1222, 0.623, -0.4009, 0.6315, 0.4847, 0.2333, 0.7168, 0.3905, 0.6486, -1.2319, 0.2771, -1.7377, 1.3747, -0.5651, -1.3178, -0.1484, -1.7937, 1.234, -0.8336, 0.9132, -0.991, 0.9301, 0.0717, -0.1241, 0.1123, -0.3122, 1.2468, 0.3915, 1.1732, -1.4986, 0.4402, -0.3996, 1.2854, -0.6386, -1.3546, -0.1376, -2.0574, 1.0119, -0.9988, 0.1319, -1.6982, 0.6043, 0.2489, 0.2883, -0.1637, -0.6638, 1.5629, -0.1349, 1.6612, -0.3945, 1.0254, -0.9419, 1.1655, -1.027, -1.4333, 0.3298, -2.4603, 0.8441, -0.576, -0.0653, -1.1618, 0.6867, -0.6298, 0.4637, 0.3421, 0.2183, 1.6551, -0.8553, 2.0461, -0.2015, 1.2998, -1.4847, 1.0665, -1.2583, -1.2745, 0.6419, -2.4529, 0.6389, -0.2419, 0.2274, -0.8015, 1.1263, -1.1165, 0.4109, 0.3869, 0.2074, 1.8659, -1.2266, 1.88, -0.4931, 0.5399, -1.6658, 0.9895, -1.2665, -1.2376, 0.5151, -2.1188, 0.1447, 0.0746, 0.4486, -0.5808, 1.5214, -0.8779, 0.6793, -0.6203, 0.5159, 2.0831, -1.2702, 0.9109, -0.7328, 0.0044, -2.4109, 0.8654, -1.197, -1.1669, 0.3824, -1.6291, 0.0141, 0.2164, 0.2907, -1.273, 0.983, -0.9891, 0.6275, -1.1764, 0.5957, 1.5011, -1.1129, 0.7633, 0.2316, 0.7575, -2.277, 0.7732, -1.1324, -0.7495, 0.151, -1.813, 0.3572, -0.6966, 0.4343, -0.9657, 0.0976, -0.9516, 0.1584, -0.949, -0.1581, 0.8213, -0.7967, 0.7247, 0.7829, 0.7027, -1.922, 0.76, -1.6336, -0.3498, 0.5047, -2.3347, 0.8909, -0.9142, 1.24, -0.7062, 0.7048, -0.6144, 0.2554, -1.5244, -0.5785, 1.0239, 0.0008, 0.7024, -0.2611, 0.0189, -1.9661, 0.8084, -2.3774, -0.3299, 0.8829, -2.7822, 0.9421, -0.8457, 2.1751, -0.8165, 1.8656, -0.4111, 0.3296, -1.6896, -0.5524, 0.9473, 0.2547, 1.0341, -0.8018, 0.0863, -1.8424, 0.7837, -2.8225, -0.2891, 1.1776, -2.9058, 1.0858, -0.6483, 2.4805, -1.5039, 2.0847, 0.1265, 0.8228, -2.1863, -1.4983, 0.2554, 0.1034, 2.0793, -0.1815, 1.047, -1.4431, 0.752, -2.973, -0.314, 1.4473, -2.8402, 1.1235, -0.6414, 2.5554, -1.7063, 2.2159, 0.1693, 0.1694, -2.7977, -2.0719, 0.1934, -0.3632, 2.035, 0.0003, 1.6357, -0.562, 0.7417, -2.9335, -0.4384, 1.3681, -2.8286, 1.1323, -0.7559, 2.2781, -1.6015, 2.1837, 0.2151, 0.3377, -1.8345, -1.9484, -0.5116, -0.4594, 1.1204, -0.0439, 1.4094, -0.2111, 0.657, -2.9391, -0.3299, 1.3441, -2.9624, 1.1556, -0.4365, 2.5738, -1.0891, 2.0438, 0.193, 0.949, -1.2231, -2.1006, -0.9053, -0.5764, 0.2563, -0.0886, 0.357, 0.6838, 0.6127, -2.8981, -0.2034, 1.5128, -3.0442, 1.0551, -0.2508, 2.5584, -1.2328, 1.6989, 0.5975, 1.4216, -1.2437, -2.3588, -0.6439, -0.1887, 0.8479, -0.1078, -0.0457, 0.7145, 0.5745, -2.9069, -0.2316, 1.6105, -3.0384, 1.2452, -0.1514, 2.6252, -0.9543, 1.5125, 0.7008, 1.5576, -0.9053, -2.9573, -1.5222, -0.3068, 0.8224, -0.7949, -0.0229, 0.4506, 0.5793, -2.9142, -0.2464, 1.7348, -3.0595, 0.8552, -0.7727, 1.7855, -0.8734, 1.4776, 0.7844, 1.1744, -1.0609, -2.7794, -1.7363, -0.4107, 1.0955, -0.3742, 0.2497, 0.1313, 0.5791, -2.9161, -0.4097, 1.7958, -3.2576, 0.7367, -0.1986, 1.7733, -0.5567, 0.9562, 0.2773, 0.9935, -0.5213, -1.8885, -0.7895, -0.5427, 1.3643, 0.1227, -0.3726, -0.4172, 0.5708, -2.5037, -0.4242, 1.4887, -3.117, 0.5997, -0.1375, 1.6138, -0.2914, 0.3967, -0.4461, -0.0234, -0.4767, -2.0528, -0.5402, -0.6048, 0.9965, 0.2161, 0.47, 0.0264, 0.4986, -1.6151, -0.4875, 0.6253, -2.4157, 0.1607, -0.5264, 1.3873, 0.6826, -0.1104, -1.123, -0.5949, 0.6732, -1.6417, -0.3748, -0.075, 0.7431, -0.9783, 0.2106, 0.5498, 0.3174, -1.1602, -0.5213, -0.0204, -1.5592, -0.3128, -0.8659, 1.5165, 1.2677, -0.4305, -1.6437, -0.522, 0.6305, -1.6449, -0.5727, -0.2313, 1.0541, -1.9479, -0.1982, 0.0711, 0.153, -1.2331, -0.454, 0.0359, -0.9133, -0.0004, -0.6887, 1.4394, 0.6289, -0.5691, -1.0295, -0.2815, -0.0788, -2.271, -0.5318, -0.833, 0.9568, -1.6954, -0.5219, 0.0474, 0.0456, -0.9201, -0.3239, -0.0278, -0.9439, -0.3796, -0.141, 1.1287, 0.8314, -0.2444, 0.1888, -0.1895, -0.7263, -1.4671, -0.1271, -0.6175, 0.6731, -1.7832, -0.6299, 0.4767, 0.0007, -0.7597, -0.3499, 0.0376, -0.7199, -0.805, -0.5097, 0.546, 0.6194, -0.0365, 0.3923, -0.7504, -0.7955, -1.5306, -0.6544, -0.8538, 0.9915, -0.5823, -0.2612, 0.5548, 0.0164, -0.6741, -0.4926, -0.1115, -0.4621, -0.9105, -0.8914, 0.4399, 0.7683, -0.0043, -0.055, -1.2257, -0.9475, -1.849, -0.7389, -0.9951, 0.6834, -0.1837, -0.6147, -0.0125, -0.0409, -0.5336, -0.5128, -0.2182, -0.2352, -0.9162, -0.446, 0.4628, 0.828, 0.0851, -0.1197, -0.6681, -0.0857, -1.733, -0.9551, -1.4454, -0.2133, 0.3196, -0.6791, 0.0155, 0.0257, -0.5947, -0.3017, -0.0867, 0.0678, -1.1077, -1.3682, 0.1127, -0.8614, 0.0412, -1.0241, -0.1928, 0.4292, -2.0859, -0.521, -1.1647, 0.2625, 0.0935, -0.047, 0.61, 0.2924, -0.8542, -0.1082, 0.0662, -0.6213, -1.1458, -1.9765, -0.4961, -0.6312, -0.1462, -2.0565, -0.1417, -0.5845, -1.4811, -0.771, -0.0384, 1.1379, -0.638, -0.0877, 0.6639, 0.8358, -1.9466, -0.0877, 0.3964, -1.867, -0.8912, -0.8143, -0.9248, -0.429, 0.1397, -1.4932, 0.4277, -1.2292, -0.1208, -1.4953, 1.1601, 0.7051, -0.3854, -0.7344, 0.4588, 1.0417, -2.4667, -0.0673, 0.7483, -2.0353, -0.3408, -0.7304, -0.9027, -1.5721, 0.7361, -0.6554, 0.3417, -1.4963, -0.112, -1.6496, 0.8058, 0.5938, 0.4148, -1.0383, -0.1943, 1.1057, -2.3647, -0.3728, 0.7827, -2.289, 0.1086, -1.2059, -0.2404, -1.8473, 1.6819, -0.0759, 0.1759, -0.8955, -0.087, -0.7532, 0.7252, 1.5318, 0.581, -1.0097, -1.3751, 1.0596, -2.1784, -0.7745, 1.2343, -2.6337, 0.4523, -2.1442, 0.3008, -1.8358, 2.0402, 0.1794, 0.0218, -0.5679, -0.5393, -0.0054, -0.2509, 1.8807, -0.524, 0.1823, -2.1867, 1.1112, -2.1784, -0.7133, 1.1335, -2.5449, 0.7184, -2.0793, 0.173, -1.9429, 2.1363, 0.1614, 0.7365, -0.7828, -0.9212, -0.5194, -0.8159, 2.0279, -0.6041, 0.5221, -1.2508, 1.1577, -2.1848, -0.6597, 1.1076, -2.6527, 0.5933, -2.126, 0.1855, -2.2102, 2.3439, -0.5518, 0.2963, -0.7091, -0.8816, 0.0793, -0.4278, 1.4839, -0.7977, 0.0884, -1.5677, 1.16, -2.1304, -0.6021, 1.1747, -2.822, 0.6185, -2.1492, 0.5486, -2.0298, 1.8973, -0.8785, -0.5778, -1.4374, -0.649, 0.6166, -0.1985, 1.1988, -0.8325, -0.0996, -2.1812, 1.0975, -2.4062, -0.7466, 1.0866, -2.7506, 1.2828, -1.6524, 0.955, -2.2965, 1.1991, -1.2001, -0.3402, -1.483, -0.9962, -0.1531, -0.9013, 0.9711, -0.754, -0.0246, -2.0944, 1.0316, -2.6833, -0.6013, 1.248, -2.7526, 1.7229, -1.671, 1.0159, -2.1352, 1.3834, -0.7624, 0.5847, -0.9252, -0.7733, -1.0645, -1.0871, 1.1603, -0.9917, -0.016, -1.5764, 1.0235, -2.9442, -0.6095, 1.2781, -3.007, 1.577, -1.5349, 1.2328, -2.2237, 1.4139, -0.1815, 0.9755, -1.0688, -0.5306, -0.5684, -0.3521, 1.1183, -1.5544, -0.2465, -1.2139, 0.9836, -3.3285, -0.9047, 1.2735, -2.8237, 1.562, -1.3599, 1.3387, -2.3036, 1.1369, -0.6028, 0.5315, -1.9986, -0.8497, -0.6971, -0.8578, -0.0871, -2.332, -0.09, -1.776, 1.0352, -3.3976, -0.9161, 1.0947, -2.9031, 1.4542, -1.6677, 1.7744, -1.916, 1.0368, -1.0287, 0.4422, -2.0855, -0.4212, -0.4267, 0.1076, 0.022, -2.5636, -0.0979, -1.7279, 1.0477, -3.4952, -0.6953, 1.0973, -3.2085, 1.14, -1.6535, 1.9375, -1.8646, 0.5019, -1.6037, 0.5455, -1.9345, -0.0599, -0.7172, 0.6131, -0.1736, -2.0407, -0.5253, -1.3355, 0.9281, -3.6175, -0.5329, 1.5349, -3.3856, 0.9369, -1.4826, 1.9739, -1.5461, 0.0484, -1.8927, 0.3312, -2.3792, -0.4102, -1.0777, 0.2851, -1.1739, -2.1211, -0.6847, -0.7285, 0.7771, -3.5808, -0.4177, 1.6093, -3.0107, 1.0615, -1.685, 2.0563, -0.7548, -0.2134, -1.9666, -0.0673, -2.8023, -0.5252, -0.737, 0.9389, -0.6882, -2.4728, -0.6082, -0.4749, 0.5843, -3.3369, -0.5275, 1.2726, -2.5653, 1.4472, -1.6018, 2.1614, 0.1717, -0.457, -0.7628, -0.1282, -2.7853, -1.1149, -0.334, 0.7208, -0.0276, -2.4073, -0.5295, -1.0341, 0.2958, -2.5024, -0.8208, 0.7443, -1.6674, 0.7418, -0.994, 1.9157, 1.0066, -0.6267, 1.1016, 0.0358, -2.5032, -1.6049, 0.0427, 0.4991, -0.1571, -1.5529, -0.0373, -1.2994, 0.0481, -1.555, -1.0768, 0.3577, -0.7799, -0.571, -0.0741, 1.3749, 0.7439, -0.8975, 1.7932, 0.0055, -1.9222, -1.5566, 0.0002, 0.5704, -0.2266, -0.988, -0.4719, -1.2483, -0.0023, -1.4869, -1.0435, 0.0729, -0.4834, -0.8614, -0.3296, 1.1438, -0.0195, -0.7424, 1.3921, 0.2758, -1.3166, -1.1331, -0.4049, 0.4389, -0.0967, -0.8513, -0.8612, -1.2289, 0.291, -2.4551, -0.2387, 0.0681, -0.6744, 0.0207, -1.0757, 1.4108, -1.3645, 0.8597, -0.4158, 1.1415, -2.1523, -0.1782, -0.986, 0.3466, -0.5872, -0.984, 0.0669, -0.7815, 0.5789, -2.7316, -0.0711, 0.659, -1.3141, 0.4754, -1.0854, 1.0406, -2.1095, 1.3985, -1.0681, 0.6341, -2.199, -0.2585, -1.437, -0.265, -0.2046, -0.1693, 0.2136, -0.182, 0.7113, -2.4653, -0.2108, 0.9223, -1.5629, 1.269, -0.4525, 1.1817, -2.7572, 0.5623, -0.8341, 0.1249, -1.4886, -0.844, -1.3202, -0.637, -0.4235, -0.8142, 0.4959, -0.5224, 0.8632, -1.9906, -0.6106, 0.8821, -1.5688, 1.6258, -0.4336, 0.958, -2.391, 0.3555, -1.5894, -0.6501, -1.0229, -1.4061, -1.1649, -1.3356, -0.6095, -1.3722, 1.1901, -1.1869, 0.935, -1.6474, -0.9848, 0.3971, -1.6735, 1.4129, -0.0665, 0.4725, -2.15, -0.089, -2.4393, -0.4516, -0.7965, -2.1064, -1.3116, -0.9625, 0.9887, -0.511, 1.5771, -0.6528, 0.9545, -1.6089, -1.2212, -0.0225, -2.0555, 1.4535, -0.03, -0.0527, -2.222, -0.5056, -2.0519, -0.4911, -0.3829, -1.5869, -1.2229, -0.432, 1.3679, 0.1234, 1.2171, -0.4676, 0.9919, -1.802, -1.3844, -0.3313, -2.3472, 1.2141, -1.0263, -0.7282, -2.5293, -0.2965, -1.682, -0.4292, 0.0424, -1.2721, -1.0685, -0.3775, 1.4513, 0.3678, 1.4687, 0.1703, 1.0422, -1.6654, -1.3734, -0.4486, -2.2948, 0.6245, -1.6476, -0.8371, -2.2101, 0.2774, -2.281, -0.5399, -0.3456, -1.1997, -1.2902, -0.3632, 1.149, -0.3392, 1.1236, 0.6334, 0.9459, -1.3118, -1.2601, -0.1253, -1.9819, 0.3011, -1.555, -1.3305, -2.0323, 0.1346, -2.4562, -0.0444, -0.0634, -0.9448, -1.1602, 0.0091, 1.5844, 0.1258, 1.3614, 0.5058, 0.8547, -1.2645, -1.2381, 0.2477, -1.6764, 0.5804, -1.4902, -1.9739, -1.8473, -0.0336, -2.5171, -0.6509, -0.2378, -0.3902, -1.0539, 0.2173, 1.5939, 0.717, 1.3015, 0.1378, 0.9699, -1.4455, -1.1521, 0.695, -1.9063, 0.8942, -1.9088, -1.7716, -1.5691, -0.3611, -2.596, -1.2935, -0.383, -0.8329, -0.7656, -0.1154, 1.1535, 0.8875, 1.1161, 0.23, 1.0256, -1.6255, -1.2071, 0.5638, -2.0126, 1.0898, -2.2668, -1.6296, -1.6624, -0.5771, -2.2808, -0.9186, -0.1037, -0.905, -0.6106, 0.1303, 1.4542, 0.8042, 1.0086, 0.2103, 1.2078, -0.9505, -0.489, 1.2515, -1.057, 2.3063, -0.405, -0.487, -0.8494, 0.535, -1.1866, 0.0934, 0.1402, 0.0573, -0.8643, 0.3304, 1.9335, 0.3754, 0.656, -0.488, 1.3987, -0.7683, -0.0686, 1.7698, -0.9065, 2.7148, 0.408, 0.2036, -0.4263, 1.7146, -0.7987, 1.0297, 0.1552, 0.5024, -0.9302, 0.6974, 2.7158, 0.6924, 1.5672, -0.7401, 1.3995, -0.7242, 0.061, 1.5823, -0.8714, 2.7083, 0.3947, 0.7295, -0.258, 1.7072, -0.6888, 1.417, 0.7817, 1.1686, 0.6513, 1.5657, 2.1721, -0.0297, 0.8802, -0.9993, 1.3093, -0.763, -0.0314, 1.4229, -0.7626, 2.6581, 0.4067, 1.1056, -0.2264, 1.4267, -0.0655, 1.1, 0.7565, 0.6155, 1.3108, 1.5972, 1.8828, 0.4318, 0.5904, -1.1851, 1.2245, -0.8436, -0.1474, 1.3239, -0.5259, 2.8818, 0.1517, 0.8032, -0.5344, 1.2522, 0.1166, 1.1981, 1.15, 0.7459, 1.3959, 1.2003, 2.7539, 0.8901, 0.891, -0.4566, 1.1086, -0.8883, -0.1462, 1.1094, -0.429, 2.8567, -0.1397, 0.5374, -0.7978, 0.7922, -0.1075, 0.8824, 1.4596, 1.1513, 1.7349, 1.2734, 3.1228, 1.4453, 1.3079, 0.8537, 1.0457, -0.7889, -0.0575, 1.2317, -0.2523, 2.5571, -0.1018, 0.615, -1.3428, 0.3847, -0.1564, 0.8101, 1.7698, 1.4314, 1.681, 1.4191, 3.1234, 2.0906, 1.831, 0.5503, 0.9707, -0.8141, -0.0538, 1.019, -0.5051, 2.3499, 0.3637, 0.2173, -1.9519, 0.1772, -0.2719, 0.963, 1.4847, 1.9001, 1.548, 1.5224, 2.9959, 2.2401, 2.3293, 0.7818, 0.852, -0.8804, -0.1568, 0.7996, -0.5958, 2.0765, 0.0744, -0.069, -1.6837, -0.4152, -1.0449, 0.4884, 1.4362, 2.107, 1.5466, 1.114, 1.9232, 1.3235, 2.128, 0.8189, 0.6746, -1.0716, -0.0884, 0.5313, -1.0645, 1.5172, -0.7221, -0.1202, -1.5589, -1.5744, -2.29, -0.9104, 0.7185, 1.6722, 1.258, 0.322, 0.269, 0.3172, 1.7307, 0.9389, 0.6213, -0.8812, 0.1721, 0.1208, -1.8206, 0.9687, -0.8878, 0.0268, -1.368, -1.7061, -2.7084, -1.0599, -0.1037, 0.6477, 1.1966, 0.1327, 0.3702, 0.079, 1.1483, 0.5902, 0.5084, -0.5762, 0.4705, 0.253, -2.105, 0.5277, -0.6684, -0.5127, -0.7249, -1.2395, -2.7746, -1.214, -0.4719, -0.4972, 1.09, 0.4768, 0.5249, 0.947, 0.9869, -0.3749, 0.3187, -0.5672, 0.5111, 0.4277, -1.7417, 0.2459, -0.4132, -0.2768, -0.8809, -1.1816, -2.2458, -1.5608, -0.9953, -1.6097, 1.2206, -0.0096, 0.4288, 1.8545, 1.2879, -0.7024, 0.2275, -0.5496, 0.5973, 0.4745, -1.5402, -0.0885, -0.517, 0.22, -0.8568, -0.3747, -0.7104, -1.027, -1.0787, -1.902, 0.5423, -1.0795, 0.0538, 1.9625, 1.4081, -0.272, 0.0858, -0.6442, 0.5679, 0.4134, -1.2936, -0.5631, -1.1144, -0.5017, -1.8215, -0.5629, -0.3807, -0.773, -0.1566, -0.8901, 0.2836, -1.6609, -0.3706, 1.4633, 1.2671, 0.1104, -0.09, -0.5575, 0.2973, 0.1006, -0.633, -0.6684, -0.7135, -0.8347, -1.1896, -0.5094, -0.7851, -0.6642, -0.1033, -0.2893, 0.1192, -1.3531, -0.5523, 0.4186, 1.1911, 0.4554, -0.2233, -0.5011, 0.2164, 0.3245, -0.086, -0.7286, -0.0663, -0.2461, -0.5963, -0.9095, -1.0418, -1.3269, -0.24, 0.4863, 0.9901, -0.2441, -0.6334, 0.1644, 0.7113, 0.5633, -0.1757, -0.6262, 0.0074, 0.5407, 0.083, -0.5029, 0.1357, 0.3296, -0.82, -1.0768, -0.39, -0.8403, 0.2651, 0.3223, 1.435, 0.0372, -0.7608, -0.0895, 0.3865, 0.6413, -0.1491, -0.8334, -0.0261, 0.5295, -0.1898, -0.2244, -0.0435, 0.5192, -0.4572, -1.1105, -0.1708, -0.5065, -0.1996, 0.4175, 1.2616, -0.1364, -0.6469, -0.1749, 0.833, 0.587, -0.0755, -1.2371, -0.1144, 0.4122, -0.5615, 0.3261, -0.2752, 0.9844, -0.497, -1.1584, -0.7426, -0.634, -1.1021, -0.6362, 0.871, 0.0846, 0.977, 0.4445, 1.0869, -0.2953, 0.0284, -1.4086, -0.0098, 0.1328, -0.8691, 0.6062, -0.2404, 1.3807, -0.8103, -0.6744, -0.9886, -0.5228, -0.7313, 0.1204, 1.4475, -0.2346, 0.9888, 0.9068, 0.5087, -0.7545, 0.09, -1.5794, 0.0963, 0.2855, -0.9284, 0.5651, -0.3627, 1.285, -0.9869, -0.3927, -0.7929, -0.1126, -0.6163, 0.2052, 1.188, -0.568, 0.3728, 0.8322, 0.1772, -1.079, 0.1389, -1.7409, 0.2138, 0.2255, -0.9663, 0.36, -0.1998, 1.2938, -0.8646, -0.1434, -0.8127, -0.0318, -1.0184, -0.0517, -0.2131, -0.1694, -0.089, 0.8032, -0.4043, -0.986, 0.1013, -1.7432, 0.1689, 0.0662, -0.8048, 0.3943, -0.2477, 0.9727, -0.9289, -0.0686, -1.0204, 0.013, -0.5737, 0.2645, -1.3067, -0.4694, -0.4276, 1.41, -0.2093, -0.8855, 0.0368, -1.7152, -0.0385, -0.0489, -0.8979, 0.3705, -0.7159, 0.7514, -0.4328, 0.0819, -0.7944, 0.1218, 0.2299, 0.7, -0.9824, -0.3268, 0.1066, 1.1969, -0.3484, -1.5325, -0.0902, -1.5819, -0.0065, -0.0462, -0.8965, 0.4698, -0.6829, 0.6761, -0.6661, -0.0571, -0.3648, -0.1218, 0.0853, -0.3899, -0.4935, -0.0334, 0.6479, 1.1693, 0.1202, -1.6503, -0.2333, -1.4564, -0.0169, -0.2506, -0.6236, 0.8352, -0.1951, 0.3631, -0.8324, -0.2212, -0.1857, -0.1716, 0.343, -0.1201, -0.055, -0.0547, 0.4849, 0.7027, 0.3432, -1.2059, -0.4092, -1.1782, 0.0417, -0.5394, -0.641, 0.4754, 0.3854, 0.6205, -0.4271, -0.5597, -0.0398, -0.4904, 0.0682, -0.0259, -0.156, 0.394, 0.8875, 0.4875, 0.4865, -0.8691, -0.6818, -0.7771, 0.0682, -0.725, -0.1888, -0.3768, 0.54, 0.2824, -0.5766, -1.2488, 0.2801, -0.1368, 0.049, -0.2937, -0.4236, 0.652, 1.118, 0.506, 0.1784, -0.4806, -0.7029, -0.4818, -0.0269, -1.0586, -0.0094, -0.6999, 0.9396, 0.255, -0.4718, -1.0967, 0.4687, 0.5916, 1.0765, 0.3845, -0.3223, 0.9205, 1.0404, 0.7535, 0.5338, -0.0124, -0.5207, -0.3959, -0.7154, -1.4352, 0.5948, -0.35, 1.1522, 0.1847, -0.2621, -0.6115, 0.5925, 0.8086, 1.0333, 0.7196, 0.3997, 0.6188, 0.5211, 0.3088, -0.0847, -0.4274, 0.598, -0.9823, 0.1187, 0.7043, -0.4826, 1.0517, 1.2729, 0.2123, 0.3309, 0.3096, -0.6312, 0.0432, -1.1369, -0.0462, -1.343, 0.3173, 0.5676, 0.2459, 1.0088, -0.0128, 1.3226, -1.2291, 0.6029, 1.6209, -0.9676, 1.2593, 1.2875, 0.8046, 0.7768, 1.1898, -0.7335, 0.3927, -1.1446, 0.4591, -0.7244, 1.2718, 0.9697, 0.7731, 1.0438, 0.1966, 1.4237, -1.1691, 0.5055, 1.7048, -1.2378, 1.6375, 1.1334, 1.7243, 0.7325, 1.5577, -1.1162, 0.8851, -0.6421, 0.7761, 0.7321, 1.9449, 0.9148, -0.3719, 0.4876, -0.5303, 1.2742, -1.1567, 0.4367, 1.8172, -1.1589, 1.928, 0.7123, 2.6559, 0.0369, 1.7416, -0.6626, 1.048, 0.5277, 0.8949, 1.8704, 1.3266, 1.5898, -1.037, -0.3702, -1.5601, 1.1522, -1.3067, 0.4729, 1.5021, -1.2667, 1.9551, 0.3487, 3.1265, 0.1927, 1.7143, 0.3693, 1.9204, 1.096, 0.6323, 1.2297, 0.7774, 2.3725, -0.7195, 0.1155, -1.4727, 1.0122, -1.3006, 0.2671, 0.8771, -1.2923, 1.9148, 0.2318, 2.8031, 0.434, 1.6368, 0.6228, 1.47, 0.5886, 0.1453, 0.8456, 0.4934, 2.2972, -0.1611, 0.5681, -1.0746, 0.8823, -0.9674, 0.291, 1.1291, -1.4111, 1.4908, 0.585, 2.2325, 1.5195, 1.7459, 0.9643, 1.1507, 0.3111, -0.496, 1.0259, 0.8402, 2.2837, 0.3745, 0.2183, -0.3495, 0.8402, -0.6895, 0.1534, 1.1146, -1.7346, 0.9586, 0.6645, 2.0555, 1.8326, 1.1372, 0.7961, 1.415, 0.8249, -1.1897, 1.5058, 1.1996, 2.7669, 0.5351, 0.6545, 0.5203, 0.9309, -1.0774, -0.0772, 1.1124, -2.2963, 0.7735, 0.1894, 2.2563, 1.6653, 1.2295, 0.1854, 0.6237, 1.5744, -0.7114, 2.0062, 1.4556, 3.0824, 0.7004, 0.8649, 0.6172, 1.0626, -1.5353, -0.2328, 1.1448, -3.2724, 0.1802, -1.0075, 1.9925, 1.4722, 1.2373, -0.9518, -0.6733, 2.4865, 1.0006, 2.8164, 0.5411, 2.1711, -0.0313, -0.1649, -0.4055, 1.0453, -1.8201, -0.4873, 1.0142, -3.3471, 0.3502, -1.0129, 1.6591, 0.3075, 0.2317, -1.5545, -1.0153, 2.4042, 1.0645, 2.2797, -0.0435, 1.2656, -0.3397, -0.5982, -0.8435, 1.0163, -1.6821, -0.7028, 1.019, -2.7745, 0.2802, -0.5765, 1.6399, -0.2222, -0.0942, -1.3589, -0.2707, 2.5261, 0.7779, 1.7061, -1.2249, 0.2753, 0.295, -0.2284, -0.3725, 0.9619, -1.3216, -0.7065, 0.5523, -2.1504, -0.1958, -0.6028, 1.2897, -0.8645, 0.1412, -1.8866, -0.1265, 2.2711, 0.1488, 1.1169, -1.0053, 0.334, 1.3422, 0.1611, -1.2086, 0.8825, -1.8151, -0.1796, 0.517, -2.5854, 0.8784, -1.3351, 1.1497, -0.9351, -0.0217, -1.7518, 0.0298, 0.9266, -0.4114, -0.0726, -1.131, 0.842, 0.5581, 0.9343, -1.9138, 0.7649, -2.6762, 0.4368, 0.7314, -3.3746, 1.751, -1.4691, 1.4555, -1.0731, 0.5051, -0.9281, 1.0462, -0.2103, -1.3108, -0.2312, -0.6908, 0.6438, -0.6035, 0.6339, -1.7754, 0.786, -3.1514, 0.6156, 0.9702, -3.4889, 2.3852, -0.5865, 2.4781, -1.1548, 0.5747, -0.9631, 1.2281, -0.9368, -1.439, 0.3216, 0.0838, 0.9221, -0.9493, 0.6617, -1.4711, 0.8338, -3.3939, 0.5852, 1.0407, -3.7705, 2.474, -0.442, 3.1351, -0.8497, 0.3216, -0.8935, 1.0574, -1.6943, -1.2132, 0.8625, 0.2185, 1.0696, -0.4465, 1.523, -1.6919, 0.8405, -3.4931, 0.5986, 0.8426, -4.2255, 2.416, -0.3076, 3.6913, -0.9117, 0.4433, -0.1389, 1.6479, -1.6847, -1.4597, 0.5409, -0.9696, 0.3489, 0.0093, 2.3065, -1.8264, 0.8364, -3.6501, 0.3973, 0.5605, -4.2918, 2.5611, 0.1624, 4.2112, -1.3501, 0.5048, -0.1207, 1.6129, -2.0093, -2.6173, -0.3373, -0.7928, 0.4663, -0.5325, 2.0416, -1.6212, 0.8724, -3.4509, 0.2718, 0.5143, -3.9033, 2.4479, 0.1496, 4.0652, -1.4376, 0.4925, -0.7792, 1.3403, -1.6411, -2.6129, -0.6613, -0.0097, 0.994, -0.7669, 1.5603, -1.3301, 0.9331, -2.991, 0.3979, 0.794, -3.391, 2.4683, 0.431, 3.8331, -1.0309, 1.4592, -0.4811, 1.3769, -0.5142, -1.7105, -0.2822, -0.1874, 0.9035, -1.0541, 0.5013, -1.5916, 0.882, -2.6107, 0.612, 1.4018, -2.5925, 2.5684, 0.1014, 3.2032, -0.685, 1.937, 0.1138, 1.4019, 0.6935, 0.196, 0.9863, -0.5055, 0.4438, -1.4782, 0.403, -1.5814, 0.9392, -1.84, 0.4309, 1.3377, -1.8772, 2.2282, 0.0851, 2.567, -0.5085, 1.2025, -0.1812, 0.9245, 0.781, 0.5053, 1.1645, -0.5418, -0.5363, -1.445, 2.1709, -0.3913, 0.891, -1.1871, -0.1705, 1.0155, -1.1539, 1.3524, 0.5179, 2.0579, 0.3092, 0.7163, -0.8766, 0.5858, 0.724, 0.3198, 0.2781, -0.4701, -0.0086, -1.0412, 2.29, 0.645, 0.8391, -0.9518, -0.4119, 0.9632, -0.8804, 0.7645, 0.3255, 1.6155, 0.6684, 0.5781, -0.853, 0.404, 0.7049, 0.4638, -0.0074, 0.1536, 0.4761, 0.0926, 1.771, 0.5702, 0.7288, -0.8559, -0.3149, 0.886, -0.6163, 0.7234, 0.2, 1.7566, 0.4041, 0.3229, -0.1768, 0.2682, 0.2563, 0.1378, 0.033, 0.5824, 0.3792, 0.3073, 1.3307, -0.0656, 0.5337, -0.7431, 0.0097, 0.5426, -0.316, 0.4589, -0.3612, 1.8919, 0.1799, 0.3292, 0.2757, 1.2201, -0.0596, 0.1325, 0.3297, -0.1543, -0.1813, 0.204, 2.3266, 0.1275, 0.274, -0.4945, 0.2072, 0.0509, -0.1198, 0.3098, -0.2269, 1.9934, 0.6127, 0.7448, 0.5998, 0.7797, -0.4709, 0.0045, 0.3804, -0.8918, -0.504, 0.5191, 2.6212, -0.1689, 0.0646, -0.4127, 0.223, 0.2178, -0.3046, 0.6912, 0.5805, 1.6253, 1.289, 0.827, 1.5944, -0.7039, -0.0773, -0.0937, -0.3317, -1.6958, 0.0824, 1.0555, 1.7838, -0.0976, -0.0823, -0.5005, 0.316, 0.6685, 0.0627, 0.533, 0.4231, 0.9776, 1.1727, 0.5452, 1.5978, -0.8948, 0.4912, 0.7221, -0.1729, -1.7608, -0.6128, 0.6145, 1.0769, -0.1973, -0.4155, -1.1648, 0.304, 0.7134, 0.1466, 0.7424, 0.5053, 0.5248, 0.4862, 0.0756, 1.3596, -0.7844, 0.7675, 1.2274, -0.6181, -1.8648, -1.0819, -0.0358, 1.072, -0.6627, 0.3666, -1.189, -0.3293, -0.2224, -1.3648, -0.1574, 0.2406, 0.0093, -0.352, 0.0638, -0.6619, -1.9095, -1.4406, 0.0445, -0.9525, 0.389, -0.5226, -0.8403, -0.4182, -1.281, 1.4968, -1.1336, -0.5462, -0.0643, -1.722, -0.0546, -0.1571, 0.1018, -0.0841, 0.8844, -0.8025, -0.1321, -1.6859, 0.0887, -0.7715, 1.8454, 0.091, -0.8339, -0.99, -1.6337, 1.7253, -0.9894, -0.6456, 0.084, -1.6879, 0.4817, -0.4775, 0.3588, -0.3574, 0.8894, -0.9354, 0.3496, -1.5236, -0.1283, -0.5743, 1.5566, 0.3512, -0.5493, -1.0422, -1.9134, 1.614, -0.7072, -1.0157, 0.236, -1.8029, 1.6474, -0.3948, 0.9432, -1.4396, 0.4416, -0.8797, 0.6491, 0.0692, 0.4253, 0.8993, 0.685, 0.8517, -0.6253, 0.1077, -2.3374, 1.5351, -0.7946, -1.2238, 0.0009, -1.6059, 1.5958, -0.6129, 0.8688, -1.2279, 0.5079, -0.4038, 0.6662, 1.0768, 0.7526, 1.0539, 0.4115, 1.2559, -0.6284, -0.0529, -1.487, 1.4458, -0.8888, -1.43, -0.1458, -2.0932, 1.177, -0.4033, 0.6702, -1.0036, 0.2627, 0.1494, 0.7803, 0.772, 0.2923, 0.6804, 0.3112, 1.1736, -0.8031, 0.0616, -0.7942, 1.2051, -1.1235, -1.3272, 0.076, -2.3833, 1.116, 0.3514, 1.0238, -0.629, -0.578, -0.074, 0.5481, 0.2845, 0.2461, 0.4423, 0.2053, 1.1484, -0.9603, 0.2335, -1.5567, 1.0374, -1.382, -1.3116, 0.1841, -2.6062, 0.8403, 0.5858, 1.6429, 1.3085, -0.4038, -0.2307, -0.3769, -0.3185, 0.9012, -0.6568, 0.0577, 0.77, -1.404, 0.0249, -1.3308, 0.9965, -1.3344, -1.2281, 0.2985, -2.8445, 0.5012, 0.6277, 1.5768, 1.726, -0.8138, -0.0601, -0.6653, -0.0218, 1.9426, -0.5523, 0.3547, 0.7709, -1.1372, -0.255, -1.0934, 0.9642, -1.3738, -1.3261, 0.3767, -2.9003, 0.5356, 0.9177, 1.4567, 1.3741, -1.0356, 0.0776, -0.9785, 0.2244, 1.8168, -0.0564, 0.4365, 0.1289, -1.8679, -0.5584, -0.7362, 0.8878, -1.3764, -1.4539, 0.3648, -2.8834, 0.4051, 0.7101, 1.4702, 2.0519, -0.9104, 0.3943, -1.3884, 0.425, 1.0882, -0.375, 0.135, 0.3332, -1.2841, -0.6182, 0.51, 0.8365, -1.3941, -1.4363, 0.4532, -3.0796, 0.2379, 0.3502, 1.3079, 2.4391, -0.7326, 0.8961, -2.0571, 0.8595, 1.5752, 0.3918, -0.6794, -0.3566, -0.895, -0.6121, 0.7459, 0.8492, -1.7555, -1.2991, 0.3409, -3.1528, 0.2008, -0.112, 1.6658, 2.1209, 0.0022, 0.4885, -2.1169, 0.5318, 1.7967, 1.4723, -0.4102, 0.0271, -1.4679, -0.2307, 0.767, 0.8642, -1.9925, -1.3098, 0.4204, -3.1342, 0.0733, -0.3141, 1.6269, 1.5607, 0.2656, 0.3662, -2.4215, -0.0565, 0.7906, 1.2386, -0.3281, 0.4338, -1.682, -0.221, 1.3303, 0.9058, -1.9783, -1.1835, 0.386, -2.9568, 0.2212, -0.4797, 1.4088, 1.0482, 0.7537, 0.3954, -1.8445, -0.627, 0.2038, 0.9912, -0.5542, 0.4068, -2.2158, -0.21, 1.2398, 0.9093, -1.9948, -1.1126, 0.1697, -3.0204, 0.1904, -0.6529, 1.3315, 0.5508, 1.6431, 0.0217, -1.0039, -0.4891, -0.007, 1.2787, -0.328, 0.9904, -2.4332, -0.5695, 1.0765, 0.9215, -1.7091, -0.9739, 0.1192, -3.0821, -0.0588, -1.1468, 0.6008, -0.3547, 1.3959, -0.2811, -0.839, -0.1867, 0.8827, 1.7833, -0.2133, 1.4815, -2.7525, -1.5469, 0.1746, 0.7248, -1.2428, -0.8628, 0.1353, -2.4824, -0.518, -1.2627, 0.0565, -0.4842, 0.1761, -0.0416, -0.6065, -0.381, 0.8207, 1.9225, 0.3415, 1.6001, -2.4352, -1.7236, 0.0082, 0.2268, -0.8224, -0.5604, -0.0869, -1.5987, -0.9492, -0.5078, 0.0332, -0.1281, -0.3252, 1.4468, 0.3357, -0.7017, -0.4368, 0.7212, 0.4283, 0.9078, -1.6321, -1.4381, 0.8286, -0.0911, -0.6602, -0.4532, -0.4274, -1.218, -1.0193, 0.6433, 0.5005, 0.5288, -0.1255, 2.3217, 0.6325, -0.483, -1.326, -0.9525, 0.2948, -0.0812, -0.7485, -1.0548, 0.3612, -0.1757, -0.8951, -0.4481, -0.2777, -1.2756, -1.2552, 0.2088, 0.2188, 0.7268, 0.022, 2.2258, 0.7296, -0.1137, -1.2015, -0.9451, 0.3332, 0.5307, 0.2421, -0.8618, -0.8645, -0.2738, -0.9815, -0.5082, -0.4675, -1.2887, -1.3713, 0.5077, 0.4508, 0.5271, -0.3051, 1.751, 0.0583, -1.0238, -1.451, -1.0215, 0.0962, 0.4249, 0.1214, -0.9639, -0.8335, -0.2556, -0.8746, -0.5566, -0.5126, -1.4643, -1.4044, 0.8874, 0.6941, 0.9663, -0.2771, 1.5738, -0.0468, -1.3609, -1.7803, -1.3467, -0.0026, 0.0211, -0.5858, -1.4892, -0.8774, -0.2536, -0.8531, -0.6528, -0.5968, -1.3178, -1.6518, 0.6563, 0.6269, 1.0551, -0.3043, 1.2339, -0.1698, -1.4815, -1.6743, -0.9912, 0.1833, 0.2199, -0.6271, -1.681, -1.5538, -0.2169, -1.1595, -0.2926, -0.677, -0.9051, -1.3295, 0.0926, 1.0175, 0.2457, 0.1618, 0.5925, 0.0377, -1.7234, -0.847, -0.771, 0.865, 0.7233, -0.1253, -1.3477, -1.7687, 0.3511, -1.7208, 0.4358, -0.2246, -0.4797, 0.2799, -0.3021, 2.5045, -0.9345, 0.8467, -0.9303, 0.4578, -1.4763, 0.4302, 0.2939, 1.5549, -0.393, -0.0118, -0.7443, -0.0213]).reshape(1, -1)

# Find the nearest neighbors
distances, indices = odel.kneighbors(query_features_2d)


# Print the indices of nearest neighbors
print("Indices of nearest neighbors:", indices)

# Optionally, you can print the distances as well
print("Distances to nearest neighbors:", distances)
# Retrieve labels of nearest neighbors using list comprehension
nearest_neighbor_labels = [train_labels[i] for i in indices[0]]

# Print the labels of nearest neighbors
print("Labels of nearest neighbors:", nearest_neighbor_labels)


Indices of nearest neighbors: [[ 8  7 16 53 11]]
Distances to nearest neighbors: [[2.90739311e-03 1.27337164e+02 1.28563800e+02 1.32281625e+02
  1.33552524e+02]]
Labels of nearest neighbors: ['000002', '000135', '000155', '000237', '000161']


K MEAN CLUSTERING

In [10]:
df

DataFrame[Feature: vector, Label: string]

In [13]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("KMeans Clustering") \
    .getOrCreate()

# Step 2: Read the Parquet file into a PySpark DataFrame
df = spark.read.parquet("dataFrame")

# Step 3: Assemble features into a single vector column
assembler = VectorAssembler(inputCols=["Feature"], outputCol="features")
df_assembled = assembler.transform(df)

# Define the number of clusters
k = 5  # You can adjust this value as needed

# Step 4: Trains a k-means model
kmeans = KMeans().setK(k).setSeed(1)
model = kmeans.fit(df_assembled)

# Step 5: Make predictions
predictions = model.transform(df_assembled)

# Step 6: Shows the result
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

# Optionally, you can explore the clusters and their members
predictions.select("Label", "prediction").show()


24/05/12 14:57:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/12 14:57:48 WARN DAGScheduler: Broadcasting large task binary with size 1000.0 KiB


Cluster Centers: 
[-0.1428032  -0.4593219  -0.68772249 ... -0.76118296  0.10703895
 -0.21033532]
[0.50359598 0.33533784 0.1243724  ... 0.29548825 0.28319363 0.15379789]
[-0.13111757 -0.21710865 -0.16304273 ... -0.30768954 -0.39656685
 -0.13188878]
[-0.65211845 -0.23837246 -0.01386523 ... -0.07705997  0.02465256
 -0.13874892]
[ 0.24884372 -0.02370173  0.63841237 ...  0.35710878  0.31952052
  0.87379204]
+------+----------+
| Label|prediction|
+------+----------+
|000176|         1|
|000177|         1|
|000178|         2|
|000179|         1|
|000181|         1|
|000182|         1|
|000183|         2|
|000184|         1|
|000185|         1|
|000188|         3|
|000189|         2|
|000190|         2|
|000191|         3|
|000192|         3|
|000193|         2|
|000194|         2|
|000195|         3|
|000196|         2|
|000197|         2|
|000198|         2|
+------+----------+
only showing top 20 rows



In [14]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

# Evaluate the model
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))


24/05/12 14:59:03 WARN DAGScheduler: Broadcasting large task binary with size 1004.4 KiB


Silhouette with squared euclidean distance = 0.107831203617332
